In [ ]:
import os
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from transformers import DistilBertModel, DistilBertTokenizer
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Constants
NUM_CLASSES = 4
MAX_TEXT_LENGTH = 24
BATCH_SIZE = 32
EPOCHS = 10
IMAGE_SIZE = 224
LEARNING_RATE = 2e-5

In [ ]:
# Paths (Replace with actual dataset paths)
TRAIN_PATH = "/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train"
VAL_PATH = "/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Val"
TEST_PATH = "/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Test"

In [ ]:
# Text Preprocessing
def extract_text_from_path(file_path):
    file_name = os.path.basename(file_path)
    file_name_no_ext, _ = os.path.splitext(file_name)
    text = file_name_no_ext.replace('_', ' ')
    return re.sub(r'\d+', '', text)  # Remove numbers

In [ ]:
# Load dataset paths
def get_files_with_labels(root_path):
    image_paths, texts, labels = [], [], []
    class_folders = sorted(os.listdir(root_path))
    label_map = {class_name: idx for idx, class_name in enumerate(class_folders)}

    for class_name in class_folders:
        class_path = os.path.join(root_path, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(file_path)
                    texts.append(extract_text_from_path(file_path))
                    labels.append(label_map[class_name])
    return image_paths, texts, labels

In [ ]:
# Image Transformations
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Dataset Class
class HybridDataset(Dataset):
    def __init__(self, image_paths, texts, labels, tokenizer, transform, max_len=MAX_TEXT_LENGTH):
        self.image_paths = image_paths
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load image
        image = Image.open(self.image_paths[idx]).convert('RGB')
        image = self.transform(image)

        # Process text
        encoding = self.tokenizer(
            self.texts[idx], padding='max_length', truncation=True,
            max_length=self.max_len, return_tensors='pt'
        )

        return {
            'image': image,
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
# Fusion Module
class AttentionFusion(nn.Module):
    def __init__(self, img_dim=1024, text_dim=1024):
        super().__init__()
        self.attn = nn.Linear(img_dim + text_dim, 1)

    def forward(self, img_features, text_features):
        weights = torch.sigmoid(self.attn(torch.cat([img_features, text_features], dim=1)))
        return weights * img_features + (1 - weights) * text_features

In [ ]:
# Hybrid Model
class HybridModel(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()
        self.image_model = models.resnet50(weights="IMAGENET1K_V2")
        self.image_model = nn.Sequential(*list(self.image_model.children())[:-1])
        self.image_fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )

        self.text_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.text_fc = nn.Sequential(
            nn.Linear(768, 1024),
            nn.BatchNorm1d(1024), 
            nn.ReLU()
        )

        self.fusion = AttentionFusion(img_dim=1024, text_dim=1024)
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, images, input_ids, attention_mask):
        img_features = self.image_model(images).squeeze()
        # Ensure img_features has the right shape
        if len(img_features.shape) == 1:
            img_features = img_features.unsqueeze(0)
        img_features = self.image_fc(img_features)
        
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)[0]
        text_features = self.text_fc(text_output[:, 0, :])
        fused_features = self.fusion(img_features, text_features)
        return self.classifier(fused_features)

In [ ]:
# Training and Evaluation Functions
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=EPOCHS):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss, correct_train, total_train = 0, 0, 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, input_ids, attn_mask, labels = batch['image'].to(device), batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(images, input_ids, attn_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = outputs.argmax(dim=1)
            correct_train += (predictions == labels).sum().item()
            total_train += labels.size(0)

        train_acc = correct_train / total_train
        train_loss /= len(train_loader)

        model.eval()
        val_loss, correct_val, total_val = 0, 0, 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                images, input_ids, attn_mask, labels = batch['image'].to(device), batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
                outputs = model(images, input_ids, attn_mask)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                predictions = outputs.argmax(dim=1)
                correct_val += (predictions == labels).sum().item()
                total_val += labels.size(0)

        val_acc = correct_val / total_val
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        scheduler.step()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_hybrid_model.pth')
            print("Saved Best Model!")

    model.load_state_dict(torch.load('best_hybrid_model.pth'))
    return model

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            images, input_ids, attn_mask, labels = batch['image'].to(device), batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
            outputs = model(images, input_ids, attn_mask)
            preds = outputs.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = (np.array(all_preds) == np.array(all_labels)).mean()
    print(f"Test Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(all_labels, all_preds))

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    return all_preds, all_labels

In [ ]:
# Main Execution
if __name__ == "__main__":
    # Load tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    # Load datasets
    train_image_paths, train_texts, train_labels = get_files_with_labels(TRAIN_PATH)
    val_image_paths, val_texts, val_labels = get_files_with_labels(VAL_PATH)
    test_image_paths, test_texts, test_labels = get_files_with_labels(TEST_PATH)

    train_dataset = HybridDataset(train_image_paths, train_texts, train_labels, tokenizer, train_transform)
    val_dataset = HybridDataset(val_image_paths, val_texts, val_labels, tokenizer, test_transform)
    test_dataset = HybridDataset(test_image_paths, test_texts, test_labels, tokenizer, test_transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    # Initialize model
    model = HybridModel().to(device)

    # Define optimizer and scheduler
    optimizer = optim.AdamW([
        {"params": model.image_model.parameters(), "lr": 1e-4},
        {"params": model.text_model.parameters(), "lr": 5e-6},
        {"params": model.classifier.parameters(), "lr": 1e-4},
    ])
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("Starting training...")
    model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=EPOCHS)

    # Evaluate on test set
    print("Evaluating on test set...")
    evaluate_model(model, test_loader)

/home/daniel.ikponmwen1/software/miniforge3/envs/pytorch/lib/python3.12/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using device: cuda
Starting training...

Epoch 1/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 1/10:   0%|          | 1/364 [00:08<53:11,  8.79s/it]
Epoch 1/10:   1%|          | 2/364 [00:09<23:16,  3.86s/it]
Epoch 1/10:   1%|          | 3/364 [00:09<13:44,  2.28s/it]
Epoch 1/10:   1%|          | 4/364 [00:09<09:13,  1.54s/it]
Epoch 1/10:   1%|▏         | 5/364 [00:10<07:20,  1.23s/it]
Epoch 1/10:   2%|▏         | 6/364 [00:11<05:35,  1.07it/s]
Epoch 1/10:   2%|▏         | 7/364 [00:11<04:29,  1.32it/s]
Epoch 1/10:   2%|▏         | 8/364 [00:11<03:50,  1.54it/s]
Epoch 1/10:   2%|▏         | 9/364 [00:16<10:19,  1.74s/it]
Epoch 1/10:   3%|▎         | 10/364 [00:16<07:51,  1.33s/it]
Epoch 1/10:   3%|▎         | 11/364 [00:16<06:08,  1.04s/it]
Epoch 1/10:   3%|▎         | 12/364 [00:17<04:56,  1.19it/s]
Epoch 1/10:   4%|▎         | 13/364 [00:20<10:00,  1.71s/it]
Epoch 1/10:   4%|▍         | 14/364 [00:21<07:48,  1.34s/it]
Epoch 1/10:   4%|▍         | 15/364 [00:21<06:09,  1.06s/it]
Epoch 1/10:   4%|▍         | 16/364 [00:22<04:58,  1.16it/s]
Epoch 1/10:   5%|▍         | 17/364 [00:26<10:16,  1.78s/it]
Epoch 1/10:   5%|▍         | 18/364 [00:26<07:58,  1.38s/it]
Epoch 1/10:   5%|▌         | 19/364 [00:26<06:16,  1.09s/it]
Epoch 1/10:   5%|▌         | 20/364 [00:27<05:03,  1.13it/s]
Epoch 1/10:   6%|▌         | 21/364 [00:31<09:48,  1.71s/it]
Epoch 1/10:   6%|▌         | 22/364 [00:31<08:30,  1.49s/it]
Epoch 1/10:   6%|▋         | 23/364 [00:32<06:38,  1.17s/it]
Epoch 1/10:   7%|▋         | 24/364 [00:32<05:16,  1.07it/s]
Epoch 1/10:   7%|▋         | 25/364 [00:36<09:31,  1.69s/it]
Epoch 1/10:   7%|▋         | 26/364 [00:36<07:46,  1.38s/it]
Epoch 1/10:   7%|▋         | 27/364 [00:37<06:05,  1.09s/it]
Epoch 1/10:   8%|▊         | 28/364 [00:37<04:55,  1.14it/s]
Epoch 1/10:   8%|▊         | 29/364 [00:41<09:35,  1.72s/it]
Epoch 1/10:   8%|▊         | 30/364 [00:42<07:55,  1.42s/it]
Epoch 1/10:   9%|▊         | 31/364 [00:42<06:11,  1.12s/it]
Epoch 1/10:   9%|▉         | 32/364 [00:42<04:59,  1.11it/s]
Epoch 1/10:   9%|▉         | 33/364 [00:46<09:38,  1.75s/it]
Epoch 1/10:   9%|▉         | 34/364 [00:47<07:34,  1.38s/it]
Epoch 1/10:  10%|▉         | 35/364 [00:47<05:54,  1.08s/it]
Epoch 1/10:  10%|▉         | 36/364 [00:47<04:48,  1.14it/s]
Epoch 1/10:  10%|█         | 37/364 [00:51<09:19,  1.71s/it]
Epoch 1/10:  10%|█         | 38/364 [00:52<07:48,  1.44s/it]
Epoch 1/10:  11%|█         | 39/364 [00:52<06:06,  1.13s/it]
Epoch 1/10:  11%|█         | 40/364 [00:53<04:55,  1.10it/s]
Epoch 1/10:  11%|█▏        | 41/364 [00:56<09:01,  1.68s/it]
Epoch 1/10:  12%|█▏        | 42/364 [00:57<07:37,  1.42s/it]
Epoch 1/10:  12%|█▏        | 43/364 [00:57<05:57,  1.11s/it]
Epoch 1/10:  12%|█▏        | 44/364 [00:58<04:47,  1.11it/s]
Epoch 1/10:  12%|█▏        | 45/364 [01:01<08:42,  1.64s/it]
Epoch 1/10:  13%|█▎        | 46/364 [01:02<07:38,  1.44s/it]
Epoch 1/10:  13%|█▎        | 47/364 [01:03<05:57,  1.13s/it]
Epoch 1/10:  13%|█▎        | 48/364 [01:03<04:49,  1.09it/s]
Epoch 1/10:  13%|█▎        | 49/364 [01:06<08:27,  1.61s/it]
Epoch 1/10:  14%|█▎        | 50/364 [01:07<07:32,  1.44s/it]
Epoch 1/10:  14%|█▍        | 51/364 [01:08<05:51,  1.12s/it]
Epoch 1/10:  14%|█▍        | 52/364 [01:08<04:40,  1.11it/s]
Epoch 1/10:  15%|█▍        | 53/364 [01:11<08:19,  1.61s/it]
Epoch 1/10:  15%|█▍        | 54/364 [01:12<07:02,  1.36s/it]
Epoch 1/10:  15%|█▌        | 55/364 [01:12<05:31,  1.07s/it]
Epoch 1/10:  15%|█▌        | 56/364 [01:13<04:27,  1.15it/s]
Epoch 1/10:  16%|█▌        | 57/364 [01:16<08:43,  1.70s/it]
Epoch 1/10:  16%|█▌        | 58/364 [01:17<07:00,  1.37s/it]
Epoch 1/10:  16%|█▌        | 59/364 [01:17<05:29,  1.08s/it]
Epoch 1/10:  16%|█▋        | 60/364 [01:18<04:26,  1.14it/s]
Epoch 1/10:  17%|█▋        | 61/364 [01:22<08:49,  1.75s/it]
Epoch 1/10:  17%|█▋        | 62/364 [01:22<06:47,  1.35s/it]
Epoch 1/10:  17%|█▋        | 63/364 [01:22<05:19,  1.06s/it]
Epoch 1/10:  18%|█▊        | 64/364 [01:23<04:24,  1.13it/s]
Epoch 1/10:  18%|█▊        | 65/364 [01:27<08:35,  1.73s/it]
Epoch 1/10:  18%|█▊        | 66/364 [01:27<06:36,  1.33s/it]
Epoch 1/10:  18%|█▊        | 67/364 [01:27<05:12,  1.05s/it]
Epoch 1/10:  19%|█▊        | 68/364 [01:28<04:18,  1.14it/s]
Epoch 1/10:  19%|█▉        | 69/364 [01:32<08:39,  1.76s/it]
Epoch 1/10:  19%|█▉        | 70/364 [01:32<06:38,  1.35s/it]
Epoch 1/10:  20%|█▉        | 71/364 [01:33<05:11,  1.06s/it]
Epoch 1/10:  20%|█▉        | 72/364 [01:33<04:17,  1.14it/s]
Epoch 1/10:  20%|██        | 73/364 [01:37<08:12,  1.69s/it]
Epoch 1/10:  20%|██        | 74/364 [01:37<06:47,  1.41s/it]
Epoch 1/10:  21%|██        | 75/364 [01:38<05:20,  1.11s/it]
Epoch 1/10:  21%|██        | 76/364 [01:38<04:16,  1.12it/s]
Epoch 1/10:  21%|██        | 77/364 [01:42<07:57,  1.66s/it]
Epoch 1/10:  21%|██▏       | 78/364 [01:42<06:54,  1.45s/it]
Epoch 1/10:  22%|██▏       | 79/364 [01:43<05:23,  1.14s/it]
Epoch 1/10:  22%|██▏       | 80/364 [01:43<04:21,  1.09it/s]
Epoch 1/10:  22%|██▏       | 81/364 [01:47<07:52,  1.67s/it]
Epoch 1/10:  23%|██▎       | 82/364 [01:48<07:00,  1.49s/it]
Epoch 1/10:  23%|██▎       | 83/364 [01:48<05:26,  1.16s/it]
Epoch 1/10:  23%|██▎       | 84/364 [01:49<04:21,  1.07it/s]
Epoch 1/10:  23%|██▎       | 85/364 [01:52<07:20,  1.58s/it]
Epoch 1/10:  24%|██▎       | 86/364 [01:53<06:51,  1.48s/it]
Epoch 1/10:  24%|██▍       | 87/364 [01:53<05:20,  1.16s/it]
Epoch 1/10:  24%|██▍       | 88/364 [01:54<04:16,  1.08it/s]
Epoch 1/10:  24%|██▍       | 89/364 [01:57<07:01,  1.53s/it]
Epoch 1/10:  25%|██▍       | 90/364 [01:58<06:30,  1.43s/it]
Epoch 1/10:  25%|██▌       | 91/364 [01:58<05:05,  1.12s/it]
Epoch 1/10:  25%|██▌       | 92/364 [01:59<04:05,  1.11it/s]
Epoch 1/10:  26%|██▌       | 93/364 [02:02<07:10,  1.59s/it]
Epoch 1/10:  26%|██▌       | 94/364 [02:03<06:30,  1.45s/it]
Epoch 1/10:  26%|██▌       | 95/364 [02:03<05:03,  1.13s/it]
Epoch 1/10:  26%|██▋       | 96/364 [02:04<04:08,  1.08it/s]
Epoch 1/10:  27%|██▋       | 97/364 [02:07<06:44,  1.52s/it]
Epoch 1/10:  27%|██▋       | 98/364 [02:08<06:30,  1.47s/it]
Epoch 1/10:  27%|██▋       | 99/364 [02:08<05:04,  1.15s/it]
Epoch 1/10:  27%|██▋       | 100/364 [02:09<04:29,  1.02s/it]
Epoch 1/10:  28%|██▊       | 101/364 [02:12<06:34,  1.50s/it]
Epoch 1/10:  28%|██▊       | 102/364 [02:13<06:02,  1.38s/it]
Epoch 1/10:  28%|██▊       | 103/364 [02:13<04:44,  1.09s/it]
Epoch 1/10:  29%|██▊       | 104/364 [02:14<04:27,  1.03s/it]
Epoch 1/10:  29%|██▉       | 105/364 [02:17<06:27,  1.50s/it]
Epoch 1/10:  29%|██▉       | 106/364 [02:18<05:46,  1.34s/it]
Epoch 1/10:  29%|██▉       | 107/364 [02:18<04:33,  1.06s/it]
Epoch 1/10:  30%|██▉       | 108/364 [02:19<04:29,  1.05s/it]
Epoch 1/10:  30%|██▉       | 109/364 [02:22<06:26,  1.51s/it]
Epoch 1/10:  30%|███       | 110/364 [02:23<05:41,  1.34s/it]
Epoch 1/10:  30%|███       | 111/364 [02:23<04:28,  1.06s/it]
Epoch 1/10:  31%|███       | 112/364 [02:24<04:37,  1.10s/it]
Epoch 1/10:  31%|███       | 113/364 [02:27<06:09,  1.47s/it]
Epoch 1/10:  31%|███▏      | 114/364 [02:28<05:42,  1.37s/it]
Epoch 1/10:  32%|███▏      | 115/364 [02:28<04:28,  1.08s/it]
Epoch 1/10:  32%|███▏      | 116/364 [02:30<04:48,  1.16s/it]
Epoch 1/10:  32%|███▏      | 117/364 [02:32<06:16,  1.53s/it]
Epoch 1/10:  32%|███▏      | 118/364 [02:33<05:17,  1.29s/it]
Epoch 1/10:  33%|███▎      | 119/364 [02:33<04:10,  1.02s/it]
Epoch 1/10:  33%|███▎      | 120/364 [02:34<04:34,  1.13s/it]
Epoch 1/10:  33%|███▎      | 121/364 [02:37<06:17,  1.55s/it]
Epoch 1/10:  34%|███▎      | 122/364 [02:38<05:24,  1.34s/it]
Epoch 1/10:  34%|███▍      | 123/364 [02:38<04:15,  1.06s/it]
Epoch 1/10:  34%|███▍      | 124/364 [02:39<04:23,  1.10s/it]
Epoch 1/10:  34%|███▍      | 125/364 [02:42<06:05,  1.53s/it]
Epoch 1/10:  35%|███▍      | 126/364 [02:43<05:19,  1.34s/it]
Epoch 1/10:  35%|███▍      | 127/364 [02:43<04:11,  1.06s/it]
Epoch 1/10:  35%|███▌      | 128/364 [02:45<04:27,  1.13s/it]
Epoch 1/10:  35%|███▌      | 129/364 [02:47<06:16,  1.60s/it]
Epoch 1/10:  36%|███▌      | 130/364 [02:48<05:01,  1.29s/it]
Epoch 1/10:  36%|███▌      | 131/364 [02:48<03:58,  1.02s/it]
Epoch 1/10:  36%|███▋      | 132/364 [02:50<04:31,  1.17s/it]
Epoch 1/10:  37%|███▋      | 133/364 [02:52<06:10,  1.61s/it]
Epoch 1/10:  37%|███▋      | 134/364 [02:53<04:48,  1.26s/it]
Epoch 1/10:  37%|███▋      | 135/364 [02:53<03:55,  1.03s/it]
Epoch 1/10:  37%|███▋      | 136/364 [02:55<04:33,  1.20s/it]
Epoch 1/10:  38%|███▊      | 137/364 [02:57<05:50,  1.54s/it]
Epoch 1/10:  38%|███▊      | 138/364 [02:58<04:32,  1.21s/it]
Epoch 1/10:  38%|███▊      | 139/364 [02:59<04:14,  1.13s/it]
Epoch 1/10:  38%|███▊      | 140/364 [03:00<04:31,  1.21s/it]
Epoch 1/10:  39%|███▊      | 141/364 [03:02<05:41,  1.53s/it]
Epoch 1/10:  39%|███▉      | 142/364 [03:03<04:26,  1.20s/it]
Epoch 1/10:  39%|███▉      | 143/364 [03:03<03:51,  1.05s/it]
Epoch 1/10:  40%|███▉      | 144/364 [03:05<04:49,  1.32s/it]
Epoch 1/10:  40%|███▉      | 145/364 [03:07<05:19,  1.46s/it]
Epoch 1/10:  40%|████      | 146/364 [03:08<04:09,  1.14s/it]
Epoch 1/10:  40%|████      | 147/364 [03:08<03:48,  1.05s/it]
Epoch 1/10:  41%|████      | 148/364 [03:11<05:00,  1.39s/it]
Epoch 1/10:  41%|████      | 149/364 [03:12<05:13,  1.46s/it]
Epoch 1/10:  41%|████      | 150/364 [03:13<04:05,  1.15s/it]
Epoch 1/10:  41%|████▏     | 151/364 [03:14<03:53,  1.10s/it]
Epoch 1/10:  42%|████▏     | 152/364 [03:16<05:03,  1.43s/it]
Epoch 1/10:  42%|████▏     | 153/364 [03:17<04:59,  1.42s/it]
Epoch 1/10:  42%|████▏     | 154/364 [03:18<04:03,  1.16s/it]
Epoch 1/10:  43%|████▎     | 155/364 [03:19<04:00,  1.15s/it]
Epoch 1/10:  43%|████▎     | 156/364 [03:21<04:59,  1.44s/it]
Epoch 1/10:  43%|████▎     | 157/364 [03:22<04:36,  1.34s/it]
Epoch 1/10:  43%|████▎     | 158/364 [03:23<03:53,  1.13s/it]
Epoch 1/10:  44%|████▎     | 159/364 [03:24<03:50,  1.13s/it]
Epoch 1/10:  44%|████▍     | 160/364 [03:26<04:43,  1.39s/it]
Epoch 1/10:  44%|████▍     | 161/364 [03:27<04:12,  1.24s/it]
Epoch 1/10:  45%|████▍     | 162/364 [03:28<04:01,  1.19s/it]
Epoch 1/10:  45%|████▍     | 163/364 [03:29<04:01,  1.20s/it]
Epoch 1/10:  45%|████▌     | 164/364 [03:31<04:43,  1.42s/it]
Epoch 1/10:  45%|████▌     | 165/364 [03:32<04:16,  1.29s/it]
Epoch 1/10:  46%|████▌     | 166/364 [03:33<04:01,  1.22s/it]
Epoch 1/10:  46%|████▌     | 167/364 [03:34<03:39,  1.12s/it]
Epoch 1/10:  46%|████▌     | 168/364 [03:36<04:35,  1.40s/it]
Epoch 1/10:  46%|████▋     | 169/364 [03:37<04:28,  1.38s/it]
Epoch 1/10:  47%|████▋     | 170/364 [03:38<03:56,  1.22s/it]
Epoch 1/10:  47%|████▋     | 171/364 [03:39<03:44,  1.16s/it]
Epoch 1/10:  47%|████▋     | 172/364 [03:41<04:17,  1.34s/it]
Epoch 1/10:  48%|████▊     | 173/364 [03:42<04:05,  1.28s/it]
Epoch 1/10:  48%|████▊     | 174/364 [03:43<03:53,  1.23s/it]
Epoch 1/10:  48%|████▊     | 175/364 [03:44<03:27,  1.10s/it]
Epoch 1/10:  48%|████▊     | 176/364 [03:46<04:24,  1.41s/it]
Epoch 1/10:  49%|████▊     | 177/364 [03:47<04:14,  1.36s/it]
Epoch 1/10:  49%|████▉     | 178/364 [03:48<03:38,  1.18s/it]
Epoch 1/10:  49%|████▉     | 179/364 [03:49<03:19,  1.08s/it]
Epoch 1/10:  49%|████▉     | 180/364 [03:51<04:13,  1.38s/it]
Epoch 1/10:  50%|████▉     | 181/364 [03:53<04:22,  1.43s/it]
Epoch 1/10:  50%|█████     | 182/364 [03:53<03:45,  1.24s/it]
Epoch 1/10:  50%|█████     | 183/364 [03:54<03:07,  1.03s/it]
Epoch 1/10:  51%|█████     | 184/364 [03:56<04:22,  1.46s/it]
Epoch 1/10:  51%|█████     | 185/364 [03:58<04:16,  1.43s/it]
Epoch 1/10:  51%|█████     | 186/364 [03:58<03:21,  1.13s/it]
Epoch 1/10:  51%|█████▏    | 187/364 [03:59<03:02,  1.03s/it]
Epoch 1/10:  52%|█████▏    | 188/364 [04:02<04:26,  1.51s/it]
Epoch 1/10:  52%|█████▏    | 189/364 [04:03<04:04,  1.40s/it]
Epoch 1/10:  52%|█████▏    | 190/364 [04:03<03:10,  1.09s/it]
Epoch 1/10:  52%|█████▏    | 191/364 [04:04<02:48,  1.03it/s]
Epoch 1/10:  53%|█████▎    | 192/364 [04:07<04:39,  1.62s/it]
Epoch 1/10:  53%|█████▎    | 193/364 [04:08<03:57,  1.39s/it]
Epoch 1/10:  53%|█████▎    | 194/364 [04:08<03:05,  1.09s/it]
Epoch 1/10:  54%|█████▎    | 195/364 [04:09<02:49,  1.01s/it]
Epoch 1/10:  54%|█████▍    | 196/364 [04:12<04:15,  1.52s/it]
Epoch 1/10:  54%|█████▍    | 197/364 [04:13<03:54,  1.40s/it]
Epoch 1/10:  54%|█████▍    | 198/364 [04:14<03:25,  1.24s/it]
Epoch 1/10:  55%|█████▍    | 199/364 [04:14<02:44,  1.00it/s]
Epoch 1/10:  55%|█████▍    | 200/364 [04:17<04:02,  1.48s/it]
Epoch 1/10:  55%|█████▌    | 201/364 [04:18<03:48,  1.40s/it]
Epoch 1/10:  55%|█████▌    | 202/364 [04:19<03:12,  1.19s/it]
Epoch 1/10:  56%|█████▌    | 203/364 [04:19<02:39,  1.01it/s]
Epoch 1/10:  56%|█████▌    | 204/364 [04:22<03:58,  1.49s/it]
Epoch 1/10:  56%|█████▋    | 205/364 [04:23<03:44,  1.41s/it]
Epoch 1/10:  57%|█████▋    | 206/364 [04:24<03:16,  1.24s/it]
Epoch 1/10:  57%|█████▋    | 207/364 [04:24<02:36,  1.00it/s]
Epoch 1/10:  57%|█████▋    | 208/364 [04:27<03:50,  1.48s/it]
Epoch 1/10:  57%|█████▋    | 209/364 [04:28<03:38,  1.41s/it]
Epoch 1/10:  58%|█████▊    | 210/364 [04:29<03:18,  1.29s/it]
Epoch 1/10:  58%|█████▊    | 211/364 [04:30<02:38,  1.04s/it]
Epoch 1/10:  58%|█████▊    | 212/364 [04:32<03:32,  1.40s/it]
Epoch 1/10:  59%|█████▊    | 213/364 [04:33<03:29,  1.38s/it]
Epoch 1/10:  59%|█████▉    | 214/364 [04:34<03:17,  1.32s/it]
Epoch 1/10:  59%|█████▉    | 215/364 [04:35<02:35,  1.04s/it]
Epoch 1/10:  59%|█████▉    | 216/364 [04:37<03:15,  1.32s/it]
Epoch 1/10:  60%|█████▉    | 217/364 [04:39<03:33,  1.45s/it]
Epoch 1/10:  60%|█████▉    | 218/364 [04:39<03:07,  1.28s/it]
Epoch 1/10:  60%|██████    | 219/364 [04:40<02:28,  1.03s/it]
Epoch 1/10:  60%|██████    | 220/364 [04:42<03:21,  1.40s/it]
Epoch 1/10:  61%|██████    | 221/364 [04:44<03:26,  1.44s/it]
Epoch 1/10:  61%|██████    | 222/364 [04:44<02:54,  1.23s/it]
Epoch 1/10:  61%|██████▏   | 223/364 [04:45<02:19,  1.01it/s]
Epoch 1/10:  62%|██████▏   | 224/364 [04:47<03:02,  1.31s/it]
Epoch 1/10:  62%|██████▏   | 225/364 [04:49<03:28,  1.50s/it]
Epoch 1/10:  62%|██████▏   | 226/364 [04:49<02:47,  1.22s/it]
Epoch 1/10:  62%|██████▏   | 227/364 [04:50<02:14,  1.02it/s]
Epoch 1/10:  63%|██████▎   | 228/364 [04:52<02:51,  1.26s/it]
Epoch 1/10:  63%|██████▎   | 229/364 [04:54<03:32,  1.57s/it]
Epoch 1/10:  63%|██████▎   | 230/364 [04:55<02:52,  1.29s/it]
Epoch 1/10:  63%|██████▎   | 231/364 [04:55<02:16,  1.03s/it]
Epoch 1/10:  64%|██████▎   | 232/364 [04:57<02:46,  1.27s/it]
Epoch 1/10:  64%|██████▍   | 233/364 [04:59<03:18,  1.52s/it]
Epoch 1/10:  64%|██████▍   | 234/364 [05:00<02:43,  1.25s/it]
Epoch 1/10:  65%|██████▍   | 235/364 [05:00<02:09,  1.01s/it]
Epoch 1/10:  65%|██████▍   | 236/364 [05:02<02:24,  1.13s/it]
Epoch 1/10:  65%|██████▌   | 237/364 [05:04<03:14,  1.54s/it]
Epoch 1/10:  65%|██████▌   | 238/364 [05:05<02:45,  1.32s/it]
Epoch 1/10:  66%|██████▌   | 239/364 [05:05<02:11,  1.05s/it]
Epoch 1/10:  66%|██████▌   | 240/364 [05:07<02:28,  1.19s/it]
Epoch 1/10:  66%|██████▌   | 241/364 [05:09<03:01,  1.48s/it]
Epoch 1/10:  66%|██████▋   | 242/364 [05:10<02:34,  1.27s/it]
Epoch 1/10:  67%|██████▋   | 243/364 [05:10<02:02,  1.01s/it]
Epoch 1/10:  67%|██████▋   | 244/364 [05:12<02:21,  1.18s/it]
Epoch 1/10:  67%|██████▋   | 245/364 [05:14<02:49,  1.43s/it]
Epoch 1/10:  68%|██████▊   | 246/364 [05:15<02:40,  1.36s/it]
Epoch 1/10:  68%|██████▊   | 247/364 [05:15<02:05,  1.08s/it]
Epoch 1/10:  68%|██████▊   | 248/364 [05:17<02:25,  1.26s/it]
Epoch 1/10:  68%|██████▊   | 249/364 [05:18<02:25,  1.26s/it]
Epoch 1/10:  69%|██████▊   | 250/364 [05:20<02:32,  1.34s/it]
Epoch 1/10:  69%|██████▉   | 251/364 [05:20<02:00,  1.07s/it]
Epoch 1/10:  69%|██████▉   | 252/364 [05:22<02:20,  1.25s/it]
Epoch 1/10:  70%|██████▉   | 253/364 [05:23<02:29,  1.35s/it]
Epoch 1/10:  70%|██████▉   | 254/364 [05:25<02:25,  1.33s/it]
Epoch 1/10:  70%|███████   | 255/364 [05:25<01:54,  1.05s/it]
Epoch 1/10:  70%|███████   | 256/364 [05:27<02:25,  1.35s/it]
Epoch 1/10:  71%|███████   | 257/364 [05:28<02:18,  1.30s/it]
Epoch 1/10:  71%|███████   | 258/364 [05:30<02:27,  1.39s/it]
Epoch 1/10:  71%|███████   | 259/364 [05:30<01:55,  1.10s/it]
Epoch 1/10:  71%|███████▏  | 260/364 [05:33<02:25,  1.40s/it]
Epoch 1/10:  72%|███████▏  | 261/364 [05:33<02:01,  1.18s/it]
Epoch 1/10:  72%|███████▏  | 262/364 [05:35<02:17,  1.35s/it]
Epoch 1/10:  72%|███████▏  | 263/364 [05:35<01:48,  1.07s/it]
Epoch 1/10:  73%|███████▎  | 264/364 [05:38<02:23,  1.43s/it]
Epoch 1/10:  73%|███████▎  | 265/364 [05:38<01:53,  1.15s/it]
Epoch 1/10:  73%|███████▎  | 266/364 [05:40<02:13,  1.37s/it]
Epoch 1/10:  73%|███████▎  | 267/364 [05:40<01:45,  1.08s/it]
Epoch 1/10:  74%|███████▎  | 268/364 [05:43<02:17,  1.43s/it]
Epoch 1/10:  74%|███████▍  | 269/364 [05:43<01:46,  1.12s/it]
Epoch 1/10:  74%|███████▍  | 270/364 [05:45<02:13,  1.42s/it]
Epoch 1/10:  74%|███████▍  | 271/364 [05:46<01:44,  1.12s/it]
Epoch 1/10:  75%|███████▍  | 272/364 [05:48<02:16,  1.48s/it]
Epoch 1/10:  75%|███████▌  | 273/364 [05:48<01:45,  1.16s/it]
Epoch 1/10:  75%|███████▌  | 274/364 [05:50<02:05,  1.39s/it]
Epoch 1/10:  76%|███████▌  | 275/364 [05:51<01:37,  1.10s/it]
Epoch 1/10:  76%|███████▌  | 276/364 [05:53<02:11,  1.49s/it]
Epoch 1/10:  76%|███████▌  | 277/364 [05:53<01:42,  1.17s/it]
Epoch 1/10:  76%|███████▋  | 278/364 [05:56<02:02,  1.43s/it]
Epoch 1/10:  77%|███████▋  | 279/364 [05:56<01:36,  1.13s/it]
Epoch 1/10:  77%|███████▋  | 280/364 [05:58<02:05,  1.50s/it]
Epoch 1/10:  77%|███████▋  | 281/364 [05:59<01:37,  1.17s/it]
Epoch 1/10:  77%|███████▋  | 282/364 [06:00<01:50,  1.35s/it]
Epoch 1/10:  78%|███████▊  | 283/364 [06:01<01:26,  1.07s/it]
Epoch 1/10:  78%|███████▊  | 284/364 [06:03<01:54,  1.43s/it]
Epoch 1/10:  78%|███████▊  | 285/364 [06:04<01:28,  1.12s/it]
Epoch 1/10:  79%|███████▊  | 286/364 [06:06<01:57,  1.51s/it]
Epoch 1/10:  79%|███████▉  | 287/364 [06:06<01:30,  1.18s/it]
Epoch 1/10:  79%|███████▉  | 288/364 [06:08<01:49,  1.44s/it]
Epoch 1/10:  79%|███████▉  | 289/364 [06:09<01:24,  1.13s/it]
Epoch 1/10:  80%|███████▉  | 290/364 [06:11<01:54,  1.54s/it]
Epoch 1/10:  80%|███████▉  | 291/364 [06:12<01:27,  1.20s/it]
Epoch 1/10:  80%|████████  | 292/364 [06:14<01:42,  1.42s/it]
Epoch 1/10:  80%|████████  | 293/364 [06:14<01:19,  1.12s/it]
Epoch 1/10:  81%|████████  | 294/364 [06:17<01:45,  1.51s/it]
Epoch 1/10:  81%|████████  | 295/364 [06:17<01:21,  1.19s/it]
Epoch 1/10:  81%|████████▏ | 296/364 [06:19<01:34,  1.39s/it]
Epoch 1/10:  82%|████████▏ | 297/364 [06:19<01:13,  1.10s/it]
Epoch 1/10:  82%|████████▏ | 298/364 [06:21<01:32,  1.40s/it]
Epoch 1/10:  82%|████████▏ | 299/364 [06:22<01:12,  1.11s/it]
Epoch 1/10:  82%|████████▏ | 300/364 [06:24<01:34,  1.47s/it]
Epoch 1/10:  83%|████████▎ | 301/364 [06:25<01:12,  1.16s/it]
Epoch 1/10:  83%|████████▎ | 302/364 [06:27<01:30,  1.46s/it]
Epoch 1/10:  83%|████████▎ | 303/364 [06:27<01:10,  1.15s/it]
Epoch 1/10:  84%|████████▎ | 304/364 [06:29<01:25,  1.42s/it]
Epoch 1/10:  84%|████████▍ | 305/364 [06:30<01:05,  1.12s/it]
Epoch 1/10:  84%|████████▍ | 306/364 [06:32<01:24,  1.46s/it]
Epoch 1/10:  84%|████████▍ | 307/364 [06:32<01:05,  1.15s/it]
Epoch 1/10:  85%|████████▍ | 308/364 [06:35<01:23,  1.49s/it]
Epoch 1/10:  85%|████████▍ | 309/364 [06:35<01:04,  1.17s/it]
Epoch 1/10:  85%|████████▌ | 310/364 [06:37<01:15,  1.40s/it]
Epoch 1/10:  85%|████████▌ | 311/364 [06:37<00:58,  1.11s/it]
Epoch 1/10:  86%|████████▌ | 312/364 [06:40<01:15,  1.45s/it]
Epoch 1/10:  86%|████████▌ | 313/364 [06:40<00:57,  1.14s/it]
Epoch 1/10:  86%|████████▋ | 314/364 [06:42<01:10,  1.41s/it]
Epoch 1/10:  87%|████████▋ | 315/364 [06:42<00:54,  1.11s/it]
Epoch 1/10:  87%|████████▋ | 316/364 [06:45<01:11,  1.49s/it]
Epoch 1/10:  87%|████████▋ | 317/364 [06:45<00:55,  1.17s/it]
Epoch 1/10:  87%|████████▋ | 318/364 [06:47<01:03,  1.38s/it]
Epoch 1/10:  88%|████████▊ | 319/364 [06:48<00:49,  1.10s/it]
Epoch 1/10:  88%|████████▊ | 320/364 [06:50<01:05,  1.50s/it]
Epoch 1/10:  88%|████████▊ | 321/364 [06:50<00:50,  1.18s/it]
Epoch 1/10:  88%|████████▊ | 322/364 [06:52<00:58,  1.40s/it]
Epoch 1/10:  89%|████████▊ | 323/364 [06:53<00:45,  1.11s/it]
Epoch 1/10:  89%|████████▉ | 324/364 [06:55<01:00,  1.52s/it]
Epoch 1/10:  89%|████████▉ | 325/364 [06:56<00:46,  1.19s/it]
Epoch 1/10:  90%|████████▉ | 326/364 [06:57<00:48,  1.27s/it]
Epoch 1/10:  90%|████████▉ | 327/364 [06:58<00:37,  1.01s/it]
Epoch 1/10:  90%|█████████ | 328/364 [07:00<00:56,  1.58s/it]
Epoch 1/10:  90%|█████████ | 329/364 [07:01<00:43,  1.23s/it]
Epoch 1/10:  91%|█████████ | 330/364 [07:02<00:44,  1.31s/it]
Epoch 1/10:  91%|█████████ | 331/364 [07:03<00:34,  1.04s/it]
Epoch 1/10:  91%|█████████ | 332/364 [07:05<00:47,  1.48s/it]
Epoch 1/10:  91%|█████████▏| 333/364 [07:06<00:36,  1.16s/it]
Epoch 1/10:  92%|█████████▏| 334/364 [07:08<00:44,  1.48s/it]
Epoch 1/10:  92%|█████████▏| 335/364 [07:08<00:33,  1.16s/it]
Epoch 1/10:  92%|█████████▏| 336/364 [07:11<00:45,  1.63s/it]
Epoch 1/10:  93%|█████████▎| 337/364 [07:11<00:34,  1.27s/it]
Epoch 1/10:  93%|█████████▎| 338/364 [07:13<00:37,  1.42s/it]
Epoch 1/10:  93%|█████████▎| 339/364 [07:14<00:27,  1.12s/it]
Epoch 1/10:  93%|█████████▎| 340/364 [07:16<00:34,  1.44s/it]
Epoch 1/10:  94%|█████████▎| 341/364 [07:16<00:26,  1.13s/it]
Epoch 1/10:  94%|█████████▍| 342/364 [07:18<00:31,  1.42s/it]
Epoch 1/10:  94%|█████████▍| 343/364 [07:19<00:23,  1.13s/it]
Epoch 1/10:  95%|█████████▍| 344/364 [07:21<00:31,  1.57s/it]
Epoch 1/10:  95%|█████████▍| 345/364 [07:22<00:23,  1.23s/it]
Epoch 1/10:  95%|█████████▌| 346/364 [07:24<00:24,  1.38s/it]
Epoch 1/10:  95%|█████████▌| 347/364 [07:24<00:18,  1.09s/it]
Epoch 1/10:  96%|█████████▌| 348/364 [07:27<00:24,  1.52s/it]
Epoch 1/10:  96%|█████████▌| 349/364 [07:27<00:17,  1.19s/it]
Epoch 1/10:  96%|█████████▌| 350/364 [07:29<00:18,  1.34s/it]
Epoch 1/10:  96%|█████████▋| 351/364 [07:29<00:13,  1.07s/it]
Epoch 1/10:  97%|█████████▋| 352/364 [07:31<00:17,  1.47s/it]
Epoch 1/10:  97%|█████████▋| 353/364 [07:32<00:12,  1.16s/it]
Epoch 1/10:  97%|█████████▋| 354/364 [07:33<00:12,  1.28s/it]
Epoch 1/10:  98%|█████████▊| 355/364 [07:34<00:09,  1.03s/it]
Epoch 1/10:  98%|█████████▊| 356/364 [07:37<00:12,  1.55s/it]
Epoch 1/10:  98%|█████████▊| 357/364 [07:37<00:08,  1.22s/it]
Epoch 1/10:  98%|█████████▊| 358/364 [07:38<00:07,  1.24s/it]
Epoch 1/10:  99%|█████████▊| 359/364 [07:39<00:04,  1.00it/s]
Epoch 1/10:  99%|█████████▉| 360/364 [07:41<00:05,  1.38s/it]
Epoch 1/10:  99%|█████████▉| 361/364 [07:42<00:03,  1.09s/it]
Epoch 1/10:  99%|█████████▉| 362/364 [07:42<00:02,  1.05s/it]
Epoch 1/10: 100%|█████████▉| 363/364 [07:43<00:00,  1.17it/s]
Epoch 1/10: 100%|██████████| 364/364 [07:44<00:00,  1.17it/s]
Epoch 1/10: 100%|██████████| 364/364 [07:44<00:00,  1.28s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:05<04:42,  5.05s/it]
Validating:   4%|▎         | 2/57 [00:05<01:59,  2.17s/it]
Validating:   5%|▌         | 3/57 [00:05<01:06,  1.24s/it]
Validating:   7%|▋         | 4/57 [00:05<00:42,  1.24it/s]
Validating:   9%|▉         | 5/57 [00:09<01:40,  1.93s/it]
Validating:  11%|█         | 6/57 [00:09<01:07,  1.33s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:47,  1.06it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:33,  1.44it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:19,  1.65s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:56,  1.20s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:40,  1.15it/s]
Validating:  21%|██        | 12/57 [00:14<00:29,  1.53it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:14,  1.69s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:52,  1.23s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:37,  1.11it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:27,  1.48it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:05,  1.63s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:46,  1.19s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:33,  1.13it/s]
Validating:  35%|███▌      | 20/57 [00:22<00:24,  1.51it/s]
Validating:  37%|███▋      | 21/57 [00:26<00:58,  1.62s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:43,  1.23s/it]
Validating:  40%|████      | 23/57 [00:27<00:30,  1.11it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:22,  1.47it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:52,  1.66s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:38,  1.25s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:27,  1.09it/s]
Validating:  49%|████▉     | 28/57 [00:31<00:20,  1.45it/s]
Validating:  51%|█████     | 29/57 [00:35<00:45,  1.61s/it]
Validating:  53%|█████▎    | 30/57 [00:35<00:31,  1.18s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:22,  1.15it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.54it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:38,  1.62s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:29,  1.27s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:20,  1.07it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.44it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:33,  1.69s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:25,  1.36s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:17,  1.01it/s]
Validating:  70%|███████   | 40/57 [00:45<00:12,  1.36it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:26,  1.64s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:20,  1.39s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:14,  1.01s/it]
Validating:  77%|███████▋  | 44/57 [00:50<00:09,  1.32it/s]
Validating:  79%|███████▉  | 45/57 [00:54<00:18,  1.57s/it]
Validating:  81%|████████  | 46/57 [00:54<00:15,  1.39s/it]
Validating:  82%|████████▏ | 47/57 [00:55<00:10,  1.03s/it]
Validating:  84%|████████▍ | 48/57 [00:55<00:06,  1.32it/s]
Validating:  86%|████████▌ | 49/57 [00:58<00:10,  1.36s/it]
Validating:  88%|████████▊ | 50/57 [00:59<00:08,  1.28s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:05,  1.06it/s]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.42it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:05,  1.26s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.04s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.30it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.75it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 1: Train Loss: 0.6137, Train Acc: 0.7771 | Val Loss: 0.3753, Val Acc: 0.8667
Saved Best Model!

Epoch 2/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 2/10:   0%|          | 1/364 [00:05<34:28,  5.70s/it]
Epoch 2/10:   1%|          | 2/364 [00:06<15:37,  2.59s/it]
Epoch 2/10:   1%|          | 3/364 [00:06<09:36,  1.60s/it]
Epoch 2/10:   1%|          | 4/364 [00:06<06:43,  1.12s/it]
Epoch 2/10:   1%|▏         | 5/364 [00:11<13:56,  2.33s/it]
Epoch 2/10:   2%|▏         | 6/364 [00:11<09:59,  1.67s/it]
Epoch 2/10:   2%|▏         | 7/364 [00:12<07:30,  1.26s/it]
Epoch 2/10:   2%|▏         | 8/364 [00:12<05:52,  1.01it/s]
Epoch 2/10:   2%|▏         | 9/364 [00:16<11:26,  1.93s/it]
Epoch 2/10:   3%|▎         | 10/364 [00:17<08:38,  1.46s/it]
Epoch 2/10:   3%|▎         | 11/364 [00:17<06:42,  1.14s/it]
Epoch 2/10:   3%|▎         | 12/364 [00:17<05:20,  1.10it/s]
Epoch 2/10:   4%|▎         | 13/364 [00:21<10:31,  1.80s/it]
Epoch 2/10:   4%|▍         | 14/364 [00:22<08:03,  1.38s/it]
Epoch 2/10:   4%|▍         | 15/364 [00:22<06:21,  1.09s/it]
Epoch 2/10:   4%|▍         | 16/364 [00:22<05:10,  1.12it/s]
Epoch 2/10:   5%|▍         | 17/364 [00:27<10:51,  1.88s/it]
Epoch 2/10:   5%|▍         | 18/364 [00:27<08:18,  1.44s/it]
Epoch 2/10:   5%|▌         | 19/364 [00:27<06:30,  1.13s/it]
Epoch 2/10:   5%|▌         | 20/364 [00:28<05:14,  1.09it/s]
Epoch 2/10:   6%|▌         | 21/364 [00:32<10:10,  1.78s/it]
Epoch 2/10:   6%|▌         | 22/364 [00:32<07:49,  1.37s/it]
Epoch 2/10:   6%|▋         | 23/364 [00:32<06:10,  1.09s/it]
Epoch 2/10:   7%|▋         | 24/364 [00:33<05:00,  1.13it/s]
Epoch 2/10:   7%|▋         | 25/364 [00:37<10:25,  1.85s/it]
Epoch 2/10:   7%|▋         | 26/364 [00:37<07:59,  1.42s/it]
Epoch 2/10:   7%|▋         | 27/364 [00:38<06:15,  1.11s/it]
Epoch 2/10:   8%|▊         | 28/364 [00:38<05:06,  1.10it/s]
Epoch 2/10:   8%|▊         | 29/364 [00:42<09:15,  1.66s/it]
Epoch 2/10:   8%|▊         | 30/364 [00:42<07:09,  1.29s/it]
Epoch 2/10:   9%|▊         | 31/364 [00:43<05:42,  1.03s/it]
Epoch 2/10:   9%|▉         | 32/364 [00:43<04:39,  1.19it/s]
Epoch 2/10:   9%|▉         | 33/364 [00:47<09:21,  1.70s/it]
Epoch 2/10:   9%|▉         | 34/364 [00:47<07:12,  1.31s/it]
Epoch 2/10:  10%|▉         | 35/364 [00:47<05:44,  1.05s/it]
Epoch 2/10:  10%|▉         | 36/364 [00:48<04:41,  1.17it/s]
Epoch 2/10:  10%|█         | 37/364 [00:52<10:07,  1.86s/it]
Epoch 2/10:  10%|█         | 38/364 [00:52<07:43,  1.42s/it]
Epoch 2/10:  11%|█         | 39/364 [00:53<06:04,  1.12s/it]
Epoch 2/10:  11%|█         | 40/364 [00:53<04:55,  1.10it/s]
Epoch 2/10:  11%|█▏        | 41/364 [00:57<10:04,  1.87s/it]
Epoch 2/10:  12%|█▏        | 42/364 [00:58<07:44,  1.44s/it]
Epoch 2/10:  12%|█▏        | 43/364 [00:58<06:04,  1.13s/it]
Epoch 2/10:  12%|█▏        | 44/364 [00:59<04:54,  1.09it/s]
Epoch 2/10:  12%|█▏        | 45/364 [01:03<09:33,  1.80s/it]
Epoch 2/10:  13%|█▎        | 46/364 [01:03<07:21,  1.39s/it]
Epoch 2/10:  13%|█▎        | 47/364 [01:03<05:47,  1.09s/it]
Epoch 2/10:  13%|█▎        | 48/364 [01:04<04:44,  1.11it/s]
Epoch 2/10:  13%|█▎        | 49/364 [01:07<08:49,  1.68s/it]
Epoch 2/10:  14%|█▎        | 50/364 [01:08<06:49,  1.31s/it]
Epoch 2/10:  14%|█▍        | 51/364 [01:08<05:26,  1.04s/it]
Epoch 2/10:  14%|█▍        | 52/364 [01:09<04:29,  1.16it/s]
Epoch 2/10:  15%|█▍        | 53/364 [01:12<08:38,  1.67s/it]
Epoch 2/10:  15%|█▍        | 54/364 [01:13<06:41,  1.29s/it]
Epoch 2/10:  15%|█▌        | 55/364 [01:13<05:19,  1.03s/it]
Epoch 2/10:  15%|█▌        | 56/364 [01:13<04:23,  1.17it/s]
Epoch 2/10:  16%|█▌        | 57/364 [01:17<08:31,  1.67s/it]
Epoch 2/10:  16%|█▌        | 58/364 [01:17<06:35,  1.29s/it]
Epoch 2/10:  16%|█▌        | 59/364 [01:18<05:15,  1.03s/it]
Epoch 2/10:  16%|█▋        | 60/364 [01:18<04:18,  1.18it/s]
Epoch 2/10:  17%|█▋        | 61/364 [01:22<08:08,  1.61s/it]
Epoch 2/10:  17%|█▋        | 62/364 [01:22<06:18,  1.25s/it]
Epoch 2/10:  17%|█▋        | 63/364 [01:23<05:02,  1.01s/it]
Epoch 2/10:  18%|█▊        | 64/364 [01:23<04:09,  1.20it/s]
Epoch 2/10:  18%|█▊        | 65/364 [01:27<08:32,  1.71s/it]
Epoch 2/10:  18%|█▊        | 66/364 [01:27<06:34,  1.33s/it]
Epoch 2/10:  18%|█▊        | 67/364 [01:28<05:31,  1.12s/it]
Epoch 2/10:  19%|█▊        | 68/364 [01:28<04:28,  1.10it/s]
Epoch 2/10:  19%|█▉        | 69/364 [01:31<07:51,  1.60s/it]
Epoch 2/10:  19%|█▉        | 70/364 [01:32<06:07,  1.25s/it]
Epoch 2/10:  20%|█▉        | 71/364 [01:33<05:39,  1.16s/it]
Epoch 2/10:  20%|█▉        | 72/364 [01:33<04:34,  1.06it/s]
Epoch 2/10:  20%|██        | 73/364 [01:36<07:57,  1.64s/it]
Epoch 2/10:  20%|██        | 74/364 [01:37<06:10,  1.28s/it]
Epoch 2/10:  21%|██        | 75/364 [01:38<05:42,  1.18s/it]
Epoch 2/10:  21%|██        | 76/364 [01:38<04:36,  1.04it/s]
Epoch 2/10:  21%|██        | 77/364 [01:41<07:40,  1.60s/it]
Epoch 2/10:  21%|██▏       | 78/364 [01:42<05:57,  1.25s/it]
Epoch 2/10:  22%|██▏       | 79/364 [01:43<05:53,  1.24s/it]
Epoch 2/10:  22%|██▏       | 80/364 [01:44<04:44,  1.00s/it]
Epoch 2/10:  22%|██▏       | 81/364 [01:46<07:19,  1.55s/it]
Epoch 2/10:  23%|██▎       | 82/364 [01:47<05:42,  1.22s/it]
Epoch 2/10:  23%|██▎       | 83/364 [01:48<05:52,  1.25s/it]
Epoch 2/10:  23%|██▎       | 84/364 [01:49<04:42,  1.01s/it]
Epoch 2/10:  23%|██▎       | 85/364 [01:51<07:20,  1.58s/it]
Epoch 2/10:  24%|██▎       | 86/364 [01:52<05:44,  1.24s/it]
Epoch 2/10:  24%|██▍       | 87/364 [01:53<05:27,  1.18s/it]
Epoch 2/10:  24%|██▍       | 88/364 [01:53<04:22,  1.05it/s]
Epoch 2/10:  24%|██▍       | 89/364 [01:57<07:35,  1.66s/it]
Epoch 2/10:  25%|██▍       | 90/364 [01:57<05:52,  1.29s/it]
Epoch 2/10:  25%|██▌       | 91/364 [01:58<05:38,  1.24s/it]
Epoch 2/10:  25%|██▌       | 92/364 [01:59<04:30,  1.01it/s]
Epoch 2/10:  26%|██▌       | 93/364 [02:02<07:14,  1.60s/it]
Epoch 2/10:  26%|██▌       | 94/364 [02:02<05:36,  1.25s/it]
Epoch 2/10:  26%|██▌       | 95/364 [02:03<05:17,  1.18s/it]
Epoch 2/10:  26%|██▋       | 96/364 [02:04<04:16,  1.05it/s]
Epoch 2/10:  27%|██▋       | 97/364 [02:07<07:24,  1.66s/it]
Epoch 2/10:  27%|██▋       | 98/364 [02:07<05:45,  1.30s/it]
Epoch 2/10:  27%|██▋       | 99/364 [02:08<05:06,  1.16s/it]
Epoch 2/10:  27%|██▋       | 100/364 [02:09<04:07,  1.07it/s]
Epoch 2/10:  28%|██▊       | 101/364 [02:12<07:03,  1.61s/it]
Epoch 2/10:  28%|██▊       | 102/364 [02:12<05:32,  1.27s/it]
Epoch 2/10:  28%|██▊       | 103/364 [02:13<04:53,  1.12s/it]
Epoch 2/10:  29%|██▊       | 104/364 [02:13<03:58,  1.09it/s]
Epoch 2/10:  29%|██▉       | 105/364 [02:17<07:20,  1.70s/it]
Epoch 2/10:  29%|██▉       | 106/364 [02:17<05:42,  1.33s/it]
Epoch 2/10:  29%|██▉       | 107/364 [02:18<04:31,  1.06s/it]
Epoch 2/10:  30%|██▉       | 108/364 [02:18<03:41,  1.15it/s]
Epoch 2/10:  30%|██▉       | 109/364 [02:22<07:50,  1.84s/it]
Epoch 2/10:  30%|███       | 110/364 [02:23<05:59,  1.42s/it]
Epoch 2/10:  30%|███       | 111/364 [02:23<04:42,  1.12s/it]
Epoch 2/10:  31%|███       | 112/364 [02:24<03:48,  1.10it/s]
Epoch 2/10:  31%|███       | 113/364 [02:28<07:35,  1.82s/it]
Epoch 2/10:  31%|███▏      | 114/364 [02:28<05:49,  1.40s/it]
Epoch 2/10:  32%|███▏      | 115/364 [02:28<04:34,  1.10s/it]
Epoch 2/10:  32%|███▏      | 116/364 [02:29<03:43,  1.11it/s]
Epoch 2/10:  32%|███▏      | 117/364 [02:33<07:18,  1.78s/it]
Epoch 2/10:  32%|███▏      | 118/364 [02:33<05:37,  1.37s/it]
Epoch 2/10:  33%|███▎      | 119/364 [02:34<04:26,  1.09s/it]
Epoch 2/10:  33%|███▎      | 120/364 [02:34<03:36,  1.13it/s]
Epoch 2/10:  33%|███▎      | 121/364 [02:38<07:35,  1.88s/it]
Epoch 2/10:  34%|███▎      | 122/364 [02:39<05:49,  1.44s/it]
Epoch 2/10:  34%|███▍      | 123/364 [02:39<04:34,  1.14s/it]
Epoch 2/10:  34%|███▍      | 124/364 [02:39<03:42,  1.08it/s]
Epoch 2/10:  34%|███▍      | 125/364 [02:43<07:13,  1.81s/it]
Epoch 2/10:  35%|███▍      | 126/364 [02:44<05:32,  1.40s/it]
Epoch 2/10:  35%|███▍      | 127/364 [02:44<04:22,  1.11s/it]
Epoch 2/10:  35%|███▌      | 128/364 [02:45<03:31,  1.12it/s]
Epoch 2/10:  35%|███▌      | 129/364 [02:49<07:04,  1.81s/it]
Epoch 2/10:  36%|███▌      | 130/364 [02:49<05:25,  1.39s/it]
Epoch 2/10:  36%|███▌      | 131/364 [02:49<04:16,  1.10s/it]
Epoch 2/10:  36%|███▋      | 132/364 [02:50<03:27,  1.12it/s]
Epoch 2/10:  37%|███▋      | 133/364 [02:54<06:49,  1.77s/it]
Epoch 2/10:  37%|███▋      | 134/364 [02:54<05:13,  1.36s/it]
Epoch 2/10:  37%|███▋      | 135/364 [02:54<04:06,  1.08s/it]
Epoch 2/10:  37%|███▋      | 136/364 [02:55<03:21,  1.13it/s]
Epoch 2/10:  38%|███▊      | 137/364 [02:58<06:20,  1.68s/it]
Epoch 2/10:  38%|███▊      | 138/364 [02:59<04:54,  1.31s/it]
Epoch 2/10:  38%|███▊      | 139/364 [02:59<03:55,  1.05s/it]
Epoch 2/10:  38%|███▊      | 140/364 [03:00<03:12,  1.16it/s]
Epoch 2/10:  39%|███▊      | 141/364 [03:04<06:39,  1.79s/it]
Epoch 2/10:  39%|███▉      | 142/364 [03:04<05:06,  1.38s/it]
Epoch 2/10:  39%|███▉      | 143/364 [03:04<04:00,  1.09s/it]
Epoch 2/10:  40%|███▉      | 144/364 [03:05<03:15,  1.12it/s]
Epoch 2/10:  40%|███▉      | 145/364 [03:09<06:23,  1.75s/it]
Epoch 2/10:  40%|████      | 146/364 [03:09<04:55,  1.35s/it]
Epoch 2/10:  40%|████      | 147/364 [03:10<03:53,  1.08s/it]
Epoch 2/10:  41%|████      | 148/364 [03:10<03:10,  1.13it/s]
Epoch 2/10:  41%|████      | 149/364 [03:14<06:10,  1.72s/it]
Epoch 2/10:  41%|████      | 150/364 [03:14<04:45,  1.33s/it]
Epoch 2/10:  41%|████▏     | 151/364 [03:14<03:44,  1.05s/it]
Epoch 2/10:  42%|████▏     | 152/364 [03:15<03:03,  1.16it/s]
Epoch 2/10:  42%|████▏     | 153/364 [03:19<06:23,  1.82s/it]
Epoch 2/10:  42%|████▏     | 154/364 [03:19<04:54,  1.40s/it]
Epoch 2/10:  43%|████▎     | 155/364 [03:20<03:52,  1.11s/it]
Epoch 2/10:  43%|████▎     | 156/364 [03:20<03:06,  1.11it/s]
Epoch 2/10:  43%|████▎     | 157/364 [03:24<06:27,  1.87s/it]
Epoch 2/10:  43%|████▎     | 158/364 [03:25<04:57,  1.44s/it]
Epoch 2/10:  44%|████▎     | 159/364 [03:25<03:53,  1.14s/it]
Epoch 2/10:  44%|████▍     | 160/364 [03:26<03:08,  1.08it/s]
Epoch 2/10:  44%|████▍     | 161/364 [03:30<06:09,  1.82s/it]
Epoch 2/10:  45%|████▍     | 162/364 [03:30<04:43,  1.41s/it]
Epoch 2/10:  45%|████▍     | 163/364 [03:30<03:44,  1.12s/it]
Epoch 2/10:  45%|████▌     | 164/364 [03:31<03:00,  1.11it/s]
Epoch 2/10:  45%|████▌     | 165/364 [03:35<05:50,  1.76s/it]
Epoch 2/10:  46%|████▌     | 166/364 [03:35<04:29,  1.36s/it]
Epoch 2/10:  46%|████▌     | 167/364 [03:35<03:32,  1.08s/it]
Epoch 2/10:  46%|████▌     | 168/364 [03:36<02:53,  1.13it/s]
Epoch 2/10:  46%|████▋     | 169/364 [03:40<05:45,  1.77s/it]
Epoch 2/10:  47%|████▋     | 170/364 [03:40<04:25,  1.37s/it]
Epoch 2/10:  47%|████▋     | 171/364 [03:41<03:28,  1.08s/it]
Epoch 2/10:  47%|████▋     | 172/364 [03:41<02:49,  1.13it/s]
Epoch 2/10:  48%|████▊     | 173/364 [03:45<05:40,  1.79s/it]
Epoch 2/10:  48%|████▊     | 174/364 [03:45<04:22,  1.38s/it]
Epoch 2/10:  48%|████▊     | 175/364 [03:46<03:27,  1.10s/it]
Epoch 2/10:  48%|████▊     | 176/364 [03:46<02:48,  1.12it/s]
Epoch 2/10:  49%|████▊     | 177/364 [03:50<05:47,  1.86s/it]
Epoch 2/10:  49%|████▉     | 178/364 [03:51<04:25,  1.43s/it]
Epoch 2/10:  49%|████▉     | 179/364 [03:51<03:29,  1.13s/it]
Epoch 2/10:  49%|████▉     | 180/364 [03:52<02:49,  1.09it/s]
Epoch 2/10:  50%|████▉     | 181/364 [03:55<05:25,  1.78s/it]
Epoch 2/10:  50%|█████     | 182/364 [03:56<04:09,  1.37s/it]
Epoch 2/10:  50%|█████     | 183/364 [03:56<03:15,  1.08s/it]
Epoch 2/10:  51%|█████     | 184/364 [03:57<02:39,  1.13it/s]
Epoch 2/10:  51%|█████     | 185/364 [04:00<05:21,  1.80s/it]
Epoch 2/10:  51%|█████     | 186/364 [04:01<04:06,  1.38s/it]
Epoch 2/10:  51%|█████▏    | 187/364 [04:01<03:14,  1.10s/it]
Epoch 2/10:  52%|█████▏    | 188/364 [04:02<02:37,  1.12it/s]
Epoch 2/10:  52%|█████▏    | 189/364 [04:06<05:27,  1.87s/it]
Epoch 2/10:  52%|█████▏    | 190/364 [04:06<04:10,  1.44s/it]
Epoch 2/10:  52%|█████▏    | 191/364 [04:07<03:15,  1.13s/it]
Epoch 2/10:  53%|█████▎    | 192/364 [04:07<02:38,  1.09it/s]
Epoch 2/10:  53%|█████▎    | 193/364 [04:11<05:02,  1.77s/it]
Epoch 2/10:  53%|█████▎    | 194/364 [04:11<03:52,  1.37s/it]
Epoch 2/10:  54%|█████▎    | 195/364 [04:12<03:03,  1.08s/it]
Epoch 2/10:  54%|█████▍    | 196/364 [04:12<02:28,  1.13it/s]
Epoch 2/10:  54%|█████▍    | 197/364 [04:16<04:56,  1.77s/it]
Epoch 2/10:  54%|█████▍    | 198/364 [04:16<03:46,  1.37s/it]
Epoch 2/10:  55%|█████▍    | 199/364 [04:17<02:59,  1.09s/it]
Epoch 2/10:  55%|█████▍    | 200/364 [04:17<02:26,  1.12it/s]
Epoch 2/10:  55%|█████▌    | 201/364 [04:21<04:51,  1.79s/it]
Epoch 2/10:  55%|█████▌    | 202/364 [04:22<03:43,  1.38s/it]
Epoch 2/10:  56%|█████▌    | 203/364 [04:22<02:56,  1.09s/it]
Epoch 2/10:  56%|█████▌    | 204/364 [04:23<02:23,  1.12it/s]
Epoch 2/10:  56%|█████▋    | 205/364 [04:26<04:32,  1.71s/it]
Epoch 2/10:  57%|█████▋    | 206/364 [04:27<03:29,  1.33s/it]
Epoch 2/10:  57%|█████▋    | 207/364 [04:27<02:45,  1.06s/it]
Epoch 2/10:  57%|█████▋    | 208/364 [04:27<02:14,  1.16it/s]
Epoch 2/10:  57%|█████▋    | 209/364 [04:31<04:32,  1.76s/it]
Epoch 2/10:  58%|█████▊    | 210/364 [04:32<03:28,  1.35s/it]
Epoch 2/10:  58%|█████▊    | 211/364 [04:32<02:44,  1.08s/it]
Epoch 2/10:  58%|█████▊    | 212/364 [04:33<02:14,  1.13it/s]
Epoch 2/10:  59%|█████▊    | 213/364 [04:36<04:22,  1.74s/it]
Epoch 2/10:  59%|█████▉    | 214/364 [04:37<03:21,  1.34s/it]
Epoch 2/10:  59%|█████▉    | 215/364 [04:37<02:38,  1.07s/it]
Epoch 2/10:  59%|█████▉    | 216/364 [04:38<02:10,  1.14it/s]
Epoch 2/10:  60%|█████▉    | 217/364 [04:41<04:20,  1.77s/it]
Epoch 2/10:  60%|█████▉    | 218/364 [04:42<03:19,  1.37s/it]
Epoch 2/10:  60%|██████    | 219/364 [04:42<02:37,  1.09s/it]
Epoch 2/10:  60%|██████    | 220/364 [04:43<02:07,  1.13it/s]
Epoch 2/10:  61%|██████    | 221/364 [04:46<04:08,  1.74s/it]
Epoch 2/10:  61%|██████    | 222/364 [04:47<03:09,  1.34s/it]
Epoch 2/10:  61%|██████▏   | 223/364 [04:47<02:29,  1.06s/it]
Epoch 2/10:  62%|██████▏   | 224/364 [04:48<02:01,  1.15it/s]
Epoch 2/10:  62%|██████▏   | 225/364 [04:52<04:10,  1.80s/it]
Epoch 2/10:  62%|██████▏   | 226/364 [04:52<03:11,  1.39s/it]
Epoch 2/10:  62%|██████▏   | 227/364 [04:52<02:30,  1.10s/it]
Epoch 2/10:  63%|██████▎   | 228/364 [04:53<02:01,  1.12it/s]
Epoch 2/10:  63%|██████▎   | 229/364 [04:56<03:51,  1.71s/it]
Epoch 2/10:  63%|██████▎   | 230/364 [04:57<02:57,  1.33s/it]
Epoch 2/10:  63%|██████▎   | 231/364 [04:57<02:20,  1.06s/it]
Epoch 2/10:  64%|██████▎   | 232/364 [04:58<01:54,  1.15it/s]
Epoch 2/10:  64%|██████▍   | 233/364 [05:02<03:47,  1.74s/it]
Epoch 2/10:  64%|██████▍   | 234/364 [05:02<02:54,  1.34s/it]
Epoch 2/10:  65%|██████▍   | 235/364 [05:02<02:17,  1.07s/it]
Epoch 2/10:  65%|██████▍   | 236/364 [05:03<01:51,  1.14it/s]
Epoch 2/10:  65%|██████▌   | 237/364 [05:07<03:38,  1.72s/it]
Epoch 2/10:  65%|██████▌   | 238/364 [05:07<02:48,  1.34s/it]
Epoch 2/10:  66%|██████▌   | 239/364 [05:07<02:13,  1.07s/it]
Epoch 2/10:  66%|██████▌   | 240/364 [05:08<01:49,  1.13it/s]
Epoch 2/10:  66%|██████▌   | 241/364 [05:12<03:46,  1.84s/it]
Epoch 2/10:  66%|██████▋   | 242/364 [05:12<02:53,  1.42s/it]
Epoch 2/10:  67%|██████▋   | 243/364 [05:13<02:15,  1.12s/it]
Epoch 2/10:  67%|██████▋   | 244/364 [05:13<01:50,  1.09it/s]
Epoch 2/10:  67%|██████▋   | 245/364 [05:17<03:32,  1.79s/it]
Epoch 2/10:  68%|██████▊   | 246/364 [05:17<02:42,  1.38s/it]
Epoch 2/10:  68%|██████▊   | 247/364 [05:18<02:07,  1.09s/it]
Epoch 2/10:  68%|██████▊   | 248/364 [05:18<01:42,  1.13it/s]
Epoch 2/10:  68%|██████▊   | 249/364 [05:22<03:32,  1.85s/it]
Epoch 2/10:  69%|██████▊   | 250/364 [05:23<02:41,  1.42s/it]
Epoch 2/10:  69%|██████▉   | 251/364 [05:23<02:07,  1.12s/it]
Epoch 2/10:  69%|██████▉   | 252/364 [05:24<01:42,  1.09it/s]
Epoch 2/10:  70%|██████▉   | 253/364 [05:27<03:09,  1.71s/it]
Epoch 2/10:  70%|██████▉   | 254/364 [05:28<02:25,  1.32s/it]
Epoch 2/10:  70%|███████   | 255/364 [05:28<01:54,  1.05s/it]
Epoch 2/10:  70%|███████   | 256/364 [05:29<01:33,  1.15it/s]
Epoch 2/10:  71%|███████   | 257/364 [05:32<03:07,  1.75s/it]
Epoch 2/10:  71%|███████   | 258/364 [05:33<02:23,  1.36s/it]
Epoch 2/10:  71%|███████   | 259/364 [05:33<01:52,  1.07s/it]
Epoch 2/10:  71%|███████▏  | 260/364 [05:34<01:31,  1.14it/s]
Epoch 2/10:  72%|███████▏  | 261/364 [05:38<03:07,  1.82s/it]
Epoch 2/10:  72%|███████▏  | 262/364 [05:38<02:22,  1.40s/it]
Epoch 2/10:  72%|███████▏  | 263/364 [05:38<01:51,  1.11s/it]
Epoch 2/10:  73%|███████▎  | 264/364 [05:39<01:30,  1.11it/s]
Epoch 2/10:  73%|███████▎  | 265/364 [05:42<02:38,  1.60s/it]
Epoch 2/10:  73%|███████▎  | 266/364 [05:43<02:02,  1.25s/it]
Epoch 2/10:  73%|███████▎  | 267/364 [05:43<01:37,  1.01s/it]
Epoch 2/10:  74%|███████▎  | 268/364 [05:43<01:20,  1.20it/s]
Epoch 2/10:  74%|███████▍  | 269/364 [05:47<02:36,  1.65s/it]
Epoch 2/10:  74%|███████▍  | 270/364 [05:47<02:00,  1.28s/it]
Epoch 2/10:  74%|███████▍  | 271/364 [05:48<01:35,  1.03s/it]
Epoch 2/10:  75%|███████▍  | 272/364 [05:48<01:18,  1.18it/s]
Epoch 2/10:  75%|███████▌  | 273/364 [05:52<02:38,  1.74s/it]
Epoch 2/10:  75%|███████▌  | 274/364 [05:53<02:01,  1.35s/it]
Epoch 2/10:  76%|███████▌  | 275/364 [05:53<01:35,  1.07s/it]
Epoch 2/10:  76%|███████▌  | 276/364 [05:53<01:16,  1.15it/s]
Epoch 2/10:  76%|███████▌  | 277/364 [05:57<02:38,  1.82s/it]
Epoch 2/10:  76%|███████▋  | 278/364 [05:58<02:00,  1.41s/it]
Epoch 2/10:  77%|███████▋  | 279/364 [05:58<01:34,  1.11s/it]
Epoch 2/10:  77%|███████▋  | 280/364 [05:59<01:15,  1.11it/s]
Epoch 2/10:  77%|███████▋  | 281/364 [06:03<02:30,  1.82s/it]
Epoch 2/10:  77%|███████▋  | 282/364 [06:03<01:54,  1.40s/it]
Epoch 2/10:  78%|███████▊  | 283/364 [06:03<01:29,  1.11s/it]
Epoch 2/10:  78%|███████▊  | 284/364 [06:04<01:12,  1.10it/s]
Epoch 2/10:  78%|███████▊  | 285/364 [06:07<02:15,  1.72s/it]
Epoch 2/10:  79%|███████▊  | 286/364 [06:08<01:43,  1.33s/it]
Epoch 2/10:  79%|███████▉  | 287/364 [06:08<01:21,  1.06s/it]
Epoch 2/10:  79%|███████▉  | 288/364 [06:09<01:06,  1.14it/s]
Epoch 2/10:  79%|███████▉  | 289/364 [06:12<02:04,  1.66s/it]
Epoch 2/10:  80%|███████▉  | 290/364 [06:13<01:35,  1.28s/it]
Epoch 2/10:  80%|███████▉  | 291/364 [06:13<01:14,  1.02s/it]
Epoch 2/10:  80%|████████  | 292/364 [06:14<01:00,  1.18it/s]
Epoch 2/10:  80%|████████  | 293/364 [06:18<02:10,  1.84s/it]
Epoch 2/10:  81%|████████  | 294/364 [06:18<01:38,  1.41s/it]
Epoch 2/10:  81%|████████  | 295/364 [06:19<01:16,  1.11s/it]
Epoch 2/10:  81%|████████▏ | 296/364 [06:19<01:02,  1.09it/s]
Epoch 2/10:  82%|████████▏ | 297/364 [06:23<02:00,  1.79s/it]
Epoch 2/10:  82%|████████▏ | 298/364 [06:23<01:31,  1.38s/it]
Epoch 2/10:  82%|████████▏ | 299/364 [06:24<01:11,  1.10s/it]
Epoch 2/10:  82%|████████▏ | 300/364 [06:24<00:57,  1.11it/s]
Epoch 2/10:  83%|████████▎ | 301/364 [06:28<01:52,  1.78s/it]
Epoch 2/10:  83%|████████▎ | 302/364 [06:28<01:25,  1.37s/it]
Epoch 2/10:  83%|████████▎ | 303/364 [06:29<01:06,  1.09s/it]
Epoch 2/10:  84%|████████▎ | 304/364 [06:29<00:53,  1.13it/s]
Epoch 2/10:  84%|████████▍ | 305/364 [06:33<01:46,  1.80s/it]
Epoch 2/10:  84%|████████▍ | 306/364 [06:34<01:20,  1.39s/it]
Epoch 2/10:  84%|████████▍ | 307/364 [06:34<01:02,  1.10s/it]
Epoch 2/10:  85%|████████▍ | 308/364 [06:34<00:50,  1.11it/s]
Epoch 2/10:  85%|████████▍ | 309/364 [06:39<01:43,  1.89s/it]
Epoch 2/10:  85%|████████▌ | 310/364 [06:39<01:18,  1.45s/it]
Epoch 2/10:  85%|████████▌ | 311/364 [06:40<01:00,  1.15s/it]
Epoch 2/10:  86%|████████▌ | 312/364 [06:40<00:48,  1.07it/s]
Epoch 2/10:  86%|████████▌ | 313/364 [06:44<01:30,  1.76s/it]
Epoch 2/10:  86%|████████▋ | 314/364 [06:44<01:08,  1.37s/it]
Epoch 2/10:  87%|████████▋ | 315/364 [06:45<00:53,  1.08s/it]
Epoch 2/10:  87%|████████▋ | 316/364 [06:45<00:42,  1.13it/s]
Epoch 2/10:  87%|████████▋ | 317/364 [06:49<01:22,  1.75s/it]
Epoch 2/10:  87%|████████▋ | 318/364 [06:49<01:02,  1.35s/it]
Epoch 2/10:  88%|████████▊ | 319/364 [06:50<00:48,  1.07s/it]
Epoch 2/10:  88%|████████▊ | 320/364 [06:50<00:38,  1.14it/s]
Epoch 2/10:  88%|████████▊ | 321/364 [06:54<01:20,  1.87s/it]
Epoch 2/10:  88%|████████▊ | 322/364 [06:55<01:00,  1.43s/it]
Epoch 2/10:  89%|████████▊ | 323/364 [06:55<00:46,  1.13s/it]
Epoch 2/10:  89%|████████▉ | 324/364 [06:55<00:36,  1.09it/s]
Epoch 2/10:  89%|████████▉ | 325/364 [06:59<01:11,  1.84s/it]
Epoch 2/10:  90%|████████▉ | 326/364 [07:00<00:53,  1.42s/it]
Epoch 2/10:  90%|████████▉ | 327/364 [07:00<00:41,  1.12s/it]
Epoch 2/10:  90%|█████████ | 328/364 [07:01<00:32,  1.10it/s]
Epoch 2/10:  90%|█████████ | 329/364 [07:05<01:05,  1.88s/it]
Epoch 2/10:  91%|█████████ | 330/364 [07:05<00:49,  1.44s/it]
Epoch 2/10:  91%|█████████ | 331/364 [07:06<00:37,  1.14s/it]
Epoch 2/10:  91%|█████████ | 332/364 [07:06<00:29,  1.08it/s]
Epoch 2/10:  91%|█████████▏| 333/364 [07:10<00:54,  1.75s/it]
Epoch 2/10:  92%|█████████▏| 334/364 [07:10<00:40,  1.36s/it]
Epoch 2/10:  92%|█████████▏| 335/364 [07:11<00:31,  1.07s/it]
Epoch 2/10:  92%|█████████▏| 336/364 [07:11<00:24,  1.14it/s]
Epoch 2/10:  93%|█████████▎| 337/364 [07:15<00:45,  1.69s/it]
Epoch 2/10:  93%|█████████▎| 338/364 [07:15<00:33,  1.31s/it]
Epoch 2/10:  93%|█████████▎| 339/364 [07:15<00:25,  1.04s/it]
Epoch 2/10:  93%|█████████▎| 340/364 [07:16<00:20,  1.17it/s]
Epoch 2/10:  94%|█████████▎| 341/364 [07:20<00:42,  1.86s/it]
Epoch 2/10:  94%|█████████▍| 342/364 [07:20<00:31,  1.43s/it]
Epoch 2/10:  94%|█████████▍| 343/364 [07:21<00:23,  1.13s/it]
Epoch 2/10:  95%|█████████▍| 344/364 [07:21<00:18,  1.09it/s]
Epoch 2/10:  95%|█████████▍| 345/364 [07:25<00:33,  1.78s/it]
Epoch 2/10:  95%|█████████▌| 346/364 [07:26<00:24,  1.37s/it]
Epoch 2/10:  95%|█████████▌| 347/364 [07:26<00:18,  1.09s/it]
Epoch 2/10:  96%|█████████▌| 348/364 [07:26<00:14,  1.12it/s]
Epoch 2/10:  96%|█████████▌| 349/364 [07:30<00:26,  1.75s/it]
Epoch 2/10:  96%|█████████▌| 350/364 [07:31<00:18,  1.36s/it]
Epoch 2/10:  96%|█████████▋| 351/364 [07:31<00:13,  1.07s/it]
Epoch 2/10:  97%|█████████▋| 352/364 [07:31<00:10,  1.13it/s]
Epoch 2/10:  97%|█████████▋| 353/364 [07:36<00:20,  1.84s/it]
Epoch 2/10:  97%|█████████▋| 354/364 [07:36<00:14,  1.42s/it]
Epoch 2/10:  98%|█████████▊| 355/364 [07:36<00:10,  1.12s/it]
Epoch 2/10:  98%|█████████▊| 356/364 [07:37<00:07,  1.09it/s]
Epoch 2/10:  98%|█████████▊| 357/364 [07:41<00:12,  1.76s/it]
Epoch 2/10:  98%|█████████▊| 358/364 [07:41<00:08,  1.35s/it]
Epoch 2/10:  99%|█████████▊| 359/364 [07:41<00:05,  1.08s/it]
Epoch 2/10:  99%|█████████▉| 360/364 [07:42<00:03,  1.14it/s]
Epoch 2/10:  99%|█████████▉| 361/364 [07:44<00:03,  1.15s/it]
Epoch 2/10:  99%|█████████▉| 362/364 [07:44<00:01,  1.08it/s]
Epoch 2/10: 100%|█████████▉| 363/364 [07:44<00:00,  1.30it/s]
Epoch 2/10: 100%|██████████| 364/364 [07:45<00:00,  1.69it/s]
Epoch 2/10: 100%|██████████| 364/364 [07:45<00:00,  1.28s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:05<04:52,  5.23s/it]
Validating:   4%|▎         | 2/57 [00:05<02:03,  2.25s/it]
Validating:   5%|▌         | 3/57 [00:05<01:09,  1.28s/it]
Validating:   7%|▋         | 4/57 [00:05<00:44,  1.20it/s]
Validating:   9%|▉         | 5/57 [00:09<01:42,  1.98s/it]
Validating:  11%|█         | 6/57 [00:09<01:09,  1.36s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:48,  1.03it/s]
Validating:  14%|█▍        | 8/57 [00:10<00:34,  1.42it/s]
Validating:  16%|█▌        | 9/57 [00:14<01:22,  1.71s/it]
Validating:  18%|█▊        | 10/57 [00:14<00:58,  1.24s/it]
Validating:  19%|█▉        | 11/57 [00:14<00:41,  1.11it/s]
Validating:  21%|██        | 12/57 [00:14<00:30,  1.49it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:18,  1.78s/it]
Validating:  25%|██▍       | 14/57 [00:19<00:55,  1.30s/it]
Validating:  26%|██▋       | 15/57 [00:19<00:40,  1.05it/s]
Validating:  28%|██▊       | 16/57 [00:19<00:29,  1.41it/s]
Validating:  30%|██▉       | 17/57 [00:23<01:05,  1.65s/it]
Validating:  32%|███▏      | 18/57 [00:23<00:46,  1.19s/it]
Validating:  33%|███▎      | 19/57 [00:23<00:33,  1.15it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:23,  1.54it/s]
Validating:  37%|███▋      | 21/57 [00:27<01:01,  1.72s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:43,  1.25s/it]
Validating:  40%|████      | 23/57 [00:28<00:30,  1.10it/s]
Validating:  42%|████▏     | 24/57 [00:28<00:22,  1.48it/s]
Validating:  44%|████▍     | 25/57 [00:32<00:55,  1.74s/it]
Validating:  46%|████▌     | 26/57 [00:32<00:39,  1.26s/it]
Validating:  47%|████▋     | 27/57 [00:32<00:27,  1.08it/s]
Validating:  49%|████▉     | 28/57 [00:32<00:20,  1.45it/s]
Validating:  51%|█████     | 29/57 [00:36<00:46,  1.67s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:32,  1.21s/it]
Validating:  54%|█████▍    | 31/57 [00:37<00:23,  1.13it/s]
Validating:  56%|█████▌    | 32/57 [00:37<00:16,  1.52it/s]
Validating:  58%|█████▊    | 33/57 [00:41<00:40,  1.68s/it]
Validating:  60%|█████▉    | 34/57 [00:41<00:28,  1.22s/it]
Validating:  61%|██████▏   | 35/57 [00:41<00:19,  1.11it/s]
Validating:  63%|██████▎   | 36/57 [00:41<00:14,  1.49it/s]
Validating:  65%|██████▍   | 37/57 [00:46<00:35,  1.78s/it]
Validating:  67%|██████▋   | 38/57 [00:46<00:24,  1.30s/it]
Validating:  68%|██████▊   | 39/57 [00:46<00:17,  1.05it/s]
Validating:  70%|███████   | 40/57 [00:46<00:12,  1.41it/s]
Validating:  72%|███████▏  | 41/57 [00:50<00:27,  1.75s/it]
Validating:  74%|███████▎  | 42/57 [00:51<00:20,  1.35s/it]
Validating:  75%|███████▌  | 43/57 [00:51<00:13,  1.01it/s]
Validating:  77%|███████▋  | 44/57 [00:51<00:09,  1.35it/s]
Validating:  79%|███████▉  | 45/57 [00:55<00:19,  1.62s/it]
Validating:  81%|████████  | 46/57 [00:55<00:14,  1.34s/it]
Validating:  82%|████████▏ | 47/57 [00:55<00:09,  1.02it/s]
Validating:  84%|████████▍ | 48/57 [00:56<00:06,  1.37it/s]
Validating:  86%|████████▌ | 49/57 [00:59<00:11,  1.41s/it]
Validating:  88%|████████▊ | 50/57 [01:00<00:09,  1.34s/it]
Validating:  89%|████████▉ | 51/57 [01:00<00:05,  1.02it/s]
Validating:  91%|█████████ | 52/57 [01:00<00:03,  1.37it/s]
Validating:  93%|█████████▎| 53/57 [01:02<00:04,  1.25s/it]
Validating:  95%|█████████▍| 54/57 [01:03<00:03,  1.07s/it]
Validating:  96%|█████████▋| 55/57 [01:03<00:01,  1.27it/s]
Validating:  98%|█████████▊| 56/57 [01:03<00:00,  1.71it/s]
Validating: 100%|██████████| 57/57 [01:03<00:00,  1.12s/it]
Epoch 2: Train Loss: 0.3287, Train Acc: 0.8829 | Val Loss: 0.3185, Val Acc: 0.8889
Saved Best Model!

Epoch 3/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 3/10:   0%|          | 1/364 [00:05<34:50,  5.76s/it]
Epoch 3/10:   1%|          | 2/364 [00:06<15:47,  2.62s/it]
Epoch 3/10:   1%|          | 3/364 [00:06<09:44,  1.62s/it]
Epoch 3/10:   1%|          | 4/364 [00:07<06:52,  1.15s/it]
Epoch 3/10:   1%|▏         | 5/364 [00:10<11:58,  2.00s/it]
Epoch 3/10:   2%|▏         | 6/364 [00:10<08:44,  1.47s/it]
Epoch 3/10:   2%|▏         | 7/364 [00:11<06:39,  1.12s/it]
Epoch 3/10:   2%|▏         | 8/364 [00:11<05:19,  1.11it/s]
Epoch 3/10:   2%|▏         | 9/364 [00:15<09:55,  1.68s/it]
Epoch 3/10:   3%|▎         | 10/364 [00:16<09:09,  1.55s/it]
Epoch 3/10:   3%|▎         | 11/364 [00:16<07:05,  1.20s/it]
Epoch 3/10:   3%|▎         | 12/364 [00:17<05:39,  1.04it/s]
Epoch 3/10:   4%|▎         | 13/364 [00:20<08:56,  1.53s/it]
Epoch 3/10:   4%|▍         | 14/364 [00:21<08:52,  1.52s/it]
Epoch 3/10:   4%|▍         | 15/364 [00:22<06:54,  1.19s/it]
Epoch 3/10:   4%|▍         | 16/364 [00:22<05:32,  1.05it/s]
Epoch 3/10:   5%|▍         | 17/364 [00:25<08:48,  1.52s/it]
Epoch 3/10:   5%|▍         | 18/364 [00:26<08:49,  1.53s/it]
Epoch 3/10:   5%|▌         | 19/364 [00:27<06:53,  1.20s/it]
Epoch 3/10:   5%|▌         | 20/364 [00:27<05:33,  1.03it/s]
Epoch 3/10:   6%|▌         | 21/364 [00:30<08:21,  1.46s/it]
Epoch 3/10:   6%|▌         | 22/364 [00:31<08:16,  1.45s/it]
Epoch 3/10:   6%|▋         | 23/364 [00:32<06:30,  1.15s/it]
Epoch 3/10:   7%|▋         | 24/364 [00:32<05:17,  1.07it/s]
Epoch 3/10:   7%|▋         | 25/364 [00:35<08:38,  1.53s/it]
Epoch 3/10:   7%|▋         | 26/364 [00:36<08:13,  1.46s/it]
Epoch 3/10:   7%|▋         | 27/364 [00:37<06:27,  1.15s/it]
Epoch 3/10:   8%|▊         | 28/364 [00:37<05:13,  1.07it/s]
Epoch 3/10:   8%|▊         | 29/364 [00:40<08:57,  1.60s/it]
Epoch 3/10:   8%|▊         | 30/364 [00:42<08:23,  1.51s/it]
Epoch 3/10:   9%|▊         | 31/364 [00:42<06:33,  1.18s/it]
Epoch 3/10:   9%|▉         | 32/364 [00:43<05:17,  1.04it/s]
Epoch 3/10:   9%|▉         | 33/364 [00:46<09:02,  1.64s/it]
Epoch 3/10:   9%|▉         | 34/364 [00:47<08:05,  1.47s/it]
Epoch 3/10:  10%|▉         | 35/364 [00:47<06:18,  1.15s/it]
Epoch 3/10:  10%|▉         | 36/364 [00:48<05:07,  1.07it/s]
Epoch 3/10:  10%|█         | 37/364 [00:51<09:11,  1.69s/it]
Epoch 3/10:  10%|█         | 38/364 [00:52<08:15,  1.52s/it]
Epoch 3/10:  11%|█         | 39/364 [00:53<06:29,  1.20s/it]
Epoch 3/10:  11%|█         | 40/364 [00:53<05:13,  1.03it/s]
Epoch 3/10:  11%|█▏        | 41/364 [00:56<08:03,  1.50s/it]
Epoch 3/10:  12%|█▏        | 42/364 [00:58<08:28,  1.58s/it]
Epoch 3/10:  12%|█▏        | 43/364 [00:58<06:36,  1.24s/it]
Epoch 3/10:  12%|█▏        | 44/364 [00:58<05:19,  1.00it/s]
Epoch 3/10:  12%|█▏        | 45/364 [01:01<07:11,  1.35s/it]
Epoch 3/10:  13%|█▎        | 46/364 [01:03<08:30,  1.60s/it]
Epoch 3/10:  13%|█▎        | 47/364 [01:03<06:36,  1.25s/it]
Epoch 3/10:  13%|█▎        | 48/364 [01:04<05:16,  1.00s/it]
Epoch 3/10:  13%|█▎        | 49/364 [01:05<06:29,  1.24s/it]
Epoch 3/10:  14%|█▎        | 50/364 [01:08<08:12,  1.57s/it]
Epoch 3/10:  14%|█▍        | 51/364 [01:08<06:25,  1.23s/it]
Epoch 3/10:  14%|█▍        | 52/364 [01:09<05:10,  1.01it/s]
Epoch 3/10:  15%|█▍        | 53/364 [01:11<06:26,  1.24s/it]
Epoch 3/10:  15%|█▍        | 54/364 [01:13<08:11,  1.58s/it]
Epoch 3/10:  15%|█▌        | 55/364 [01:13<06:20,  1.23s/it]
Epoch 3/10:  15%|█▌        | 56/364 [01:14<05:04,  1.01it/s]
Epoch 3/10:  16%|█▌        | 57/364 [01:15<06:02,  1.18s/it]
Epoch 3/10:  16%|█▌        | 58/364 [01:18<08:33,  1.68s/it]
Epoch 3/10:  16%|█▌        | 59/364 [01:19<06:37,  1.30s/it]
Epoch 3/10:  16%|█▋        | 60/364 [01:19<05:16,  1.04s/it]
Epoch 3/10:  17%|█▋        | 61/364 [01:21<06:16,  1.24s/it]
Epoch 3/10:  17%|█▋        | 62/364 [01:24<08:52,  1.76s/it]
Epoch 3/10:  17%|█▋        | 63/364 [01:24<06:49,  1.36s/it]
Epoch 3/10:  18%|█▊        | 64/364 [01:25<05:24,  1.08s/it]
Epoch 3/10:  18%|█▊        | 65/364 [01:26<05:51,  1.17s/it]
Epoch 3/10:  18%|█▊        | 66/364 [01:29<08:40,  1.75s/it]
Epoch 3/10:  18%|█▊        | 67/364 [01:30<06:40,  1.35s/it]
Epoch 3/10:  19%|█▊        | 68/364 [01:30<05:17,  1.07s/it]
Epoch 3/10:  19%|█▉        | 69/364 [01:31<05:47,  1.18s/it]
Epoch 3/10:  19%|█▉        | 70/364 [01:34<07:56,  1.62s/it]
Epoch 3/10:  20%|█▉        | 71/364 [01:34<06:10,  1.27s/it]
Epoch 3/10:  20%|█▉        | 72/364 [01:35<04:57,  1.02s/it]
Epoch 3/10:  20%|██        | 73/364 [01:37<06:13,  1.28s/it]
Epoch 3/10:  20%|██        | 74/364 [01:39<07:47,  1.61s/it]
Epoch 3/10:  21%|██        | 75/364 [01:40<06:03,  1.26s/it]
Epoch 3/10:  21%|██        | 76/364 [01:40<04:49,  1.01s/it]
Epoch 3/10:  21%|██        | 77/364 [01:42<06:01,  1.26s/it]
Epoch 3/10:  21%|██▏       | 78/364 [01:44<07:55,  1.66s/it]
Epoch 3/10:  22%|██▏       | 79/364 [01:45<06:07,  1.29s/it]
Epoch 3/10:  22%|██▏       | 80/364 [01:45<04:51,  1.03s/it]
Epoch 3/10:  22%|██▏       | 81/364 [01:47<06:13,  1.32s/it]
Epoch 3/10:  23%|██▎       | 82/364 [01:50<08:01,  1.71s/it]
Epoch 3/10:  23%|██▎       | 83/364 [01:50<06:11,  1.32s/it]
Epoch 3/10:  23%|██▎       | 84/364 [01:51<04:55,  1.06s/it]
Epoch 3/10:  23%|██▎       | 85/364 [01:52<05:39,  1.22s/it]
Epoch 3/10:  24%|██▎       | 86/364 [01:55<07:52,  1.70s/it]
Epoch 3/10:  24%|██▍       | 87/364 [01:56<06:05,  1.32s/it]
Epoch 3/10:  24%|██▍       | 88/364 [01:56<04:50,  1.05s/it]
Epoch 3/10:  24%|██▍       | 89/364 [01:58<05:41,  1.24s/it]
Epoch 3/10:  25%|██▍       | 90/364 [02:00<07:29,  1.64s/it]
Epoch 3/10:  25%|██▌       | 91/364 [02:01<05:51,  1.29s/it]
Epoch 3/10:  25%|██▌       | 92/364 [02:01<04:40,  1.03s/it]
Epoch 3/10:  26%|██▌       | 93/364 [02:03<05:23,  1.19s/it]
Epoch 3/10:  26%|██▌       | 94/364 [02:05<07:20,  1.63s/it]
Epoch 3/10:  26%|██▌       | 95/364 [02:06<05:41,  1.27s/it]
Epoch 3/10:  26%|██▋       | 96/364 [02:06<04:32,  1.02s/it]
Epoch 3/10:  27%|██▋       | 97/364 [02:08<05:30,  1.24s/it]
Epoch 3/10:  27%|██▋       | 98/364 [02:11<07:30,  1.70s/it]
Epoch 3/10:  27%|██▋       | 99/364 [02:11<05:46,  1.31s/it]
Epoch 3/10:  27%|██▋       | 100/364 [02:12<04:36,  1.05s/it]
Epoch 3/10:  28%|██▊       | 101/364 [02:13<04:47,  1.09s/it]
Epoch 3/10:  28%|██▊       | 102/364 [02:16<07:46,  1.78s/it]
Epoch 3/10:  28%|██▊       | 103/364 [02:17<06:00,  1.38s/it]
Epoch 3/10:  29%|██▊       | 104/364 [02:17<04:44,  1.09s/it]
Epoch 3/10:  29%|██▉       | 105/364 [02:18<04:23,  1.02s/it]
Epoch 3/10:  29%|██▉       | 106/364 [02:21<06:59,  1.63s/it]
Epoch 3/10:  29%|██▉       | 107/364 [02:21<05:25,  1.27s/it]
Epoch 3/10:  30%|██▉       | 108/364 [02:22<04:19,  1.01s/it]
Epoch 3/10:  30%|██▉       | 109/364 [02:23<04:56,  1.16s/it]
Epoch 3/10:  30%|███       | 110/364 [02:26<06:34,  1.55s/it]
Epoch 3/10:  30%|███       | 111/364 [02:26<05:06,  1.21s/it]
Epoch 3/10:  31%|███       | 112/364 [02:27<04:06,  1.02it/s]
Epoch 3/10:  31%|███       | 113/364 [02:29<05:20,  1.28s/it]
Epoch 3/10:  31%|███▏      | 114/364 [02:31<06:23,  1.53s/it]
Epoch 3/10:  32%|███▏      | 115/364 [02:31<04:58,  1.20s/it]
Epoch 3/10:  32%|███▏      | 116/364 [02:32<04:00,  1.03it/s]
Epoch 3/10:  32%|███▏      | 117/364 [02:34<05:39,  1.37s/it]
Epoch 3/10:  32%|███▏      | 118/364 [02:36<06:34,  1.60s/it]
Epoch 3/10:  33%|███▎      | 119/364 [02:37<05:06,  1.25s/it]
Epoch 3/10:  33%|███▎      | 120/364 [02:37<04:05,  1.00s/it]
Epoch 3/10:  33%|███▎      | 121/364 [02:39<05:04,  1.25s/it]
Epoch 3/10:  34%|███▎      | 122/364 [02:41<06:40,  1.66s/it]
Epoch 3/10:  34%|███▍      | 123/364 [02:42<05:11,  1.29s/it]
Epoch 3/10:  34%|███▍      | 124/364 [02:42<04:08,  1.04s/it]
Epoch 3/10:  34%|███▍      | 125/364 [02:44<04:42,  1.18s/it]
Epoch 3/10:  35%|███▍      | 126/364 [02:47<06:37,  1.67s/it]
Epoch 3/10:  35%|███▍      | 127/364 [02:47<05:07,  1.30s/it]
Epoch 3/10:  35%|███▌      | 128/364 [02:47<04:05,  1.04s/it]
Epoch 3/10:  35%|███▌      | 129/364 [02:49<04:43,  1.21s/it]
Epoch 3/10:  36%|███▌      | 130/364 [02:52<06:36,  1.70s/it]
Epoch 3/10:  36%|███▌      | 131/364 [02:52<05:06,  1.32s/it]
Epoch 3/10:  36%|███▋      | 132/364 [02:53<04:03,  1.05s/it]
Epoch 3/10:  37%|███▋      | 133/364 [02:54<04:33,  1.18s/it]
Epoch 3/10:  37%|███▋      | 134/364 [02:57<06:18,  1.65s/it]
Epoch 3/10:  37%|███▋      | 135/364 [02:57<04:53,  1.28s/it]
Epoch 3/10:  37%|███▋      | 136/364 [02:58<03:53,  1.02s/it]
Epoch 3/10:  38%|███▊      | 137/364 [02:59<03:58,  1.05s/it]
Epoch 3/10:  38%|███▊      | 138/364 [03:02<06:09,  1.63s/it]
Epoch 3/10:  38%|███▊      | 139/364 [03:02<04:47,  1.28s/it]
Epoch 3/10:  38%|███▊      | 140/364 [03:03<03:48,  1.02s/it]
Epoch 3/10:  39%|███▊      | 141/364 [03:04<03:29,  1.07it/s]
Epoch 3/10:  39%|███▉      | 142/364 [03:07<06:18,  1.71s/it]
Epoch 3/10:  39%|███▉      | 143/364 [03:07<04:51,  1.32s/it]
Epoch 3/10:  40%|███▉      | 144/364 [03:08<03:53,  1.06s/it]
Epoch 3/10:  40%|███▉      | 145/364 [03:08<03:14,  1.13it/s]
Epoch 3/10:  40%|████      | 146/364 [03:12<06:26,  1.77s/it]
Epoch 3/10:  40%|████      | 147/364 [03:13<04:58,  1.37s/it]
Epoch 3/10:  41%|████      | 148/364 [03:13<03:55,  1.09s/it]
Epoch 3/10:  41%|████      | 149/364 [03:14<03:15,  1.10it/s]
Epoch 3/10:  41%|████      | 150/364 [03:17<06:07,  1.72s/it]
Epoch 3/10:  41%|████▏     | 151/364 [03:18<04:44,  1.33s/it]
Epoch 3/10:  42%|████▏     | 152/364 [03:18<03:45,  1.06s/it]
Epoch 3/10:  42%|████▏     | 153/364 [03:19<03:23,  1.04it/s]
Epoch 3/10:  42%|████▏     | 154/364 [03:22<05:57,  1.70s/it]
Epoch 3/10:  43%|████▎     | 155/364 [03:23<04:35,  1.32s/it]
Epoch 3/10:  43%|████▎     | 156/364 [03:23<03:39,  1.05s/it]
Epoch 3/10:  43%|████▎     | 157/364 [03:24<03:16,  1.05it/s]
Epoch 3/10:  43%|████▎     | 158/364 [03:27<05:52,  1.71s/it]
Epoch 3/10:  44%|████▎     | 159/364 [03:28<04:33,  1.33s/it]
Epoch 3/10:  44%|████▍     | 160/364 [03:28<03:36,  1.06s/it]
Epoch 3/10:  44%|████▍     | 161/364 [03:29<03:25,  1.01s/it]
Epoch 3/10:  45%|████▍     | 162/364 [03:32<05:37,  1.67s/it]
Epoch 3/10:  45%|████▍     | 163/364 [03:33<04:20,  1.30s/it]
Epoch 3/10:  45%|████▌     | 164/364 [03:33<03:28,  1.04s/it]
Epoch 3/10:  45%|████▌     | 165/364 [03:34<03:31,  1.06s/it]
Epoch 3/10:  46%|████▌     | 166/364 [03:37<05:27,  1.65s/it]
Epoch 3/10:  46%|████▌     | 167/364 [03:38<04:14,  1.29s/it]
Epoch 3/10:  46%|████▌     | 168/364 [03:38<03:22,  1.03s/it]
Epoch 3/10:  46%|████▋     | 169/364 [03:39<03:21,  1.03s/it]
Epoch 3/10:  47%|████▋     | 170/364 [03:42<05:13,  1.61s/it]
Epoch 3/10:  47%|████▋     | 171/364 [03:43<04:02,  1.26s/it]
Epoch 3/10:  47%|████▋     | 172/364 [03:43<03:13,  1.01s/it]
Epoch 3/10:  48%|████▊     | 173/364 [03:44<03:25,  1.08s/it]
Epoch 3/10:  48%|████▊     | 174/364 [03:47<04:57,  1.56s/it]
Epoch 3/10:  48%|████▊     | 175/364 [03:47<03:50,  1.22s/it]
Epoch 3/10:  48%|████▊     | 176/364 [03:48<03:06,  1.01it/s]
Epoch 3/10:  49%|████▊     | 177/364 [03:49<03:31,  1.13s/it]
Epoch 3/10:  49%|████▉     | 178/364 [03:52<04:58,  1.60s/it]
Epoch 3/10:  49%|████▉     | 179/364 [03:52<03:51,  1.25s/it]
Epoch 3/10:  49%|████▉     | 180/364 [03:53<03:05,  1.01s/it]
Epoch 3/10:  50%|████▉     | 181/364 [03:55<03:43,  1.22s/it]
Epoch 3/10:  50%|█████     | 182/364 [03:57<04:37,  1.52s/it]
Epoch 3/10:  50%|█████     | 183/364 [03:57<03:36,  1.20s/it]
Epoch 3/10:  51%|█████     | 184/364 [03:58<02:54,  1.03it/s]
Epoch 3/10:  51%|█████     | 185/364 [04:00<03:53,  1.31s/it]
Epoch 3/10:  51%|█████     | 186/364 [04:02<04:25,  1.49s/it]
Epoch 3/10:  51%|█████▏    | 187/364 [04:02<03:27,  1.17s/it]
Epoch 3/10:  52%|█████▏    | 188/364 [04:03<02:47,  1.05it/s]
Epoch 3/10:  52%|█████▏    | 189/364 [04:05<03:58,  1.36s/it]
Epoch 3/10:  52%|█████▏    | 190/364 [04:07<04:15,  1.47s/it]
Epoch 3/10:  52%|█████▏    | 191/364 [04:07<03:20,  1.16s/it]
Epoch 3/10:  53%|█████▎    | 192/364 [04:07<02:42,  1.06it/s]
Epoch 3/10:  53%|█████▎    | 193/364 [04:10<04:00,  1.40s/it]
Epoch 3/10:  53%|█████▎    | 194/364 [04:11<03:50,  1.36s/it]
Epoch 3/10:  54%|█████▎    | 195/364 [04:12<03:01,  1.08s/it]
Epoch 3/10:  54%|█████▍    | 196/364 [04:12<02:27,  1.14it/s]
Epoch 3/10:  54%|█████▍    | 197/364 [04:15<04:18,  1.55s/it]
Epoch 3/10:  54%|█████▍    | 198/364 [04:16<03:55,  1.42s/it]
Epoch 3/10:  55%|█████▍    | 199/364 [04:17<03:05,  1.13s/it]
Epoch 3/10:  55%|█████▍    | 200/364 [04:17<02:31,  1.08it/s]
Epoch 3/10:  55%|█████▌    | 201/364 [04:20<04:03,  1.50s/it]
Epoch 3/10:  55%|█████▌    | 202/364 [04:21<03:44,  1.38s/it]
Epoch 3/10:  56%|█████▌    | 203/364 [04:22<02:57,  1.10s/it]
Epoch 3/10:  56%|█████▌    | 204/364 [04:22<02:24,  1.11it/s]
Epoch 3/10:  56%|█████▋    | 205/364 [04:25<04:02,  1.53s/it]
Epoch 3/10:  57%|█████▋    | 206/364 [04:26<03:50,  1.46s/it]
Epoch 3/10:  57%|█████▋    | 207/364 [04:27<03:00,  1.15s/it]
Epoch 3/10:  57%|█████▋    | 208/364 [04:27<02:26,  1.06it/s]
Epoch 3/10:  57%|█████▋    | 209/364 [04:30<04:03,  1.57s/it]
Epoch 3/10:  58%|█████▊    | 210/364 [04:31<03:40,  1.43s/it]
Epoch 3/10:  58%|█████▊    | 211/364 [04:32<02:52,  1.13s/it]
Epoch 3/10:  58%|█████▊    | 212/364 [04:32<02:19,  1.09it/s]
Epoch 3/10:  59%|█████▊    | 213/364 [04:35<03:56,  1.57s/it]
Epoch 3/10:  59%|█████▉    | 214/364 [04:37<03:43,  1.49s/it]
Epoch 3/10:  59%|█████▉    | 215/364 [04:37<02:54,  1.17s/it]
Epoch 3/10:  59%|█████▉    | 216/364 [04:37<02:20,  1.05it/s]
Epoch 3/10:  60%|█████▉    | 217/364 [04:41<03:55,  1.60s/it]
Epoch 3/10:  60%|█████▉    | 218/364 [04:42<03:35,  1.47s/it]
Epoch 3/10:  60%|██████    | 219/364 [04:42<02:48,  1.16s/it]
Epoch 3/10:  60%|██████    | 220/364 [04:43<02:16,  1.05it/s]
Epoch 3/10:  61%|██████    | 221/364 [04:45<03:33,  1.49s/it]
Epoch 3/10:  61%|██████    | 222/364 [04:47<03:27,  1.46s/it]
Epoch 3/10:  61%|██████▏   | 223/364 [04:47<02:42,  1.15s/it]
Epoch 3/10:  62%|██████▏   | 224/364 [04:48<02:11,  1.07it/s]
Epoch 3/10:  62%|██████▏   | 225/364 [04:51<03:49,  1.65s/it]
Epoch 3/10:  62%|██████▏   | 226/364 [04:52<03:29,  1.52s/it]
Epoch 3/10:  62%|██████▏   | 227/364 [04:53<02:43,  1.19s/it]
Epoch 3/10:  63%|██████▎   | 228/364 [04:53<02:11,  1.03it/s]
Epoch 3/10:  63%|██████▎   | 229/364 [04:56<03:39,  1.62s/it]
Epoch 3/10:  63%|██████▎   | 230/364 [04:57<03:16,  1.47s/it]
Epoch 3/10:  63%|██████▎   | 231/364 [04:58<02:33,  1.15s/it]
Epoch 3/10:  64%|██████▎   | 232/364 [04:58<02:03,  1.07it/s]
Epoch 3/10:  64%|██████▍   | 233/364 [05:01<03:26,  1.58s/it]
Epoch 3/10:  64%|██████▍   | 234/364 [05:02<03:13,  1.49s/it]
Epoch 3/10:  65%|██████▍   | 235/364 [05:03<02:31,  1.17s/it]
Epoch 3/10:  65%|██████▍   | 236/364 [05:03<02:01,  1.05it/s]
Epoch 3/10:  65%|██████▌   | 237/364 [05:07<03:26,  1.62s/it]
Epoch 3/10:  65%|██████▌   | 238/364 [05:08<03:02,  1.45s/it]
Epoch 3/10:  66%|██████▌   | 239/364 [05:08<02:22,  1.14s/it]
Epoch 3/10:  66%|██████▌   | 240/364 [05:08<01:54,  1.08it/s]
Epoch 3/10:  66%|██████▌   | 241/364 [05:11<03:07,  1.53s/it]
Epoch 3/10:  66%|██████▋   | 242/364 [05:13<03:04,  1.51s/it]
Epoch 3/10:  67%|██████▋   | 243/364 [05:13<02:23,  1.19s/it]
Epoch 3/10:  67%|██████▋   | 244/364 [05:14<01:55,  1.04it/s]
Epoch 3/10:  67%|██████▋   | 245/364 [05:16<02:44,  1.38s/it]
Epoch 3/10:  68%|██████▊   | 246/364 [05:18<03:13,  1.64s/it]
Epoch 3/10:  68%|██████▊   | 247/364 [05:19<02:29,  1.28s/it]
Epoch 3/10:  68%|██████▊   | 248/364 [05:19<01:59,  1.03s/it]
Epoch 3/10:  68%|██████▊   | 249/364 [05:21<02:25,  1.26s/it]
Epoch 3/10:  69%|██████▊   | 250/364 [05:23<02:50,  1.49s/it]
Epoch 3/10:  69%|██████▉   | 251/364 [05:23<02:12,  1.18s/it]
Epoch 3/10:  69%|██████▉   | 252/364 [05:24<01:46,  1.05it/s]
Epoch 3/10:  70%|██████▉   | 253/364 [05:26<02:29,  1.34s/it]
Epoch 3/10:  70%|██████▉   | 254/364 [05:28<02:59,  1.63s/it]
Epoch 3/10:  70%|███████   | 255/364 [05:29<02:18,  1.27s/it]
Epoch 3/10:  70%|███████   | 256/364 [05:29<01:49,  1.02s/it]
Epoch 3/10:  71%|███████   | 257/364 [05:31<02:18,  1.30s/it]
Epoch 3/10:  71%|███████   | 258/364 [05:33<02:34,  1.46s/it]
Epoch 3/10:  71%|███████   | 259/364 [05:33<02:00,  1.15s/it]
Epoch 3/10:  71%|███████▏  | 260/364 [05:34<01:36,  1.08it/s]
Epoch 3/10:  72%|███████▏  | 261/364 [05:36<02:18,  1.34s/it]
Epoch 3/10:  72%|███████▏  | 262/364 [05:38<02:30,  1.47s/it]
Epoch 3/10:  72%|███████▏  | 263/364 [05:38<01:57,  1.16s/it]
Epoch 3/10:  73%|███████▎  | 264/364 [05:39<01:33,  1.06it/s]
Epoch 3/10:  73%|███████▎  | 265/364 [05:41<02:21,  1.43s/it]
Epoch 3/10:  73%|███████▎  | 266/364 [05:43<02:37,  1.60s/it]
Epoch 3/10:  73%|███████▎  | 267/364 [05:44<02:01,  1.25s/it]
Epoch 3/10:  74%|███████▎  | 268/364 [05:44<01:35,  1.00it/s]
Epoch 3/10:  74%|███████▍  | 269/364 [05:46<02:07,  1.34s/it]
Epoch 3/10:  74%|███████▍  | 270/364 [05:49<02:36,  1.67s/it]
Epoch 3/10:  74%|███████▍  | 271/364 [05:49<02:01,  1.30s/it]
Epoch 3/10:  75%|███████▍  | 272/364 [05:50<01:35,  1.04s/it]
Epoch 3/10:  75%|███████▌  | 273/364 [05:52<01:55,  1.27s/it]
Epoch 3/10:  75%|███████▌  | 274/364 [05:54<02:25,  1.62s/it]
Epoch 3/10:  76%|███████▌  | 275/364 [05:54<01:51,  1.26s/it]
Epoch 3/10:  76%|███████▌  | 276/364 [05:55<01:28,  1.01s/it]
Epoch 3/10:  76%|███████▌  | 277/364 [05:57<01:48,  1.25s/it]
Epoch 3/10:  76%|███████▋  | 278/364 [05:59<02:19,  1.62s/it]
Epoch 3/10:  77%|███████▋  | 279/364 [06:00<01:47,  1.26s/it]
Epoch 3/10:  77%|███████▋  | 280/364 [06:00<01:24,  1.01s/it]
Epoch 3/10:  77%|███████▋  | 281/364 [06:02<01:48,  1.31s/it]
Epoch 3/10:  77%|███████▋  | 282/364 [06:04<02:11,  1.60s/it]
Epoch 3/10:  78%|███████▊  | 283/364 [06:05<01:40,  1.24s/it]
Epoch 3/10:  78%|███████▊  | 284/364 [06:05<01:19,  1.00it/s]
Epoch 3/10:  78%|███████▊  | 285/364 [06:07<01:49,  1.39s/it]
Epoch 3/10:  79%|███████▊  | 286/364 [06:09<01:59,  1.53s/it]
Epoch 3/10:  79%|███████▉  | 287/364 [06:10<01:32,  1.20s/it]
Epoch 3/10:  79%|███████▉  | 288/364 [06:10<01:13,  1.04it/s]
Epoch 3/10:  79%|███████▉  | 289/364 [06:12<01:43,  1.38s/it]
Epoch 3/10:  80%|███████▉  | 290/364 [06:14<01:54,  1.55s/it]
Epoch 3/10:  80%|███████▉  | 291/364 [06:15<01:28,  1.21s/it]
Epoch 3/10:  80%|████████  | 292/364 [06:15<01:10,  1.03it/s]
Epoch 3/10:  80%|████████  | 293/364 [06:17<01:34,  1.33s/it]
Epoch 3/10:  81%|████████  | 294/364 [06:19<01:49,  1.56s/it]
Epoch 3/10:  81%|████████  | 295/364 [06:20<01:24,  1.22s/it]
Epoch 3/10:  81%|████████▏ | 296/364 [06:20<01:07,  1.01it/s]
Epoch 3/10:  82%|████████▏ | 297/364 [06:23<01:33,  1.40s/it]
Epoch 3/10:  82%|████████▏ | 298/364 [06:24<01:38,  1.49s/it]
Epoch 3/10:  82%|████████▏ | 299/364 [06:25<01:15,  1.17s/it]
Epoch 3/10:  82%|████████▏ | 300/364 [06:25<01:00,  1.06it/s]
Epoch 3/10:  83%|████████▎ | 301/364 [06:28<01:39,  1.57s/it]
Epoch 3/10:  83%|████████▎ | 302/364 [06:29<01:26,  1.40s/it]
Epoch 3/10:  83%|████████▎ | 303/364 [06:30<01:07,  1.11s/it]
Epoch 3/10:  84%|████████▎ | 304/364 [06:30<00:54,  1.11it/s]
Epoch 3/10:  84%|████████▍ | 305/364 [06:33<01:33,  1.58s/it]
Epoch 3/10:  84%|████████▍ | 306/364 [06:34<01:21,  1.41s/it]
Epoch 3/10:  84%|████████▍ | 307/364 [06:35<01:03,  1.12s/it]
Epoch 3/10:  85%|████████▍ | 308/364 [06:35<00:50,  1.11it/s]
Epoch 3/10:  85%|████████▍ | 309/364 [06:38<01:27,  1.58s/it]
Epoch 3/10:  85%|████████▌ | 310/364 [06:39<01:13,  1.36s/it]
Epoch 3/10:  85%|████████▌ | 311/364 [06:40<00:57,  1.08s/it]
Epoch 3/10:  86%|████████▌ | 312/364 [06:40<00:46,  1.13it/s]
Epoch 3/10:  86%|████████▌ | 313/364 [06:43<01:22,  1.61s/it]
Epoch 3/10:  86%|████████▋ | 314/364 [06:44<01:11,  1.43s/it]
Epoch 3/10:  87%|████████▋ | 315/364 [06:45<00:55,  1.13s/it]
Epoch 3/10:  87%|████████▋ | 316/364 [06:45<00:44,  1.09it/s]
Epoch 3/10:  87%|████████▋ | 317/364 [06:48<01:15,  1.60s/it]
Epoch 3/10:  87%|████████▋ | 318/364 [06:49<01:01,  1.34s/it]
Epoch 3/10:  88%|████████▊ | 319/364 [06:50<00:47,  1.06s/it]
Epoch 3/10:  88%|████████▊ | 320/364 [06:50<00:38,  1.14it/s]
Epoch 3/10:  88%|████████▊ | 321/364 [06:54<01:12,  1.69s/it]
Epoch 3/10:  88%|████████▊ | 322/364 [06:54<00:56,  1.34s/it]
Epoch 3/10:  89%|████████▊ | 323/364 [06:55<00:43,  1.07s/it]
Epoch 3/10:  89%|████████▉ | 324/364 [06:55<00:34,  1.15it/s]
Epoch 3/10:  89%|████████▉ | 325/364 [06:58<01:05,  1.68s/it]
Epoch 3/10:  90%|████████▉ | 326/364 [06:59<00:56,  1.48s/it]
Epoch 3/10:  90%|████████▉ | 327/364 [07:00<00:43,  1.17s/it]
Epoch 3/10:  90%|█████████ | 328/364 [07:00<00:34,  1.06it/s]
Epoch 3/10:  90%|█████████ | 329/364 [07:03<00:54,  1.55s/it]
Epoch 3/10:  91%|█████████ | 330/364 [07:05<00:51,  1.52s/it]
Epoch 3/10:  91%|█████████ | 331/364 [07:05<00:39,  1.19s/it]
Epoch 3/10:  91%|█████████ | 332/364 [07:06<00:30,  1.04it/s]
Epoch 3/10:  91%|█████████▏| 333/364 [07:08<00:46,  1.51s/it]
Epoch 3/10:  92%|█████████▏| 334/364 [07:10<00:46,  1.55s/it]
Epoch 3/10:  92%|█████████▏| 335/364 [07:10<00:35,  1.21s/it]
Epoch 3/10:  92%|█████████▏| 336/364 [07:11<00:27,  1.03it/s]
Epoch 3/10:  93%|█████████▎| 337/364 [07:13<00:39,  1.46s/it]
Epoch 3/10:  93%|█████████▎| 338/364 [07:15<00:39,  1.54s/it]
Epoch 3/10:  93%|█████████▎| 339/364 [07:16<00:29,  1.20s/it]
Epoch 3/10:  93%|█████████▎| 340/364 [07:16<00:23,  1.03it/s]
Epoch 3/10:  94%|█████████▎| 341/364 [07:19<00:34,  1.51s/it]
Epoch 3/10:  94%|█████████▍| 342/364 [07:20<00:31,  1.43s/it]
Epoch 3/10:  94%|█████████▍| 343/364 [07:20<00:23,  1.13s/it]
Epoch 3/10:  95%|█████████▍| 344/364 [07:21<00:18,  1.09it/s]
Epoch 3/10:  95%|█████████▍| 345/364 [07:24<00:28,  1.51s/it]
Epoch 3/10:  95%|█████████▌| 346/364 [07:25<00:27,  1.54s/it]
Epoch 3/10:  95%|█████████▌| 347/364 [07:26<00:20,  1.21s/it]
Epoch 3/10:  96%|█████████▌| 348/364 [07:26<00:15,  1.02it/s]
Epoch 3/10:  96%|█████████▌| 349/364 [07:29<00:21,  1.41s/it]
Epoch 3/10:  96%|█████████▌| 350/364 [07:31<00:22,  1.60s/it]
Epoch 3/10:  96%|█████████▋| 351/364 [07:31<00:16,  1.25s/it]
Epoch 3/10:  97%|█████████▋| 352/364 [07:32<00:12,  1.01s/it]
Epoch 3/10:  97%|█████████▋| 353/364 [07:34<00:13,  1.27s/it]
Epoch 3/10:  97%|█████████▋| 354/364 [07:36<00:16,  1.67s/it]
Epoch 3/10:  98%|█████████▊| 355/364 [07:37<00:11,  1.29s/it]
Epoch 3/10:  98%|█████████▊| 356/364 [07:37<00:08,  1.03s/it]
Epoch 3/10:  98%|█████████▊| 357/364 [07:38<00:08,  1.17s/it]
Epoch 3/10:  98%|█████████▊| 358/364 [07:41<00:10,  1.68s/it]
Epoch 3/10:  99%|█████████▊| 359/364 [07:42<00:06,  1.30s/it]
Epoch 3/10:  99%|█████████▉| 360/364 [07:42<00:04,  1.03s/it]
Epoch 3/10:  99%|█████████▉| 361/364 [07:43<00:02,  1.02it/s]
Epoch 3/10:  99%|█████████▉| 362/364 [07:45<00:02,  1.17s/it]
Epoch 3/10: 100%|█████████▉| 363/364 [07:45<00:00,  1.06it/s]
Epoch 3/10: 100%|██████████| 364/364 [07:45<00:00,  1.40it/s]
Epoch 3/10: 100%|██████████| 364/364 [07:45<00:00,  1.28s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:35,  4.92s/it]
Validating:   4%|▎         | 2/57 [00:05<01:56,  2.12s/it]
Validating:   5%|▌         | 3/57 [00:05<01:06,  1.22s/it]
Validating:   7%|▋         | 4/57 [00:05<00:42,  1.25it/s]
Validating:   9%|▉         | 5/57 [00:09<01:38,  1.90s/it]
Validating:  11%|█         | 6/57 [00:09<01:09,  1.36s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:48,  1.03it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:34,  1.42it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:16,  1.59s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:57,  1.23s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:41,  1.12it/s]
Validating:  21%|██        | 12/57 [00:14<00:30,  1.49it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:14,  1.69s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:53,  1.23s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:38,  1.10it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:27,  1.47it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:05,  1.65s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:47,  1.21s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:33,  1.12it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:24,  1.50it/s]
Validating:  37%|███▋      | 21/57 [00:26<00:59,  1.64s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:45,  1.30s/it]
Validating:  40%|████      | 23/57 [00:27<00:32,  1.06it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:23,  1.41it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:52,  1.64s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:40,  1.29s/it]
Validating:  47%|████▋     | 27/57 [00:32<00:28,  1.05it/s]
Validating:  49%|████▉     | 28/57 [00:32<00:20,  1.42it/s]
Validating:  51%|█████     | 29/57 [00:36<00:46,  1.65s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:34,  1.29s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:24,  1.06it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:17,  1.41it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:39,  1.66s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:28,  1.23s/it]
Validating:  61%|██████▏   | 35/57 [00:41<00:19,  1.10it/s]
Validating:  63%|██████▎   | 36/57 [00:41<00:14,  1.47it/s]
Validating:  65%|██████▍   | 37/57 [00:45<00:36,  1.82s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:24,  1.31s/it]
Validating:  68%|██████▊   | 39/57 [00:46<00:17,  1.04it/s]
Validating:  70%|███████   | 40/57 [00:46<00:12,  1.39it/s]
Validating:  72%|███████▏  | 41/57 [00:50<00:28,  1.79s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:20,  1.38s/it]
Validating:  75%|███████▌  | 43/57 [00:51<00:14,  1.01s/it]
Validating:  77%|███████▋  | 44/57 [00:51<00:09,  1.33it/s]
Validating:  79%|███████▉  | 45/57 [00:55<00:20,  1.68s/it]
Validating:  81%|████████  | 46/57 [00:55<00:15,  1.39s/it]
Validating:  82%|████████▏ | 47/57 [00:55<00:10,  1.01s/it]
Validating:  84%|████████▍ | 48/57 [00:56<00:06,  1.32it/s]
Validating:  86%|████████▌ | 49/57 [00:59<00:11,  1.41s/it]
Validating:  88%|████████▊ | 50/57 [01:00<00:09,  1.33s/it]
Validating:  89%|████████▉ | 51/57 [01:00<00:05,  1.03it/s]
Validating:  91%|█████████ | 52/57 [01:00<00:03,  1.38it/s]
Validating:  93%|█████████▎| 53/57 [01:02<00:04,  1.07s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:02,  1.09it/s]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.47it/s]
Validating:  98%|█████████▊| 56/57 [01:03<00:00,  1.96it/s]
Validating: 100%|██████████| 57/57 [01:03<00:00,  1.11s/it]
Epoch 3: Train Loss: 0.2434, Train Acc: 0.9127 | Val Loss: 0.3131, Val Acc: 0.8922
Saved Best Model!

Epoch 4/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 4/10:   0%|          | 1/364 [00:05<33:07,  5.48s/it]
Epoch 4/10:   1%|          | 2/364 [00:05<15:07,  2.51s/it]
Epoch 4/10:   1%|          | 3/364 [00:06<09:20,  1.55s/it]
Epoch 4/10:   1%|          | 4/364 [00:06<06:40,  1.11s/it]
Epoch 4/10:   1%|▏         | 5/364 [00:10<12:47,  2.14s/it]
Epoch 4/10:   2%|▏         | 6/364 [00:11<09:17,  1.56s/it]
Epoch 4/10:   2%|▏         | 7/364 [00:11<07:04,  1.19s/it]
Epoch 4/10:   2%|▏         | 8/364 [00:11<05:34,  1.06it/s]
Epoch 4/10:   2%|▏         | 9/364 [00:16<11:31,  1.95s/it]
Epoch 4/10:   3%|▎         | 10/364 [00:16<08:40,  1.47s/it]
Epoch 4/10:   3%|▎         | 11/364 [00:16<06:44,  1.14s/it]
Epoch 4/10:   3%|▎         | 12/364 [00:17<05:25,  1.08it/s]
Epoch 4/10:   4%|▎         | 13/364 [00:21<11:11,  1.91s/it]
Epoch 4/10:   4%|▍         | 14/364 [00:21<08:30,  1.46s/it]
Epoch 4/10:   4%|▍         | 15/364 [00:22<06:40,  1.15s/it]
Epoch 4/10:   4%|▍         | 16/364 [00:22<05:22,  1.08it/s]
Epoch 4/10:   5%|▍         | 17/364 [00:26<10:27,  1.81s/it]
Epoch 4/10:   5%|▍         | 18/364 [00:27<08:01,  1.39s/it]
Epoch 4/10:   5%|▌         | 19/364 [00:27<06:17,  1.09s/it]
Epoch 4/10:   5%|▌         | 20/364 [00:27<05:05,  1.12it/s]
Epoch 4/10:   6%|▌         | 21/364 [00:31<10:17,  1.80s/it]
Epoch 4/10:   6%|▌         | 22/364 [00:32<07:54,  1.39s/it]
Epoch 4/10:   6%|▋         | 23/364 [00:32<06:14,  1.10s/it]
Epoch 4/10:   7%|▋         | 24/364 [00:33<05:05,  1.11it/s]
Epoch 4/10:   7%|▋         | 25/364 [00:36<09:32,  1.69s/it]
Epoch 4/10:   7%|▋         | 26/364 [00:37<07:20,  1.30s/it]
Epoch 4/10:   7%|▋         | 27/364 [00:37<05:50,  1.04s/it]
Epoch 4/10:   8%|▊         | 28/364 [00:37<04:49,  1.16it/s]
Epoch 4/10:   8%|▊         | 29/364 [00:41<09:28,  1.70s/it]
Epoch 4/10:   8%|▊         | 30/364 [00:42<07:21,  1.32s/it]
Epoch 4/10:   9%|▊         | 31/364 [00:42<05:51,  1.06s/it]
Epoch 4/10:   9%|▉         | 32/364 [00:42<04:46,  1.16it/s]
Epoch 4/10:   9%|▉         | 33/364 [00:46<09:40,  1.75s/it]
Epoch 4/10:   9%|▉         | 34/364 [00:47<07:27,  1.36s/it]
Epoch 4/10:  10%|▉         | 35/364 [00:47<05:54,  1.08s/it]
Epoch 4/10:  10%|▉         | 36/364 [00:47<04:50,  1.13it/s]
Epoch 4/10:  10%|█         | 37/364 [00:51<09:52,  1.81s/it]
Epoch 4/10:  10%|█         | 38/364 [00:52<07:33,  1.39s/it]
Epoch 4/10:  11%|█         | 39/364 [00:52<05:56,  1.10s/it]
Epoch 4/10:  11%|█         | 40/364 [00:53<04:51,  1.11it/s]
Epoch 4/10:  11%|█▏        | 41/364 [00:57<09:57,  1.85s/it]
Epoch 4/10:  12%|█▏        | 42/364 [00:57<07:39,  1.43s/it]
Epoch 4/10:  12%|█▏        | 43/364 [00:58<06:00,  1.12s/it]
Epoch 4/10:  12%|█▏        | 44/364 [00:58<04:51,  1.10it/s]
Epoch 4/10:  12%|█▏        | 45/364 [01:02<09:09,  1.72s/it]
Epoch 4/10:  13%|█▎        | 46/364 [01:02<07:04,  1.33s/it]
Epoch 4/10:  13%|█▎        | 47/364 [01:02<05:34,  1.06s/it]
Epoch 4/10:  13%|█▎        | 48/364 [01:03<04:35,  1.15it/s]
Epoch 4/10:  13%|█▎        | 49/364 [01:07<09:11,  1.75s/it]
Epoch 4/10:  14%|█▎        | 50/364 [01:07<07:05,  1.36s/it]
Epoch 4/10:  14%|█▍        | 51/364 [01:08<05:36,  1.07s/it]
Epoch 4/10:  14%|█▍        | 52/364 [01:08<04:34,  1.14it/s]
Epoch 4/10:  15%|█▍        | 53/364 [01:12<09:12,  1.78s/it]
Epoch 4/10:  15%|█▍        | 54/364 [01:12<07:02,  1.36s/it]
Epoch 4/10:  15%|█▌        | 55/364 [01:13<05:35,  1.08s/it]
Epoch 4/10:  15%|█▌        | 56/364 [01:13<04:34,  1.12it/s]
Epoch 4/10:  16%|█▌        | 57/364 [01:17<08:50,  1.73s/it]
Epoch 4/10:  16%|█▌        | 58/364 [01:17<06:48,  1.33s/it]
Epoch 4/10:  16%|█▌        | 59/364 [01:18<05:23,  1.06s/it]
Epoch 4/10:  16%|█▋        | 60/364 [01:18<04:26,  1.14it/s]
Epoch 4/10:  17%|█▋        | 61/364 [01:22<08:20,  1.65s/it]
Epoch 4/10:  17%|█▋        | 62/364 [01:22<06:32,  1.30s/it]
Epoch 4/10:  17%|█▋        | 63/364 [01:23<05:12,  1.04s/it]
Epoch 4/10:  18%|█▊        | 64/364 [01:23<04:14,  1.18it/s]
Epoch 4/10:  18%|█▊        | 65/364 [01:27<08:56,  1.79s/it]
Epoch 4/10:  18%|█▊        | 66/364 [01:27<06:52,  1.38s/it]
Epoch 4/10:  18%|█▊        | 67/364 [01:28<05:27,  1.10s/it]
Epoch 4/10:  19%|█▊        | 68/364 [01:28<04:26,  1.11it/s]
Epoch 4/10:  19%|█▉        | 69/364 [01:32<08:46,  1.78s/it]
Epoch 4/10:  19%|█▉        | 70/364 [01:32<06:46,  1.38s/it]
Epoch 4/10:  20%|█▉        | 71/364 [01:33<05:21,  1.10s/it]
Epoch 4/10:  20%|█▉        | 72/364 [01:33<04:24,  1.11it/s]
Epoch 4/10:  20%|██        | 73/364 [01:37<08:51,  1.83s/it]
Epoch 4/10:  20%|██        | 74/364 [01:38<06:47,  1.40s/it]
Epoch 4/10:  21%|██        | 75/364 [01:38<05:21,  1.11s/it]
Epoch 4/10:  21%|██        | 76/364 [01:39<04:18,  1.11it/s]
Epoch 4/10:  21%|██        | 77/364 [01:43<08:39,  1.81s/it]
Epoch 4/10:  21%|██▏       | 78/364 [01:43<06:38,  1.39s/it]
Epoch 4/10:  22%|██▏       | 79/364 [01:43<05:12,  1.10s/it]
Epoch 4/10:  22%|██▏       | 80/364 [01:44<04:12,  1.12it/s]
Epoch 4/10:  22%|██▏       | 81/364 [01:47<08:08,  1.73s/it]
Epoch 4/10:  23%|██▎       | 82/364 [01:48<06:16,  1.34s/it]
Epoch 4/10:  23%|██▎       | 83/364 [01:48<04:59,  1.06s/it]
Epoch 4/10:  23%|██▎       | 84/364 [01:49<04:03,  1.15it/s]
Epoch 4/10:  23%|██▎       | 85/364 [01:53<08:30,  1.83s/it]
Epoch 4/10:  24%|██▎       | 86/364 [01:53<06:32,  1.41s/it]
Epoch 4/10:  24%|██▍       | 87/364 [01:54<05:08,  1.11s/it]
Epoch 4/10:  24%|██▍       | 88/364 [01:54<04:11,  1.10it/s]
Epoch 4/10:  24%|██▍       | 89/364 [01:58<08:16,  1.80s/it]
Epoch 4/10:  25%|██▍       | 90/364 [01:58<06:21,  1.39s/it]
Epoch 4/10:  25%|██▌       | 91/364 [01:59<05:00,  1.10s/it]
Epoch 4/10:  25%|██▌       | 92/364 [01:59<04:05,  1.11it/s]
Epoch 4/10:  26%|██▌       | 93/364 [02:03<07:57,  1.76s/it]
Epoch 4/10:  26%|██▌       | 94/364 [02:03<06:09,  1.37s/it]
Epoch 4/10:  26%|██▌       | 95/364 [02:04<04:53,  1.09s/it]
Epoch 4/10:  26%|██▋       | 96/364 [02:04<03:57,  1.13it/s]
Epoch 4/10:  27%|██▋       | 97/364 [02:08<07:20,  1.65s/it]
Epoch 4/10:  27%|██▋       | 98/364 [02:08<05:40,  1.28s/it]
Epoch 4/10:  27%|██▋       | 99/364 [02:09<04:32,  1.03s/it]
Epoch 4/10:  27%|██▋       | 100/364 [02:09<03:47,  1.16it/s]
Epoch 4/10:  28%|██▊       | 101/364 [02:13<07:41,  1.76s/it]
Epoch 4/10:  28%|██▊       | 102/364 [02:13<05:56,  1.36s/it]
Epoch 4/10:  28%|██▊       | 103/364 [02:14<04:42,  1.08s/it]
Epoch 4/10:  29%|██▊       | 104/364 [02:14<03:51,  1.12it/s]
Epoch 4/10:  29%|██▉       | 105/364 [02:18<08:01,  1.86s/it]
Epoch 4/10:  29%|██▉       | 106/364 [02:19<06:08,  1.43s/it]
Epoch 4/10:  29%|██▉       | 107/364 [02:19<04:50,  1.13s/it]
Epoch 4/10:  30%|██▉       | 108/364 [02:20<03:54,  1.09it/s]
Epoch 4/10:  30%|██▉       | 109/364 [02:24<07:41,  1.81s/it]
Epoch 4/10:  30%|███       | 110/364 [02:24<05:52,  1.39s/it]
Epoch 4/10:  30%|███       | 111/364 [02:24<04:38,  1.10s/it]
Epoch 4/10:  31%|███       | 112/364 [02:25<03:46,  1.11it/s]
Epoch 4/10:  31%|███       | 113/364 [02:29<07:49,  1.87s/it]
Epoch 4/10:  31%|███▏      | 114/364 [02:29<05:59,  1.44s/it]
Epoch 4/10:  32%|███▏      | 115/364 [02:30<04:43,  1.14s/it]
Epoch 4/10:  32%|███▏      | 116/364 [02:30<03:48,  1.09it/s]
Epoch 4/10:  32%|███▏      | 117/364 [02:34<07:00,  1.70s/it]
Epoch 4/10:  32%|███▏      | 118/364 [02:34<05:26,  1.33s/it]
Epoch 4/10:  33%|███▎      | 119/364 [02:35<04:19,  1.06s/it]
Epoch 4/10:  33%|███▎      | 120/364 [02:35<03:32,  1.15it/s]
Epoch 4/10:  33%|███▎      | 121/364 [02:39<06:50,  1.69s/it]
Epoch 4/10:  34%|███▎      | 122/364 [02:39<05:17,  1.31s/it]
Epoch 4/10:  34%|███▍      | 123/364 [02:40<04:12,  1.05s/it]
Epoch 4/10:  34%|███▍      | 124/364 [02:40<03:24,  1.17it/s]
Epoch 4/10:  34%|███▍      | 125/364 [02:44<07:08,  1.79s/it]
Epoch 4/10:  35%|███▍      | 126/364 [02:44<05:28,  1.38s/it]
Epoch 4/10:  35%|███▍      | 127/364 [02:45<04:19,  1.09s/it]
Epoch 4/10:  35%|███▌      | 128/364 [02:45<03:29,  1.12it/s]
Epoch 4/10:  35%|███▌      | 129/364 [02:49<07:07,  1.82s/it]
Epoch 4/10:  36%|███▌      | 130/364 [02:50<05:29,  1.41s/it]
Epoch 4/10:  36%|███▌      | 131/364 [02:50<04:19,  1.11s/it]
Epoch 4/10:  36%|███▋      | 132/364 [02:50<03:30,  1.10it/s]
Epoch 4/10:  37%|███▋      | 133/364 [02:54<06:55,  1.80s/it]
Epoch 4/10:  37%|███▋      | 134/364 [02:55<05:20,  1.40s/it]
Epoch 4/10:  37%|███▋      | 135/364 [02:55<04:13,  1.11s/it]
Epoch 4/10:  37%|███▋      | 136/364 [02:56<03:26,  1.10it/s]
Epoch 4/10:  38%|███▊      | 137/364 [02:59<06:22,  1.69s/it]
Epoch 4/10:  38%|███▊      | 138/364 [03:00<04:55,  1.31s/it]
Epoch 4/10:  38%|███▊      | 139/364 [03:00<04:12,  1.12s/it]
Epoch 4/10:  38%|███▊      | 140/364 [03:01<03:23,  1.10it/s]
Epoch 4/10:  39%|███▊      | 141/364 [03:04<06:14,  1.68s/it]
Epoch 4/10:  39%|███▉      | 142/364 [03:05<04:50,  1.31s/it]
Epoch 4/10:  39%|███▉      | 143/364 [03:06<04:31,  1.23s/it]
Epoch 4/10:  40%|███▉      | 144/364 [03:06<03:37,  1.01it/s]
Epoch 4/10:  40%|███▉      | 145/364 [03:09<05:48,  1.59s/it]
Epoch 4/10:  40%|████      | 146/364 [03:10<04:30,  1.24s/it]
Epoch 4/10:  40%|████      | 147/364 [03:11<04:54,  1.36s/it]
Epoch 4/10:  41%|████      | 148/364 [03:12<03:54,  1.09s/it]
Epoch 4/10:  41%|████      | 149/364 [03:14<05:01,  1.40s/it]
Epoch 4/10:  41%|████      | 150/364 [03:14<03:57,  1.11s/it]
Epoch 4/10:  41%|████▏     | 151/364 [03:16<04:48,  1.36s/it]
Epoch 4/10:  42%|████▏     | 152/364 [03:17<03:47,  1.07s/it]
Epoch 4/10:  42%|████▏     | 153/364 [03:19<05:27,  1.55s/it]
Epoch 4/10:  42%|████▏     | 154/364 [03:20<04:15,  1.22s/it]
Epoch 4/10:  43%|████▎     | 155/364 [03:21<04:54,  1.41s/it]
Epoch 4/10:  43%|████▎     | 156/364 [03:22<03:52,  1.12s/it]
Epoch 4/10:  43%|████▎     | 157/364 [03:24<05:08,  1.49s/it]
Epoch 4/10:  43%|████▎     | 158/364 [03:25<04:01,  1.17s/it]
Epoch 4/10:  44%|████▎     | 159/364 [03:26<04:13,  1.24s/it]
Epoch 4/10:  44%|████▍     | 160/364 [03:26<03:21,  1.01it/s]
Epoch 4/10:  44%|████▍     | 161/364 [03:30<05:26,  1.61s/it]
Epoch 4/10:  45%|████▍     | 162/364 [03:30<04:12,  1.25s/it]
Epoch 4/10:  45%|████▍     | 163/364 [03:31<03:47,  1.13s/it]
Epoch 4/10:  45%|████▌     | 164/364 [03:31<03:05,  1.08it/s]
Epoch 4/10:  45%|████▌     | 165/364 [03:35<05:37,  1.70s/it]
Epoch 4/10:  46%|████▌     | 166/364 [03:35<04:20,  1.32s/it]
Epoch 4/10:  46%|████▌     | 167/364 [03:36<03:56,  1.20s/it]
Epoch 4/10:  46%|████▌     | 168/364 [03:37<03:09,  1.03it/s]
Epoch 4/10:  46%|████▋     | 169/364 [03:40<05:13,  1.61s/it]
Epoch 4/10:  47%|████▋     | 170/364 [03:40<04:03,  1.25s/it]
Epoch 4/10:  47%|████▋     | 171/364 [03:41<03:49,  1.19s/it]
Epoch 4/10:  47%|████▋     | 172/364 [03:42<03:04,  1.04it/s]
Epoch 4/10:  48%|████▊     | 173/364 [03:45<05:01,  1.58s/it]
Epoch 4/10:  48%|████▊     | 174/364 [03:45<03:54,  1.23s/it]
Epoch 4/10:  48%|████▊     | 175/364 [03:46<03:46,  1.20s/it]
Epoch 4/10:  48%|████▊     | 176/364 [03:47<03:01,  1.04it/s]
Epoch 4/10:  49%|████▊     | 177/364 [03:50<05:00,  1.61s/it]
Epoch 4/10:  49%|████▉     | 178/364 [03:50<03:52,  1.25s/it]
Epoch 4/10:  49%|████▉     | 179/364 [03:51<03:35,  1.17s/it]
Epoch 4/10:  49%|████▉     | 180/364 [03:51<02:53,  1.06it/s]
Epoch 4/10:  50%|████▉     | 181/364 [03:54<04:46,  1.57s/it]
Epoch 4/10:  50%|█████     | 182/364 [03:55<03:42,  1.22s/it]
Epoch 4/10:  50%|█████     | 183/364 [03:56<03:20,  1.11s/it]
Epoch 4/10:  51%|█████     | 184/364 [03:56<02:42,  1.11it/s]
Epoch 4/10:  51%|█████     | 185/364 [04:00<05:06,  1.71s/it]
Epoch 4/10:  51%|█████     | 186/364 [04:00<03:56,  1.33s/it]
Epoch 4/10:  51%|█████▏    | 187/364 [04:01<03:36,  1.22s/it]
Epoch 4/10:  52%|█████▏    | 188/364 [04:02<02:52,  1.02it/s]
Epoch 4/10:  52%|█████▏    | 189/364 [04:05<04:51,  1.66s/it]
Epoch 4/10:  52%|█████▏    | 190/364 [04:05<03:45,  1.29s/it]
Epoch 4/10:  52%|█████▏    | 191/364 [04:07<03:45,  1.30s/it]
Epoch 4/10:  53%|█████▎    | 192/364 [04:07<02:59,  1.04s/it]
Epoch 4/10:  53%|█████▎    | 193/364 [04:10<04:35,  1.61s/it]
Epoch 4/10:  53%|█████▎    | 194/364 [04:10<03:34,  1.26s/it]
Epoch 4/10:  54%|█████▎    | 195/364 [04:12<03:26,  1.22s/it]
Epoch 4/10:  54%|█████▍    | 196/364 [04:12<02:45,  1.02it/s]
Epoch 4/10:  54%|█████▍    | 197/364 [04:15<04:29,  1.62s/it]
Epoch 4/10:  54%|█████▍    | 198/364 [04:15<03:29,  1.26s/it]
Epoch 4/10:  55%|█████▍    | 199/364 [04:16<03:11,  1.16s/it]
Epoch 4/10:  55%|█████▍    | 200/364 [04:17<02:34,  1.06it/s]
Epoch 4/10:  55%|█████▌    | 201/364 [04:20<04:23,  1.62s/it]
Epoch 4/10:  55%|█████▌    | 202/364 [04:20<03:25,  1.27s/it]
Epoch 4/10:  56%|█████▌    | 203/364 [04:21<03:07,  1.16s/it]
Epoch 4/10:  56%|█████▌    | 204/364 [04:22<02:30,  1.06it/s]
Epoch 4/10:  56%|█████▋    | 205/364 [04:25<04:17,  1.62s/it]
Epoch 4/10:  57%|█████▋    | 206/364 [04:25<03:19,  1.27s/it]
Epoch 4/10:  57%|█████▋    | 207/364 [04:27<03:19,  1.27s/it]
Epoch 4/10:  57%|█████▋    | 208/364 [04:27<02:38,  1.02s/it]
Epoch 4/10:  57%|█████▋    | 209/364 [04:30<03:47,  1.47s/it]
Epoch 4/10:  58%|█████▊    | 210/364 [04:30<02:58,  1.16s/it]
Epoch 4/10:  58%|█████▊    | 211/364 [04:32<03:38,  1.43s/it]
Epoch 4/10:  58%|█████▊    | 212/364 [04:33<02:52,  1.13s/it]
Epoch 4/10:  59%|█████▊    | 213/364 [04:34<03:18,  1.32s/it]
Epoch 4/10:  59%|█████▉    | 214/364 [04:35<02:40,  1.07s/it]
Epoch 4/10:  59%|█████▉    | 215/364 [04:37<03:47,  1.53s/it]
Epoch 4/10:  59%|█████▉    | 216/364 [04:38<02:57,  1.20s/it]
Epoch 4/10:  60%|█████▉    | 217/364 [04:39<03:09,  1.29s/it]
Epoch 4/10:  60%|█████▉    | 218/364 [04:40<02:31,  1.04s/it]
Epoch 4/10:  60%|██████    | 219/364 [04:42<03:34,  1.48s/it]
Epoch 4/10:  60%|██████    | 220/364 [04:43<02:48,  1.17s/it]
Epoch 4/10:  61%|██████    | 221/364 [04:45<03:12,  1.35s/it]
Epoch 4/10:  61%|██████    | 222/364 [04:45<02:49,  1.20s/it]
Epoch 4/10:  61%|██████▏   | 223/364 [04:47<03:04,  1.31s/it]
Epoch 4/10:  62%|██████▏   | 224/364 [04:47<02:25,  1.04s/it]
Epoch 4/10:  62%|██████▏   | 225/364 [04:49<03:05,  1.34s/it]
Epoch 4/10:  62%|██████▏   | 226/364 [04:51<03:06,  1.35s/it]
Epoch 4/10:  62%|██████▏   | 227/364 [04:52<02:53,  1.27s/it]
Epoch 4/10:  63%|██████▎   | 228/364 [04:52<02:18,  1.02s/it]
Epoch 4/10:  63%|██████▎   | 229/364 [04:54<02:54,  1.29s/it]
Epoch 4/10:  63%|██████▎   | 230/364 [04:56<03:01,  1.35s/it]
Epoch 4/10:  63%|██████▎   | 231/364 [04:57<03:12,  1.45s/it]
Epoch 4/10:  64%|██████▎   | 232/364 [04:58<02:30,  1.14s/it]
Epoch 4/10:  64%|██████▍   | 233/364 [05:00<02:53,  1.33s/it]
Epoch 4/10:  64%|██████▍   | 234/364 [05:01<02:56,  1.36s/it]
Epoch 4/10:  65%|██████▍   | 235/364 [05:02<02:52,  1.34s/it]
Epoch 4/10:  65%|██████▍   | 236/364 [05:03<02:16,  1.06s/it]
Epoch 4/10:  65%|██████▌   | 237/364 [05:05<02:54,  1.38s/it]
Epoch 4/10:  65%|██████▌   | 238/364 [05:06<02:56,  1.40s/it]
Epoch 4/10:  66%|██████▌   | 239/364 [05:07<02:46,  1.33s/it]
Epoch 4/10:  66%|██████▌   | 240/364 [05:08<02:11,  1.06s/it]
Epoch 4/10:  66%|██████▌   | 241/364 [05:10<02:55,  1.42s/it]
Epoch 4/10:  66%|██████▋   | 242/364 [05:11<02:43,  1.34s/it]
Epoch 4/10:  67%|██████▋   | 243/364 [05:12<02:23,  1.19s/it]
Epoch 4/10:  67%|██████▋   | 244/364 [05:13<01:55,  1.04it/s]
Epoch 4/10:  67%|██████▋   | 245/364 [05:15<02:40,  1.35s/it]
Epoch 4/10:  68%|██████▊   | 246/364 [05:16<02:41,  1.37s/it]
Epoch 4/10:  68%|██████▊   | 247/364 [05:17<02:24,  1.24s/it]
Epoch 4/10:  68%|██████▊   | 248/364 [05:18<01:55,  1.01it/s]
Epoch 4/10:  68%|██████▊   | 249/364 [05:20<02:30,  1.31s/it]
Epoch 4/10:  69%|██████▊   | 250/364 [05:22<02:52,  1.52s/it]
Epoch 4/10:  69%|██████▉   | 251/364 [05:22<02:15,  1.20s/it]
Epoch 4/10:  69%|██████▉   | 252/364 [05:23<01:48,  1.03it/s]
Epoch 4/10:  70%|██████▉   | 253/364 [05:25<02:31,  1.37s/it]
Epoch 4/10:  70%|██████▉   | 254/364 [05:27<02:47,  1.52s/it]
Epoch 4/10:  70%|███████   | 255/364 [05:27<02:10,  1.19s/it]
Epoch 4/10:  70%|███████   | 256/364 [05:28<01:44,  1.04it/s]
Epoch 4/10:  71%|███████   | 257/364 [05:30<02:33,  1.43s/it]
Epoch 4/10:  71%|███████   | 258/364 [05:32<02:50,  1.61s/it]
Epoch 4/10:  71%|███████   | 259/364 [05:33<02:11,  1.25s/it]
Epoch 4/10:  71%|███████▏  | 260/364 [05:33<01:44,  1.00s/it]
Epoch 4/10:  72%|███████▏  | 261/364 [05:35<02:19,  1.35s/it]
Epoch 4/10:  72%|███████▏  | 262/364 [05:37<02:42,  1.59s/it]
Epoch 4/10:  72%|███████▏  | 263/364 [05:38<02:05,  1.24s/it]
Epoch 4/10:  73%|███████▎  | 264/364 [05:38<01:39,  1.00it/s]
Epoch 4/10:  73%|███████▎  | 265/364 [05:41<02:21,  1.43s/it]
Epoch 4/10:  73%|███████▎  | 266/364 [05:42<02:30,  1.54s/it]
Epoch 4/10:  73%|███████▎  | 267/364 [05:43<01:56,  1.21s/it]
Epoch 4/10:  74%|███████▎  | 268/364 [05:43<01:33,  1.03it/s]
Epoch 4/10:  74%|███████▍  | 269/364 [05:46<02:21,  1.49s/it]
Epoch 4/10:  74%|███████▍  | 270/364 [05:47<02:10,  1.38s/it]
Epoch 4/10:  74%|███████▍  | 271/364 [05:48<01:42,  1.10s/it]
Epoch 4/10:  75%|███████▍  | 272/364 [05:48<01:22,  1.11it/s]
Epoch 4/10:  75%|███████▌  | 273/364 [05:51<02:30,  1.65s/it]
Epoch 4/10:  75%|███████▌  | 274/364 [05:52<01:56,  1.29s/it]
Epoch 4/10:  76%|███████▌  | 275/364 [05:52<01:31,  1.03s/it]
Epoch 4/10:  76%|███████▌  | 276/364 [05:53<01:14,  1.18it/s]
Epoch 4/10:  76%|███████▌  | 277/364 [05:56<02:26,  1.68s/it]
Epoch 4/10:  76%|███████▋  | 278/364 [05:57<01:52,  1.31s/it]
Epoch 4/10:  77%|███████▋  | 279/364 [05:57<01:29,  1.05s/it]
Epoch 4/10:  77%|███████▋  | 280/364 [05:58<01:12,  1.16it/s]
Epoch 4/10:  77%|███████▋  | 281/364 [06:01<02:22,  1.71s/it]
Epoch 4/10:  77%|███████▋  | 282/364 [06:02<01:55,  1.41s/it]
Epoch 4/10:  78%|███████▊  | 283/364 [06:02<01:30,  1.11s/it]
Epoch 4/10:  78%|███████▊  | 284/364 [06:03<01:12,  1.10it/s]
Epoch 4/10:  78%|███████▊  | 285/364 [06:06<02:11,  1.66s/it]
Epoch 4/10:  79%|███████▊  | 286/364 [06:07<01:51,  1.42s/it]
Epoch 4/10:  79%|███████▉  | 287/364 [06:08<01:26,  1.13s/it]
Epoch 4/10:  79%|███████▉  | 288/364 [06:08<01:09,  1.10it/s]
Epoch 4/10:  79%|███████▉  | 289/364 [06:12<02:08,  1.71s/it]
Epoch 4/10:  80%|███████▉  | 290/364 [06:12<01:41,  1.38s/it]
Epoch 4/10:  80%|███████▉  | 291/364 [06:13<01:19,  1.09s/it]
Epoch 4/10:  80%|████████  | 292/364 [06:13<01:04,  1.12it/s]
Epoch 4/10:  80%|████████  | 293/364 [06:16<01:55,  1.62s/it]
Epoch 4/10:  81%|████████  | 294/364 [06:17<01:40,  1.44s/it]
Epoch 4/10:  81%|████████  | 295/364 [06:18<01:18,  1.14s/it]
Epoch 4/10:  81%|████████▏ | 296/364 [06:18<01:03,  1.07it/s]
Epoch 4/10:  82%|████████▏ | 297/364 [06:21<01:47,  1.61s/it]
Epoch 4/10:  82%|████████▏ | 298/364 [06:22<01:26,  1.31s/it]
Epoch 4/10:  82%|████████▏ | 299/364 [06:22<01:07,  1.05s/it]
Epoch 4/10:  82%|████████▏ | 300/364 [06:23<00:55,  1.16it/s]
Epoch 4/10:  83%|████████▎ | 301/364 [06:27<01:50,  1.75s/it]
Epoch 4/10:  83%|████████▎ | 302/364 [06:27<01:23,  1.35s/it]
Epoch 4/10:  83%|████████▎ | 303/364 [06:28<01:05,  1.08s/it]
Epoch 4/10:  84%|████████▎ | 304/364 [06:28<00:52,  1.13it/s]
Epoch 4/10:  84%|████████▍ | 305/364 [06:32<01:47,  1.82s/it]
Epoch 4/10:  84%|████████▍ | 306/364 [06:32<01:21,  1.41s/it]
Epoch 4/10:  84%|████████▍ | 307/364 [06:33<01:03,  1.11s/it]
Epoch 4/10:  85%|████████▍ | 308/364 [06:33<00:50,  1.11it/s]
Epoch 4/10:  85%|████████▍ | 309/364 [06:37<01:37,  1.78s/it]
Epoch 4/10:  85%|████████▌ | 310/364 [06:38<01:14,  1.38s/it]
Epoch 4/10:  85%|████████▌ | 311/364 [06:38<00:57,  1.09s/it]
Epoch 4/10:  86%|████████▌ | 312/364 [06:38<00:46,  1.12it/s]
Epoch 4/10:  86%|████████▌ | 313/364 [06:42<01:25,  1.68s/it]
Epoch 4/10:  86%|████████▋ | 314/364 [06:42<01:05,  1.30s/it]
Epoch 4/10:  87%|████████▋ | 315/364 [06:43<00:51,  1.04s/it]
Epoch 4/10:  87%|████████▋ | 316/364 [06:43<00:44,  1.08it/s]
Epoch 4/10:  87%|████████▋ | 317/364 [06:47<01:14,  1.58s/it]
Epoch 4/10:  87%|████████▋ | 318/364 [06:47<00:56,  1.23s/it]
Epoch 4/10:  88%|████████▊ | 319/364 [06:47<00:44,  1.02it/s]
Epoch 4/10:  88%|████████▊ | 320/364 [06:49<00:46,  1.06s/it]
Epoch 4/10:  88%|████████▊ | 321/364 [06:52<01:11,  1.67s/it]
Epoch 4/10:  88%|████████▊ | 322/364 [06:52<00:54,  1.29s/it]
Epoch 4/10:  89%|████████▊ | 323/364 [06:53<00:42,  1.03s/it]
Epoch 4/10:  89%|████████▉ | 324/364 [06:54<00:41,  1.03s/it]
Epoch 4/10:  89%|████████▉ | 325/364 [06:57<01:05,  1.68s/it]
Epoch 4/10:  90%|████████▉ | 326/364 [06:57<00:49,  1.30s/it]
Epoch 4/10:  90%|████████▉ | 327/364 [06:58<00:38,  1.03s/it]
Epoch 4/10:  90%|█████████ | 328/364 [06:59<00:40,  1.13s/it]
Epoch 4/10:  90%|█████████ | 329/364 [07:02<01:02,  1.78s/it]
Epoch 4/10:  91%|█████████ | 330/364 [07:03<00:46,  1.37s/it]
Epoch 4/10:  91%|█████████ | 331/364 [07:03<00:35,  1.09s/it]
Epoch 4/10:  91%|█████████ | 332/364 [07:04<00:35,  1.11s/it]
Epoch 4/10:  91%|█████████▏| 333/364 [07:07<00:52,  1.71s/it]
Epoch 4/10:  92%|█████████▏| 334/364 [07:08<00:39,  1.32s/it]
Epoch 4/10:  92%|█████████▏| 335/364 [07:08<00:30,  1.05s/it]
Epoch 4/10:  92%|█████████▏| 336/364 [07:09<00:28,  1.03s/it]
Epoch 4/10:  93%|█████████▎| 337/364 [07:12<00:46,  1.72s/it]
Epoch 4/10:  93%|█████████▎| 338/364 [07:13<00:34,  1.33s/it]
Epoch 4/10:  93%|█████████▎| 339/364 [07:13<00:26,  1.06s/it]
Epoch 4/10:  93%|█████████▎| 340/364 [07:14<00:24,  1.03s/it]
Epoch 4/10:  94%|█████████▎| 341/364 [07:17<00:37,  1.65s/it]
Epoch 4/10:  94%|█████████▍| 342/364 [07:18<00:28,  1.28s/it]
Epoch 4/10:  94%|█████████▍| 343/364 [07:18<00:21,  1.02s/it]
Epoch 4/10:  95%|█████████▍| 344/364 [07:19<00:18,  1.06it/s]
Epoch 4/10:  95%|█████████▍| 345/364 [07:23<00:34,  1.82s/it]
Epoch 4/10:  95%|█████████▌| 346/364 [07:23<00:25,  1.40s/it]
Epoch 4/10:  95%|█████████▌| 347/364 [07:24<00:18,  1.11s/it]
Epoch 4/10:  96%|█████████▌| 348/364 [07:24<00:14,  1.11it/s]
Epoch 4/10:  96%|█████████▌| 349/364 [07:28<00:27,  1.84s/it]
Epoch 4/10:  96%|█████████▌| 350/364 [07:29<00:19,  1.42s/it]
Epoch 4/10:  96%|█████████▋| 351/364 [07:29<00:14,  1.12s/it]
Epoch 4/10:  97%|█████████▋| 352/364 [07:29<00:10,  1.09it/s]
Epoch 4/10:  97%|█████████▋| 353/364 [07:34<00:20,  1.88s/it]
Epoch 4/10:  97%|█████████▋| 354/364 [07:34<00:14,  1.44s/it]
Epoch 4/10:  98%|█████████▊| 355/364 [07:34<00:10,  1.14s/it]
Epoch 4/10:  98%|█████████▊| 356/364 [07:35<00:07,  1.08it/s]
Epoch 4/10:  98%|█████████▊| 357/364 [07:39<00:13,  1.91s/it]
Epoch 4/10:  98%|█████████▊| 358/364 [07:39<00:08,  1.46s/it]
Epoch 4/10:  99%|█████████▊| 359/364 [07:40<00:05,  1.15s/it]
Epoch 4/10:  99%|█████████▉| 360/364 [07:40<00:03,  1.07it/s]
Epoch 4/10:  99%|█████████▉| 361/364 [07:43<00:04,  1.35s/it]
Epoch 4/10:  99%|█████████▉| 362/364 [07:43<00:02,  1.06s/it]
Epoch 4/10: 100%|█████████▉| 363/364 [07:43<00:00,  1.16it/s]
Epoch 4/10: 100%|██████████| 364/364 [07:44<00:00,  1.52it/s]
Epoch 4/10: 100%|██████████| 364/364 [07:44<00:00,  1.28s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:37,  4.95s/it]
Validating:   4%|▎         | 2/57 [00:05<01:56,  2.12s/it]
Validating:   5%|▌         | 3/57 [00:05<01:05,  1.21s/it]
Validating:   7%|▋         | 4/57 [00:05<00:41,  1.27it/s]
Validating:   9%|▉         | 5/57 [00:09<01:42,  1.97s/it]
Validating:  11%|█         | 6/57 [00:09<01:09,  1.36s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:48,  1.03it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:34,  1.44it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:20,  1.68s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:56,  1.21s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:40,  1.14it/s]
Validating:  21%|██        | 12/57 [00:14<00:29,  1.53it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:17,  1.76s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:54,  1.28s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:39,  1.07it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:28,  1.43it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:05,  1.63s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:46,  1.19s/it]
Validating:  33%|███▎      | 19/57 [00:23<00:33,  1.13it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:24,  1.52it/s]
Validating:  37%|███▋      | 21/57 [00:27<00:59,  1.66s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:42,  1.22s/it]
Validating:  40%|████      | 23/57 [00:27<00:30,  1.11it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:22,  1.49it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:52,  1.65s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:37,  1.20s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:26,  1.13it/s]
Validating:  49%|████▉     | 28/57 [00:32<00:19,  1.49it/s]
Validating:  51%|█████     | 29/57 [00:35<00:44,  1.61s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:33,  1.23s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:23,  1.11it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.48it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:38,  1.60s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:28,  1.24s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:20,  1.09it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.45it/s]
Validating:  65%|██████▍   | 37/57 [00:45<00:34,  1.72s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:24,  1.27s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:16,  1.07it/s]
Validating:  70%|███████   | 40/57 [00:45<00:11,  1.44it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:27,  1.72s/it]
Validating:  74%|███████▎  | 42/57 [00:49<00:18,  1.26s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:12,  1.09it/s]
Validating:  77%|███████▋  | 44/57 [00:50<00:08,  1.47it/s]
Validating:  79%|███████▉  | 45/57 [00:54<00:20,  1.70s/it]
Validating:  81%|████████  | 46/57 [00:54<00:13,  1.26s/it]
Validating:  82%|████████▏ | 47/57 [00:54<00:09,  1.08it/s]
Validating:  84%|████████▍ | 48/57 [00:54<00:06,  1.43it/s]
Validating:  86%|████████▌ | 49/57 [00:58<00:12,  1.51s/it]
Validating:  88%|████████▊ | 50/57 [00:58<00:08,  1.25s/it]
Validating:  89%|████████▉ | 51/57 [00:58<00:05,  1.08it/s]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.45it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:05,  1.27s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:02,  1.03it/s]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.40it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.88it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.09s/it]
Epoch 4: Train Loss: 0.1852, Train Acc: 0.9340 | Val Loss: 0.3099, Val Acc: 0.9000
Saved Best Model!

Epoch 5/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 5/10:   0%|          | 1/364 [00:05<34:57,  5.78s/it]
Epoch 5/10:   1%|          | 2/364 [00:06<15:57,  2.65s/it]
Epoch 5/10:   1%|          | 3/364 [00:06<09:47,  1.63s/it]
Epoch 5/10:   1%|          | 4/364 [00:07<06:55,  1.15s/it]
Epoch 5/10:   1%|▏         | 5/364 [00:11<12:57,  2.17s/it]
Epoch 5/10:   2%|▏         | 6/364 [00:11<09:19,  1.56s/it]
Epoch 5/10:   2%|▏         | 7/364 [00:11<07:03,  1.19s/it]
Epoch 5/10:   2%|▏         | 8/364 [00:12<05:35,  1.06it/s]
Epoch 5/10:   2%|▏         | 9/364 [00:16<11:02,  1.87s/it]
Epoch 5/10:   3%|▎         | 10/364 [00:16<08:24,  1.43s/it]
Epoch 5/10:   3%|▎         | 11/364 [00:17<06:33,  1.12s/it]
Epoch 5/10:   3%|▎         | 12/364 [00:17<05:19,  1.10it/s]
Epoch 5/10:   4%|▎         | 13/364 [00:21<10:16,  1.76s/it]
Epoch 5/10:   4%|▍         | 14/364 [00:21<08:30,  1.46s/it]
Epoch 5/10:   4%|▍         | 15/364 [00:22<06:38,  1.14s/it]
Epoch 5/10:   4%|▍         | 16/364 [00:22<05:19,  1.09it/s]
Epoch 5/10:   5%|▍         | 17/364 [00:26<10:05,  1.74s/it]
Epoch 5/10:   5%|▍         | 18/364 [00:26<07:48,  1.35s/it]
Epoch 5/10:   5%|▌         | 19/364 [00:27<06:11,  1.08s/it]
Epoch 5/10:   5%|▌         | 20/364 [00:27<05:01,  1.14it/s]
Epoch 5/10:   6%|▌         | 21/364 [00:31<10:29,  1.83s/it]
Epoch 5/10:   6%|▌         | 22/364 [00:32<08:02,  1.41s/it]
Epoch 5/10:   6%|▋         | 23/364 [00:32<06:20,  1.12s/it]
Epoch 5/10:   7%|▋         | 24/364 [00:33<05:07,  1.10it/s]
Epoch 5/10:   7%|▋         | 25/364 [00:37<10:58,  1.94s/it]
Epoch 5/10:   7%|▋         | 26/364 [00:37<08:21,  1.48s/it]
Epoch 5/10:   7%|▋         | 27/364 [00:38<06:35,  1.17s/it]
Epoch 5/10:   8%|▊         | 28/364 [00:38<05:17,  1.06it/s]
Epoch 5/10:   8%|▊         | 29/364 [00:42<10:03,  1.80s/it]
Epoch 5/10:   8%|▊         | 30/364 [00:42<07:44,  1.39s/it]
Epoch 5/10:   9%|▊         | 31/364 [00:43<06:04,  1.09s/it]
Epoch 5/10:   9%|▉         | 32/364 [00:43<04:54,  1.13it/s]
Epoch 5/10:   9%|▉         | 33/364 [00:47<10:00,  1.81s/it]
Epoch 5/10:   9%|▉         | 34/364 [00:48<07:38,  1.39s/it]
Epoch 5/10:  10%|▉         | 35/364 [00:48<06:01,  1.10s/it]
Epoch 5/10:  10%|▉         | 36/364 [00:48<04:54,  1.12it/s]
Epoch 5/10:  10%|█         | 37/364 [00:52<09:51,  1.81s/it]
Epoch 5/10:  10%|█         | 38/364 [00:53<07:35,  1.40s/it]
Epoch 5/10:  11%|█         | 39/364 [00:53<05:58,  1.10s/it]
Epoch 5/10:  11%|█         | 40/364 [00:54<04:53,  1.10it/s]
Epoch 5/10:  11%|█▏        | 41/364 [00:57<09:31,  1.77s/it]
Epoch 5/10:  12%|█▏        | 42/364 [00:58<07:20,  1.37s/it]
Epoch 5/10:  12%|█▏        | 43/364 [00:58<05:51,  1.09s/it]
Epoch 5/10:  12%|█▏        | 44/364 [00:59<04:47,  1.11it/s]
Epoch 5/10:  12%|█▏        | 45/364 [01:02<09:12,  1.73s/it]
Epoch 5/10:  13%|█▎        | 46/364 [01:04<08:26,  1.59s/it]
Epoch 5/10:  13%|█▎        | 47/364 [01:04<06:32,  1.24s/it]
Epoch 5/10:  13%|█▎        | 48/364 [01:05<05:16,  1.00s/it]
Epoch 5/10:  13%|█▎        | 49/364 [01:07<08:07,  1.55s/it]
Epoch 5/10:  14%|█▎        | 50/364 [01:09<08:25,  1.61s/it]
Epoch 5/10:  14%|█▍        | 51/364 [01:10<06:31,  1.25s/it]
Epoch 5/10:  14%|█▍        | 52/364 [01:10<05:11,  1.00it/s]
Epoch 5/10:  15%|█▍        | 53/364 [01:13<07:52,  1.52s/it]
Epoch 5/10:  15%|█▍        | 54/364 [01:15<08:16,  1.60s/it]
Epoch 5/10:  15%|█▌        | 55/364 [01:15<06:25,  1.25s/it]
Epoch 5/10:  15%|█▌        | 56/364 [01:15<05:09,  1.00s/it]
Epoch 5/10:  16%|█▌        | 57/364 [01:18<07:36,  1.49s/it]
Epoch 5/10:  16%|█▌        | 58/364 [01:20<07:38,  1.50s/it]
Epoch 5/10:  16%|█▌        | 59/364 [01:20<06:00,  1.18s/it]
Epoch 5/10:  16%|█▋        | 60/364 [01:20<04:50,  1.05it/s]
Epoch 5/10:  17%|█▋        | 61/364 [01:23<07:40,  1.52s/it]
Epoch 5/10:  17%|█▋        | 62/364 [01:25<07:41,  1.53s/it]
Epoch 5/10:  17%|█▋        | 63/364 [01:25<05:59,  1.20s/it]
Epoch 5/10:  18%|█▊        | 64/364 [01:26<04:47,  1.04it/s]
Epoch 5/10:  18%|█▊        | 65/364 [01:29<07:51,  1.58s/it]
Epoch 5/10:  18%|█▊        | 66/364 [01:30<07:51,  1.58s/it]
Epoch 5/10:  18%|█▊        | 67/364 [01:31<06:06,  1.23s/it]
Epoch 5/10:  19%|█▊        | 68/364 [01:31<04:53,  1.01it/s]
Epoch 5/10:  19%|█▉        | 69/364 [01:34<07:06,  1.44s/it]
Epoch 5/10:  19%|█▉        | 70/364 [01:36<08:05,  1.65s/it]
Epoch 5/10:  20%|█▉        | 71/364 [01:36<06:16,  1.29s/it]
Epoch 5/10:  20%|█▉        | 72/364 [01:37<05:01,  1.03s/it]
Epoch 5/10:  20%|██        | 73/364 [01:39<06:53,  1.42s/it]
Epoch 5/10:  20%|██        | 74/364 [01:41<07:18,  1.51s/it]
Epoch 5/10:  21%|██        | 75/364 [01:41<05:40,  1.18s/it]
Epoch 5/10:  21%|██        | 76/364 [01:41<04:37,  1.04it/s]
Epoch 5/10:  21%|██        | 77/364 [01:44<06:58,  1.46s/it]
Epoch 5/10:  21%|██▏       | 78/364 [01:46<07:35,  1.59s/it]
Epoch 5/10:  22%|██▏       | 79/364 [01:46<05:53,  1.24s/it]
Epoch 5/10:  22%|██▏       | 80/364 [01:47<04:41,  1.01it/s]
Epoch 5/10:  22%|██▏       | 81/364 [01:49<06:17,  1.34s/it]
Epoch 5/10:  23%|██▎       | 82/364 [01:51<07:44,  1.65s/it]
Epoch 5/10:  23%|██▎       | 83/364 [01:52<05:59,  1.28s/it]
Epoch 5/10:  23%|██▎       | 84/364 [01:52<04:47,  1.03s/it]
Epoch 5/10:  23%|██▎       | 85/364 [01:54<06:04,  1.31s/it]
Epoch 5/10:  24%|██▎       | 86/364 [01:57<07:31,  1.62s/it]
Epoch 5/10:  24%|██▍       | 87/364 [01:57<05:50,  1.26s/it]
Epoch 5/10:  24%|██▍       | 88/364 [01:57<04:39,  1.01s/it]
Epoch 5/10:  24%|██▍       | 89/364 [01:59<05:53,  1.29s/it]
Epoch 5/10:  25%|██▍       | 90/364 [02:02<07:09,  1.57s/it]
Epoch 5/10:  25%|██▌       | 91/364 [02:02<05:35,  1.23s/it]
Epoch 5/10:  25%|██▌       | 92/364 [02:02<04:30,  1.01it/s]
Epoch 5/10:  26%|██▌       | 93/364 [02:05<06:17,  1.39s/it]
Epoch 5/10:  26%|██▌       | 94/364 [02:07<07:14,  1.61s/it]
Epoch 5/10:  26%|██▌       | 95/364 [02:07<05:37,  1.25s/it]
Epoch 5/10:  26%|██▋       | 96/364 [02:08<04:29,  1.00s/it]
Epoch 5/10:  27%|██▋       | 97/364 [02:10<05:57,  1.34s/it]
Epoch 5/10:  27%|██▋       | 98/364 [02:12<07:10,  1.62s/it]
Epoch 5/10:  27%|██▋       | 99/364 [02:12<05:33,  1.26s/it]
Epoch 5/10:  27%|██▋       | 100/364 [02:13<04:25,  1.00s/it]
Epoch 5/10:  28%|██▊       | 101/364 [02:15<05:38,  1.29s/it]
Epoch 5/10:  28%|██▊       | 102/364 [02:18<07:35,  1.74s/it]
Epoch 5/10:  28%|██▊       | 103/364 [02:18<05:51,  1.35s/it]
Epoch 5/10:  29%|██▊       | 104/364 [02:19<04:39,  1.07s/it]
Epoch 5/10:  29%|██▉       | 105/364 [02:20<04:52,  1.13s/it]
Epoch 5/10:  29%|██▉       | 106/364 [02:23<07:25,  1.73s/it]
Epoch 5/10:  29%|██▉       | 107/364 [02:23<05:42,  1.33s/it]
Epoch 5/10:  30%|██▉       | 108/364 [02:24<04:30,  1.06s/it]
Epoch 5/10:  30%|██▉       | 109/364 [02:25<04:43,  1.11s/it]
Epoch 5/10:  30%|███       | 110/364 [02:28<06:54,  1.63s/it]
Epoch 5/10:  30%|███       | 111/364 [02:28<05:22,  1.27s/it]
Epoch 5/10:  31%|███       | 112/364 [02:29<04:16,  1.02s/it]
Epoch 5/10:  31%|███       | 113/364 [02:30<04:42,  1.13s/it]
Epoch 5/10:  31%|███▏      | 114/364 [02:33<06:58,  1.67s/it]
Epoch 5/10:  32%|███▏      | 115/364 [02:33<05:23,  1.30s/it]
Epoch 5/10:  32%|███▏      | 116/364 [02:34<04:16,  1.04s/it]
Epoch 5/10:  32%|███▏      | 117/364 [02:35<04:49,  1.17s/it]
Epoch 5/10:  32%|███▏      | 118/364 [02:38<07:15,  1.77s/it]
Epoch 5/10:  33%|███▎      | 119/364 [02:39<05:33,  1.36s/it]
Epoch 5/10:  33%|███▎      | 120/364 [02:39<04:22,  1.08s/it]
Epoch 5/10:  33%|███▎      | 121/364 [02:40<04:26,  1.10s/it]
Epoch 5/10:  34%|███▎      | 122/364 [02:43<06:42,  1.66s/it]
Epoch 5/10:  34%|███▍      | 123/364 [02:44<05:11,  1.29s/it]
Epoch 5/10:  34%|███▍      | 124/364 [02:44<04:08,  1.03s/it]
Epoch 5/10:  34%|███▍      | 125/364 [02:45<04:13,  1.06s/it]
Epoch 5/10:  35%|███▍      | 126/364 [02:49<06:44,  1.70s/it]
Epoch 5/10:  35%|███▍      | 127/364 [02:49<05:13,  1.32s/it]
Epoch 5/10:  35%|███▌      | 128/364 [02:49<04:08,  1.05s/it]
Epoch 5/10:  35%|███▌      | 129/364 [02:51<04:17,  1.10s/it]
Epoch 5/10:  36%|███▌      | 130/364 [02:54<06:28,  1.66s/it]
Epoch 5/10:  36%|███▌      | 131/364 [02:54<05:02,  1.30s/it]
Epoch 5/10:  36%|███▋      | 132/364 [02:55<04:01,  1.04s/it]
Epoch 5/10:  37%|███▋      | 133/364 [02:56<04:57,  1.29s/it]
Epoch 5/10:  37%|███▋      | 134/364 [02:59<06:15,  1.63s/it]
Epoch 5/10:  37%|███▋      | 135/364 [02:59<04:52,  1.28s/it]
Epoch 5/10:  37%|███▋      | 136/364 [03:00<03:53,  1.02s/it]
Epoch 5/10:  38%|███▊      | 137/364 [03:01<04:31,  1.20s/it]
Epoch 5/10:  38%|███▊      | 138/364 [03:04<06:20,  1.68s/it]
Epoch 5/10:  38%|███▊      | 139/364 [03:05<04:54,  1.31s/it]
Epoch 5/10:  38%|███▊      | 140/364 [03:05<03:54,  1.05s/it]
Epoch 5/10:  39%|███▊      | 141/364 [03:06<04:19,  1.16s/it]
Epoch 5/10:  39%|███▉      | 142/364 [03:09<06:20,  1.71s/it]
Epoch 5/10:  39%|███▉      | 143/364 [03:10<04:52,  1.33s/it]
Epoch 5/10:  40%|███▉      | 144/364 [03:10<03:53,  1.06s/it]
Epoch 5/10:  40%|███▉      | 145/364 [03:12<04:52,  1.34s/it]
Epoch 5/10:  40%|████      | 146/364 [03:14<05:03,  1.39s/it]
Epoch 5/10:  40%|████      | 147/364 [03:14<04:00,  1.11s/it]
Epoch 5/10:  41%|████      | 148/364 [03:15<03:16,  1.10it/s]
Epoch 5/10:  41%|████      | 149/364 [03:17<05:14,  1.46s/it]
Epoch 5/10:  41%|████      | 150/364 [03:19<05:30,  1.54s/it]
Epoch 5/10:  41%|████▏     | 151/364 [03:20<04:17,  1.21s/it]
Epoch 5/10:  42%|████▏     | 152/364 [03:20<03:35,  1.02s/it]
Epoch 5/10:  42%|████▏     | 153/364 [03:22<04:36,  1.31s/it]
Epoch 5/10:  42%|████▏     | 154/364 [03:24<05:21,  1.53s/it]
Epoch 5/10:  43%|████▎     | 155/364 [03:25<04:11,  1.20s/it]
Epoch 5/10:  43%|████▎     | 156/364 [03:25<03:39,  1.06s/it]
Epoch 5/10:  43%|████▎     | 157/364 [03:27<04:30,  1.31s/it]
Epoch 5/10:  43%|████▎     | 158/364 [03:30<05:28,  1.60s/it]
Epoch 5/10:  44%|████▎     | 159/364 [03:30<04:15,  1.25s/it]
Epoch 5/10:  44%|████▍     | 160/364 [03:31<03:34,  1.05s/it]
Epoch 5/10:  44%|████▍     | 161/364 [03:32<04:12,  1.24s/it]
Epoch 5/10:  45%|████▍     | 162/364 [03:35<05:27,  1.62s/it]
Epoch 5/10:  45%|████▍     | 163/364 [03:35<04:12,  1.26s/it]
Epoch 5/10:  45%|████▌     | 164/364 [03:36<03:22,  1.01s/it]
Epoch 5/10:  45%|████▌     | 165/364 [03:37<03:49,  1.16s/it]
Epoch 5/10:  46%|████▌     | 166/364 [03:40<05:39,  1.71s/it]
Epoch 5/10:  46%|████▌     | 167/364 [03:41<04:21,  1.33s/it]
Epoch 5/10:  46%|████▌     | 168/364 [03:41<03:29,  1.07s/it]
Epoch 5/10:  46%|████▋     | 169/364 [03:42<03:33,  1.09s/it]
Epoch 5/10:  47%|████▋     | 170/364 [03:45<05:33,  1.72s/it]
Epoch 5/10:  47%|████▋     | 171/364 [03:46<04:17,  1.33s/it]
Epoch 5/10:  47%|████▋     | 172/364 [03:46<03:24,  1.06s/it]
Epoch 5/10:  48%|████▊     | 173/364 [03:48<03:46,  1.18s/it]
Epoch 5/10:  48%|████▊     | 174/364 [03:50<05:14,  1.66s/it]
Epoch 5/10:  48%|████▊     | 175/364 [03:51<04:04,  1.29s/it]
Epoch 5/10:  48%|████▊     | 176/364 [03:51<03:14,  1.04s/it]
Epoch 5/10:  49%|████▊     | 177/364 [03:53<03:42,  1.19s/it]
Epoch 5/10:  49%|████▉     | 178/364 [03:55<04:52,  1.57s/it]
Epoch 5/10:  49%|████▉     | 179/364 [03:56<03:47,  1.23s/it]
Epoch 5/10:  49%|████▉     | 180/364 [03:56<03:03,  1.00it/s]
Epoch 5/10:  50%|████▉     | 181/364 [03:58<03:45,  1.23s/it]
Epoch 5/10:  50%|█████     | 182/364 [04:00<04:39,  1.54s/it]
Epoch 5/10:  50%|█████     | 183/364 [04:01<03:37,  1.20s/it]
Epoch 5/10:  51%|█████     | 184/364 [04:01<03:03,  1.02s/it]
Epoch 5/10:  51%|█████     | 185/364 [04:03<03:41,  1.24s/it]
Epoch 5/10:  51%|█████     | 186/364 [04:06<04:57,  1.67s/it]
Epoch 5/10:  51%|█████▏    | 187/364 [04:06<03:50,  1.30s/it]
Epoch 5/10:  52%|█████▏    | 188/364 [04:07<03:03,  1.04s/it]
Epoch 5/10:  52%|█████▏    | 189/364 [04:08<03:35,  1.23s/it]
Epoch 5/10:  52%|█████▏    | 190/364 [04:11<04:49,  1.67s/it]
Epoch 5/10:  52%|█████▏    | 191/364 [04:11<03:43,  1.29s/it]
Epoch 5/10:  53%|█████▎    | 192/364 [04:12<02:58,  1.04s/it]
Epoch 5/10:  53%|█████▎    | 193/364 [04:14<03:43,  1.30s/it]
Epoch 5/10:  53%|█████▎    | 194/364 [04:16<04:48,  1.70s/it]
Epoch 5/10:  54%|█████▎    | 195/364 [04:17<03:42,  1.32s/it]
Epoch 5/10:  54%|█████▍    | 196/364 [04:17<02:55,  1.05s/it]
Epoch 5/10:  54%|█████▍    | 197/364 [04:19<03:25,  1.23s/it]
Epoch 5/10:  54%|█████▍    | 198/364 [04:22<04:40,  1.69s/it]
Epoch 5/10:  55%|█████▍    | 199/364 [04:22<03:36,  1.31s/it]
Epoch 5/10:  55%|█████▍    | 200/364 [04:22<02:52,  1.05s/it]
Epoch 5/10:  55%|█████▌    | 201/364 [04:24<03:16,  1.21s/it]
Epoch 5/10:  55%|█████▌    | 202/364 [04:27<04:21,  1.61s/it]
Epoch 5/10:  56%|█████▌    | 203/364 [04:27<03:22,  1.26s/it]
Epoch 5/10:  56%|█████▌    | 204/364 [04:27<02:42,  1.01s/it]
Epoch 5/10:  56%|█████▋    | 205/364 [04:29<03:24,  1.29s/it]
Epoch 5/10:  57%|█████▋    | 206/364 [04:32<04:12,  1.60s/it]
Epoch 5/10:  57%|█████▋    | 207/364 [04:32<03:15,  1.24s/it]
Epoch 5/10:  57%|█████▋    | 208/364 [04:33<02:36,  1.00s/it]
Epoch 5/10:  57%|█████▋    | 209/364 [04:34<03:15,  1.26s/it]
Epoch 5/10:  58%|█████▊    | 210/364 [04:37<04:08,  1.62s/it]
Epoch 5/10:  58%|█████▊    | 211/364 [04:37<03:12,  1.26s/it]
Epoch 5/10:  58%|█████▊    | 212/364 [04:38<02:34,  1.01s/it]
Epoch 5/10:  59%|█████▊    | 213/364 [04:39<02:57,  1.18s/it]
Epoch 5/10:  59%|█████▉    | 214/364 [04:42<04:03,  1.62s/it]
Epoch 5/10:  59%|█████▉    | 215/364 [04:42<03:08,  1.26s/it]
Epoch 5/10:  59%|█████▉    | 216/364 [04:43<02:30,  1.02s/it]
Epoch 5/10:  60%|█████▉    | 217/364 [04:45<03:00,  1.23s/it]
Epoch 5/10:  60%|█████▉    | 218/364 [04:47<04:04,  1.67s/it]
Epoch 5/10:  60%|██████    | 219/364 [04:48<03:08,  1.30s/it]
Epoch 5/10:  60%|██████    | 220/364 [04:48<02:29,  1.04s/it]
Epoch 5/10:  61%|██████    | 221/364 [04:49<02:35,  1.09s/it]
Epoch 5/10:  61%|██████    | 222/364 [04:53<04:06,  1.74s/it]
Epoch 5/10:  61%|██████▏   | 223/364 [04:53<03:09,  1.35s/it]
Epoch 5/10:  62%|██████▏   | 224/364 [04:53<02:29,  1.07s/it]
Epoch 5/10:  62%|██████▏   | 225/364 [04:55<02:29,  1.08s/it]
Epoch 5/10:  62%|██████▏   | 226/364 [04:58<04:00,  1.74s/it]
Epoch 5/10:  62%|██████▏   | 227/364 [04:58<03:04,  1.34s/it]
Epoch 5/10:  63%|██████▎   | 228/364 [04:59<02:25,  1.07s/it]
Epoch 5/10:  63%|██████▎   | 229/364 [05:00<02:36,  1.16s/it]
Epoch 5/10:  63%|██████▎   | 230/364 [05:03<03:41,  1.65s/it]
Epoch 5/10:  63%|██████▎   | 231/364 [05:03<02:50,  1.28s/it]
Epoch 5/10:  64%|██████▎   | 232/364 [05:04<02:16,  1.03s/it]
Epoch 5/10:  64%|██████▍   | 233/364 [05:05<02:44,  1.25s/it]
Epoch 5/10:  64%|██████▍   | 234/364 [05:08<03:42,  1.71s/it]
Epoch 5/10:  65%|██████▍   | 235/364 [05:09<02:51,  1.33s/it]
Epoch 5/10:  65%|██████▍   | 236/364 [05:09<02:16,  1.06s/it]
Epoch 5/10:  65%|██████▌   | 237/364 [05:11<02:32,  1.20s/it]
Epoch 5/10:  65%|██████▌   | 238/364 [05:14<03:36,  1.72s/it]
Epoch 5/10:  66%|██████▌   | 239/364 [05:14<02:46,  1.33s/it]
Epoch 5/10:  66%|██████▌   | 240/364 [05:14<02:11,  1.06s/it]
Epoch 5/10:  66%|██████▌   | 241/364 [05:16<02:15,  1.10s/it]
Epoch 5/10:  66%|██████▋   | 242/364 [05:19<03:32,  1.74s/it]
Epoch 5/10:  67%|██████▋   | 243/364 [05:19<02:43,  1.35s/it]
Epoch 5/10:  67%|██████▋   | 244/364 [05:20<02:09,  1.08s/it]
Epoch 5/10:  67%|██████▋   | 245/364 [05:21<02:16,  1.15s/it]
Epoch 5/10:  68%|██████▊   | 246/364 [05:24<03:08,  1.60s/it]
Epoch 5/10:  68%|██████▊   | 247/364 [05:24<02:25,  1.24s/it]
Epoch 5/10:  68%|██████▊   | 248/364 [05:25<01:56,  1.00s/it]
Epoch 5/10:  68%|██████▊   | 249/364 [05:26<02:25,  1.27s/it]
Epoch 5/10:  69%|██████▊   | 250/364 [05:29<02:59,  1.58s/it]
Epoch 5/10:  69%|██████▉   | 251/364 [05:29<02:19,  1.23s/it]
Epoch 5/10:  69%|██████▉   | 252/364 [05:30<01:51,  1.01it/s]
Epoch 5/10:  70%|██████▉   | 253/364 [05:31<02:19,  1.25s/it]
Epoch 5/10:  70%|██████▉   | 254/364 [05:34<03:12,  1.75s/it]
Epoch 5/10:  70%|███████   | 255/364 [05:35<02:27,  1.35s/it]
Epoch 5/10:  70%|███████   | 256/364 [05:35<01:56,  1.08s/it]
Epoch 5/10:  71%|███████   | 257/364 [05:37<02:05,  1.17s/it]
Epoch 5/10:  71%|███████   | 258/364 [05:39<02:54,  1.65s/it]
Epoch 5/10:  71%|███████   | 259/364 [05:40<02:15,  1.29s/it]
Epoch 5/10:  71%|███████▏  | 260/364 [05:40<01:46,  1.03s/it]
Epoch 5/10:  72%|███████▏  | 261/364 [05:42<01:55,  1.12s/it]
Epoch 5/10:  72%|███████▏  | 262/364 [05:45<03:00,  1.77s/it]
Epoch 5/10:  72%|███████▏  | 263/364 [05:45<02:18,  1.37s/it]
Epoch 5/10:  73%|███████▎  | 264/364 [05:46<01:48,  1.08s/it]
Epoch 5/10:  73%|███████▎  | 265/364 [05:47<01:43,  1.05s/it]
Epoch 5/10:  73%|███████▎  | 266/364 [05:50<02:57,  1.81s/it]
Epoch 5/10:  73%|███████▎  | 267/364 [05:51<02:15,  1.39s/it]
Epoch 5/10:  74%|███████▎  | 268/364 [05:51<01:46,  1.10s/it]
Epoch 5/10:  74%|███████▍  | 269/364 [05:52<01:25,  1.10it/s]
Epoch 5/10:  74%|███████▍  | 270/364 [05:55<02:45,  1.76s/it]
Epoch 5/10:  74%|███████▍  | 271/364 [05:56<02:06,  1.36s/it]
Epoch 5/10:  75%|███████▍  | 272/364 [05:56<01:39,  1.08s/it]
Epoch 5/10:  75%|███████▌  | 273/364 [05:57<01:29,  1.02it/s]
Epoch 5/10:  75%|███████▌  | 274/364 [06:01<02:38,  1.76s/it]
Epoch 5/10:  76%|███████▌  | 275/364 [06:01<02:01,  1.36s/it]
Epoch 5/10:  76%|███████▌  | 276/364 [06:01<01:35,  1.08s/it]
Epoch 5/10:  76%|███████▌  | 277/364 [06:02<01:24,  1.03it/s]
Epoch 5/10:  76%|███████▋  | 278/364 [06:06<02:34,  1.80s/it]
Epoch 5/10:  77%|███████▋  | 279/364 [06:06<01:57,  1.38s/it]
Epoch 5/10:  77%|███████▋  | 280/364 [06:07<01:32,  1.10s/it]
Epoch 5/10:  77%|███████▋  | 281/364 [06:07<01:21,  1.02it/s]
Epoch 5/10:  77%|███████▋  | 282/364 [06:11<02:30,  1.83s/it]
Epoch 5/10:  78%|███████▊  | 283/364 [06:12<01:54,  1.41s/it]
Epoch 5/10:  78%|███████▊  | 284/364 [06:12<01:29,  1.11s/it]
Epoch 5/10:  78%|███████▊  | 285/364 [06:13<01:21,  1.03s/it]
Epoch 5/10:  79%|███████▊  | 286/364 [06:16<02:13,  1.72s/it]
Epoch 5/10:  79%|███████▉  | 287/364 [06:17<01:42,  1.33s/it]
Epoch 5/10:  79%|███████▉  | 288/364 [06:17<01:20,  1.06s/it]
Epoch 5/10:  79%|███████▉  | 289/364 [06:18<01:17,  1.04s/it]
Epoch 5/10:  80%|███████▉  | 290/364 [06:21<02:08,  1.74s/it]
Epoch 5/10:  80%|███████▉  | 291/364 [06:22<01:38,  1.35s/it]
Epoch 5/10:  80%|████████  | 292/364 [06:22<01:16,  1.07s/it]
Epoch 5/10:  80%|████████  | 293/364 [06:23<01:11,  1.01s/it]
Epoch 5/10:  81%|████████  | 294/364 [06:27<02:03,  1.77s/it]
Epoch 5/10:  81%|████████  | 295/364 [06:27<01:34,  1.36s/it]
Epoch 5/10:  81%|████████▏ | 296/364 [06:28<01:13,  1.08s/it]
Epoch 5/10:  82%|████████▏ | 297/364 [06:28<01:08,  1.03s/it]
Epoch 5/10:  82%|████████▏ | 298/364 [06:32<01:52,  1.70s/it]
Epoch 5/10:  82%|████████▏ | 299/364 [06:32<01:25,  1.32s/it]
Epoch 5/10:  82%|████████▏ | 300/364 [06:33<01:06,  1.05s/it]
Epoch 5/10:  83%|████████▎ | 301/364 [06:33<01:03,  1.01s/it]
Epoch 5/10:  83%|████████▎ | 302/364 [06:37<01:51,  1.80s/it]
Epoch 5/10:  83%|████████▎ | 303/364 [06:38<01:24,  1.39s/it]
Epoch 5/10:  84%|████████▎ | 304/364 [06:38<01:06,  1.10s/it]
Epoch 5/10:  84%|████████▍ | 305/364 [06:39<01:00,  1.02s/it]
Epoch 5/10:  84%|████████▍ | 306/364 [06:42<01:41,  1.75s/it]
Epoch 5/10:  84%|████████▍ | 307/364 [06:43<01:16,  1.35s/it]
Epoch 5/10:  85%|████████▍ | 308/364 [06:43<00:59,  1.07s/it]
Epoch 5/10:  85%|████████▍ | 309/364 [06:44<00:52,  1.04it/s]
Epoch 5/10:  85%|████████▌ | 310/364 [06:47<01:35,  1.77s/it]
Epoch 5/10:  85%|████████▌ | 311/364 [06:48<01:12,  1.37s/it]
Epoch 5/10:  86%|████████▌ | 312/364 [06:48<00:56,  1.08s/it]
Epoch 5/10:  86%|████████▌ | 313/364 [06:49<00:51,  1.01s/it]
Epoch 5/10:  86%|████████▋ | 314/364 [06:53<01:31,  1.82s/it]
Epoch 5/10:  87%|████████▋ | 315/364 [06:53<01:09,  1.41s/it]
Epoch 5/10:  87%|████████▋ | 316/364 [06:54<00:53,  1.11s/it]
Epoch 5/10:  87%|████████▋ | 317/364 [06:54<00:42,  1.10it/s]
Epoch 5/10:  87%|████████▋ | 318/364 [06:58<01:24,  1.83s/it]
Epoch 5/10:  88%|████████▊ | 319/364 [06:59<01:03,  1.41s/it]
Epoch 5/10:  88%|████████▊ | 320/364 [06:59<00:48,  1.11s/it]
Epoch 5/10:  88%|████████▊ | 321/364 [06:59<00:38,  1.11it/s]
Epoch 5/10:  88%|████████▊ | 322/364 [07:03<01:16,  1.81s/it]
Epoch 5/10:  89%|████████▊ | 323/364 [07:04<00:57,  1.39s/it]
Epoch 5/10:  89%|████████▉ | 324/364 [07:04<00:44,  1.11s/it]
Epoch 5/10:  89%|████████▉ | 325/364 [07:05<00:36,  1.07it/s]
Epoch 5/10:  90%|████████▉ | 326/364 [07:09<01:14,  1.97s/it]
Epoch 5/10:  90%|████████▉ | 327/364 [07:10<00:55,  1.51s/it]
Epoch 5/10:  90%|█████████ | 328/364 [07:10<00:42,  1.18s/it]
Epoch 5/10:  90%|█████████ | 329/364 [07:10<00:33,  1.05it/s]
Epoch 5/10:  91%|█████████ | 330/364 [07:14<01:00,  1.79s/it]
Epoch 5/10:  91%|█████████ | 331/364 [07:15<00:45,  1.38s/it]
Epoch 5/10:  91%|█████████ | 332/364 [07:15<00:35,  1.09s/it]
Epoch 5/10:  91%|█████████▏| 333/364 [07:15<00:27,  1.12it/s]
Epoch 5/10:  92%|█████████▏| 334/364 [07:19<00:55,  1.84s/it]
Epoch 5/10:  92%|█████████▏| 335/364 [07:20<00:40,  1.41s/it]
Epoch 5/10:  92%|█████████▏| 336/364 [07:20<00:31,  1.11s/it]
Epoch 5/10:  93%|█████████▎| 337/364 [07:21<00:24,  1.10it/s]
Epoch 5/10:  93%|█████████▎| 338/364 [07:24<00:43,  1.69s/it]
Epoch 5/10:  93%|█████████▎| 339/364 [07:25<00:32,  1.31s/it]
Epoch 5/10:  93%|█████████▎| 340/364 [07:25<00:25,  1.04s/it]
Epoch 5/10:  94%|█████████▎| 341/364 [07:26<00:21,  1.07it/s]
Epoch 5/10:  94%|█████████▍| 342/364 [07:29<00:36,  1.67s/it]
Epoch 5/10:  94%|█████████▍| 343/364 [07:30<00:27,  1.29s/it]
Epoch 5/10:  95%|█████████▍| 344/364 [07:30<00:20,  1.04s/it]
Epoch 5/10:  95%|█████████▍| 345/364 [07:31<00:20,  1.10s/it]
Epoch 5/10:  95%|█████████▌| 346/364 [07:34<00:29,  1.64s/it]
Epoch 5/10:  95%|█████████▌| 347/364 [07:34<00:21,  1.27s/it]
Epoch 5/10:  96%|█████████▌| 348/364 [07:35<00:16,  1.02s/it]
Epoch 5/10:  96%|█████████▌| 349/364 [07:37<00:18,  1.23s/it]
Epoch 5/10:  96%|█████████▌| 350/364 [07:39<00:22,  1.63s/it]
Epoch 5/10:  96%|█████████▋| 351/364 [07:40<00:16,  1.27s/it]
Epoch 5/10:  97%|█████████▋| 352/364 [07:40<00:12,  1.03s/it]
Epoch 5/10:  97%|█████████▋| 353/364 [07:42<00:14,  1.31s/it]
Epoch 5/10:  97%|█████████▋| 354/364 [07:44<00:15,  1.59s/it]
Epoch 5/10:  98%|█████████▊| 355/364 [07:45<00:11,  1.24s/it]
Epoch 5/10:  98%|█████████▊| 356/364 [07:45<00:07,  1.01it/s]
Epoch 5/10:  98%|█████████▊| 357/364 [07:47<00:09,  1.32s/it]
Epoch 5/10:  98%|█████████▊| 358/364 [07:49<00:09,  1.55s/it]
Epoch 5/10:  99%|█████████▊| 359/364 [07:50<00:06,  1.22s/it]
Epoch 5/10:  99%|█████████▉| 360/364 [07:50<00:03,  1.02it/s]
Epoch 5/10:  99%|█████████▉| 361/364 [07:52<00:03,  1.15s/it]
Epoch 5/10:  99%|█████████▉| 362/364 [07:53<00:02,  1.07s/it]
Epoch 5/10: 100%|█████████▉| 363/364 [07:53<00:00,  1.15it/s]
Epoch 5/10: 100%|██████████| 364/364 [07:53<00:00,  1.50it/s]
Epoch 5/10: 100%|██████████| 364/364 [07:53<00:00,  1.30s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:05<05:01,  5.39s/it]
Validating:   4%|▎         | 2/57 [00:05<02:07,  2.31s/it]
Validating:   5%|▌         | 3/57 [00:05<01:11,  1.33s/it]
Validating:   7%|▋         | 4/57 [00:05<00:45,  1.16it/s]
Validating:   9%|▉         | 5/57 [00:09<01:44,  2.01s/it]
Validating:  11%|█         | 6/57 [00:10<01:10,  1.38s/it]
Validating:  12%|█▏        | 7/57 [00:10<00:48,  1.02it/s]
Validating:  14%|█▍        | 8/57 [00:10<00:34,  1.41it/s]
Validating:  16%|█▌        | 9/57 [00:14<01:22,  1.71s/it]
Validating:  18%|█▊        | 10/57 [00:14<00:57,  1.23s/it]
Validating:  19%|█▉        | 11/57 [00:14<00:41,  1.12it/s]
Validating:  21%|██        | 12/57 [00:14<00:29,  1.51it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:17,  1.76s/it]
Validating:  25%|██▍       | 14/57 [00:19<00:54,  1.28s/it]
Validating:  26%|██▋       | 15/57 [00:19<00:39,  1.08it/s]
Validating:  28%|██▊       | 16/57 [00:19<00:28,  1.44it/s]
Validating:  30%|██▉       | 17/57 [00:23<01:08,  1.70s/it]
Validating:  32%|███▏      | 18/57 [00:23<00:48,  1.24s/it]
Validating:  33%|███▎      | 19/57 [00:23<00:34,  1.09it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:25,  1.48it/s]
Validating:  37%|███▋      | 21/57 [00:27<01:00,  1.69s/it]
Validating:  39%|███▊      | 22/57 [00:28<00:42,  1.23s/it]
Validating:  40%|████      | 23/57 [00:28<00:30,  1.12it/s]
Validating:  42%|████▏     | 24/57 [00:28<00:21,  1.51it/s]
Validating:  44%|████▍     | 25/57 [00:32<00:56,  1.77s/it]
Validating:  46%|████▌     | 26/57 [00:32<00:39,  1.29s/it]
Validating:  47%|████▋     | 27/57 [00:32<00:28,  1.06it/s]
Validating:  49%|████▉     | 28/57 [00:33<00:20,  1.44it/s]
Validating:  51%|█████     | 29/57 [00:37<00:47,  1.70s/it]
Validating:  53%|█████▎    | 30/57 [00:37<00:33,  1.24s/it]
Validating:  54%|█████▍    | 31/57 [00:37<00:23,  1.11it/s]
Validating:  56%|█████▌    | 32/57 [00:37<00:16,  1.49it/s]
Validating:  58%|█████▊    | 33/57 [00:41<00:38,  1.62s/it]
Validating:  60%|█████▉    | 34/57 [00:41<00:27,  1.18s/it]
Validating:  61%|██████▏   | 35/57 [00:41<00:19,  1.15it/s]
Validating:  63%|██████▎   | 36/57 [00:41<00:13,  1.53it/s]
Validating:  65%|██████▍   | 37/57 [00:46<00:35,  1.77s/it]
Validating:  67%|██████▋   | 38/57 [00:46<00:24,  1.29s/it]
Validating:  68%|██████▊   | 39/57 [00:46<00:17,  1.05it/s]
Validating:  70%|███████   | 40/57 [00:46<00:12,  1.41it/s]
Validating:  72%|███████▏  | 41/57 [00:50<00:28,  1.78s/it]
Validating:  74%|███████▎  | 42/57 [00:51<00:19,  1.29s/it]
Validating:  75%|███████▌  | 43/57 [00:51<00:13,  1.06it/s]
Validating:  77%|███████▋  | 44/57 [00:51<00:09,  1.43it/s]
Validating:  79%|███████▉  | 45/57 [00:55<00:20,  1.74s/it]
Validating:  81%|████████  | 46/57 [00:55<00:13,  1.27s/it]
Validating:  82%|████████▏ | 47/57 [00:55<00:09,  1.07it/s]
Validating:  84%|████████▍ | 48/57 [00:55<00:06,  1.45it/s]
Validating:  86%|████████▌ | 49/57 [00:59<00:12,  1.59s/it]
Validating:  88%|████████▊ | 50/57 [00:59<00:08,  1.17s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:05,  1.17it/s]
Validating:  91%|█████████ | 52/57 [01:00<00:03,  1.56it/s]
Validating:  93%|█████████▎| 53/57 [01:03<00:05,  1.49s/it]
Validating:  95%|█████████▍| 54/57 [01:03<00:03,  1.09s/it]
Validating:  96%|█████████▋| 55/57 [01:03<00:01,  1.26it/s]
Validating:  98%|█████████▊| 56/57 [01:03<00:00,  1.69it/s]
Validating: 100%|██████████| 57/57 [01:04<00:00,  1.12s/it]
Epoch 5: Train Loss: 0.1401, Train Acc: 0.9509 | Val Loss: 0.3557, Val Acc: 0.8883

Epoch 6/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 6/10:   0%|          | 1/364 [00:06<36:30,  6.03s/it]
Epoch 6/10:   1%|          | 2/364 [00:06<16:26,  2.72s/it]
Epoch 6/10:   1%|          | 3/364 [00:06<10:05,  1.68s/it]
Epoch 6/10:   1%|          | 4/364 [00:07<07:05,  1.18s/it]
Epoch 6/10:   1%|▏         | 5/364 [00:11<13:34,  2.27s/it]
Epoch 6/10:   2%|▏         | 6/364 [00:11<09:44,  1.63s/it]
Epoch 6/10:   2%|▏         | 7/364 [00:12<07:21,  1.24s/it]
Epoch 6/10:   2%|▏         | 8/364 [00:12<05:49,  1.02it/s]
Epoch 6/10:   2%|▏         | 9/364 [00:16<11:29,  1.94s/it]
Epoch 6/10:   3%|▎         | 10/364 [00:17<08:39,  1.47s/it]
Epoch 6/10:   3%|▎         | 11/364 [00:17<06:46,  1.15s/it]
Epoch 6/10:   3%|▎         | 12/364 [00:18<05:26,  1.08it/s]
Epoch 6/10:   4%|▎         | 13/364 [00:22<10:53,  1.86s/it]
Epoch 6/10:   4%|▍         | 14/364 [00:22<08:19,  1.43s/it]
Epoch 6/10:   4%|▍         | 15/364 [00:22<06:31,  1.12s/it]
Epoch 6/10:   4%|▍         | 16/364 [00:23<05:16,  1.10it/s]
Epoch 6/10:   5%|▍         | 17/364 [00:27<10:47,  1.87s/it]
Epoch 6/10:   5%|▍         | 18/364 [00:27<08:16,  1.44s/it]
Epoch 6/10:   5%|▌         | 19/364 [00:28<06:31,  1.13s/it]
Epoch 6/10:   5%|▌         | 20/364 [00:28<05:14,  1.09it/s]
Epoch 6/10:   6%|▌         | 21/364 [00:32<10:52,  1.90s/it]
Epoch 6/10:   6%|▌         | 22/364 [00:33<08:17,  1.45s/it]
Epoch 6/10:   6%|▋         | 23/364 [00:33<06:29,  1.14s/it]
Epoch 6/10:   7%|▋         | 24/364 [00:34<05:15,  1.08it/s]
Epoch 6/10:   7%|▋         | 25/364 [00:38<10:45,  1.91s/it]
Epoch 6/10:   7%|▋         | 26/364 [00:38<08:14,  1.46s/it]
Epoch 6/10:   7%|▋         | 27/364 [00:39<06:27,  1.15s/it]
Epoch 6/10:   8%|▊         | 28/364 [00:39<05:13,  1.07it/s]
Epoch 6/10:   8%|▊         | 29/364 [00:43<09:56,  1.78s/it]
Epoch 6/10:   8%|▊         | 30/364 [00:43<07:41,  1.38s/it]
Epoch 6/10:   9%|▊         | 31/364 [00:44<06:03,  1.09s/it]
Epoch 6/10:   9%|▉         | 32/364 [00:44<04:56,  1.12it/s]
Epoch 6/10:   9%|▉         | 33/364 [00:48<09:50,  1.78s/it]
Epoch 6/10:   9%|▉         | 34/364 [00:48<07:33,  1.38s/it]
Epoch 6/10:  10%|▉         | 35/364 [00:49<05:57,  1.09s/it]
Epoch 6/10:  10%|▉         | 36/364 [00:49<04:49,  1.13it/s]
Epoch 6/10:  10%|█         | 37/364 [00:53<10:11,  1.87s/it]
Epoch 6/10:  10%|█         | 38/364 [00:54<07:48,  1.44s/it]
Epoch 6/10:  11%|█         | 39/364 [00:54<06:07,  1.13s/it]
Epoch 6/10:  11%|█         | 40/364 [00:55<04:57,  1.09it/s]
Epoch 6/10:  11%|█▏        | 41/364 [00:58<09:33,  1.77s/it]
Epoch 6/10:  12%|█▏        | 42/364 [00:59<07:21,  1.37s/it]
Epoch 6/10:  12%|█▏        | 43/364 [00:59<05:47,  1.08s/it]
Epoch 6/10:  12%|█▏        | 44/364 [01:00<04:42,  1.13it/s]
Epoch 6/10:  12%|█▏        | 45/364 [01:04<09:52,  1.86s/it]
Epoch 6/10:  13%|█▎        | 46/364 [01:04<07:34,  1.43s/it]
Epoch 6/10:  13%|█▎        | 47/364 [01:05<05:56,  1.12s/it]
Epoch 6/10:  13%|█▎        | 48/364 [01:05<04:48,  1.10it/s]
Epoch 6/10:  13%|█▎        | 49/364 [01:09<09:36,  1.83s/it]
Epoch 6/10:  14%|█▎        | 50/364 [01:10<07:23,  1.41s/it]
Epoch 6/10:  14%|█▍        | 51/364 [01:10<05:51,  1.12s/it]
Epoch 6/10:  14%|█▍        | 52/364 [01:10<04:43,  1.10it/s]
Epoch 6/10:  15%|█▍        | 53/364 [01:15<09:52,  1.90s/it]
Epoch 6/10:  15%|█▍        | 54/364 [01:15<07:31,  1.46s/it]
Epoch 6/10:  15%|█▌        | 55/364 [01:15<05:54,  1.15s/it]
Epoch 6/10:  15%|█▌        | 56/364 [01:16<04:45,  1.08it/s]
Epoch 6/10:  16%|█▌        | 57/364 [01:20<09:03,  1.77s/it]
Epoch 6/10:  16%|█▌        | 58/364 [01:20<06:59,  1.37s/it]
Epoch 6/10:  16%|█▌        | 59/364 [01:20<05:30,  1.08s/it]
Epoch 6/10:  16%|█▋        | 60/364 [01:21<04:31,  1.12it/s]
Epoch 6/10:  17%|█▋        | 61/364 [01:25<09:03,  1.79s/it]
Epoch 6/10:  17%|█▋        | 62/364 [01:25<06:56,  1.38s/it]
Epoch 6/10:  17%|█▋        | 63/364 [01:26<05:31,  1.10s/it]
Epoch 6/10:  18%|█▊        | 64/364 [01:26<04:28,  1.12it/s]
Epoch 6/10:  18%|█▊        | 65/364 [01:30<08:43,  1.75s/it]
Epoch 6/10:  18%|█▊        | 66/364 [01:30<06:42,  1.35s/it]
Epoch 6/10:  18%|█▊        | 67/364 [01:31<05:19,  1.07s/it]
Epoch 6/10:  19%|█▊        | 68/364 [01:31<04:18,  1.14it/s]
Epoch 6/10:  19%|█▉        | 69/364 [01:35<08:54,  1.81s/it]
Epoch 6/10:  19%|█▉        | 70/364 [01:35<06:48,  1.39s/it]
Epoch 6/10:  20%|█▉        | 71/364 [01:36<05:20,  1.09s/it]
Epoch 6/10:  20%|█▉        | 72/364 [01:36<04:21,  1.12it/s]
Epoch 6/10:  20%|██        | 73/364 [01:41<09:29,  1.96s/it]
Epoch 6/10:  20%|██        | 74/364 [01:41<07:13,  1.50s/it]
Epoch 6/10:  21%|██        | 75/364 [01:42<05:39,  1.17s/it]
Epoch 6/10:  21%|██        | 76/364 [01:42<04:34,  1.05it/s]
Epoch 6/10:  21%|██        | 77/364 [01:46<08:59,  1.88s/it]
Epoch 6/10:  21%|██▏       | 78/364 [01:46<06:51,  1.44s/it]
Epoch 6/10:  22%|██▏       | 79/364 [01:47<05:23,  1.14s/it]
Epoch 6/10:  22%|██▏       | 80/364 [01:47<04:21,  1.09it/s]
Epoch 6/10:  22%|██▏       | 81/364 [01:51<07:59,  1.69s/it]
Epoch 6/10:  23%|██▎       | 82/364 [01:51<06:09,  1.31s/it]
Epoch 6/10:  23%|██▎       | 83/364 [01:52<04:53,  1.04s/it]
Epoch 6/10:  23%|██▎       | 84/364 [01:52<04:00,  1.16it/s]
Epoch 6/10:  23%|██▎       | 85/364 [01:56<08:48,  1.89s/it]
Epoch 6/10:  24%|██▎       | 86/364 [01:57<06:43,  1.45s/it]
Epoch 6/10:  24%|██▍       | 87/364 [01:57<05:17,  1.14s/it]
Epoch 6/10:  24%|██▍       | 88/364 [01:58<04:16,  1.07it/s]
Epoch 6/10:  24%|██▍       | 89/364 [02:01<07:58,  1.74s/it]
Epoch 6/10:  25%|██▍       | 90/364 [02:02<06:07,  1.34s/it]
Epoch 6/10:  25%|██▌       | 91/364 [02:02<04:51,  1.07s/it]
Epoch 6/10:  25%|██▌       | 92/364 [02:03<03:57,  1.15it/s]
Epoch 6/10:  26%|██▌       | 93/364 [02:06<07:59,  1.77s/it]
Epoch 6/10:  26%|██▌       | 94/364 [02:07<06:10,  1.37s/it]
Epoch 6/10:  26%|██▌       | 95/364 [02:07<04:53,  1.09s/it]
Epoch 6/10:  26%|██▋       | 96/364 [02:08<03:58,  1.13it/s]
Epoch 6/10:  27%|██▋       | 97/364 [02:12<07:50,  1.76s/it]
Epoch 6/10:  27%|██▋       | 98/364 [02:12<06:02,  1.36s/it]
Epoch 6/10:  27%|██▋       | 99/364 [02:12<04:46,  1.08s/it]
Epoch 6/10:  27%|██▋       | 100/364 [02:13<03:53,  1.13it/s]
Epoch 6/10:  28%|██▊       | 101/364 [02:17<07:53,  1.80s/it]
Epoch 6/10:  28%|██▊       | 102/364 [02:17<06:02,  1.38s/it]
Epoch 6/10:  28%|██▊       | 103/364 [02:18<04:48,  1.10s/it]
Epoch 6/10:  29%|██▊       | 104/364 [02:18<03:53,  1.11it/s]
Epoch 6/10:  29%|██▉       | 105/364 [02:22<07:47,  1.81s/it]
Epoch 6/10:  29%|██▉       | 106/364 [02:22<05:58,  1.39s/it]
Epoch 6/10:  29%|██▉       | 107/364 [02:23<04:40,  1.09s/it]
Epoch 6/10:  30%|██▉       | 108/364 [02:23<03:48,  1.12it/s]
Epoch 6/10:  30%|██▉       | 109/364 [02:27<07:40,  1.80s/it]
Epoch 6/10:  30%|███       | 110/364 [02:28<05:54,  1.39s/it]
Epoch 6/10:  30%|███       | 111/364 [02:28<04:38,  1.10s/it]
Epoch 6/10:  31%|███       | 112/364 [02:28<03:45,  1.12it/s]
Epoch 6/10:  31%|███       | 113/364 [02:32<07:45,  1.86s/it]
Epoch 6/10:  31%|███▏      | 114/364 [02:33<05:56,  1.43s/it]
Epoch 6/10:  32%|███▏      | 115/364 [02:33<04:40,  1.13s/it]
Epoch 6/10:  32%|███▏      | 116/364 [02:34<03:47,  1.09it/s]
Epoch 6/10:  32%|███▏      | 117/364 [02:38<08:11,  1.99s/it]
Epoch 6/10:  32%|███▏      | 118/364 [02:39<06:13,  1.52s/it]
Epoch 6/10:  33%|███▎      | 119/364 [02:39<04:52,  1.19s/it]
Epoch 6/10:  33%|███▎      | 120/364 [02:40<03:54,  1.04it/s]
Epoch 6/10:  33%|███▎      | 121/364 [02:44<07:44,  1.91s/it]
Epoch 6/10:  34%|███▎      | 122/364 [02:44<05:52,  1.46s/it]
Epoch 6/10:  34%|███▍      | 123/364 [02:44<04:36,  1.15s/it]
Epoch 6/10:  34%|███▍      | 124/364 [02:45<03:42,  1.08it/s]
Epoch 6/10:  34%|███▍      | 125/364 [02:49<07:34,  1.90s/it]
Epoch 6/10:  35%|███▍      | 126/364 [02:49<05:46,  1.46s/it]
Epoch 6/10:  35%|███▍      | 127/364 [02:50<04:32,  1.15s/it]
Epoch 6/10:  35%|███▌      | 128/364 [02:50<03:39,  1.08it/s]
Epoch 6/10:  35%|███▌      | 129/364 [02:54<07:01,  1.80s/it]
Epoch 6/10:  36%|███▌      | 130/364 [02:55<05:23,  1.38s/it]
Epoch 6/10:  36%|███▌      | 131/364 [02:55<04:14,  1.09s/it]
Epoch 6/10:  36%|███▋      | 132/364 [02:55<03:25,  1.13it/s]
Epoch 6/10:  37%|███▋      | 133/364 [02:59<06:14,  1.62s/it]
Epoch 6/10:  37%|███▋      | 134/364 [02:59<04:50,  1.26s/it]
Epoch 6/10:  37%|███▋      | 135/364 [03:00<03:51,  1.01s/it]
Epoch 6/10:  37%|███▋      | 136/364 [03:00<03:10,  1.20it/s]
Epoch 6/10:  38%|███▊      | 137/364 [03:04<06:58,  1.85s/it]
Epoch 6/10:  38%|███▊      | 138/364 [03:05<05:21,  1.42s/it]
Epoch 6/10:  38%|███▊      | 139/364 [03:05<04:12,  1.12s/it]
Epoch 6/10:  38%|███▊      | 140/364 [03:05<03:25,  1.09it/s]
Epoch 6/10:  39%|███▊      | 141/364 [03:09<06:45,  1.82s/it]
Epoch 6/10:  39%|███▉      | 142/364 [03:10<05:10,  1.40s/it]
Epoch 6/10:  39%|███▉      | 143/364 [03:10<04:04,  1.11s/it]
Epoch 6/10:  40%|███▉      | 144/364 [03:11<03:18,  1.11it/s]
Epoch 6/10:  40%|███▉      | 145/364 [03:14<06:18,  1.73s/it]
Epoch 6/10:  40%|████      | 146/364 [03:15<04:51,  1.34s/it]
Epoch 6/10:  40%|████      | 147/364 [03:15<03:49,  1.06s/it]
Epoch 6/10:  41%|████      | 148/364 [03:16<03:08,  1.14it/s]
Epoch 6/10:  41%|████      | 149/364 [03:19<06:21,  1.77s/it]
Epoch 6/10:  41%|████      | 150/364 [03:20<04:53,  1.37s/it]
Epoch 6/10:  41%|████▏     | 151/364 [03:20<03:51,  1.09s/it]
Epoch 6/10:  42%|████▏     | 152/364 [03:21<03:08,  1.12it/s]
Epoch 6/10:  42%|████▏     | 153/364 [03:24<06:07,  1.74s/it]
Epoch 6/10:  42%|████▏     | 154/364 [03:25<04:42,  1.34s/it]
Epoch 6/10:  43%|████▎     | 155/364 [03:25<03:43,  1.07s/it]
Epoch 6/10:  43%|████▎     | 156/364 [03:26<03:02,  1.14it/s]
Epoch 6/10:  43%|████▎     | 157/364 [03:29<05:50,  1.69s/it]
Epoch 6/10:  43%|████▎     | 158/364 [03:30<04:30,  1.31s/it]
Epoch 6/10:  44%|████▎     | 159/364 [03:30<03:35,  1.05s/it]
Epoch 6/10:  44%|████▍     | 160/364 [03:31<02:56,  1.16it/s]
Epoch 6/10:  44%|████▍     | 161/364 [03:34<05:35,  1.65s/it]
Epoch 6/10:  45%|████▍     | 162/364 [03:35<04:19,  1.28s/it]
Epoch 6/10:  45%|████▍     | 163/364 [03:35<03:26,  1.03s/it]
Epoch 6/10:  45%|████▌     | 164/364 [03:35<02:49,  1.18it/s]
Epoch 6/10:  45%|████▌     | 165/364 [03:40<06:02,  1.82s/it]
Epoch 6/10:  46%|████▌     | 166/364 [03:40<04:38,  1.40s/it]
Epoch 6/10:  46%|████▌     | 167/364 [03:40<03:38,  1.11s/it]
Epoch 6/10:  46%|████▌     | 168/364 [03:41<02:56,  1.11it/s]
Epoch 6/10:  46%|████▋     | 169/364 [03:45<05:46,  1.78s/it]
Epoch 6/10:  47%|████▋     | 170/364 [03:45<04:26,  1.37s/it]
Epoch 6/10:  47%|████▋     | 171/364 [03:45<03:30,  1.09s/it]
Epoch 6/10:  47%|████▋     | 172/364 [03:46<02:50,  1.12it/s]
Epoch 6/10:  48%|████▊     | 173/364 [03:50<06:06,  1.92s/it]
Epoch 6/10:  48%|████▊     | 174/364 [03:51<04:39,  1.47s/it]
Epoch 6/10:  48%|████▊     | 175/364 [03:51<03:38,  1.16s/it]
Epoch 6/10:  48%|████▊     | 176/364 [03:51<02:55,  1.07it/s]
Epoch 6/10:  49%|████▊     | 177/364 [03:55<05:39,  1.82s/it]
Epoch 6/10:  49%|████▉     | 178/364 [03:56<04:20,  1.40s/it]
Epoch 6/10:  49%|████▉     | 179/364 [03:56<03:24,  1.11s/it]
Epoch 6/10:  49%|████▉     | 180/364 [03:57<02:47,  1.10it/s]
Epoch 6/10:  50%|████▉     | 181/364 [04:01<05:36,  1.84s/it]
Epoch 6/10:  50%|█████     | 182/364 [04:01<04:17,  1.42s/it]
Epoch 6/10:  50%|█████     | 183/364 [04:02<03:23,  1.13s/it]
Epoch 6/10:  51%|█████     | 184/364 [04:02<02:45,  1.09it/s]
Epoch 6/10:  51%|█████     | 185/364 [04:06<05:36,  1.88s/it]
Epoch 6/10:  51%|█████     | 186/364 [04:07<04:17,  1.45s/it]
Epoch 6/10:  51%|█████▏    | 187/364 [04:07<03:21,  1.14s/it]
Epoch 6/10:  52%|█████▏    | 188/364 [04:07<02:41,  1.09it/s]
Epoch 6/10:  52%|█████▏    | 189/364 [04:11<05:25,  1.86s/it]
Epoch 6/10:  52%|█████▏    | 190/364 [04:12<04:08,  1.43s/it]
Epoch 6/10:  52%|█████▏    | 191/364 [04:12<03:15,  1.13s/it]
Epoch 6/10:  53%|█████▎    | 192/364 [04:13<02:37,  1.09it/s]
Epoch 6/10:  53%|█████▎    | 193/364 [04:16<05:02,  1.77s/it]
Epoch 6/10:  53%|█████▎    | 194/364 [04:17<03:52,  1.37s/it]
Epoch 6/10:  54%|█████▎    | 195/364 [04:17<03:03,  1.08s/it]
Epoch 6/10:  54%|█████▍    | 196/364 [04:18<02:30,  1.12it/s]
Epoch 6/10:  54%|█████▍    | 197/364 [04:22<05:15,  1.89s/it]
Epoch 6/10:  54%|█████▍    | 198/364 [04:22<04:00,  1.45s/it]
Epoch 6/10:  55%|█████▍    | 199/364 [04:23<03:08,  1.14s/it]
Epoch 6/10:  55%|█████▍    | 200/364 [04:23<02:32,  1.08it/s]
Epoch 6/10:  55%|█████▌    | 201/364 [04:27<04:41,  1.73s/it]
Epoch 6/10:  55%|█████▌    | 202/364 [04:27<03:36,  1.34s/it]
Epoch 6/10:  56%|█████▌    | 203/364 [04:28<02:51,  1.07s/it]
Epoch 6/10:  56%|█████▌    | 204/364 [04:28<02:20,  1.14it/s]
Epoch 6/10:  56%|█████▋    | 205/364 [04:32<04:47,  1.81s/it]
Epoch 6/10:  57%|█████▋    | 206/364 [04:33<03:39,  1.39s/it]
Epoch 6/10:  57%|█████▋    | 207/364 [04:33<02:53,  1.10s/it]
Epoch 6/10:  57%|█████▋    | 208/364 [04:33<02:21,  1.11it/s]
Epoch 6/10:  57%|█████▋    | 209/364 [04:37<04:21,  1.69s/it]
Epoch 6/10:  58%|█████▊    | 210/364 [04:37<03:20,  1.30s/it]
Epoch 6/10:  58%|█████▊    | 211/364 [04:38<02:38,  1.04s/it]
Epoch 6/10:  58%|█████▊    | 212/364 [04:38<02:09,  1.17it/s]
Epoch 6/10:  59%|█████▊    | 213/364 [04:43<04:47,  1.90s/it]
Epoch 6/10:  59%|█████▉    | 214/364 [04:43<03:40,  1.47s/it]
Epoch 6/10:  59%|█████▉    | 215/364 [04:43<02:52,  1.16s/it]
Epoch 6/10:  59%|█████▉    | 216/364 [04:44<02:19,  1.06it/s]
Epoch 6/10:  60%|█████▉    | 217/364 [04:48<04:28,  1.83s/it]
Epoch 6/10:  60%|█████▉    | 218/364 [04:48<03:26,  1.41s/it]
Epoch 6/10:  60%|██████    | 219/364 [04:49<02:42,  1.12s/it]
Epoch 6/10:  60%|██████    | 220/364 [04:49<02:11,  1.10it/s]
Epoch 6/10:  61%|██████    | 221/364 [04:53<04:17,  1.80s/it]
Epoch 6/10:  61%|██████    | 222/364 [04:53<03:17,  1.39s/it]
Epoch 6/10:  61%|██████▏   | 223/364 [04:54<02:35,  1.10s/it]
Epoch 6/10:  62%|██████▏   | 224/364 [04:54<02:05,  1.12it/s]
Epoch 6/10:  62%|██████▏   | 225/364 [04:58<04:05,  1.76s/it]
Epoch 6/10:  62%|██████▏   | 226/364 [04:58<03:08,  1.36s/it]
Epoch 6/10:  62%|██████▏   | 227/364 [04:59<02:28,  1.08s/it]
Epoch 6/10:  63%|██████▎   | 228/364 [04:59<02:00,  1.13it/s]
Epoch 6/10:  63%|██████▎   | 229/364 [05:03<04:13,  1.88s/it]
Epoch 6/10:  63%|██████▎   | 230/364 [05:04<03:13,  1.45s/it]
Epoch 6/10:  63%|██████▎   | 231/364 [05:04<02:32,  1.15s/it]
Epoch 6/10:  64%|██████▎   | 232/364 [05:05<02:02,  1.08it/s]
Epoch 6/10:  64%|██████▍   | 233/364 [05:09<03:58,  1.82s/it]
Epoch 6/10:  64%|██████▍   | 234/364 [05:09<03:03,  1.41s/it]
Epoch 6/10:  65%|██████▍   | 235/364 [05:10<02:38,  1.23s/it]
Epoch 6/10:  65%|██████▍   | 236/364 [05:10<02:06,  1.01it/s]
Epoch 6/10:  65%|██████▌   | 237/364 [05:14<03:36,  1.70s/it]
Epoch 6/10:  65%|██████▌   | 238/364 [05:14<02:46,  1.32s/it]
Epoch 6/10:  66%|██████▌   | 239/364 [05:15<02:11,  1.05s/it]
Epoch 6/10:  66%|██████▌   | 240/364 [05:15<01:47,  1.16it/s]
Epoch 6/10:  66%|██████▌   | 241/364 [05:19<03:30,  1.71s/it]
Epoch 6/10:  66%|██████▋   | 242/364 [05:19<02:42,  1.33s/it]
Epoch 6/10:  67%|██████▋   | 243/364 [05:20<02:08,  1.06s/it]
Epoch 6/10:  67%|██████▋   | 244/364 [05:20<01:44,  1.15it/s]
Epoch 6/10:  67%|██████▋   | 245/364 [05:24<03:26,  1.73s/it]
Epoch 6/10:  68%|██████▊   | 246/364 [05:24<02:38,  1.35s/it]
Epoch 6/10:  68%|██████▊   | 247/364 [05:25<02:05,  1.07s/it]
Epoch 6/10:  68%|██████▊   | 248/364 [05:25<01:41,  1.14it/s]
Epoch 6/10:  68%|██████▊   | 249/364 [05:29<03:27,  1.81s/it]
Epoch 6/10:  69%|██████▊   | 250/364 [05:29<02:38,  1.39s/it]
Epoch 6/10:  69%|██████▉   | 251/364 [05:30<02:04,  1.10s/it]
Epoch 6/10:  69%|██████▉   | 252/364 [05:30<01:39,  1.12it/s]
Epoch 6/10:  70%|██████▉   | 253/364 [05:34<03:13,  1.75s/it]
Epoch 6/10:  70%|██████▉   | 254/364 [05:34<02:28,  1.35s/it]
Epoch 6/10:  70%|███████   | 255/364 [05:35<01:57,  1.08s/it]
Epoch 6/10:  70%|███████   | 256/364 [05:35<01:35,  1.13it/s]
Epoch 6/10:  71%|███████   | 257/364 [05:39<03:15,  1.82s/it]
Epoch 6/10:  71%|███████   | 258/364 [05:40<02:28,  1.40s/it]
Epoch 6/10:  71%|███████   | 259/364 [05:40<01:56,  1.11s/it]
Epoch 6/10:  71%|███████▏  | 260/364 [05:41<01:33,  1.11it/s]
Epoch 6/10:  72%|███████▏  | 261/364 [05:45<03:12,  1.87s/it]
Epoch 6/10:  72%|███████▏  | 262/364 [05:45<02:26,  1.44s/it]
Epoch 6/10:  72%|███████▏  | 263/364 [05:46<01:54,  1.14s/it]
Epoch 6/10:  73%|███████▎  | 264/364 [05:46<01:32,  1.08it/s]
Epoch 6/10:  73%|███████▎  | 265/364 [05:50<02:55,  1.77s/it]
Epoch 6/10:  73%|███████▎  | 266/364 [05:50<02:14,  1.37s/it]
Epoch 6/10:  73%|███████▎  | 267/364 [05:51<01:45,  1.09s/it]
Epoch 6/10:  74%|███████▎  | 268/364 [05:51<01:25,  1.12it/s]
Epoch 6/10:  74%|███████▍  | 269/364 [05:55<02:44,  1.73s/it]
Epoch 6/10:  74%|███████▍  | 270/364 [05:55<02:06,  1.34s/it]
Epoch 6/10:  74%|███████▍  | 271/364 [05:56<01:39,  1.07s/it]
Epoch 6/10:  75%|███████▍  | 272/364 [05:56<01:21,  1.13it/s]
Epoch 6/10:  75%|███████▌  | 273/364 [06:00<02:36,  1.72s/it]
Epoch 6/10:  75%|███████▌  | 274/364 [06:00<01:59,  1.33s/it]
Epoch 6/10:  76%|███████▌  | 275/364 [06:01<01:34,  1.07s/it]
Epoch 6/10:  76%|███████▌  | 276/364 [06:01<01:16,  1.15it/s]
Epoch 6/10:  76%|███████▌  | 277/364 [06:05<02:27,  1.70s/it]
Epoch 6/10:  76%|███████▋  | 278/364 [06:05<01:53,  1.32s/it]
Epoch 6/10:  77%|███████▋  | 279/364 [06:05<01:29,  1.05s/it]
Epoch 6/10:  77%|███████▋  | 280/364 [06:06<01:12,  1.16it/s]
Epoch 6/10:  77%|███████▋  | 281/364 [06:10<02:23,  1.73s/it]
Epoch 6/10:  77%|███████▋  | 282/364 [06:10<01:50,  1.34s/it]
Epoch 6/10:  78%|███████▊  | 283/364 [06:11<01:26,  1.07s/it]
Epoch 6/10:  78%|███████▊  | 284/364 [06:11<01:10,  1.13it/s]
Epoch 6/10:  78%|███████▊  | 285/364 [06:15<02:25,  1.84s/it]
Epoch 6/10:  79%|███████▊  | 286/364 [06:15<01:49,  1.41s/it]
Epoch 6/10:  79%|███████▉  | 287/364 [06:16<01:25,  1.11s/it]
Epoch 6/10:  79%|███████▉  | 288/364 [06:16<01:08,  1.10it/s]
Epoch 6/10:  79%|███████▉  | 289/364 [06:20<02:16,  1.81s/it]
Epoch 6/10:  80%|███████▉  | 290/364 [06:21<01:43,  1.40s/it]
Epoch 6/10:  80%|███████▉  | 291/364 [06:21<01:20,  1.11s/it]
Epoch 6/10:  80%|████████  | 292/364 [06:22<01:05,  1.10it/s]
Epoch 6/10:  80%|████████  | 293/364 [06:26<02:12,  1.87s/it]
Epoch 6/10:  81%|████████  | 294/364 [06:26<01:40,  1.43s/it]
Epoch 6/10:  81%|████████  | 295/364 [06:26<01:17,  1.13s/it]
Epoch 6/10:  81%|████████▏ | 296/364 [06:27<01:02,  1.09it/s]
Epoch 6/10:  82%|████████▏ | 297/364 [06:31<01:57,  1.75s/it]
Epoch 6/10:  82%|████████▏ | 298/364 [06:31<01:29,  1.35s/it]
Epoch 6/10:  82%|████████▏ | 299/364 [06:31<01:09,  1.07s/it]
Epoch 6/10:  82%|████████▏ | 300/364 [06:32<00:56,  1.14it/s]
Epoch 6/10:  83%|████████▎ | 301/364 [06:36<01:53,  1.81s/it]
Epoch 6/10:  83%|████████▎ | 302/364 [06:36<01:26,  1.39s/it]
Epoch 6/10:  83%|████████▎ | 303/364 [06:37<01:07,  1.11s/it]
Epoch 6/10:  84%|████████▎ | 304/364 [06:37<00:54,  1.10it/s]
Epoch 6/10:  84%|████████▍ | 305/364 [06:41<01:49,  1.85s/it]
Epoch 6/10:  84%|████████▍ | 306/364 [06:42<01:22,  1.42s/it]
Epoch 6/10:  84%|████████▍ | 307/364 [06:42<01:04,  1.13s/it]
Epoch 6/10:  85%|████████▍ | 308/364 [06:42<00:51,  1.09it/s]
Epoch 6/10:  85%|████████▍ | 309/364 [06:46<01:31,  1.66s/it]
Epoch 6/10:  85%|████████▌ | 310/364 [06:46<01:09,  1.29s/it]
Epoch 6/10:  85%|████████▌ | 311/364 [06:47<00:54,  1.03s/it]
Epoch 6/10:  86%|████████▌ | 312/364 [06:47<00:44,  1.18it/s]
Epoch 6/10:  86%|████████▌ | 313/364 [06:51<01:26,  1.70s/it]
Epoch 6/10:  86%|████████▋ | 314/364 [06:51<01:06,  1.32s/it]
Epoch 6/10:  87%|████████▋ | 315/364 [06:52<00:51,  1.05s/it]
Epoch 6/10:  87%|████████▋ | 316/364 [06:52<00:41,  1.16it/s]
Epoch 6/10:  87%|████████▋ | 317/364 [06:55<01:13,  1.57s/it]
Epoch 6/10:  87%|████████▋ | 318/364 [06:56<00:56,  1.23s/it]
Epoch 6/10:  88%|████████▊ | 319/364 [06:56<00:44,  1.01it/s]
Epoch 6/10:  88%|████████▊ | 320/364 [06:57<00:36,  1.22it/s]
Epoch 6/10:  88%|████████▊ | 321/364 [07:00<01:13,  1.70s/it]
Epoch 6/10:  88%|████████▊ | 322/364 [07:01<00:55,  1.32s/it]
Epoch 6/10:  89%|████████▊ | 323/364 [07:01<00:43,  1.05s/it]
Epoch 6/10:  89%|████████▉ | 324/364 [07:02<00:34,  1.16it/s]
Epoch 6/10:  89%|████████▉ | 325/364 [07:06<01:09,  1.79s/it]
Epoch 6/10:  90%|████████▉ | 326/364 [07:06<00:52,  1.38s/it]
Epoch 6/10:  90%|████████▉ | 327/364 [07:06<00:40,  1.09s/it]
Epoch 6/10:  90%|█████████ | 328/364 [07:07<00:32,  1.11it/s]
Epoch 6/10:  90%|█████████ | 329/364 [07:11<01:00,  1.72s/it]
Epoch 6/10:  91%|█████████ | 330/364 [07:11<00:45,  1.33s/it]
Epoch 6/10:  91%|█████████ | 331/364 [07:11<00:35,  1.06s/it]
Epoch 6/10:  91%|█████████ | 332/364 [07:12<00:27,  1.14it/s]
Epoch 6/10:  91%|█████████▏| 333/364 [07:16<00:54,  1.75s/it]
Epoch 6/10:  92%|█████████▏| 334/364 [07:16<00:40,  1.35s/it]
Epoch 6/10:  92%|█████████▏| 335/364 [07:16<00:31,  1.07s/it]
Epoch 6/10:  92%|█████████▏| 336/364 [07:17<00:24,  1.13it/s]
Epoch 6/10:  93%|█████████▎| 337/364 [07:21<00:46,  1.71s/it]
Epoch 6/10:  93%|█████████▎| 338/364 [07:21<00:34,  1.33s/it]
Epoch 6/10:  93%|█████████▎| 339/364 [07:21<00:26,  1.06s/it]
Epoch 6/10:  93%|█████████▎| 340/364 [07:22<00:20,  1.15it/s]
Epoch 6/10:  94%|█████████▎| 341/364 [07:26<00:40,  1.76s/it]
Epoch 6/10:  94%|█████████▍| 342/364 [07:26<00:30,  1.37s/it]
Epoch 6/10:  94%|█████████▍| 343/364 [07:27<00:22,  1.08s/it]
Epoch 6/10:  95%|█████████▍| 344/364 [07:27<00:17,  1.13it/s]
Epoch 6/10:  95%|█████████▍| 345/364 [07:31<00:31,  1.68s/it]
Epoch 6/10:  95%|█████████▌| 346/364 [07:31<00:24,  1.37s/it]
Epoch 6/10:  95%|█████████▌| 347/364 [07:32<00:18,  1.09s/it]
Epoch 6/10:  96%|█████████▌| 348/364 [07:32<00:14,  1.12it/s]
Epoch 6/10:  96%|█████████▌| 349/364 [07:36<00:26,  1.74s/it]
Epoch 6/10:  96%|█████████▌| 350/364 [07:36<00:18,  1.35s/it]
Epoch 6/10:  96%|█████████▋| 351/364 [07:37<00:13,  1.07s/it]
Epoch 6/10:  97%|█████████▋| 352/364 [07:37<00:10,  1.12it/s]
Epoch 6/10:  97%|█████████▋| 353/364 [07:41<00:18,  1.71s/it]
Epoch 6/10:  97%|█████████▋| 354/364 [07:41<00:13,  1.33s/it]
Epoch 6/10:  98%|█████████▊| 355/364 [07:42<00:09,  1.06s/it]
Epoch 6/10:  98%|█████████▊| 356/364 [07:42<00:06,  1.14it/s]
Epoch 6/10:  98%|█████████▊| 357/364 [07:46<00:11,  1.68s/it]
Epoch 6/10:  98%|█████████▊| 358/364 [07:46<00:07,  1.30s/it]
Epoch 6/10:  99%|█████████▊| 359/364 [07:46<00:05,  1.04s/it]
Epoch 6/10:  99%|█████████▉| 360/364 [07:47<00:03,  1.17it/s]
Epoch 6/10:  99%|█████████▉| 361/364 [07:49<00:03,  1.26s/it]
Epoch 6/10:  99%|█████████▉| 362/364 [07:49<00:02,  1.00s/it]
Epoch 6/10: 100%|█████████▉| 363/364 [07:50<00:00,  1.21it/s]
Epoch 6/10: 100%|██████████| 364/364 [07:50<00:00,  1.58it/s]
Epoch 6/10: 100%|██████████| 364/364 [07:50<00:00,  1.29s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:14,  4.54s/it]
Validating:   4%|▎         | 2/57 [00:05<01:57,  2.14s/it]
Validating:   5%|▌         | 3/57 [00:05<01:06,  1.23s/it]
Validating:   7%|▋         | 4/57 [00:05<00:42,  1.24it/s]
Validating:   9%|▉         | 5/57 [00:08<01:33,  1.80s/it]
Validating:  11%|█         | 6/57 [00:09<01:13,  1.43s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:50,  1.02s/it]
Validating:  14%|█▍        | 8/57 [00:09<00:36,  1.34it/s]
Validating:  16%|█▌        | 9/57 [00:12<01:10,  1.48s/it]
Validating:  18%|█▊        | 10/57 [00:13<01:00,  1.29s/it]
Validating:  19%|█▉        | 11/57 [00:14<00:43,  1.06it/s]
Validating:  21%|██        | 12/57 [00:14<00:31,  1.44it/s]
Validating:  23%|██▎       | 13/57 [00:17<01:09,  1.58s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:54,  1.26s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:38,  1.08it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:28,  1.46it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:00,  1.52s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:49,  1.27s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:35,  1.07it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:26,  1.42it/s]
Validating:  37%|███▋      | 21/57 [00:26<00:54,  1.52s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:47,  1.35s/it]
Validating:  40%|████      | 23/57 [00:27<00:33,  1.01it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:24,  1.35it/s]
Validating:  44%|████▍     | 25/57 [00:30<00:47,  1.48s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:41,  1.32s/it]
Validating:  47%|████▋     | 27/57 [00:32<00:29,  1.02it/s]
Validating:  49%|████▉     | 28/57 [00:32<00:21,  1.36it/s]
Validating:  51%|█████     | 29/57 [00:35<00:40,  1.43s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:35,  1.32s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:25,  1.04it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:18,  1.38it/s]
Validating:  58%|█████▊    | 33/57 [00:39<00:34,  1.42s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:29,  1.28s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:20,  1.05it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.41it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:30,  1.52s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:26,  1.38s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:18,  1.02s/it]
Validating:  70%|███████   | 40/57 [00:45<00:12,  1.33it/s]
Validating:  72%|███████▏  | 41/57 [00:48<00:23,  1.49s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:20,  1.40s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:14,  1.02s/it]
Validating:  77%|███████▋  | 44/57 [00:50<00:09,  1.32it/s]
Validating:  79%|███████▉  | 45/57 [00:53<00:17,  1.44s/it]
Validating:  81%|████████  | 46/57 [00:54<00:15,  1.38s/it]
Validating:  82%|████████▏ | 47/57 [00:54<00:10,  1.02s/it]
Validating:  84%|████████▍ | 48/57 [00:54<00:06,  1.33it/s]
Validating:  86%|████████▌ | 49/57 [00:57<00:10,  1.26s/it]
Validating:  88%|████████▊ | 50/57 [00:58<00:09,  1.32s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:05,  1.03it/s]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.38it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:04,  1.16s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.14s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.20it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.62it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 6: Train Loss: 0.0978, Train Acc: 0.9668 | Val Loss: 0.3398, Val Acc: 0.8956

Epoch 7/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 7/10:   0%|          | 1/364 [00:05<34:18,  5.67s/it]
Epoch 7/10:   1%|          | 2/364 [00:06<15:34,  2.58s/it]
Epoch 7/10:   1%|          | 3/364 [00:06<09:33,  1.59s/it]
Epoch 7/10:   1%|          | 4/364 [00:06<06:45,  1.13s/it]
Epoch 7/10:   1%|▏         | 5/364 [00:10<12:55,  2.16s/it]
Epoch 7/10:   2%|▏         | 6/364 [00:11<09:20,  1.56s/it]
Epoch 7/10:   2%|▏         | 7/364 [00:11<07:03,  1.19s/it]
Epoch 7/10:   2%|▏         | 8/364 [00:12<05:37,  1.05it/s]
Epoch 7/10:   2%|▏         | 9/364 [00:16<11:04,  1.87s/it]
Epoch 7/10:   3%|▎         | 10/364 [00:16<08:22,  1.42s/it]
Epoch 7/10:   3%|▎         | 11/364 [00:16<06:31,  1.11s/it]
Epoch 7/10:   3%|▎         | 12/364 [00:17<05:19,  1.10it/s]
Epoch 7/10:   4%|▎         | 13/364 [00:21<10:31,  1.80s/it]
Epoch 7/10:   4%|▍         | 14/364 [00:21<08:03,  1.38s/it]
Epoch 7/10:   4%|▍         | 15/364 [00:22<06:21,  1.09s/it]
Epoch 7/10:   4%|▍         | 16/364 [00:22<05:08,  1.13it/s]
Epoch 7/10:   5%|▍         | 17/364 [00:26<10:23,  1.80s/it]
Epoch 7/10:   5%|▍         | 18/364 [00:26<07:58,  1.38s/it]
Epoch 7/10:   5%|▌         | 19/364 [00:27<06:17,  1.09s/it]
Epoch 7/10:   5%|▌         | 20/364 [00:27<05:07,  1.12it/s]
Epoch 7/10:   6%|▌         | 21/364 [00:31<09:32,  1.67s/it]
Epoch 7/10:   6%|▌         | 22/364 [00:31<07:25,  1.30s/it]
Epoch 7/10:   6%|▋         | 23/364 [00:31<05:57,  1.05s/it]
Epoch 7/10:   7%|▋         | 24/364 [00:32<04:53,  1.16it/s]
Epoch 7/10:   7%|▋         | 25/364 [00:36<09:55,  1.76s/it]
Epoch 7/10:   7%|▋         | 26/364 [00:36<07:39,  1.36s/it]
Epoch 7/10:   7%|▋         | 27/364 [00:37<06:04,  1.08s/it]
Epoch 7/10:   8%|▊         | 28/364 [00:37<04:55,  1.14it/s]
Epoch 7/10:   8%|▊         | 29/364 [00:41<10:19,  1.85s/it]
Epoch 7/10:   8%|▊         | 30/364 [00:42<07:55,  1.42s/it]
Epoch 7/10:   9%|▊         | 31/364 [00:42<06:16,  1.13s/it]
Epoch 7/10:   9%|▉         | 32/364 [00:42<05:06,  1.08it/s]
Epoch 7/10:   9%|▉         | 33/364 [00:46<09:32,  1.73s/it]
Epoch 7/10:   9%|▉         | 34/364 [00:47<07:21,  1.34s/it]
Epoch 7/10:  10%|▉         | 35/364 [00:47<05:49,  1.06s/it]
Epoch 7/10:  10%|▉         | 36/364 [00:47<04:45,  1.15it/s]
Epoch 7/10:  10%|█         | 37/364 [00:51<09:59,  1.83s/it]
Epoch 7/10:  10%|█         | 38/364 [00:52<07:37,  1.40s/it]
Epoch 7/10:  11%|█         | 39/364 [00:52<06:00,  1.11s/it]
Epoch 7/10:  11%|█         | 40/364 [00:53<04:52,  1.11it/s]
Epoch 7/10:  11%|█▏        | 41/364 [00:56<09:14,  1.72s/it]
Epoch 7/10:  12%|█▏        | 42/364 [00:57<07:09,  1.33s/it]
Epoch 7/10:  12%|█▏        | 43/364 [00:57<05:41,  1.06s/it]
Epoch 7/10:  12%|█▏        | 44/364 [00:58<04:39,  1.14it/s]
Epoch 7/10:  12%|█▏        | 45/364 [01:01<09:07,  1.72s/it]
Epoch 7/10:  13%|█▎        | 46/364 [01:02<07:00,  1.32s/it]
Epoch 7/10:  13%|█▎        | 47/364 [01:02<05:35,  1.06s/it]
Epoch 7/10:  13%|█▎        | 48/364 [01:03<04:35,  1.15it/s]
Epoch 7/10:  13%|█▎        | 49/364 [01:06<08:53,  1.69s/it]
Epoch 7/10:  14%|█▎        | 50/364 [01:07<06:53,  1.32s/it]
Epoch 7/10:  14%|█▍        | 51/364 [01:07<05:29,  1.05s/it]
Epoch 7/10:  14%|█▍        | 52/364 [01:07<04:30,  1.15it/s]
Epoch 7/10:  15%|█▍        | 53/364 [01:11<09:10,  1.77s/it]
Epoch 7/10:  15%|█▍        | 54/364 [01:12<07:02,  1.36s/it]
Epoch 7/10:  15%|█▌        | 55/364 [01:12<05:34,  1.08s/it]
Epoch 7/10:  15%|█▌        | 56/364 [01:13<04:31,  1.13it/s]
Epoch 7/10:  16%|█▌        | 57/364 [01:17<09:19,  1.82s/it]
Epoch 7/10:  16%|█▌        | 58/364 [01:17<07:09,  1.41s/it]
Epoch 7/10:  16%|█▌        | 59/364 [01:17<05:40,  1.12s/it]
Epoch 7/10:  16%|█▋        | 60/364 [01:18<04:37,  1.10it/s]
Epoch 7/10:  17%|█▋        | 61/364 [01:22<09:01,  1.79s/it]
Epoch 7/10:  17%|█▋        | 62/364 [01:22<06:56,  1.38s/it]
Epoch 7/10:  17%|█▋        | 63/364 [01:23<05:30,  1.10s/it]
Epoch 7/10:  18%|█▊        | 64/364 [01:23<04:29,  1.11it/s]
Epoch 7/10:  18%|█▊        | 65/364 [01:27<08:45,  1.76s/it]
Epoch 7/10:  18%|█▊        | 66/364 [01:27<06:44,  1.36s/it]
Epoch 7/10:  18%|█▊        | 67/364 [01:28<05:19,  1.08s/it]
Epoch 7/10:  19%|█▊        | 68/364 [01:28<04:19,  1.14it/s]
Epoch 7/10:  19%|█▉        | 69/364 [01:32<08:44,  1.78s/it]
Epoch 7/10:  19%|█▉        | 70/364 [01:32<06:43,  1.37s/it]
Epoch 7/10:  20%|█▉        | 71/364 [01:33<05:18,  1.09s/it]
Epoch 7/10:  20%|█▉        | 72/364 [01:33<04:20,  1.12it/s]
Epoch 7/10:  20%|██        | 73/364 [01:37<08:53,  1.83s/it]
Epoch 7/10:  20%|██        | 74/364 [01:38<06:48,  1.41s/it]
Epoch 7/10:  21%|██        | 75/364 [01:38<05:21,  1.11s/it]
Epoch 7/10:  21%|██        | 76/364 [01:39<04:19,  1.11it/s]
Epoch 7/10:  21%|██        | 77/364 [01:42<08:30,  1.78s/it]
Epoch 7/10:  21%|██▏       | 78/364 [01:43<06:31,  1.37s/it]
Epoch 7/10:  22%|██▏       | 79/364 [01:43<05:10,  1.09s/it]
Epoch 7/10:  22%|██▏       | 80/364 [01:44<04:11,  1.13it/s]
Epoch 7/10:  22%|██▏       | 81/364 [01:47<08:04,  1.71s/it]
Epoch 7/10:  23%|██▎       | 82/364 [01:48<06:13,  1.33s/it]
Epoch 7/10:  23%|██▎       | 83/364 [01:48<04:58,  1.06s/it]
Epoch 7/10:  23%|██▎       | 84/364 [01:49<04:03,  1.15it/s]
Epoch 7/10:  23%|██▎       | 85/364 [01:52<07:36,  1.64s/it]
Epoch 7/10:  24%|██▎       | 86/364 [01:52<05:56,  1.28s/it]
Epoch 7/10:  24%|██▍       | 87/364 [01:53<04:44,  1.03s/it]
Epoch 7/10:  24%|██▍       | 88/364 [01:53<03:54,  1.18it/s]
Epoch 7/10:  24%|██▍       | 89/364 [01:57<07:21,  1.60s/it]
Epoch 7/10:  25%|██▍       | 90/364 [01:57<05:42,  1.25s/it]
Epoch 7/10:  25%|██▌       | 91/364 [01:57<04:33,  1.00s/it]
Epoch 7/10:  25%|██▌       | 92/364 [01:58<03:52,  1.17it/s]
Epoch 7/10:  26%|██▌       | 93/364 [02:02<07:52,  1.74s/it]
Epoch 7/10:  26%|██▌       | 94/364 [02:02<06:03,  1.35s/it]
Epoch 7/10:  26%|██▌       | 95/364 [02:03<04:48,  1.07s/it]
Epoch 7/10:  26%|██▋       | 96/364 [02:03<03:55,  1.14it/s]
Epoch 7/10:  27%|██▋       | 97/364 [02:07<08:01,  1.80s/it]
Epoch 7/10:  27%|██▋       | 98/364 [02:08<06:10,  1.39s/it]
Epoch 7/10:  27%|██▋       | 99/364 [02:08<04:51,  1.10s/it]
Epoch 7/10:  27%|██▋       | 100/364 [02:08<03:57,  1.11it/s]
Epoch 7/10:  28%|██▊       | 101/364 [02:12<08:10,  1.87s/it]
Epoch 7/10:  28%|██▊       | 102/364 [02:13<06:23,  1.46s/it]
Epoch 7/10:  28%|██▊       | 103/364 [02:13<05:00,  1.15s/it]
Epoch 7/10:  29%|██▊       | 104/364 [02:14<04:01,  1.08it/s]
Epoch 7/10:  29%|██▉       | 105/364 [02:18<07:43,  1.79s/it]
Epoch 7/10:  29%|██▉       | 106/364 [02:18<05:56,  1.38s/it]
Epoch 7/10:  29%|██▉       | 107/364 [02:18<04:39,  1.09s/it]
Epoch 7/10:  30%|██▉       | 108/364 [02:19<03:47,  1.13it/s]
Epoch 7/10:  30%|██▉       | 109/364 [02:23<07:25,  1.75s/it]
Epoch 7/10:  30%|███       | 110/364 [02:23<05:44,  1.35s/it]
Epoch 7/10:  30%|███       | 111/364 [02:23<04:31,  1.08s/it]
Epoch 7/10:  31%|███       | 112/364 [02:24<03:42,  1.13it/s]
Epoch 7/10:  31%|███       | 113/364 [02:28<07:31,  1.80s/it]
Epoch 7/10:  31%|███▏      | 114/364 [02:28<05:45,  1.38s/it]
Epoch 7/10:  32%|███▏      | 115/364 [02:29<04:35,  1.11s/it]
Epoch 7/10:  32%|███▏      | 116/364 [02:29<03:44,  1.11it/s]
Epoch 7/10:  32%|███▏      | 117/364 [02:33<06:55,  1.68s/it]
Epoch 7/10:  32%|███▏      | 118/364 [02:34<05:59,  1.46s/it]
Epoch 7/10:  33%|███▎      | 119/364 [02:34<04:41,  1.15s/it]
Epoch 7/10:  33%|███▎      | 120/364 [02:34<03:47,  1.07it/s]
Epoch 7/10:  33%|███▎      | 121/364 [02:38<06:20,  1.57s/it]
Epoch 7/10:  34%|███▎      | 122/364 [02:39<06:09,  1.53s/it]
Epoch 7/10:  34%|███▍      | 123/364 [02:39<04:49,  1.20s/it]
Epoch 7/10:  34%|███▍      | 124/364 [02:40<03:52,  1.03it/s]
Epoch 7/10:  34%|███▍      | 125/364 [02:43<06:16,  1.58s/it]
Epoch 7/10:  35%|███▍      | 126/364 [02:44<05:39,  1.43s/it]
Epoch 7/10:  35%|███▍      | 127/364 [02:44<04:25,  1.12s/it]
Epoch 7/10:  35%|███▌      | 128/364 [02:45<03:36,  1.09it/s]
Epoch 7/10:  35%|███▌      | 129/364 [02:48<06:29,  1.66s/it]
Epoch 7/10:  36%|███▌      | 130/364 [02:49<05:16,  1.35s/it]
Epoch 7/10:  36%|███▌      | 131/364 [02:49<04:11,  1.08s/it]
Epoch 7/10:  36%|███▋      | 132/364 [02:50<03:23,  1.14it/s]
Epoch 7/10:  37%|███▋      | 133/364 [02:53<06:37,  1.72s/it]
Epoch 7/10:  37%|███▋      | 134/364 [02:54<05:33,  1.45s/it]
Epoch 7/10:  37%|███▋      | 135/364 [02:55<04:22,  1.15s/it]
Epoch 7/10:  37%|███▋      | 136/364 [02:55<03:32,  1.08it/s]
Epoch 7/10:  38%|███▊      | 137/364 [02:59<06:33,  1.73s/it]
Epoch 7/10:  38%|███▊      | 138/364 [02:59<05:03,  1.34s/it]
Epoch 7/10:  38%|███▊      | 139/364 [02:59<04:00,  1.07s/it]
Epoch 7/10:  38%|███▊      | 140/364 [03:00<03:15,  1.14it/s]
Epoch 7/10:  39%|███▊      | 141/364 [03:04<06:54,  1.86s/it]
Epoch 7/10:  39%|███▉      | 142/364 [03:04<05:17,  1.43s/it]
Epoch 7/10:  39%|███▉      | 143/364 [03:05<04:10,  1.13s/it]
Epoch 7/10:  40%|███▉      | 144/364 [03:05<03:21,  1.09it/s]
Epoch 7/10:  40%|███▉      | 145/364 [03:09<06:51,  1.88s/it]
Epoch 7/10:  40%|████      | 146/364 [03:10<05:16,  1.45s/it]
Epoch 7/10:  40%|████      | 147/364 [03:10<04:06,  1.14s/it]
Epoch 7/10:  41%|████      | 148/364 [03:11<03:19,  1.08it/s]
Epoch 7/10:  41%|████      | 149/364 [03:15<06:31,  1.82s/it]
Epoch 7/10:  41%|████      | 150/364 [03:15<05:00,  1.40s/it]
Epoch 7/10:  41%|████▏     | 151/364 [03:15<03:56,  1.11s/it]
Epoch 7/10:  42%|████▏     | 152/364 [03:16<03:12,  1.10it/s]
Epoch 7/10:  42%|████▏     | 153/364 [03:20<06:10,  1.76s/it]
Epoch 7/10:  42%|████▏     | 154/364 [03:20<04:45,  1.36s/it]
Epoch 7/10:  43%|████▎     | 155/364 [03:21<03:46,  1.08s/it]
Epoch 7/10:  43%|████▎     | 156/364 [03:21<03:05,  1.12it/s]
Epoch 7/10:  43%|████▎     | 157/364 [03:25<06:00,  1.74s/it]
Epoch 7/10:  43%|████▎     | 158/364 [03:25<04:37,  1.35s/it]
Epoch 7/10:  44%|████▎     | 159/364 [03:26<03:39,  1.07s/it]
Epoch 7/10:  44%|████▍     | 160/364 [03:26<02:59,  1.13it/s]
Epoch 7/10:  44%|████▍     | 161/364 [03:30<05:41,  1.68s/it]
Epoch 7/10:  45%|████▍     | 162/364 [03:30<04:23,  1.31s/it]
Epoch 7/10:  45%|████▍     | 163/364 [03:30<03:29,  1.04s/it]
Epoch 7/10:  45%|████▌     | 164/364 [03:31<02:51,  1.17it/s]
Epoch 7/10:  45%|████▌     | 165/364 [03:35<06:11,  1.87s/it]
Epoch 7/10:  46%|████▌     | 166/364 [03:35<04:44,  1.44s/it]
Epoch 7/10:  46%|████▌     | 167/364 [03:36<03:43,  1.14s/it]
Epoch 7/10:  46%|████▌     | 168/364 [03:36<03:01,  1.08it/s]
Epoch 7/10:  46%|████▋     | 169/364 [03:40<06:02,  1.86s/it]
Epoch 7/10:  47%|████▋     | 170/364 [03:41<04:37,  1.43s/it]
Epoch 7/10:  47%|████▋     | 171/364 [03:41<03:38,  1.13s/it]
Epoch 7/10:  47%|████▋     | 172/364 [03:42<02:55,  1.09it/s]
Epoch 7/10:  48%|████▊     | 173/364 [03:45<05:37,  1.77s/it]
Epoch 7/10:  48%|████▊     | 174/364 [03:46<04:19,  1.36s/it]
Epoch 7/10:  48%|████▊     | 175/364 [03:46<03:25,  1.08s/it]
Epoch 7/10:  48%|████▊     | 176/364 [03:47<02:46,  1.13it/s]
Epoch 7/10:  49%|████▊     | 177/364 [03:50<05:26,  1.75s/it]
Epoch 7/10:  49%|████▉     | 178/364 [03:51<04:10,  1.35s/it]
Epoch 7/10:  49%|████▉     | 179/364 [03:51<03:19,  1.08s/it]
Epoch 7/10:  49%|████▉     | 180/364 [03:52<02:43,  1.13it/s]
Epoch 7/10:  50%|████▉     | 181/364 [03:55<05:16,  1.73s/it]
Epoch 7/10:  50%|█████     | 182/364 [03:56<04:03,  1.34s/it]
Epoch 7/10:  50%|█████     | 183/364 [03:56<03:11,  1.06s/it]
Epoch 7/10:  51%|█████     | 184/364 [03:57<02:36,  1.15it/s]
Epoch 7/10:  51%|█████     | 185/364 [04:00<05:05,  1.71s/it]
Epoch 7/10:  51%|█████     | 186/364 [04:01<03:58,  1.34s/it]
Epoch 7/10:  51%|█████▏    | 187/364 [04:01<03:08,  1.07s/it]
Epoch 7/10:  52%|█████▏    | 188/364 [04:02<02:33,  1.14it/s]
Epoch 7/10:  52%|█████▏    | 189/364 [04:06<05:08,  1.76s/it]
Epoch 7/10:  52%|█████▏    | 190/364 [04:06<04:02,  1.39s/it]
Epoch 7/10:  52%|█████▏    | 191/364 [04:07<03:14,  1.13s/it]
Epoch 7/10:  53%|█████▎    | 192/364 [04:07<02:38,  1.08it/s]
Epoch 7/10:  53%|█████▎    | 193/364 [04:11<05:11,  1.82s/it]
Epoch 7/10:  53%|█████▎    | 194/364 [04:11<03:58,  1.40s/it]
Epoch 7/10:  54%|█████▎    | 195/364 [04:12<03:10,  1.12s/it]
Epoch 7/10:  54%|█████▍    | 196/364 [04:12<02:34,  1.09it/s]
Epoch 7/10:  54%|█████▍    | 197/364 [04:16<04:37,  1.66s/it]
Epoch 7/10:  54%|█████▍    | 198/364 [04:16<03:50,  1.39s/it]
Epoch 7/10:  55%|█████▍    | 199/364 [04:17<03:11,  1.16s/it]
Epoch 7/10:  55%|█████▍    | 200/364 [04:17<02:34,  1.06it/s]
Epoch 7/10:  55%|█████▌    | 201/364 [04:20<04:10,  1.54s/it]
Epoch 7/10:  55%|█████▌    | 202/364 [04:22<03:57,  1.47s/it]
Epoch 7/10:  56%|█████▌    | 203/364 [04:22<03:18,  1.24s/it]
Epoch 7/10:  56%|█████▌    | 204/364 [04:23<02:38,  1.01it/s]
Epoch 7/10:  56%|█████▋    | 205/364 [04:26<04:05,  1.55s/it]
Epoch 7/10:  57%|█████▋    | 206/364 [04:26<03:28,  1.32s/it]
Epoch 7/10:  57%|█████▋    | 207/364 [04:27<03:06,  1.19s/it]
Epoch 7/10:  57%|█████▋    | 208/364 [04:28<02:29,  1.04it/s]
Epoch 7/10:  57%|█████▋    | 209/364 [04:31<04:17,  1.66s/it]
Epoch 7/10:  58%|█████▊    | 210/364 [04:32<03:19,  1.29s/it]
Epoch 7/10:  58%|█████▊    | 211/364 [04:32<02:45,  1.08s/it]
Epoch 7/10:  58%|█████▊    | 212/364 [04:33<02:15,  1.13it/s]
Epoch 7/10:  59%|█████▊    | 213/364 [04:36<04:03,  1.61s/it]
Epoch 7/10:  59%|█████▉    | 214/364 [04:37<03:26,  1.38s/it]
Epoch 7/10:  59%|█████▉    | 215/364 [04:37<02:46,  1.12s/it]
Epoch 7/10:  59%|█████▉    | 216/364 [04:38<02:15,  1.09it/s]
Epoch 7/10:  60%|█████▉    | 217/364 [04:41<04:08,  1.69s/it]
Epoch 7/10:  60%|█████▉    | 218/364 [04:42<03:19,  1.37s/it]
Epoch 7/10:  60%|██████    | 219/364 [04:42<02:40,  1.11s/it]
Epoch 7/10:  60%|██████    | 220/364 [04:43<02:10,  1.10it/s]
Epoch 7/10:  61%|██████    | 221/364 [04:46<03:54,  1.64s/it]
Epoch 7/10:  61%|██████    | 222/364 [04:47<03:20,  1.41s/it]
Epoch 7/10:  61%|██████▏   | 223/364 [04:47<02:43,  1.16s/it]
Epoch 7/10:  62%|██████▏   | 224/364 [04:48<02:12,  1.06it/s]
Epoch 7/10:  62%|██████▏   | 225/364 [04:51<03:41,  1.59s/it]
Epoch 7/10:  62%|██████▏   | 226/364 [04:52<03:10,  1.38s/it]
Epoch 7/10:  62%|██████▏   | 227/364 [04:53<02:39,  1.16s/it]
Epoch 7/10:  63%|██████▎   | 228/364 [04:53<02:08,  1.06it/s]
Epoch 7/10:  63%|██████▎   | 229/364 [04:56<03:28,  1.55s/it]
Epoch 7/10:  63%|██████▎   | 230/364 [04:57<02:59,  1.34s/it]
Epoch 7/10:  63%|██████▎   | 231/364 [04:58<02:39,  1.20s/it]
Epoch 7/10:  64%|██████▎   | 232/364 [04:58<02:08,  1.03it/s]
Epoch 7/10:  64%|██████▍   | 233/364 [05:01<03:17,  1.51s/it]
Epoch 7/10:  64%|██████▍   | 234/364 [05:02<02:55,  1.35s/it]
Epoch 7/10:  65%|██████▍   | 235/364 [05:03<02:41,  1.25s/it]
Epoch 7/10:  65%|██████▍   | 236/364 [05:03<02:09,  1.01s/it]
Epoch 7/10:  65%|██████▌   | 237/364 [05:06<03:16,  1.55s/it]
Epoch 7/10:  65%|██████▌   | 238/364 [05:07<02:51,  1.36s/it]
Epoch 7/10:  66%|██████▌   | 239/364 [05:08<02:37,  1.26s/it]
Epoch 7/10:  66%|██████▌   | 240/364 [05:09<02:05,  1.01s/it]
Epoch 7/10:  66%|██████▌   | 241/364 [05:12<03:17,  1.61s/it]
Epoch 7/10:  66%|██████▋   | 242/364 [05:12<02:43,  1.34s/it]
Epoch 7/10:  67%|██████▋   | 243/364 [05:13<02:13,  1.10s/it]
Epoch 7/10:  67%|██████▋   | 244/364 [05:13<01:48,  1.10it/s]
Epoch 7/10:  67%|██████▋   | 245/364 [05:17<03:12,  1.62s/it]
Epoch 7/10:  68%|██████▊   | 246/364 [05:18<02:49,  1.43s/it]
Epoch 7/10:  68%|██████▊   | 247/364 [05:18<02:12,  1.14s/it]
Epoch 7/10:  68%|██████▊   | 248/364 [05:18<01:47,  1.08it/s]
Epoch 7/10:  68%|██████▊   | 249/364 [05:21<03:00,  1.57s/it]
Epoch 7/10:  69%|██████▊   | 250/364 [05:22<02:41,  1.41s/it]
Epoch 7/10:  69%|██████▉   | 251/364 [05:23<02:07,  1.13s/it]
Epoch 7/10:  69%|██████▉   | 252/364 [05:23<01:43,  1.09it/s]
Epoch 7/10:  70%|██████▉   | 253/364 [05:27<02:58,  1.61s/it]
Epoch 7/10:  70%|██████▉   | 254/364 [05:27<02:25,  1.32s/it]
Epoch 7/10:  70%|███████   | 255/364 [05:28<02:04,  1.15s/it]
Epoch 7/10:  70%|███████   | 256/364 [05:28<01:40,  1.07it/s]
Epoch 7/10:  71%|███████   | 257/364 [05:31<02:46,  1.55s/it]
Epoch 7/10:  71%|███████   | 258/364 [05:32<02:08,  1.22s/it]
Epoch 7/10:  71%|███████   | 259/364 [05:33<02:11,  1.26s/it]
Epoch 7/10:  71%|███████▏  | 260/364 [05:34<01:45,  1.01s/it]
Epoch 7/10:  72%|███████▏  | 261/364 [05:36<02:38,  1.54s/it]
Epoch 7/10:  72%|███████▏  | 262/364 [05:37<02:04,  1.22s/it]
Epoch 7/10:  72%|███████▏  | 263/364 [05:39<02:18,  1.37s/it]
Epoch 7/10:  73%|███████▎  | 264/364 [05:39<01:48,  1.08s/it]
Epoch 7/10:  73%|███████▎  | 265/364 [05:41<02:27,  1.49s/it]
Epoch 7/10:  73%|███████▎  | 266/364 [05:42<01:54,  1.17s/it]
Epoch 7/10:  73%|███████▎  | 267/364 [05:44<02:09,  1.33s/it]
Epoch 7/10:  74%|███████▎  | 268/364 [05:44<01:42,  1.06s/it]
Epoch 7/10:  74%|███████▍  | 269/364 [05:47<02:33,  1.62s/it]
Epoch 7/10:  74%|███████▍  | 270/364 [05:47<01:59,  1.27s/it]
Epoch 7/10:  74%|███████▍  | 271/364 [05:48<01:46,  1.15s/it]
Epoch 7/10:  75%|███████▍  | 272/364 [05:49<01:26,  1.07it/s]
Epoch 7/10:  75%|███████▌  | 273/364 [05:52<02:38,  1.75s/it]
Epoch 7/10:  75%|███████▌  | 274/364 [05:53<02:01,  1.35s/it]
Epoch 7/10:  76%|███████▌  | 275/364 [05:53<01:41,  1.14s/it]
Epoch 7/10:  76%|███████▌  | 276/364 [05:54<01:21,  1.08it/s]
Epoch 7/10:  76%|███████▌  | 277/364 [05:57<02:23,  1.65s/it]
Epoch 7/10:  76%|███████▋  | 278/364 [05:58<01:51,  1.29s/it]
Epoch 7/10:  77%|███████▋  | 279/364 [05:58<01:32,  1.08s/it]
Epoch 7/10:  77%|███████▋  | 280/364 [05:59<01:14,  1.13it/s]
Epoch 7/10:  77%|███████▋  | 281/364 [06:02<02:15,  1.63s/it]
Epoch 7/10:  77%|███████▋  | 282/364 [06:02<01:43,  1.27s/it]
Epoch 7/10:  78%|███████▊  | 283/364 [06:03<01:36,  1.19s/it]
Epoch 7/10:  78%|███████▊  | 284/364 [06:04<01:16,  1.04it/s]
Epoch 7/10:  78%|███████▊  | 285/364 [06:07<02:07,  1.61s/it]
Epoch 7/10:  79%|███████▊  | 286/364 [06:07<01:38,  1.26s/it]
Epoch 7/10:  79%|███████▉  | 287/364 [06:09<01:35,  1.24s/it]
Epoch 7/10:  79%|███████▉  | 288/364 [06:09<01:15,  1.00it/s]
Epoch 7/10:  79%|███████▉  | 289/364 [06:11<01:46,  1.42s/it]
Epoch 7/10:  80%|███████▉  | 290/364 [06:12<01:35,  1.29s/it]
Epoch 7/10:  80%|███████▉  | 291/364 [06:14<01:36,  1.33s/it]
Epoch 7/10:  80%|████████  | 292/364 [06:14<01:16,  1.06s/it]
Epoch 7/10:  80%|████████  | 293/364 [06:16<01:34,  1.32s/it]
Epoch 7/10:  81%|████████  | 294/364 [06:18<01:31,  1.31s/it]
Epoch 7/10:  81%|████████  | 295/364 [06:19<01:26,  1.26s/it]
Epoch 7/10:  81%|████████▏ | 296/364 [06:19<01:08,  1.01s/it]
Epoch 7/10:  82%|████████▏ | 297/364 [06:21<01:29,  1.33s/it]
Epoch 7/10:  82%|████████▏ | 298/364 [06:23<01:29,  1.36s/it]
Epoch 7/10:  82%|████████▏ | 299/364 [06:24<01:20,  1.25s/it]
Epoch 7/10:  82%|████████▏ | 300/364 [06:24<01:04,  1.01s/it]
Epoch 7/10:  83%|████████▎ | 301/364 [06:26<01:28,  1.41s/it]
Epoch 7/10:  83%|████████▎ | 302/364 [06:28<01:26,  1.40s/it]
Epoch 7/10:  83%|████████▎ | 303/364 [06:29<01:16,  1.26s/it]
Epoch 7/10:  84%|████████▎ | 304/364 [06:29<01:00,  1.01s/it]
Epoch 7/10:  84%|████████▍ | 305/364 [06:31<01:12,  1.23s/it]
Epoch 7/10:  84%|████████▍ | 306/364 [06:33<01:29,  1.54s/it]
Epoch 7/10:  84%|████████▍ | 307/364 [06:34<01:08,  1.21s/it]
Epoch 7/10:  85%|████████▍ | 308/364 [06:34<00:54,  1.02it/s]
Epoch 7/10:  85%|████████▍ | 309/364 [06:36<01:02,  1.14s/it]
Epoch 7/10:  85%|████████▌ | 310/364 [06:39<01:32,  1.72s/it]
Epoch 7/10:  85%|████████▌ | 311/364 [06:39<01:10,  1.33s/it]
Epoch 7/10:  86%|████████▌ | 312/364 [06:39<00:55,  1.07s/it]
Epoch 7/10:  86%|████████▌ | 313/364 [06:41<00:57,  1.12s/it]
Epoch 7/10:  86%|████████▋ | 314/364 [06:44<01:23,  1.67s/it]
Epoch 7/10:  87%|████████▋ | 315/364 [06:44<01:03,  1.30s/it]
Epoch 7/10:  87%|████████▋ | 316/364 [06:45<00:50,  1.05s/it]
Epoch 7/10:  87%|████████▋ | 317/364 [06:46<00:55,  1.18s/it]
Epoch 7/10:  87%|████████▋ | 318/364 [06:48<01:06,  1.45s/it]
Epoch 7/10:  88%|████████▊ | 319/364 [06:49<00:51,  1.14s/it]
Epoch 7/10:  88%|████████▊ | 320/364 [06:49<00:40,  1.08it/s]
Epoch 7/10:  88%|████████▊ | 321/364 [06:51<00:58,  1.36s/it]
Epoch 7/10:  88%|████████▊ | 322/364 [06:53<01:04,  1.54s/it]
Epoch 7/10:  89%|████████▊ | 323/364 [06:54<00:49,  1.21s/it]
Epoch 7/10:  89%|████████▉ | 324/364 [06:54<00:38,  1.03it/s]
Epoch 7/10:  89%|████████▉ | 325/364 [06:57<00:54,  1.41s/it]
Epoch 7/10:  90%|████████▉ | 326/364 [06:58<00:58,  1.55s/it]
Epoch 7/10:  90%|████████▉ | 327/364 [06:59<00:44,  1.21s/it]
Epoch 7/10:  90%|█████████ | 328/364 [06:59<00:35,  1.02it/s]
Epoch 7/10:  90%|█████████ | 329/364 [07:02<00:52,  1.51s/it]
Epoch 7/10:  91%|█████████ | 330/364 [07:04<00:54,  1.60s/it]
Epoch 7/10:  91%|█████████ | 331/364 [07:04<00:41,  1.26s/it]
Epoch 7/10:  91%|█████████ | 332/364 [07:05<00:32,  1.01s/it]
Epoch 7/10:  91%|█████████▏| 333/364 [07:07<00:46,  1.49s/it]
Epoch 7/10:  92%|█████████▏| 334/364 [07:09<00:47,  1.58s/it]
Epoch 7/10:  92%|█████████▏| 335/364 [07:10<00:35,  1.24s/it]
Epoch 7/10:  92%|█████████▏| 336/364 [07:10<00:27,  1.00it/s]
Epoch 7/10:  93%|█████████▎| 337/364 [07:13<00:40,  1.51s/it]
Epoch 7/10:  93%|█████████▎| 338/364 [07:14<00:36,  1.39s/it]
Epoch 7/10:  93%|█████████▎| 339/364 [07:14<00:27,  1.11s/it]
Epoch 7/10:  93%|█████████▎| 340/364 [07:15<00:21,  1.10it/s]
Epoch 7/10:  94%|█████████▎| 341/364 [07:18<00:37,  1.62s/it]
Epoch 7/10:  94%|█████████▍| 342/364 [07:19<00:29,  1.36s/it]
Epoch 7/10:  94%|█████████▍| 343/364 [07:19<00:22,  1.08s/it]
Epoch 7/10:  95%|█████████▍| 344/364 [07:20<00:17,  1.13it/s]
Epoch 7/10:  95%|█████████▍| 345/364 [07:23<00:31,  1.66s/it]
Epoch 7/10:  95%|█████████▌| 346/364 [07:24<00:24,  1.34s/it]
Epoch 7/10:  95%|█████████▌| 347/364 [07:24<00:18,  1.07s/it]
Epoch 7/10:  96%|█████████▌| 348/364 [07:25<00:14,  1.12it/s]
Epoch 7/10:  96%|█████████▌| 349/364 [07:28<00:23,  1.60s/it]
Epoch 7/10:  96%|█████████▌| 350/364 [07:28<00:18,  1.29s/it]
Epoch 7/10:  96%|█████████▋| 351/364 [07:29<00:13,  1.03s/it]
Epoch 7/10:  97%|█████████▋| 352/364 [07:29<00:10,  1.17it/s]
Epoch 7/10:  97%|█████████▋| 353/364 [07:32<00:16,  1.53s/it]
Epoch 7/10:  97%|█████████▋| 354/364 [07:33<00:13,  1.37s/it]
Epoch 7/10:  98%|█████████▊| 355/364 [07:34<00:09,  1.09s/it]
Epoch 7/10:  98%|█████████▊| 356/364 [07:34<00:07,  1.12it/s]
Epoch 7/10:  98%|█████████▊| 357/364 [07:38<00:11,  1.62s/it]
Epoch 7/10:  98%|█████████▊| 358/364 [07:38<00:08,  1.35s/it]
Epoch 7/10:  99%|█████████▊| 359/364 [07:39<00:05,  1.09s/it]
Epoch 7/10:  99%|█████████▉| 360/364 [07:39<00:03,  1.13it/s]
Epoch 7/10:  99%|█████████▉| 361/364 [07:41<00:03,  1.28s/it]
Epoch 7/10:  99%|█████████▉| 362/364 [07:42<00:02,  1.12s/it]
Epoch 7/10: 100%|█████████▉| 363/364 [07:43<00:00,  1.10it/s]
Epoch 7/10: 100%|██████████| 364/364 [07:43<00:00,  1.45it/s]
Epoch 7/10: 100%|██████████| 364/364 [07:43<00:00,  1.27s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:31,  4.84s/it]
Validating:   4%|▎         | 2/57 [00:05<01:55,  2.10s/it]
Validating:   5%|▌         | 3/57 [00:05<01:05,  1.22s/it]
Validating:   7%|▋         | 4/57 [00:05<00:42,  1.26it/s]
Validating:   9%|▉         | 5/57 [00:09<01:42,  1.97s/it]
Validating:  11%|█         | 6/57 [00:09<01:09,  1.36s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:48,  1.04it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:34,  1.43it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:17,  1.62s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:54,  1.17s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:39,  1.17it/s]
Validating:  21%|██        | 12/57 [00:13<00:28,  1.58it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:14,  1.70s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:53,  1.24s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:38,  1.09it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:28,  1.45it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:05,  1.63s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:46,  1.18s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:33,  1.14it/s]
Validating:  35%|███▌      | 20/57 [00:22<00:24,  1.52it/s]
Validating:  37%|███▋      | 21/57 [00:26<00:57,  1.60s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:43,  1.24s/it]
Validating:  40%|████      | 23/57 [00:27<00:30,  1.10it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:22,  1.46it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:51,  1.60s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:38,  1.25s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:27,  1.08it/s]
Validating:  49%|████▉     | 28/57 [00:31<00:20,  1.43it/s]
Validating:  51%|█████     | 29/57 [00:35<00:45,  1.61s/it]
Validating:  53%|█████▎    | 30/57 [00:35<00:32,  1.20s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:23,  1.12it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.50it/s]
Validating:  58%|█████▊    | 33/57 [00:39<00:37,  1.56s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:29,  1.30s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:20,  1.05it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.41it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:33,  1.66s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:26,  1.38s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:18,  1.01s/it]
Validating:  70%|███████   | 40/57 [00:45<00:12,  1.33it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:25,  1.61s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:21,  1.41s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:14,  1.03s/it]
Validating:  77%|███████▋  | 44/57 [00:50<00:09,  1.31it/s]
Validating:  79%|███████▉  | 45/57 [00:53<00:18,  1.54s/it]
Validating:  81%|████████  | 46/57 [00:54<00:15,  1.41s/it]
Validating:  82%|████████▏ | 47/57 [00:55<00:10,  1.03s/it]
Validating:  84%|████████▍ | 48/57 [00:55<00:06,  1.31it/s]
Validating:  86%|████████▌ | 49/57 [00:57<00:10,  1.31s/it]
Validating:  88%|████████▊ | 50/57 [00:59<00:09,  1.36s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:06,  1.01s/it]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.34it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:04,  1.10s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.01s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.35it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.81it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 7: Train Loss: 0.0673, Train Acc: 0.9777 | Val Loss: 0.3481, Val Acc: 0.8983

Epoch 8/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 8/10:   0%|          | 1/364 [00:05<33:49,  5.59s/it]
Epoch 8/10:   1%|          | 2/364 [00:06<15:27,  2.56s/it]
Epoch 8/10:   1%|          | 3/364 [00:06<09:31,  1.58s/it]
Epoch 8/10:   1%|          | 4/364 [00:06<06:45,  1.13s/it]
Epoch 8/10:   1%|▏         | 5/364 [00:10<11:54,  1.99s/it]
Epoch 8/10:   2%|▏         | 6/364 [00:10<08:42,  1.46s/it]
Epoch 8/10:   2%|▏         | 7/364 [00:11<06:42,  1.13s/it]
Epoch 8/10:   2%|▏         | 8/364 [00:11<05:22,  1.10it/s]
Epoch 8/10:   2%|▏         | 9/364 [00:15<10:55,  1.85s/it]
Epoch 8/10:   3%|▎         | 10/364 [00:16<08:19,  1.41s/it]
Epoch 8/10:   3%|▎         | 11/364 [00:16<06:31,  1.11s/it]
Epoch 8/10:   3%|▎         | 12/364 [00:16<05:16,  1.11it/s]
Epoch 8/10:   4%|▎         | 13/364 [00:20<10:39,  1.82s/it]
Epoch 8/10:   4%|▍         | 14/364 [00:21<08:11,  1.40s/it]
Epoch 8/10:   4%|▍         | 15/364 [00:21<06:27,  1.11s/it]
Epoch 8/10:   4%|▍         | 16/364 [00:22<05:14,  1.11it/s]
Epoch 8/10:   5%|▍         | 17/364 [00:25<09:45,  1.69s/it]
Epoch 8/10:   5%|▍         | 18/364 [00:26<07:33,  1.31s/it]
Epoch 8/10:   5%|▌         | 19/364 [00:26<05:59,  1.04s/it]
Epoch 8/10:   5%|▌         | 20/364 [00:26<04:54,  1.17it/s]
Epoch 8/10:   6%|▌         | 21/364 [00:30<09:53,  1.73s/it]
Epoch 8/10:   6%|▌         | 22/364 [00:31<07:36,  1.34s/it]
Epoch 8/10:   6%|▋         | 23/364 [00:31<06:01,  1.06s/it]
Epoch 8/10:   7%|▋         | 24/364 [00:31<04:54,  1.15it/s]
Epoch 8/10:   7%|▋         | 25/364 [00:35<10:18,  1.82s/it]
Epoch 8/10:   7%|▋         | 26/364 [00:36<07:55,  1.41s/it]
Epoch 8/10:   7%|▋         | 27/364 [00:36<06:19,  1.13s/it]
Epoch 8/10:   8%|▊         | 28/364 [00:37<05:06,  1.10it/s]
Epoch 8/10:   8%|▊         | 29/364 [00:41<10:37,  1.90s/it]
Epoch 8/10:   8%|▊         | 30/364 [00:41<08:08,  1.46s/it]
Epoch 8/10:   9%|▊         | 31/364 [00:42<06:24,  1.16s/it]
Epoch 8/10:   9%|▉         | 32/364 [00:42<05:10,  1.07it/s]
Epoch 8/10:   9%|▉         | 33/364 [00:46<09:11,  1.67s/it]
Epoch 8/10:   9%|▉         | 34/364 [00:46<07:07,  1.30s/it]
Epoch 8/10:  10%|▉         | 35/364 [00:47<05:40,  1.04s/it]
Epoch 8/10:  10%|▉         | 36/364 [00:47<04:40,  1.17it/s]
Epoch 8/10:  10%|█         | 37/364 [00:50<08:43,  1.60s/it]
Epoch 8/10:  10%|█         | 38/364 [00:51<06:46,  1.25s/it]
Epoch 8/10:  11%|█         | 39/364 [00:51<05:24,  1.00it/s]
Epoch 8/10:  11%|█         | 40/364 [00:52<04:55,  1.10it/s]
Epoch 8/10:  11%|█▏        | 41/364 [00:55<08:57,  1.66s/it]
Epoch 8/10:  12%|█▏        | 42/364 [00:56<06:55,  1.29s/it]
Epoch 8/10:  12%|█▏        | 43/364 [00:56<05:30,  1.03s/it]
Epoch 8/10:  12%|█▏        | 44/364 [00:57<05:30,  1.03s/it]
Epoch 8/10:  12%|█▏        | 45/364 [01:00<08:36,  1.62s/it]
Epoch 8/10:  13%|█▎        | 46/364 [01:01<06:41,  1.26s/it]
Epoch 8/10:  13%|█▎        | 47/364 [01:01<05:19,  1.01s/it]
Epoch 8/10:  13%|█▎        | 48/364 [01:03<06:07,  1.16s/it]
Epoch 8/10:  13%|█▎        | 49/364 [01:05<07:57,  1.51s/it]
Epoch 8/10:  14%|█▎        | 50/364 [01:05<06:15,  1.19s/it]
Epoch 8/10:  14%|█▍        | 51/364 [01:06<05:03,  1.03it/s]
Epoch 8/10:  14%|█▍        | 52/364 [01:08<06:39,  1.28s/it]
Epoch 8/10:  15%|█▍        | 53/364 [01:10<08:21,  1.61s/it]
Epoch 8/10:  15%|█▍        | 54/364 [01:11<06:30,  1.26s/it]
Epoch 8/10:  15%|█▌        | 55/364 [01:11<05:19,  1.03s/it]
Epoch 8/10:  15%|█▌        | 56/364 [01:13<06:11,  1.21s/it]
Epoch 8/10:  16%|█▌        | 57/364 [01:15<07:58,  1.56s/it]
Epoch 8/10:  16%|█▌        | 58/364 [01:16<06:12,  1.22s/it]
Epoch 8/10:  16%|█▌        | 59/364 [01:16<05:33,  1.09s/it]
Epoch 8/10:  16%|█▋        | 60/364 [01:17<05:38,  1.11s/it]
Epoch 8/10:  17%|█▋        | 61/364 [01:20<07:36,  1.51s/it]
Epoch 8/10:  17%|█▋        | 62/364 [01:20<05:57,  1.18s/it]
Epoch 8/10:  17%|█▋        | 63/364 [01:22<06:01,  1.20s/it]
Epoch 8/10:  18%|█▊        | 64/364 [01:22<05:03,  1.01s/it]
Epoch 8/10:  18%|█▊        | 65/364 [01:25<08:22,  1.68s/it]
Epoch 8/10:  18%|█▊        | 66/364 [01:26<06:27,  1.30s/it]
Epoch 8/10:  18%|█▊        | 67/364 [01:27<05:53,  1.19s/it]
Epoch 8/10:  19%|█▊        | 68/364 [01:27<04:51,  1.02it/s]
Epoch 8/10:  19%|█▉        | 69/364 [01:30<07:58,  1.62s/it]
Epoch 8/10:  19%|█▉        | 70/364 [01:31<06:12,  1.27s/it]
Epoch 8/10:  20%|█▉        | 71/364 [01:32<06:01,  1.23s/it]
Epoch 8/10:  20%|█▉        | 72/364 [01:32<04:49,  1.01it/s]
Epoch 8/10:  20%|██        | 73/364 [01:36<08:19,  1.72s/it]
Epoch 8/10:  20%|██        | 74/364 [01:36<06:25,  1.33s/it]
Epoch 8/10:  21%|██        | 75/364 [01:37<05:31,  1.15s/it]
Epoch 8/10:  21%|██        | 76/364 [01:37<04:29,  1.07it/s]
Epoch 8/10:  21%|██        | 77/364 [01:41<08:02,  1.68s/it]
Epoch 8/10:  21%|██▏       | 78/364 [01:41<06:13,  1.31s/it]
Epoch 8/10:  22%|██▏       | 79/364 [01:42<06:00,  1.26s/it]
Epoch 8/10:  22%|██▏       | 80/364 [01:43<04:49,  1.02s/it]
Epoch 8/10:  22%|██▏       | 81/364 [01:46<08:03,  1.71s/it]
Epoch 8/10:  23%|██▎       | 82/364 [01:47<06:13,  1.33s/it]
Epoch 8/10:  23%|██▎       | 83/364 [01:48<06:13,  1.33s/it]
Epoch 8/10:  23%|██▎       | 84/364 [01:48<04:55,  1.06s/it]
Epoch 8/10:  23%|██▎       | 85/364 [01:51<07:46,  1.67s/it]
Epoch 8/10:  24%|██▎       | 86/364 [01:52<06:00,  1.30s/it]
Epoch 8/10:  24%|██▍       | 87/364 [01:53<05:48,  1.26s/it]
Epoch 8/10:  24%|██▍       | 88/364 [01:53<04:40,  1.02s/it]
Epoch 8/10:  24%|██▍       | 89/364 [01:57<07:47,  1.70s/it]
Epoch 8/10:  25%|██▍       | 90/364 [01:57<05:59,  1.31s/it]
Epoch 8/10:  25%|██▌       | 91/364 [01:58<05:05,  1.12s/it]
Epoch 8/10:  25%|██▌       | 92/364 [01:58<04:08,  1.09it/s]
Epoch 8/10:  26%|██▌       | 93/364 [02:02<07:59,  1.77s/it]
Epoch 8/10:  26%|██▌       | 94/364 [02:02<06:09,  1.37s/it]
Epoch 8/10:  26%|██▌       | 95/364 [02:03<04:52,  1.09s/it]
Epoch 8/10:  26%|██▋       | 96/364 [02:03<03:57,  1.13it/s]
Epoch 8/10:  27%|██▋       | 97/364 [02:07<07:53,  1.77s/it]
Epoch 8/10:  27%|██▋       | 98/364 [02:08<06:03,  1.37s/it]
Epoch 8/10:  27%|██▋       | 99/364 [02:08<04:47,  1.09s/it]
Epoch 8/10:  27%|██▋       | 100/364 [02:08<03:53,  1.13it/s]
Epoch 8/10:  28%|██▊       | 101/364 [02:13<08:22,  1.91s/it]
Epoch 8/10:  28%|██▊       | 102/364 [02:13<06:24,  1.47s/it]
Epoch 8/10:  28%|██▊       | 103/364 [02:14<05:01,  1.16s/it]
Epoch 8/10:  29%|██▊       | 104/364 [02:14<04:04,  1.06it/s]
Epoch 8/10:  29%|██▉       | 105/364 [02:17<07:08,  1.66s/it]
Epoch 8/10:  29%|██▉       | 106/364 [02:18<05:31,  1.29s/it]
Epoch 8/10:  29%|██▉       | 107/364 [02:18<04:24,  1.03s/it]
Epoch 8/10:  30%|██▉       | 108/364 [02:19<03:38,  1.17it/s]
Epoch 8/10:  30%|██▉       | 109/364 [02:23<07:45,  1.83s/it]
Epoch 8/10:  30%|███       | 110/364 [02:23<05:57,  1.41s/it]
Epoch 8/10:  30%|███       | 111/364 [02:24<04:41,  1.11s/it]
Epoch 8/10:  31%|███       | 112/364 [02:24<03:49,  1.10it/s]
Epoch 8/10:  31%|███       | 113/364 [02:28<07:41,  1.84s/it]
Epoch 8/10:  31%|███▏      | 114/364 [02:28<05:53,  1.41s/it]
Epoch 8/10:  32%|███▏      | 115/364 [02:29<04:38,  1.12s/it]
Epoch 8/10:  32%|███▏      | 116/364 [02:29<03:47,  1.09it/s]
Epoch 8/10:  32%|███▏      | 117/364 [02:33<07:40,  1.86s/it]
Epoch 8/10:  32%|███▏      | 118/364 [02:34<05:50,  1.43s/it]
Epoch 8/10:  33%|███▎      | 119/364 [02:34<04:34,  1.12s/it]
Epoch 8/10:  33%|███▎      | 120/364 [02:35<03:43,  1.09it/s]
Epoch 8/10:  33%|███▎      | 121/364 [02:39<07:16,  1.80s/it]
Epoch 8/10:  34%|███▎      | 122/364 [02:39<05:34,  1.38s/it]
Epoch 8/10:  34%|███▍      | 123/364 [02:39<04:22,  1.09s/it]
Epoch 8/10:  34%|███▍      | 124/364 [02:40<03:33,  1.12it/s]
Epoch 8/10:  34%|███▍      | 125/364 [02:44<07:11,  1.81s/it]
Epoch 8/10:  35%|███▍      | 126/364 [02:44<05:30,  1.39s/it]
Epoch 8/10:  35%|███▍      | 127/364 [02:45<04:20,  1.10s/it]
Epoch 8/10:  35%|███▌      | 128/364 [02:45<03:33,  1.11it/s]
Epoch 8/10:  35%|███▌      | 129/364 [02:49<07:06,  1.82s/it]
Epoch 8/10:  36%|███▌      | 130/364 [02:49<05:27,  1.40s/it]
Epoch 8/10:  36%|███▌      | 131/364 [02:50<04:18,  1.11s/it]
Epoch 8/10:  36%|███▋      | 132/364 [02:50<03:28,  1.11it/s]
Epoch 8/10:  37%|███▋      | 133/364 [02:54<07:06,  1.85s/it]
Epoch 8/10:  37%|███▋      | 134/364 [02:55<05:26,  1.42s/it]
Epoch 8/10:  37%|███▋      | 135/364 [02:55<04:16,  1.12s/it]
Epoch 8/10:  37%|███▋      | 136/364 [02:56<03:27,  1.10it/s]
Epoch 8/10:  38%|███▊      | 137/364 [02:59<06:35,  1.74s/it]
Epoch 8/10:  38%|███▊      | 138/364 [03:00<05:05,  1.35s/it]
Epoch 8/10:  38%|███▊      | 139/364 [03:00<04:01,  1.07s/it]
Epoch 8/10:  38%|███▊      | 140/364 [03:01<03:17,  1.14it/s]
Epoch 8/10:  39%|███▊      | 141/364 [03:04<06:38,  1.79s/it]
Epoch 8/10:  39%|███▉      | 142/364 [03:05<05:06,  1.38s/it]
Epoch 8/10:  39%|███▉      | 143/364 [03:05<04:02,  1.10s/it]
Epoch 8/10:  40%|███▉      | 144/364 [03:06<03:17,  1.12it/s]
Epoch 8/10:  40%|███▉      | 145/364 [03:09<06:17,  1.72s/it]
Epoch 8/10:  40%|████      | 146/364 [03:10<04:52,  1.34s/it]
Epoch 8/10:  40%|████      | 147/364 [03:10<03:50,  1.06s/it]
Epoch 8/10:  41%|████      | 148/364 [03:11<03:08,  1.15it/s]
Epoch 8/10:  41%|████      | 149/364 [03:15<06:20,  1.77s/it]
Epoch 8/10:  41%|████      | 150/364 [03:15<04:51,  1.36s/it]
Epoch 8/10:  41%|████▏     | 151/364 [03:15<03:51,  1.09s/it]
Epoch 8/10:  42%|████▏     | 152/364 [03:16<03:08,  1.13it/s]
Epoch 8/10:  42%|████▏     | 153/364 [03:20<06:05,  1.73s/it]
Epoch 8/10:  42%|████▏     | 154/364 [03:20<04:42,  1.35s/it]
Epoch 8/10:  43%|████▎     | 155/364 [03:20<03:43,  1.07s/it]
Epoch 8/10:  43%|████▎     | 156/364 [03:21<03:02,  1.14it/s]
Epoch 8/10:  43%|████▎     | 157/364 [03:25<06:01,  1.74s/it]
Epoch 8/10:  43%|████▎     | 158/364 [03:25<04:36,  1.34s/it]
Epoch 8/10:  44%|████▎     | 159/364 [03:25<03:37,  1.06s/it]
Epoch 8/10:  44%|████▍     | 160/364 [03:26<02:59,  1.14it/s]
Epoch 8/10:  44%|████▍     | 161/364 [03:30<05:58,  1.77s/it]
Epoch 8/10:  45%|████▍     | 162/364 [03:30<04:36,  1.37s/it]
Epoch 8/10:  45%|████▍     | 163/364 [03:31<03:38,  1.09s/it]
Epoch 8/10:  45%|████▌     | 164/364 [03:31<02:56,  1.13it/s]
Epoch 8/10:  45%|████▌     | 165/364 [03:34<05:33,  1.68s/it]
Epoch 8/10:  46%|████▌     | 166/364 [03:35<04:17,  1.30s/it]
Epoch 8/10:  46%|████▌     | 167/364 [03:35<03:22,  1.03s/it]
Epoch 8/10:  46%|████▌     | 168/364 [03:36<02:47,  1.17it/s]
Epoch 8/10:  46%|████▋     | 169/364 [03:40<05:59,  1.84s/it]
Epoch 8/10:  47%|████▋     | 170/364 [03:40<04:34,  1.42s/it]
Epoch 8/10:  47%|████▋     | 171/364 [03:41<03:36,  1.12s/it]
Epoch 8/10:  47%|████▋     | 172/364 [03:41<02:55,  1.09it/s]
Epoch 8/10:  48%|████▊     | 173/364 [03:45<05:48,  1.82s/it]
Epoch 8/10:  48%|████▊     | 174/364 [03:46<04:27,  1.41s/it]
Epoch 8/10:  48%|████▊     | 175/364 [03:46<03:30,  1.12s/it]
Epoch 8/10:  48%|████▊     | 176/364 [03:46<02:50,  1.10it/s]
Epoch 8/10:  49%|████▊     | 177/364 [03:50<05:21,  1.72s/it]
Epoch 8/10:  49%|████▉     | 178/364 [03:50<04:07,  1.33s/it]
Epoch 8/10:  49%|████▉     | 179/364 [03:51<03:16,  1.06s/it]
Epoch 8/10:  49%|████▉     | 180/364 [03:51<02:40,  1.15it/s]
Epoch 8/10:  50%|████▉     | 181/364 [03:55<05:32,  1.82s/it]
Epoch 8/10:  50%|█████     | 182/364 [03:56<04:15,  1.40s/it]
Epoch 8/10:  50%|█████     | 183/364 [03:56<03:20,  1.11s/it]
Epoch 8/10:  51%|█████     | 184/364 [03:57<02:42,  1.11it/s]
Epoch 8/10:  51%|█████     | 185/364 [04:01<05:24,  1.81s/it]
Epoch 8/10:  51%|█████     | 186/364 [04:01<04:07,  1.39s/it]
Epoch 8/10:  51%|█████▏    | 187/364 [04:01<03:14,  1.10s/it]
Epoch 8/10:  52%|█████▏    | 188/364 [04:02<02:37,  1.12it/s]
Epoch 8/10:  52%|█████▏    | 189/364 [04:06<05:19,  1.83s/it]
Epoch 8/10:  52%|█████▏    | 190/364 [04:06<04:04,  1.41s/it]
Epoch 8/10:  52%|█████▏    | 191/364 [04:07<03:12,  1.11s/it]
Epoch 8/10:  53%|█████▎    | 192/364 [04:07<02:36,  1.10it/s]
Epoch 8/10:  53%|█████▎    | 193/364 [04:11<04:51,  1.70s/it]
Epoch 8/10:  53%|█████▎    | 194/364 [04:11<03:44,  1.32s/it]
Epoch 8/10:  54%|█████▎    | 195/364 [04:11<02:57,  1.05s/it]
Epoch 8/10:  54%|█████▍    | 196/364 [04:12<02:25,  1.16it/s]
Epoch 8/10:  54%|█████▍    | 197/364 [04:16<04:47,  1.72s/it]
Epoch 8/10:  54%|█████▍    | 198/364 [04:16<03:41,  1.33s/it]
Epoch 8/10:  55%|█████▍    | 199/364 [04:16<02:53,  1.05s/it]
Epoch 8/10:  55%|█████▍    | 200/364 [04:17<02:21,  1.16it/s]
Epoch 8/10:  55%|█████▌    | 201/364 [04:21<04:48,  1.77s/it]
Epoch 8/10:  55%|█████▌    | 202/364 [04:21<03:41,  1.37s/it]
Epoch 8/10:  56%|█████▌    | 203/364 [04:22<02:54,  1.09s/it]
Epoch 8/10:  56%|█████▌    | 204/364 [04:22<02:22,  1.12it/s]
Epoch 8/10:  56%|█████▋    | 205/364 [04:26<04:44,  1.79s/it]
Epoch 8/10:  57%|█████▋    | 206/364 [04:26<03:37,  1.38s/it]
Epoch 8/10:  57%|█████▋    | 207/364 [04:27<02:51,  1.10s/it]
Epoch 8/10:  57%|█████▋    | 208/364 [04:27<02:20,  1.11it/s]
Epoch 8/10:  57%|█████▋    | 209/364 [04:31<04:20,  1.68s/it]
Epoch 8/10:  58%|█████▊    | 210/364 [04:31<03:22,  1.31s/it]
Epoch 8/10:  58%|█████▊    | 211/364 [04:32<02:40,  1.05s/it]
Epoch 8/10:  58%|█████▊    | 212/364 [04:32<02:10,  1.16it/s]
Epoch 8/10:  59%|█████▊    | 213/364 [04:36<04:13,  1.68s/it]
Epoch 8/10:  59%|█████▉    | 214/364 [04:36<03:14,  1.30s/it]
Epoch 8/10:  59%|█████▉    | 215/364 [04:36<02:34,  1.03s/it]
Epoch 8/10:  59%|█████▉    | 216/364 [04:37<02:06,  1.17it/s]
Epoch 8/10:  60%|█████▉    | 217/364 [04:41<04:22,  1.79s/it]
Epoch 8/10:  60%|█████▉    | 218/364 [04:41<03:21,  1.38s/it]
Epoch 8/10:  60%|██████    | 219/364 [04:42<02:38,  1.09s/it]
Epoch 8/10:  60%|██████    | 220/364 [04:42<02:08,  1.12it/s]
Epoch 8/10:  61%|██████    | 221/364 [04:46<04:25,  1.86s/it]
Epoch 8/10:  61%|██████    | 222/364 [04:47<03:22,  1.43s/it]
Epoch 8/10:  61%|██████▏   | 223/364 [04:47<02:39,  1.13s/it]
Epoch 8/10:  62%|██████▏   | 224/364 [04:48<02:08,  1.09it/s]
Epoch 8/10:  62%|██████▏   | 225/364 [04:51<04:05,  1.76s/it]
Epoch 8/10:  62%|██████▏   | 226/364 [04:52<03:07,  1.36s/it]
Epoch 8/10:  62%|██████▏   | 227/364 [04:52<02:27,  1.08s/it]
Epoch 8/10:  63%|██████▎   | 228/364 [04:53<01:59,  1.14it/s]
Epoch 8/10:  63%|██████▎   | 229/364 [04:56<03:57,  1.76s/it]
Epoch 8/10:  63%|██████▎   | 230/364 [04:57<03:03,  1.37s/it]
Epoch 8/10:  63%|██████▎   | 231/364 [04:57<02:24,  1.09s/it]
Epoch 8/10:  64%|██████▎   | 232/364 [04:58<01:57,  1.13it/s]
Epoch 8/10:  64%|██████▍   | 233/364 [05:02<04:00,  1.84s/it]
Epoch 8/10:  64%|██████▍   | 234/364 [05:02<03:03,  1.41s/it]
Epoch 8/10:  65%|██████▍   | 235/364 [05:03<02:24,  1.12s/it]
Epoch 8/10:  65%|██████▍   | 236/364 [05:03<01:57,  1.09it/s]
Epoch 8/10:  65%|██████▌   | 237/364 [05:07<03:38,  1.72s/it]
Epoch 8/10:  65%|██████▌   | 238/364 [05:07<02:48,  1.34s/it]
Epoch 8/10:  66%|██████▌   | 239/364 [05:07<02:13,  1.06s/it]
Epoch 8/10:  66%|██████▌   | 240/364 [05:08<01:47,  1.15it/s]
Epoch 8/10:  66%|██████▌   | 241/364 [05:12<03:46,  1.84s/it]
Epoch 8/10:  66%|██████▋   | 242/364 [05:12<02:53,  1.42s/it]
Epoch 8/10:  67%|██████▋   | 243/364 [05:13<02:15,  1.12s/it]
Epoch 8/10:  67%|██████▋   | 244/364 [05:13<01:49,  1.10it/s]
Epoch 8/10:  67%|██████▋   | 245/364 [05:17<03:26,  1.73s/it]
Epoch 8/10:  68%|██████▊   | 246/364 [05:17<02:38,  1.34s/it]
Epoch 8/10:  68%|██████▊   | 247/364 [05:18<02:04,  1.06s/it]
Epoch 8/10:  68%|██████▊   | 248/364 [05:18<01:41,  1.14it/s]
Epoch 8/10:  68%|██████▊   | 249/364 [05:22<03:26,  1.80s/it]
Epoch 8/10:  69%|██████▊   | 250/364 [05:23<02:38,  1.39s/it]
Epoch 8/10:  69%|██████▉   | 251/364 [05:23<02:04,  1.10s/it]
Epoch 8/10:  69%|██████▉   | 252/364 [05:23<01:40,  1.11it/s]
Epoch 8/10:  70%|██████▉   | 253/364 [05:27<03:09,  1.71s/it]
Epoch 8/10:  70%|██████▉   | 254/364 [05:27<02:25,  1.32s/it]
Epoch 8/10:  70%|███████   | 255/364 [05:28<01:54,  1.05s/it]
Epoch 8/10:  70%|███████   | 256/364 [05:28<01:32,  1.16it/s]
Epoch 8/10:  71%|███████   | 257/364 [05:32<03:08,  1.76s/it]
Epoch 8/10:  71%|███████   | 258/364 [05:33<02:24,  1.36s/it]
Epoch 8/10:  71%|███████   | 259/364 [05:33<01:53,  1.08s/it]
Epoch 8/10:  71%|███████▏  | 260/364 [05:33<01:32,  1.13it/s]
Epoch 8/10:  72%|███████▏  | 261/364 [05:37<02:52,  1.68s/it]
Epoch 8/10:  72%|███████▏  | 262/364 [05:37<02:12,  1.30s/it]
Epoch 8/10:  72%|███████▏  | 263/364 [05:38<01:45,  1.04s/it]
Epoch 8/10:  73%|███████▎  | 264/364 [05:38<01:25,  1.17it/s]
Epoch 8/10:  73%|███████▎  | 265/364 [05:42<03:01,  1.83s/it]
Epoch 8/10:  73%|███████▎  | 266/364 [05:43<02:18,  1.41s/it]
Epoch 8/10:  73%|███████▎  | 267/364 [05:43<01:47,  1.11s/it]
Epoch 8/10:  74%|███████▎  | 268/364 [05:44<01:26,  1.10it/s]
Epoch 8/10:  74%|███████▍  | 269/364 [05:47<02:48,  1.77s/it]
Epoch 8/10:  74%|███████▍  | 270/364 [05:48<02:09,  1.37s/it]
Epoch 8/10:  74%|███████▍  | 271/364 [05:48<01:41,  1.09s/it]
Epoch 8/10:  75%|███████▍  | 272/364 [05:49<01:21,  1.12it/s]
Epoch 8/10:  75%|███████▌  | 273/364 [05:52<02:39,  1.76s/it]
Epoch 8/10:  75%|███████▌  | 274/364 [05:53<02:01,  1.35s/it]
Epoch 8/10:  76%|███████▌  | 275/364 [05:53<01:36,  1.08s/it]
Epoch 8/10:  76%|███████▌  | 276/364 [05:54<01:18,  1.13it/s]
Epoch 8/10:  76%|███████▌  | 277/364 [05:57<02:30,  1.73s/it]
Epoch 8/10:  76%|███████▋  | 278/364 [05:58<01:54,  1.34s/it]
Epoch 8/10:  77%|███████▋  | 279/364 [05:58<01:31,  1.07s/it]
Epoch 8/10:  77%|███████▋  | 280/364 [05:59<01:13,  1.14it/s]
Epoch 8/10:  77%|███████▋  | 281/364 [06:02<02:19,  1.68s/it]
Epoch 8/10:  77%|███████▋  | 282/364 [06:03<01:46,  1.30s/it]
Epoch 8/10:  78%|███████▊  | 283/364 [06:03<01:23,  1.03s/it]
Epoch 8/10:  78%|███████▊  | 284/364 [06:04<01:08,  1.18it/s]
Epoch 8/10:  78%|███████▊  | 285/364 [06:08<02:24,  1.82s/it]
Epoch 8/10:  79%|███████▊  | 286/364 [06:08<01:50,  1.41s/it]
Epoch 8/10:  79%|███████▉  | 287/364 [06:09<01:26,  1.12s/it]
Epoch 8/10:  79%|███████▉  | 288/364 [06:09<01:09,  1.10it/s]
Epoch 8/10:  79%|███████▉  | 289/364 [06:13<02:07,  1.70s/it]
Epoch 8/10:  80%|███████▉  | 290/364 [06:13<01:37,  1.32s/it]
Epoch 8/10:  80%|███████▉  | 291/364 [06:13<01:16,  1.05s/it]
Epoch 8/10:  80%|████████  | 292/364 [06:14<01:02,  1.15it/s]
Epoch 8/10:  80%|████████  | 293/364 [06:18<02:10,  1.83s/it]
Epoch 8/10:  81%|████████  | 294/364 [06:18<01:38,  1.41s/it]
Epoch 8/10:  81%|████████  | 295/364 [06:19<01:17,  1.12s/it]
Epoch 8/10:  81%|████████▏ | 296/364 [06:19<01:01,  1.10it/s]
Epoch 8/10:  82%|████████▏ | 297/364 [06:23<01:55,  1.72s/it]
Epoch 8/10:  82%|████████▏ | 298/364 [06:23<01:27,  1.33s/it]
Epoch 8/10:  82%|████████▏ | 299/364 [06:24<01:08,  1.05s/it]
Epoch 8/10:  82%|████████▏ | 300/364 [06:24<00:56,  1.14it/s]
Epoch 8/10:  83%|████████▎ | 301/364 [06:28<01:48,  1.73s/it]
Epoch 8/10:  83%|████████▎ | 302/364 [06:28<01:22,  1.34s/it]
Epoch 8/10:  83%|████████▎ | 303/364 [06:29<01:04,  1.06s/it]
Epoch 8/10:  84%|████████▎ | 304/364 [06:29<00:52,  1.14it/s]
Epoch 8/10:  84%|████████▍ | 305/364 [06:33<01:43,  1.75s/it]
Epoch 8/10:  84%|████████▍ | 306/364 [06:33<01:18,  1.35s/it]
Epoch 8/10:  84%|████████▍ | 307/364 [06:34<01:01,  1.08s/it]
Epoch 8/10:  85%|████████▍ | 308/364 [06:34<00:49,  1.14it/s]
Epoch 8/10:  85%|████████▍ | 309/364 [06:38<01:41,  1.85s/it]
Epoch 8/10:  85%|████████▌ | 310/364 [06:39<01:16,  1.42s/it]
Epoch 8/10:  85%|████████▌ | 311/364 [06:39<00:59,  1.12s/it]
Epoch 8/10:  86%|████████▌ | 312/364 [06:40<00:47,  1.09it/s]
Epoch 8/10:  86%|████████▌ | 313/364 [06:43<01:30,  1.78s/it]
Epoch 8/10:  86%|████████▋ | 314/364 [06:44<01:08,  1.37s/it]
Epoch 8/10:  87%|████████▋ | 315/364 [06:44<00:53,  1.09s/it]
Epoch 8/10:  87%|████████▋ | 316/364 [06:45<00:42,  1.12it/s]
Epoch 8/10:  87%|████████▋ | 317/364 [06:48<01:21,  1.73s/it]
Epoch 8/10:  87%|████████▋ | 318/364 [06:49<01:01,  1.34s/it]
Epoch 8/10:  88%|████████▊ | 319/364 [06:49<00:48,  1.07s/it]
Epoch 8/10:  88%|████████▊ | 320/364 [06:50<00:38,  1.14it/s]
Epoch 8/10:  88%|████████▊ | 321/364 [06:53<01:14,  1.73s/it]
Epoch 8/10:  88%|████████▊ | 322/364 [06:54<00:56,  1.35s/it]
Epoch 8/10:  89%|████████▊ | 323/364 [06:54<00:43,  1.07s/it]
Epoch 8/10:  89%|████████▉ | 324/364 [06:55<00:35,  1.13it/s]
Epoch 8/10:  89%|████████▉ | 325/364 [06:59<01:12,  1.86s/it]
Epoch 8/10:  90%|████████▉ | 326/364 [06:59<00:54,  1.43s/it]
Epoch 8/10:  90%|████████▉ | 327/364 [07:00<00:41,  1.13s/it]
Epoch 8/10:  90%|█████████ | 328/364 [07:00<00:33,  1.09it/s]
Epoch 8/10:  90%|█████████ | 329/364 [07:04<00:59,  1.69s/it]
Epoch 8/10:  91%|█████████ | 330/364 [07:04<00:44,  1.31s/it]
Epoch 8/10:  91%|█████████ | 331/364 [07:04<00:34,  1.05s/it]
Epoch 8/10:  91%|█████████ | 332/364 [07:05<00:27,  1.16it/s]
Epoch 8/10:  91%|█████████▏| 333/364 [07:09<00:54,  1.75s/it]
Epoch 8/10:  92%|█████████▏| 334/364 [07:09<00:40,  1.36s/it]
Epoch 8/10:  92%|█████████▏| 335/364 [07:10<00:31,  1.08s/it]
Epoch 8/10:  92%|█████████▏| 336/364 [07:10<00:24,  1.13it/s]
Epoch 8/10:  93%|█████████▎| 337/364 [07:14<00:47,  1.74s/it]
Epoch 8/10:  93%|█████████▎| 338/364 [07:14<00:35,  1.35s/it]
Epoch 8/10:  93%|█████████▎| 339/364 [07:15<00:26,  1.07s/it]
Epoch 8/10:  93%|█████████▎| 340/364 [07:15<00:21,  1.14it/s]
Epoch 8/10:  94%|█████████▎| 341/364 [07:19<00:41,  1.82s/it]
Epoch 8/10:  94%|█████████▍| 342/364 [07:19<00:30,  1.40s/it]
Epoch 8/10:  94%|█████████▍| 343/364 [07:20<00:23,  1.10s/it]
Epoch 8/10:  95%|█████████▍| 344/364 [07:20<00:17,  1.11it/s]
Epoch 8/10:  95%|█████████▍| 345/364 [07:24<00:33,  1.74s/it]
Epoch 8/10:  95%|█████████▌| 346/364 [07:24<00:24,  1.35s/it]
Epoch 8/10:  95%|█████████▌| 347/364 [07:25<00:18,  1.09s/it]
Epoch 8/10:  96%|█████████▌| 348/364 [07:25<00:14,  1.13it/s]
Epoch 8/10:  96%|█████████▌| 349/364 [07:29<00:25,  1.72s/it]
Epoch 8/10:  96%|█████████▌| 350/364 [07:29<00:18,  1.33s/it]
Epoch 8/10:  96%|█████████▋| 351/364 [07:30<00:13,  1.06s/it]
Epoch 8/10:  97%|█████████▋| 352/364 [07:30<00:10,  1.14it/s]
Epoch 8/10:  97%|█████████▋| 353/364 [07:34<00:18,  1.71s/it]
Epoch 8/10:  97%|█████████▋| 354/364 [07:34<00:13,  1.33s/it]
Epoch 8/10:  98%|█████████▊| 355/364 [07:35<00:09,  1.06s/it]
Epoch 8/10:  98%|█████████▊| 356/364 [07:35<00:06,  1.15it/s]
Epoch 8/10:  98%|█████████▊| 357/364 [07:39<00:12,  1.74s/it]
Epoch 8/10:  98%|█████████▊| 358/364 [07:39<00:08,  1.35s/it]
Epoch 8/10:  99%|█████████▊| 359/364 [07:40<00:05,  1.07s/it]
Epoch 8/10:  99%|█████████▉| 360/364 [07:40<00:03,  1.14it/s]
Epoch 8/10:  99%|█████████▉| 361/364 [07:42<00:03,  1.20s/it]
Epoch 8/10:  99%|█████████▉| 362/364 [07:43<00:01,  1.04it/s]
Epoch 8/10: 100%|█████████▉| 363/364 [07:43<00:00,  1.26it/s]
Epoch 8/10: 100%|██████████| 364/364 [07:43<00:00,  1.64it/s]
Epoch 8/10: 100%|██████████| 364/364 [07:43<00:00,  1.27s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:35,  4.92s/it]
Validating:   4%|▎         | 2/57 [00:05<01:56,  2.12s/it]
Validating:   5%|▌         | 3/57 [00:05<01:05,  1.21s/it]
Validating:   7%|▋         | 4/57 [00:05<00:41,  1.27it/s]
Validating:   9%|▉         | 5/57 [00:09<01:44,  2.00s/it]
Validating:  11%|█         | 6/57 [00:09<01:10,  1.38s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:49,  1.02it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:34,  1.41it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:17,  1.61s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:55,  1.19s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:39,  1.15it/s]
Validating:  21%|██        | 12/57 [00:14<00:29,  1.55it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:13,  1.68s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:52,  1.22s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:37,  1.12it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:27,  1.51it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:04,  1.61s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:45,  1.18s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:32,  1.15it/s]
Validating:  35%|███▌      | 20/57 [00:22<00:23,  1.55it/s]
Validating:  37%|███▋      | 21/57 [00:26<01:00,  1.67s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:44,  1.28s/it]
Validating:  40%|████      | 23/57 [00:27<00:32,  1.06it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:23,  1.42it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:52,  1.64s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:39,  1.27s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:28,  1.07it/s]
Validating:  49%|████▉     | 28/57 [00:32<00:20,  1.42it/s]
Validating:  51%|█████     | 29/57 [00:35<00:44,  1.58s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:32,  1.21s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:23,  1.12it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.50it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:38,  1.61s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:28,  1.22s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:19,  1.10it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.48it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:34,  1.73s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:24,  1.28s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:16,  1.07it/s]
Validating:  70%|███████   | 40/57 [00:45<00:11,  1.45it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:27,  1.72s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:19,  1.33s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:13,  1.02it/s]
Validating:  77%|███████▋  | 44/57 [00:50<00:09,  1.38it/s]
Validating:  79%|███████▉  | 45/57 [00:53<00:19,  1.62s/it]
Validating:  81%|████████  | 46/57 [00:54<00:14,  1.34s/it]
Validating:  82%|████████▏ | 47/57 [00:54<00:09,  1.02it/s]
Validating:  84%|████████▍ | 48/57 [00:54<00:06,  1.37it/s]
Validating:  86%|████████▌ | 49/57 [00:57<00:10,  1.37s/it]
Validating:  88%|████████▊ | 50/57 [00:58<00:09,  1.29s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:05,  1.05it/s]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.41it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:04,  1.17s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.11s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.21it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.63it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 8: Train Loss: 0.0490, Train Acc: 0.9856 | Val Loss: 0.3649, Val Acc: 0.8900

Epoch 9/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 9/10:   0%|          | 1/364 [00:05<33:53,  5.60s/it]
Epoch 9/10:   1%|          | 2/364 [00:06<15:27,  2.56s/it]
Epoch 9/10:   1%|          | 3/364 [00:06<09:32,  1.59s/it]
Epoch 9/10:   1%|          | 4/364 [00:06<06:45,  1.13s/it]
Epoch 9/10:   1%|▏         | 5/364 [00:10<12:01,  2.01s/it]
Epoch 9/10:   2%|▏         | 6/364 [00:10<08:47,  1.47s/it]
Epoch 9/10:   2%|▏         | 7/364 [00:11<06:45,  1.13s/it]
Epoch 9/10:   2%|▏         | 8/364 [00:11<05:26,  1.09it/s]
Epoch 9/10:   2%|▏         | 9/364 [00:15<10:55,  1.85s/it]
Epoch 9/10:   3%|▎         | 10/364 [00:16<08:17,  1.40s/it]
Epoch 9/10:   3%|▎         | 11/364 [00:16<06:29,  1.10s/it]
Epoch 9/10:   3%|▎         | 12/364 [00:16<05:18,  1.11it/s]
Epoch 9/10:   4%|▎         | 13/364 [00:20<10:21,  1.77s/it]
Epoch 9/10:   4%|▍         | 14/364 [00:21<07:56,  1.36s/it]
Epoch 9/10:   4%|▍         | 15/364 [00:21<06:15,  1.08s/it]
Epoch 9/10:   4%|▍         | 16/364 [00:21<05:06,  1.14it/s]
Epoch 9/10:   5%|▍         | 17/364 [00:25<09:49,  1.70s/it]
Epoch 9/10:   5%|▍         | 18/364 [00:26<08:53,  1.54s/it]
Epoch 9/10:   5%|▌         | 19/364 [00:27<06:55,  1.20s/it]
Epoch 9/10:   5%|▌         | 20/364 [00:27<05:35,  1.03it/s]
Epoch 9/10:   6%|▌         | 21/364 [00:31<09:45,  1.71s/it]
Epoch 9/10:   6%|▌         | 22/364 [00:31<07:32,  1.32s/it]
Epoch 9/10:   6%|▋         | 23/364 [00:31<05:59,  1.05s/it]
Epoch 9/10:   7%|▋         | 24/364 [00:32<04:56,  1.15it/s]
Epoch 9/10:   7%|▋         | 25/364 [00:36<10:02,  1.78s/it]
Epoch 9/10:   7%|▋         | 26/364 [00:36<07:42,  1.37s/it]
Epoch 9/10:   7%|▋         | 27/364 [00:37<06:07,  1.09s/it]
Epoch 9/10:   8%|▊         | 28/364 [00:37<04:58,  1.13it/s]
Epoch 9/10:   8%|▊         | 29/364 [00:41<10:01,  1.79s/it]
Epoch 9/10:   8%|▊         | 30/364 [00:41<07:43,  1.39s/it]
Epoch 9/10:   9%|▊         | 31/364 [00:42<06:04,  1.10s/it]
Epoch 9/10:   9%|▉         | 32/364 [00:42<04:58,  1.11it/s]
Epoch 9/10:   9%|▉         | 33/364 [00:46<09:30,  1.72s/it]
Epoch 9/10:   9%|▉         | 34/364 [00:46<07:19,  1.33s/it]
Epoch 9/10:  10%|▉         | 35/364 [00:47<05:47,  1.06s/it]
Epoch 9/10:  10%|▉         | 36/364 [00:47<04:45,  1.15it/s]
Epoch 9/10:  10%|█         | 37/364 [00:51<09:40,  1.77s/it]
Epoch 9/10:  10%|█         | 38/364 [00:51<07:29,  1.38s/it]
Epoch 9/10:  11%|█         | 39/364 [00:52<05:55,  1.09s/it]
Epoch 9/10:  11%|█         | 40/364 [00:52<04:48,  1.12it/s]
Epoch 9/10:  11%|█▏        | 41/364 [00:56<09:21,  1.74s/it]
Epoch 9/10:  12%|█▏        | 42/364 [00:56<07:12,  1.34s/it]
Epoch 9/10:  12%|█▏        | 43/364 [00:57<05:44,  1.07s/it]
Epoch 9/10:  12%|█▏        | 44/364 [00:57<04:41,  1.14it/s]
Epoch 9/10:  12%|█▏        | 45/364 [01:01<09:25,  1.77s/it]
Epoch 9/10:  13%|█▎        | 46/364 [01:02<07:14,  1.37s/it]
Epoch 9/10:  13%|█▎        | 47/364 [01:02<05:46,  1.09s/it]
Epoch 9/10:  13%|█▎        | 48/364 [01:02<04:40,  1.13it/s]
Epoch 9/10:  13%|█▎        | 49/364 [01:06<09:00,  1.72s/it]
Epoch 9/10:  14%|█▎        | 50/364 [01:07<06:57,  1.33s/it]
Epoch 9/10:  14%|█▍        | 51/364 [01:07<05:32,  1.06s/it]
Epoch 9/10:  14%|█▍        | 52/364 [01:07<04:31,  1.15it/s]
Epoch 9/10:  15%|█▍        | 53/364 [01:11<09:12,  1.78s/it]
Epoch 9/10:  15%|█▍        | 54/364 [01:12<07:08,  1.38s/it]
Epoch 9/10:  15%|█▌        | 55/364 [01:12<05:37,  1.09s/it]
Epoch 9/10:  15%|█▌        | 56/364 [01:13<04:35,  1.12it/s]
Epoch 9/10:  16%|█▌        | 57/364 [01:16<08:56,  1.75s/it]
Epoch 9/10:  16%|█▌        | 58/364 [01:17<06:53,  1.35s/it]
Epoch 9/10:  16%|█▌        | 59/364 [01:17<05:26,  1.07s/it]
Epoch 9/10:  16%|█▋        | 60/364 [01:18<04:25,  1.14it/s]
Epoch 9/10:  17%|█▋        | 61/364 [01:22<09:27,  1.87s/it]
Epoch 9/10:  17%|█▋        | 62/364 [01:22<07:14,  1.44s/it]
Epoch 9/10:  17%|█▋        | 63/364 [01:23<05:43,  1.14s/it]
Epoch 9/10:  18%|█▊        | 64/364 [01:23<04:39,  1.07it/s]
Epoch 9/10:  18%|█▊        | 65/364 [01:27<08:48,  1.77s/it]
Epoch 9/10:  18%|█▊        | 66/364 [01:27<06:47,  1.37s/it]
Epoch 9/10:  18%|█▊        | 67/364 [01:28<05:23,  1.09s/it]
Epoch 9/10:  19%|█▊        | 68/364 [01:28<04:23,  1.12it/s]
Epoch 9/10:  19%|█▉        | 69/364 [01:32<09:00,  1.83s/it]
Epoch 9/10:  19%|█▉        | 70/364 [01:33<06:54,  1.41s/it]
Epoch 9/10:  20%|█▉        | 71/364 [01:33<05:26,  1.11s/it]
Epoch 9/10:  20%|█▉        | 72/364 [01:33<04:24,  1.10it/s]
Epoch 9/10:  20%|██        | 73/364 [01:37<08:37,  1.78s/it]
Epoch 9/10:  20%|██        | 74/364 [01:38<06:38,  1.38s/it]
Epoch 9/10:  21%|██        | 75/364 [01:38<05:13,  1.09s/it]
Epoch 9/10:  21%|██        | 76/364 [01:38<04:16,  1.12it/s]
Epoch 9/10:  21%|██        | 77/364 [01:43<08:59,  1.88s/it]
Epoch 9/10:  21%|██▏       | 78/364 [01:43<06:53,  1.45s/it]
Epoch 9/10:  22%|██▏       | 79/364 [01:44<05:24,  1.14s/it]
Epoch 9/10:  22%|██▏       | 80/364 [01:44<04:22,  1.08it/s]
Epoch 9/10:  22%|██▏       | 81/364 [01:48<09:01,  1.91s/it]
Epoch 9/10:  23%|██▎       | 82/364 [01:49<06:52,  1.46s/it]
Epoch 9/10:  23%|██▎       | 83/364 [01:49<05:24,  1.15s/it]
Epoch 9/10:  23%|██▎       | 84/364 [01:49<04:22,  1.07it/s]
Epoch 9/10:  23%|██▎       | 85/364 [01:53<08:14,  1.77s/it]
Epoch 9/10:  24%|██▎       | 86/364 [01:54<06:21,  1.37s/it]
Epoch 9/10:  24%|██▍       | 87/364 [01:54<05:03,  1.10s/it]
Epoch 9/10:  24%|██▍       | 88/364 [01:54<04:06,  1.12it/s]
Epoch 9/10:  24%|██▍       | 89/364 [01:58<07:53,  1.72s/it]
Epoch 9/10:  25%|██▍       | 90/364 [01:59<06:05,  1.33s/it]
Epoch 9/10:  25%|██▌       | 91/364 [01:59<04:49,  1.06s/it]
Epoch 9/10:  25%|██▌       | 92/364 [01:59<03:57,  1.15it/s]
Epoch 9/10:  26%|██▌       | 93/364 [02:03<07:52,  1.74s/it]
Epoch 9/10:  26%|██▌       | 94/364 [02:04<06:02,  1.34s/it]
Epoch 9/10:  26%|██▌       | 95/364 [02:04<04:47,  1.07s/it]
Epoch 9/10:  26%|██▋       | 96/364 [02:04<03:54,  1.14it/s]
Epoch 9/10:  27%|██▋       | 97/364 [02:08<07:49,  1.76s/it]
Epoch 9/10:  27%|██▋       | 98/364 [02:09<06:00,  1.36s/it]
Epoch 9/10:  27%|██▋       | 99/364 [02:09<04:45,  1.08s/it]
Epoch 9/10:  27%|██▋       | 100/364 [02:10<03:55,  1.12it/s]
Epoch 9/10:  28%|██▊       | 101/364 [02:13<07:39,  1.75s/it]
Epoch 9/10:  28%|██▊       | 102/364 [02:14<05:54,  1.35s/it]
Epoch 9/10:  28%|██▊       | 103/364 [02:14<04:40,  1.07s/it]
Epoch 9/10:  29%|██▊       | 104/364 [02:15<03:49,  1.13it/s]
Epoch 9/10:  29%|██▉       | 105/364 [02:19<07:44,  1.79s/it]
Epoch 9/10:  29%|██▉       | 106/364 [02:19<05:56,  1.38s/it]
Epoch 9/10:  29%|██▉       | 107/364 [02:19<04:42,  1.10s/it]
Epoch 9/10:  30%|██▉       | 108/364 [02:20<03:49,  1.12it/s]
Epoch 9/10:  30%|██▉       | 109/364 [02:23<07:10,  1.69s/it]
Epoch 9/10:  30%|███       | 110/364 [02:24<05:32,  1.31s/it]
Epoch 9/10:  30%|███       | 111/364 [02:24<04:23,  1.04s/it]
Epoch 9/10:  31%|███       | 112/364 [02:25<03:36,  1.16it/s]
Epoch 9/10:  31%|███       | 113/364 [02:29<07:42,  1.84s/it]
Epoch 9/10:  31%|███▏      | 114/364 [02:29<05:53,  1.42s/it]
Epoch 9/10:  32%|███▏      | 115/364 [02:30<04:38,  1.12s/it]
Epoch 9/10:  32%|███▏      | 116/364 [02:30<03:46,  1.10it/s]
Epoch 9/10:  32%|███▏      | 117/364 [02:34<07:30,  1.83s/it]
Epoch 9/10:  32%|███▏      | 118/364 [02:34<05:45,  1.41s/it]
Epoch 9/10:  33%|███▎      | 119/364 [02:35<04:33,  1.11s/it]
Epoch 9/10:  33%|███▎      | 120/364 [02:35<03:41,  1.10it/s]
Epoch 9/10:  33%|███▎      | 121/364 [02:39<07:31,  1.86s/it]
Epoch 9/10:  34%|███▎      | 122/364 [02:40<05:47,  1.43s/it]
Epoch 9/10:  34%|███▍      | 123/364 [02:40<04:33,  1.13s/it]
Epoch 9/10:  34%|███▍      | 124/364 [02:41<03:40,  1.09it/s]
Epoch 9/10:  34%|███▍      | 125/364 [02:45<07:12,  1.81s/it]
Epoch 9/10:  35%|███▍      | 126/364 [02:45<05:32,  1.40s/it]
Epoch 9/10:  35%|███▍      | 127/364 [02:45<04:21,  1.10s/it]
Epoch 9/10:  35%|███▌      | 128/364 [02:46<03:31,  1.11it/s]
Epoch 9/10:  35%|███▌      | 129/364 [02:50<06:52,  1.76s/it]
Epoch 9/10:  36%|███▌      | 130/364 [02:50<05:17,  1.36s/it]
Epoch 9/10:  36%|███▌      | 131/364 [02:50<04:10,  1.08s/it]
Epoch 9/10:  36%|███▋      | 132/364 [02:51<03:24,  1.14it/s]
Epoch 9/10:  37%|███▋      | 133/364 [02:55<06:50,  1.78s/it]
Epoch 9/10:  37%|███▋      | 134/364 [02:55<05:15,  1.37s/it]
Epoch 9/10:  37%|███▋      | 135/364 [02:56<04:09,  1.09s/it]
Epoch 9/10:  37%|███▋      | 136/364 [02:56<03:23,  1.12it/s]
Epoch 9/10:  38%|███▊      | 137/364 [03:00<06:46,  1.79s/it]
Epoch 9/10:  38%|███▊      | 138/364 [03:00<05:11,  1.38s/it]
Epoch 9/10:  38%|███▊      | 139/364 [03:01<04:05,  1.09s/it]
Epoch 9/10:  38%|███▊      | 140/364 [03:01<03:18,  1.13it/s]
Epoch 9/10:  39%|███▊      | 141/364 [03:05<06:09,  1.66s/it]
Epoch 9/10:  39%|███▉      | 142/364 [03:05<04:47,  1.29s/it]
Epoch 9/10:  39%|███▉      | 143/364 [03:05<03:48,  1.03s/it]
Epoch 9/10:  40%|███▉      | 144/364 [03:06<03:07,  1.17it/s]
Epoch 9/10:  40%|███▉      | 145/364 [03:10<06:21,  1.74s/it]
Epoch 9/10:  40%|████      | 146/364 [03:10<04:53,  1.34s/it]
Epoch 9/10:  40%|████      | 147/364 [03:11<03:51,  1.07s/it]
Epoch 9/10:  41%|████      | 148/364 [03:11<03:08,  1.14it/s]
Epoch 9/10:  41%|████      | 149/364 [03:15<06:19,  1.76s/it]
Epoch 9/10:  41%|████      | 150/364 [03:15<04:52,  1.37s/it]
Epoch 9/10:  41%|████▏     | 151/364 [03:16<03:50,  1.08s/it]
Epoch 9/10:  42%|████▏     | 152/364 [03:16<03:07,  1.13it/s]
Epoch 9/10:  42%|████▏     | 153/364 [03:20<06:05,  1.73s/it]
Epoch 9/10:  42%|████▏     | 154/364 [03:20<04:41,  1.34s/it]
Epoch 9/10:  43%|████▎     | 155/364 [03:21<03:42,  1.06s/it]
Epoch 9/10:  43%|████▎     | 156/364 [03:21<03:01,  1.14it/s]
Epoch 9/10:  43%|████▎     | 157/364 [03:25<05:50,  1.69s/it]
Epoch 9/10:  43%|████▎     | 158/364 [03:25<04:30,  1.31s/it]
Epoch 9/10:  44%|████▎     | 159/364 [03:26<03:34,  1.05s/it]
Epoch 9/10:  44%|████▍     | 160/364 [03:26<02:56,  1.15it/s]
Epoch 9/10:  44%|████▍     | 161/364 [03:30<06:03,  1.79s/it]
Epoch 9/10:  45%|████▍     | 162/364 [03:30<04:38,  1.38s/it]
Epoch 9/10:  45%|████▍     | 163/364 [03:31<03:39,  1.09s/it]
Epoch 9/10:  45%|████▌     | 164/364 [03:31<02:58,  1.12it/s]
Epoch 9/10:  45%|████▌     | 165/364 [03:35<06:00,  1.81s/it]
Epoch 9/10:  46%|████▌     | 166/364 [03:36<04:36,  1.40s/it]
Epoch 9/10:  46%|████▌     | 167/364 [03:36<03:38,  1.11s/it]
Epoch 9/10:  46%|████▌     | 168/364 [03:36<02:56,  1.11it/s]
Epoch 9/10:  46%|████▋     | 169/364 [03:40<05:36,  1.72s/it]
Epoch 9/10:  47%|████▋     | 170/364 [03:41<04:19,  1.34s/it]
Epoch 9/10:  47%|████▋     | 171/364 [03:41<03:25,  1.07s/it]
Epoch 9/10:  47%|████▋     | 172/364 [03:41<02:47,  1.14it/s]
Epoch 9/10:  48%|████▊     | 173/364 [03:45<05:32,  1.74s/it]
Epoch 9/10:  48%|████▊     | 174/364 [03:46<04:16,  1.35s/it]
Epoch 9/10:  48%|████▊     | 175/364 [03:46<03:22,  1.07s/it]
Epoch 9/10:  48%|████▊     | 176/364 [03:46<02:44,  1.14it/s]
Epoch 9/10:  49%|████▊     | 177/364 [03:50<05:37,  1.81s/it]
Epoch 9/10:  49%|████▉     | 178/364 [03:51<04:19,  1.40s/it]
Epoch 9/10:  49%|████▉     | 179/364 [03:51<03:24,  1.11s/it]
Epoch 9/10:  49%|████▉     | 180/364 [03:52<02:46,  1.11it/s]
Epoch 9/10:  50%|████▉     | 181/364 [03:56<05:33,  1.82s/it]
Epoch 9/10:  50%|█████     | 182/364 [03:56<04:15,  1.41s/it]
Epoch 9/10:  50%|█████     | 183/364 [03:57<03:21,  1.11s/it]
Epoch 9/10:  51%|█████     | 184/364 [03:57<02:44,  1.10it/s]
Epoch 9/10:  51%|█████     | 185/364 [04:00<04:57,  1.66s/it]
Epoch 9/10:  51%|█████     | 186/364 [04:01<03:49,  1.29s/it]
Epoch 9/10:  51%|█████▏    | 187/364 [04:01<03:03,  1.04s/it]
Epoch 9/10:  52%|█████▏    | 188/364 [04:02<02:31,  1.16it/s]
Epoch 9/10:  52%|█████▏    | 189/364 [04:05<05:02,  1.73s/it]
Epoch 9/10:  52%|█████▏    | 190/364 [04:06<03:52,  1.34s/it]
Epoch 9/10:  52%|█████▏    | 191/364 [04:06<03:03,  1.06s/it]
Epoch 9/10:  53%|█████▎    | 192/364 [04:07<02:29,  1.15it/s]
Epoch 9/10:  53%|█████▎    | 193/364 [04:11<05:10,  1.81s/it]
Epoch 9/10:  53%|█████▎    | 194/364 [04:11<03:58,  1.40s/it]
Epoch 9/10:  54%|█████▎    | 195/364 [04:12<03:07,  1.11s/it]
Epoch 9/10:  54%|█████▍    | 196/364 [04:12<02:32,  1.11it/s]
Epoch 9/10:  54%|█████▍    | 197/364 [04:16<04:44,  1.70s/it]
Epoch 9/10:  54%|█████▍    | 198/364 [04:16<03:39,  1.32s/it]
Epoch 9/10:  55%|█████▍    | 199/364 [04:16<02:53,  1.05s/it]
Epoch 9/10:  55%|█████▍    | 200/364 [04:17<02:21,  1.16it/s]
Epoch 9/10:  55%|█████▌    | 201/364 [04:21<05:08,  1.89s/it]
Epoch 9/10:  55%|█████▌    | 202/364 [04:22<03:55,  1.46s/it]
Epoch 9/10:  56%|█████▌    | 203/364 [04:22<03:04,  1.15s/it]
Epoch 9/10:  56%|█████▌    | 204/364 [04:22<02:28,  1.08it/s]
Epoch 9/10:  56%|█████▋    | 205/364 [04:26<04:52,  1.84s/it]
Epoch 9/10:  57%|█████▋    | 206/364 [04:27<03:43,  1.41s/it]
Epoch 9/10:  57%|█████▋    | 207/364 [04:27<02:55,  1.12s/it]
Epoch 9/10:  57%|█████▋    | 208/364 [04:28<02:22,  1.09it/s]
Epoch 9/10:  57%|█████▋    | 209/364 [04:31<04:21,  1.69s/it]
Epoch 9/10:  58%|█████▊    | 210/364 [04:32<03:21,  1.31s/it]
Epoch 9/10:  58%|█████▊    | 211/364 [04:32<02:39,  1.04s/it]
Epoch 9/10:  58%|█████▊    | 212/364 [04:32<02:09,  1.17it/s]
Epoch 9/10:  59%|█████▊    | 213/364 [04:36<04:20,  1.73s/it]
Epoch 9/10:  59%|█████▉    | 214/364 [04:37<03:20,  1.34s/it]
Epoch 9/10:  59%|█████▉    | 215/364 [04:37<02:39,  1.07s/it]
Epoch 9/10:  59%|█████▉    | 216/364 [04:38<02:10,  1.14it/s]
Epoch 9/10:  60%|█████▉    | 217/364 [04:41<04:16,  1.74s/it]
Epoch 9/10:  60%|█████▉    | 218/364 [04:42<03:15,  1.34s/it]
Epoch 9/10:  60%|██████    | 219/364 [04:42<02:35,  1.08s/it]
Epoch 9/10:  60%|██████    | 220/364 [04:43<02:06,  1.14it/s]
Epoch 9/10:  61%|██████    | 221/364 [04:46<04:02,  1.70s/it]
Epoch 9/10:  61%|██████    | 222/364 [04:47<03:06,  1.31s/it]
Epoch 9/10:  61%|██████▏   | 223/364 [04:47<02:27,  1.05s/it]
Epoch 9/10:  62%|██████▏   | 224/364 [04:47<02:00,  1.16it/s]
Epoch 9/10:  62%|██████▏   | 225/364 [04:51<03:49,  1.65s/it]
Epoch 9/10:  62%|██████▏   | 226/364 [04:51<02:56,  1.28s/it]
Epoch 9/10:  62%|██████▏   | 227/364 [04:52<02:20,  1.03s/it]
Epoch 9/10:  63%|██████▎   | 228/364 [04:52<01:54,  1.18it/s]
Epoch 9/10:  63%|██████▎   | 229/364 [04:56<03:55,  1.75s/it]
Epoch 9/10:  63%|██████▎   | 230/364 [04:57<03:01,  1.35s/it]
Epoch 9/10:  63%|██████▎   | 231/364 [04:57<02:22,  1.07s/it]
Epoch 9/10:  64%|██████▎   | 232/364 [04:57<01:56,  1.14it/s]
Epoch 9/10:  64%|██████▍   | 233/364 [05:01<03:53,  1.78s/it]
Epoch 9/10:  64%|██████▍   | 234/364 [05:02<02:59,  1.38s/it]
Epoch 9/10:  65%|██████▍   | 235/364 [05:02<02:20,  1.09s/it]
Epoch 9/10:  65%|██████▍   | 236/364 [05:03<01:54,  1.12it/s]
Epoch 9/10:  65%|██████▌   | 237/364 [05:06<03:31,  1.67s/it]
Epoch 9/10:  65%|██████▌   | 238/364 [05:06<02:43,  1.30s/it]
Epoch 9/10:  66%|██████▌   | 239/364 [05:07<02:09,  1.04s/it]
Epoch 9/10:  66%|██████▌   | 240/364 [05:07<01:45,  1.17it/s]
Epoch 9/10:  66%|██████▌   | 241/364 [05:11<03:27,  1.69s/it]
Epoch 9/10:  66%|██████▋   | 242/364 [05:11<02:39,  1.31s/it]
Epoch 9/10:  67%|██████▋   | 243/364 [05:12<02:06,  1.04s/it]
Epoch 9/10:  67%|██████▋   | 244/364 [05:12<01:43,  1.16it/s]
Epoch 9/10:  67%|██████▋   | 245/364 [05:16<03:30,  1.77s/it]
Epoch 9/10:  68%|██████▊   | 246/364 [05:17<02:41,  1.37s/it]
Epoch 9/10:  68%|██████▊   | 247/364 [05:17<02:07,  1.09s/it]
Epoch 9/10:  68%|██████▊   | 248/364 [05:17<01:43,  1.12it/s]
Epoch 9/10:  68%|██████▊   | 249/364 [05:21<03:16,  1.71s/it]
Epoch 9/10:  69%|██████▊   | 250/364 [05:21<02:30,  1.32s/it]
Epoch 9/10:  69%|██████▉   | 251/364 [05:22<02:00,  1.06s/it]
Epoch 9/10:  69%|██████▉   | 252/364 [05:22<01:38,  1.14it/s]
Epoch 9/10:  70%|██████▉   | 253/364 [05:26<03:15,  1.76s/it]
Epoch 9/10:  70%|██████▉   | 254/364 [05:27<02:30,  1.37s/it]
Epoch 9/10:  70%|███████   | 255/364 [05:27<01:58,  1.08s/it]
Epoch 9/10:  70%|███████   | 256/364 [05:27<01:35,  1.13it/s]
Epoch 9/10:  71%|███████   | 257/364 [05:32<03:17,  1.84s/it]
Epoch 9/10:  71%|███████   | 258/364 [05:32<02:30,  1.42s/it]
Epoch 9/10:  71%|███████   | 259/364 [05:32<01:57,  1.12s/it]
Epoch 9/10:  71%|███████▏  | 260/364 [05:33<01:34,  1.10it/s]
Epoch 9/10:  72%|███████▏  | 261/364 [05:37<03:05,  1.80s/it]
Epoch 9/10:  72%|███████▏  | 262/364 [05:37<02:21,  1.39s/it]
Epoch 9/10:  72%|███████▏  | 263/364 [05:38<01:51,  1.11s/it]
Epoch 9/10:  73%|███████▎  | 264/364 [05:38<01:30,  1.10it/s]
Epoch 9/10:  73%|███████▎  | 265/364 [05:42<02:53,  1.76s/it]
Epoch 9/10:  73%|███████▎  | 266/364 [05:42<02:12,  1.35s/it]
Epoch 9/10:  73%|███████▎  | 267/364 [05:43<01:44,  1.08s/it]
Epoch 9/10:  74%|███████▎  | 268/364 [05:43<01:25,  1.13it/s]
Epoch 9/10:  74%|███████▍  | 269/364 [05:47<02:50,  1.80s/it]
Epoch 9/10:  74%|███████▍  | 270/364 [05:47<02:11,  1.39s/it]
Epoch 9/10:  74%|███████▍  | 271/364 [05:48<01:42,  1.10s/it]
Epoch 9/10:  75%|███████▍  | 272/364 [05:48<01:22,  1.11it/s]
Epoch 9/10:  75%|███████▌  | 273/364 [05:52<02:43,  1.79s/it]
Epoch 9/10:  75%|███████▌  | 274/364 [05:53<02:04,  1.39s/it]
Epoch 9/10:  76%|███████▌  | 275/364 [05:53<01:37,  1.09s/it]
Epoch 9/10:  76%|███████▌  | 276/364 [05:53<01:18,  1.12it/s]
Epoch 9/10:  76%|███████▌  | 277/364 [05:57<02:38,  1.83s/it]
Epoch 9/10:  76%|███████▋  | 278/364 [05:58<02:00,  1.41s/it]
Epoch 9/10:  77%|███████▋  | 279/364 [05:58<01:35,  1.12s/it]
Epoch 9/10:  77%|███████▋  | 280/364 [05:59<01:16,  1.10it/s]
Epoch 9/10:  77%|███████▋  | 281/364 [06:03<02:36,  1.88s/it]
Epoch 9/10:  77%|███████▋  | 282/364 [06:03<01:59,  1.45s/it]
Epoch 9/10:  78%|███████▊  | 283/364 [06:04<01:32,  1.14s/it]
Epoch 9/10:  78%|███████▊  | 284/364 [06:04<01:13,  1.08it/s]
Epoch 9/10:  78%|███████▊  | 285/364 [06:08<02:26,  1.85s/it]
Epoch 9/10:  79%|███████▊  | 286/364 [06:09<01:51,  1.43s/it]
Epoch 9/10:  79%|███████▉  | 287/364 [06:09<01:27,  1.13s/it]
Epoch 9/10:  79%|███████▉  | 288/364 [06:09<01:09,  1.09it/s]
Epoch 9/10:  79%|███████▉  | 289/364 [06:13<02:11,  1.76s/it]
Epoch 9/10:  80%|███████▉  | 290/364 [06:14<01:40,  1.36s/it]
Epoch 9/10:  80%|███████▉  | 291/364 [06:14<01:18,  1.08s/it]
Epoch 9/10:  80%|████████  | 292/364 [06:14<01:03,  1.13it/s]
Epoch 9/10:  80%|████████  | 293/364 [06:18<02:02,  1.72s/it]
Epoch 9/10:  81%|████████  | 294/364 [06:19<01:33,  1.33s/it]
Epoch 9/10:  81%|████████  | 295/364 [06:19<01:13,  1.06s/it]
Epoch 9/10:  81%|████████▏ | 296/364 [06:19<00:59,  1.15it/s]
Epoch 9/10:  82%|████████▏ | 297/364 [06:23<01:53,  1.69s/it]
Epoch 9/10:  82%|████████▏ | 298/364 [06:23<01:26,  1.31s/it]
Epoch 9/10:  82%|████████▏ | 299/364 [06:24<01:08,  1.05s/it]
Epoch 9/10:  82%|████████▏ | 300/364 [06:24<00:55,  1.15it/s]
Epoch 9/10:  83%|████████▎ | 301/364 [06:28<01:53,  1.80s/it]
Epoch 9/10:  83%|████████▎ | 302/364 [06:29<01:26,  1.39s/it]
Epoch 9/10:  83%|████████▎ | 303/364 [06:29<01:07,  1.11s/it]
Epoch 9/10:  84%|████████▎ | 304/364 [06:30<00:54,  1.10it/s]
Epoch 9/10:  84%|████████▍ | 305/364 [06:33<01:46,  1.80s/it]
Epoch 9/10:  84%|████████▍ | 306/364 [06:34<01:20,  1.39s/it]
Epoch 9/10:  84%|████████▍ | 307/364 [06:34<01:02,  1.10s/it]
Epoch 9/10:  85%|████████▍ | 308/364 [06:35<00:50,  1.12it/s]
Epoch 9/10:  85%|████████▍ | 309/364 [06:39<01:37,  1.77s/it]
Epoch 9/10:  85%|████████▌ | 310/364 [06:39<01:13,  1.37s/it]
Epoch 9/10:  85%|████████▌ | 311/364 [06:39<00:57,  1.08s/it]
Epoch 9/10:  86%|████████▌ | 312/364 [06:40<00:46,  1.13it/s]
Epoch 9/10:  86%|████████▌ | 313/364 [06:43<01:22,  1.62s/it]
Epoch 9/10:  86%|████████▋ | 314/364 [06:44<01:03,  1.26s/it]
Epoch 9/10:  87%|████████▋ | 315/364 [06:44<00:49,  1.02s/it]
Epoch 9/10:  87%|████████▋ | 316/364 [06:45<00:40,  1.18it/s]
Epoch 9/10:  87%|████████▋ | 317/364 [06:48<01:23,  1.77s/it]
Epoch 9/10:  87%|████████▋ | 318/364 [06:49<01:02,  1.37s/it]
Epoch 9/10:  88%|████████▊ | 319/364 [06:49<00:48,  1.08s/it]
Epoch 9/10:  88%|████████▊ | 320/364 [06:50<00:39,  1.13it/s]
Epoch 9/10:  88%|████████▊ | 321/364 [06:54<01:17,  1.80s/it]
Epoch 9/10:  88%|████████▊ | 322/364 [06:54<00:58,  1.39s/it]
Epoch 9/10:  89%|████████▊ | 323/364 [06:54<00:45,  1.10s/it]
Epoch 9/10:  89%|████████▉ | 324/364 [06:55<00:35,  1.11it/s]
Epoch 9/10:  89%|████████▉ | 325/364 [06:59<01:09,  1.78s/it]
Epoch 9/10:  90%|████████▉ | 326/364 [06:59<00:52,  1.37s/it]
Epoch 9/10:  90%|████████▉ | 327/364 [07:00<00:40,  1.09s/it]
Epoch 9/10:  90%|█████████ | 328/364 [07:00<00:31,  1.13it/s]
Epoch 9/10:  90%|█████████ | 329/364 [07:03<00:57,  1.65s/it]
Epoch 9/10:  91%|█████████ | 330/364 [07:04<00:43,  1.29s/it]
Epoch 9/10:  91%|█████████ | 331/364 [07:04<00:33,  1.03s/it]
Epoch 9/10:  91%|█████████ | 332/364 [07:05<00:27,  1.17it/s]
Epoch 9/10:  91%|█████████▏| 333/364 [07:08<00:52,  1.68s/it]
Epoch 9/10:  92%|█████████▏| 334/364 [07:09<00:39,  1.30s/it]
Epoch 9/10:  92%|█████████▏| 335/364 [07:09<00:30,  1.04s/it]
Epoch 9/10:  92%|█████████▏| 336/364 [07:10<00:23,  1.17it/s]
Epoch 9/10:  93%|█████████▎| 337/364 [07:14<00:47,  1.77s/it]
Epoch 9/10:  93%|█████████▎| 338/364 [07:14<00:35,  1.37s/it]
Epoch 9/10:  93%|█████████▎| 339/364 [07:14<00:27,  1.08s/it]
Epoch 9/10:  93%|█████████▎| 340/364 [07:15<00:21,  1.13it/s]
Epoch 9/10:  94%|█████████▎| 341/364 [07:18<00:37,  1.63s/it]
Epoch 9/10:  94%|█████████▍| 342/364 [07:19<00:28,  1.27s/it]
Epoch 9/10:  94%|█████████▍| 343/364 [07:19<00:21,  1.03s/it]
Epoch 9/10:  95%|█████████▍| 344/364 [07:20<00:16,  1.18it/s]
Epoch 9/10:  95%|█████████▍| 345/364 [07:23<00:32,  1.70s/it]
Epoch 9/10:  95%|█████████▌| 346/364 [07:24<00:23,  1.32s/it]
Epoch 9/10:  95%|█████████▌| 347/364 [07:24<00:17,  1.05s/it]
Epoch 9/10:  96%|█████████▌| 348/364 [07:24<00:13,  1.16it/s]
Epoch 9/10:  96%|█████████▌| 349/364 [07:28<00:25,  1.73s/it]
Epoch 9/10:  96%|█████████▌| 350/364 [07:29<00:18,  1.34s/it]
Epoch 9/10:  96%|█████████▋| 351/364 [07:29<00:13,  1.07s/it]
Epoch 9/10:  97%|█████████▋| 352/364 [07:30<00:10,  1.15it/s]
Epoch 9/10:  97%|█████████▋| 353/364 [07:34<00:20,  1.83s/it]
Epoch 9/10:  97%|█████████▋| 354/364 [07:34<00:14,  1.41s/it]
Epoch 9/10:  98%|█████████▊| 355/364 [07:34<00:10,  1.12s/it]
Epoch 9/10:  98%|█████████▊| 356/364 [07:35<00:07,  1.10it/s]
Epoch 9/10:  98%|█████████▊| 357/364 [07:39<00:12,  1.85s/it]
Epoch 9/10:  98%|█████████▊| 358/364 [07:39<00:08,  1.42s/it]
Epoch 9/10:  99%|█████████▊| 359/364 [07:40<00:05,  1.12s/it]
Epoch 9/10:  99%|█████████▉| 360/364 [07:40<00:03,  1.08it/s]
Epoch 9/10:  99%|█████████▉| 361/364 [07:43<00:04,  1.36s/it]
Epoch 9/10:  99%|█████████▉| 362/364 [07:43<00:02,  1.07s/it]
Epoch 9/10: 100%|█████████▉| 363/364 [07:43<00:00,  1.14it/s]
Epoch 9/10: 100%|██████████| 364/364 [07:44<00:00,  1.50it/s]
Epoch 9/10: 100%|██████████| 364/364 [07:44<00:00,  1.28s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:30,  4.83s/it]
Validating:   4%|▎         | 2/57 [00:05<01:55,  2.09s/it]
Validating:   5%|▌         | 3/57 [00:05<01:05,  1.20s/it]
Validating:   7%|▋         | 4/57 [00:05<00:41,  1.29it/s]
Validating:   9%|▉         | 5/57 [00:09<01:38,  1.89s/it]
Validating:  11%|█         | 6/57 [00:09<01:06,  1.31s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:46,  1.07it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:33,  1.45it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:20,  1.67s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:56,  1.21s/it]
Validating:  19%|█▉        | 11/57 [00:13<00:40,  1.13it/s]
Validating:  21%|██        | 12/57 [00:13<00:29,  1.52it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:16,  1.74s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:54,  1.27s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:39,  1.07it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:28,  1.44it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:05,  1.65s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:46,  1.20s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:33,  1.14it/s]
Validating:  35%|███▌      | 20/57 [00:22<00:24,  1.51it/s]
Validating:  37%|███▋      | 21/57 [00:26<01:00,  1.67s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:42,  1.22s/it]
Validating:  40%|████      | 23/57 [00:27<00:30,  1.10it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:22,  1.48it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:53,  1.68s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:38,  1.23s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:27,  1.11it/s]
Validating:  49%|████▉     | 28/57 [00:31<00:19,  1.47it/s]
Validating:  51%|█████     | 29/57 [00:35<00:44,  1.59s/it]
Validating:  53%|█████▎    | 30/57 [00:36<00:32,  1.22s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:23,  1.11it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.48it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:38,  1.61s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:28,  1.23s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:19,  1.10it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:14,  1.47it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:34,  1.72s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:24,  1.31s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:17,  1.03it/s]
Validating:  70%|███████   | 40/57 [00:45<00:12,  1.39it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:26,  1.66s/it]
Validating:  74%|███████▎  | 42/57 [00:50<00:20,  1.34s/it]
Validating:  75%|███████▌  | 43/57 [00:50<00:13,  1.01it/s]
Validating:  77%|███████▋  | 44/57 [00:50<00:09,  1.36it/s]
Validating:  79%|███████▉  | 45/57 [00:53<00:18,  1.57s/it]
Validating:  81%|████████  | 46/57 [00:54<00:15,  1.39s/it]
Validating:  82%|████████▏ | 47/57 [00:54<00:10,  1.02s/it]
Validating:  84%|████████▍ | 48/57 [00:55<00:06,  1.33it/s]
Validating:  86%|████████▌ | 49/57 [00:57<00:10,  1.35s/it]
Validating:  88%|████████▊ | 50/57 [00:59<00:09,  1.35s/it]
Validating:  89%|████████▉ | 51/57 [00:59<00:05,  1.02it/s]
Validating:  91%|█████████ | 52/57 [00:59<00:03,  1.36it/s]
Validating:  93%|█████████▎| 53/57 [01:01<00:04,  1.20s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.01s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.34it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.79it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 9: Train Loss: 0.0388, Train Acc: 0.9892 | Val Loss: 0.3731, Val Acc: 0.8906

Epoch 10/10:   0%|          | 0/364 [00:00<?, ?it/s]
Epoch 10/10:   0%|          | 1/364 [00:05<34:19,  5.67s/it]
Epoch 10/10:   1%|          | 2/364 [00:06<15:33,  2.58s/it]
Epoch 10/10:   1%|          | 3/364 [00:06<09:35,  1.59s/it]
Epoch 10/10:   1%|          | 4/364 [00:06<06:46,  1.13s/it]
Epoch 10/10:   1%|▏         | 5/364 [00:10<12:17,  2.05s/it]
Epoch 10/10:   2%|▏         | 6/364 [00:11<08:56,  1.50s/it]
Epoch 10/10:   2%|▏         | 7/364 [00:11<06:47,  1.14s/it]
Epoch 10/10:   2%|▏         | 8/364 [00:11<05:25,  1.10it/s]
Epoch 10/10:   2%|▏         | 9/364 [00:15<10:52,  1.84s/it]
Epoch 10/10:   3%|▎         | 10/364 [00:16<08:18,  1.41s/it]
Epoch 10/10:   3%|▎         | 11/364 [00:16<06:33,  1.12s/it]
Epoch 10/10:   3%|▎         | 12/364 [00:17<05:17,  1.11it/s]
Epoch 10/10:   4%|▎         | 13/364 [00:20<10:33,  1.80s/it]
Epoch 10/10:   4%|▍         | 14/364 [00:21<08:06,  1.39s/it]
Epoch 10/10:   4%|▍         | 15/364 [00:21<06:23,  1.10s/it]
Epoch 10/10:   4%|▍         | 16/364 [00:22<05:14,  1.11it/s]
Epoch 10/10:   5%|▍         | 17/364 [00:26<10:29,  1.81s/it]
Epoch 10/10:   5%|▍         | 18/364 [00:26<08:04,  1.40s/it]
Epoch 10/10:   5%|▌         | 19/364 [00:27<06:22,  1.11s/it]
Epoch 10/10:   5%|▌         | 20/364 [00:27<05:09,  1.11it/s]
Epoch 10/10:   6%|▌         | 21/364 [00:31<10:03,  1.76s/it]
Epoch 10/10:   6%|▌         | 22/364 [00:31<07:44,  1.36s/it]
Epoch 10/10:   6%|▋         | 23/364 [00:32<06:08,  1.08s/it]
Epoch 10/10:   7%|▋         | 24/364 [00:32<05:00,  1.13it/s]
Epoch 10/10:   7%|▋         | 25/364 [00:36<09:29,  1.68s/it]
Epoch 10/10:   7%|▋         | 26/364 [00:36<07:21,  1.30s/it]
Epoch 10/10:   7%|▋         | 27/364 [00:36<05:54,  1.05s/it]
Epoch 10/10:   8%|▊         | 28/364 [00:37<04:51,  1.15it/s]
Epoch 10/10:   8%|▊         | 29/364 [00:41<09:38,  1.73s/it]
Epoch 10/10:   8%|▊         | 30/364 [00:41<07:25,  1.34s/it]
Epoch 10/10:   9%|▊         | 31/364 [00:41<05:51,  1.06s/it]
Epoch 10/10:   9%|▉         | 32/364 [00:42<04:48,  1.15it/s]
Epoch 10/10:   9%|▉         | 33/364 [00:46<09:30,  1.72s/it]
Epoch 10/10:   9%|▉         | 34/364 [00:46<07:22,  1.34s/it]
Epoch 10/10:  10%|▉         | 35/364 [00:46<05:53,  1.07s/it]
Epoch 10/10:  10%|▉         | 36/364 [00:47<04:50,  1.13it/s]
Epoch 10/10:  10%|█         | 37/364 [00:51<09:26,  1.73s/it]
Epoch 10/10:  10%|█         | 38/364 [00:51<07:18,  1.34s/it]
Epoch 10/10:  11%|█         | 39/364 [00:52<05:52,  1.08s/it]
Epoch 10/10:  11%|█         | 40/364 [00:52<04:48,  1.12it/s]
Epoch 10/10:  11%|█▏        | 41/364 [00:56<09:12,  1.71s/it]
Epoch 10/10:  12%|█▏        | 42/364 [00:56<07:10,  1.34s/it]
Epoch 10/10:  12%|█▏        | 43/364 [00:57<06:25,  1.20s/it]
Epoch 10/10:  12%|█▏        | 44/364 [00:57<05:11,  1.03it/s]
Epoch 10/10:  12%|█▏        | 45/364 [01:00<08:31,  1.60s/it]
Epoch 10/10:  13%|█▎        | 46/364 [01:01<06:43,  1.27s/it]
Epoch 10/10:  13%|█▎        | 47/364 [01:02<06:38,  1.26s/it]
Epoch 10/10:  13%|█▎        | 48/364 [01:03<05:17,  1.01s/it]
Epoch 10/10:  13%|█▎        | 49/364 [01:05<07:28,  1.42s/it]
Epoch 10/10:  14%|█▎        | 50/364 [01:06<07:17,  1.39s/it]
Epoch 10/10:  14%|█▍        | 51/364 [01:08<07:00,  1.34s/it]
Epoch 10/10:  14%|█▍        | 52/364 [01:08<05:34,  1.07s/it]
Epoch 10/10:  15%|█▍        | 53/364 [01:10<06:54,  1.33s/it]
Epoch 10/10:  15%|█▍        | 54/364 [01:12<07:38,  1.48s/it]
Epoch 10/10:  15%|█▌        | 55/364 [01:13<06:44,  1.31s/it]
Epoch 10/10:  15%|█▌        | 56/364 [01:13<05:21,  1.04s/it]
Epoch 10/10:  16%|█▌        | 57/364 [01:15<06:12,  1.21s/it]
Epoch 10/10:  16%|█▌        | 58/364 [01:17<07:37,  1.50s/it]
Epoch 10/10:  16%|█▌        | 59/364 [01:17<06:15,  1.23s/it]
Epoch 10/10:  16%|█▋        | 60/364 [01:18<05:01,  1.01it/s]
Epoch 10/10:  17%|█▋        | 61/364 [01:20<06:04,  1.20s/it]
Epoch 10/10:  17%|█▋        | 62/364 [01:22<08:03,  1.60s/it]
Epoch 10/10:  17%|█▋        | 63/364 [01:23<06:19,  1.26s/it]
Epoch 10/10:  18%|█▊        | 64/364 [01:23<05:03,  1.01s/it]
Epoch 10/10:  18%|█▊        | 65/364 [01:25<06:07,  1.23s/it]
Epoch 10/10:  18%|█▊        | 66/364 [01:27<07:58,  1.60s/it]
Epoch 10/10:  18%|█▊        | 67/364 [01:28<06:12,  1.25s/it]
Epoch 10/10:  19%|█▊        | 68/364 [01:28<04:58,  1.01s/it]
Epoch 10/10:  19%|█▉        | 69/364 [01:30<06:09,  1.25s/it]
Epoch 10/10:  19%|█▉        | 70/364 [01:32<07:47,  1.59s/it]
Epoch 10/10:  20%|█▉        | 71/364 [01:33<06:02,  1.24s/it]
Epoch 10/10:  20%|█▉        | 72/364 [01:33<04:50,  1.00it/s]
Epoch 10/10:  20%|██        | 73/364 [01:35<05:46,  1.19s/it]
Epoch 10/10:  20%|██        | 74/364 [01:38<08:06,  1.68s/it]
Epoch 10/10:  21%|██        | 75/364 [01:38<06:22,  1.32s/it]
Epoch 10/10:  21%|██        | 76/364 [01:39<05:05,  1.06s/it]
Epoch 10/10:  21%|██        | 77/364 [01:40<05:02,  1.05s/it]
Epoch 10/10:  21%|██▏       | 78/364 [01:43<07:59,  1.68s/it]
Epoch 10/10:  22%|██▏       | 79/364 [01:43<06:22,  1.34s/it]
Epoch 10/10:  22%|██▏       | 80/364 [01:44<05:02,  1.06s/it]
Epoch 10/10:  22%|██▏       | 81/364 [01:45<05:17,  1.12s/it]
Epoch 10/10:  23%|██▎       | 82/364 [01:48<07:42,  1.64s/it]
Epoch 10/10:  23%|██▎       | 83/364 [01:48<05:59,  1.28s/it]
Epoch 10/10:  23%|██▎       | 84/364 [01:49<04:47,  1.03s/it]
Epoch 10/10:  23%|██▎       | 85/364 [01:50<04:57,  1.07s/it]
Epoch 10/10:  24%|██▎       | 86/364 [01:53<07:34,  1.64s/it]
Epoch 10/10:  24%|██▍       | 87/364 [01:53<06:01,  1.30s/it]
Epoch 10/10:  24%|██▍       | 88/364 [01:54<04:47,  1.04s/it]
Epoch 10/10:  24%|██▍       | 89/364 [01:55<05:15,  1.15s/it]
Epoch 10/10:  25%|██▍       | 90/364 [01:58<07:29,  1.64s/it]
Epoch 10/10:  25%|██▌       | 91/364 [01:58<05:48,  1.28s/it]
Epoch 10/10:  25%|██▌       | 92/364 [01:59<04:36,  1.02s/it]
Epoch 10/10:  26%|██▌       | 93/364 [02:00<04:46,  1.06s/it]
Epoch 10/10:  26%|██▌       | 94/364 [02:03<07:23,  1.64s/it]
Epoch 10/10:  26%|██▌       | 95/364 [02:03<05:42,  1.28s/it]
Epoch 10/10:  26%|██▋       | 96/364 [02:04<04:33,  1.02s/it]
Epoch 10/10:  27%|██▋       | 97/364 [02:05<04:55,  1.10s/it]
Epoch 10/10:  27%|██▋       | 98/364 [02:08<07:04,  1.59s/it]
Epoch 10/10:  27%|██▋       | 99/364 [02:08<05:30,  1.25s/it]
Epoch 10/10:  27%|██▋       | 100/364 [02:09<04:25,  1.01s/it]
Epoch 10/10:  28%|██▊       | 101/364 [02:10<05:18,  1.21s/it]
Epoch 10/10:  28%|██▊       | 102/364 [02:13<07:04,  1.62s/it]
Epoch 10/10:  28%|██▊       | 103/364 [02:13<05:31,  1.27s/it]
Epoch 10/10:  29%|██▊       | 104/364 [02:14<04:25,  1.02s/it]
Epoch 10/10:  29%|██▉       | 105/364 [02:16<05:19,  1.23s/it]
Epoch 10/10:  29%|██▉       | 106/364 [02:18<06:44,  1.57s/it]
Epoch 10/10:  29%|██▉       | 107/364 [02:18<05:17,  1.24s/it]
Epoch 10/10:  30%|██▉       | 108/364 [02:19<04:14,  1.01it/s]
Epoch 10/10:  30%|██▉       | 109/364 [02:21<05:09,  1.21s/it]
Epoch 10/10:  30%|███       | 110/364 [02:23<07:04,  1.67s/it]
Epoch 10/10:  30%|███       | 111/364 [02:24<05:28,  1.30s/it]
Epoch 10/10:  31%|███       | 112/364 [02:24<04:21,  1.04s/it]
Epoch 10/10:  31%|███       | 113/364 [02:26<04:47,  1.14s/it]
Epoch 10/10:  31%|███▏      | 114/364 [02:28<06:48,  1.63s/it]
Epoch 10/10:  32%|███▏      | 115/364 [02:29<05:18,  1.28s/it]
Epoch 10/10:  32%|███▏      | 116/364 [02:29<04:14,  1.03s/it]
Epoch 10/10:  32%|███▏      | 117/364 [02:30<04:22,  1.06s/it]
Epoch 10/10:  32%|███▏      | 118/364 [02:34<07:19,  1.78s/it]
Epoch 10/10:  33%|███▎      | 119/364 [02:34<05:37,  1.38s/it]
Epoch 10/10:  33%|███▎      | 120/364 [02:35<04:26,  1.09s/it]
Epoch 10/10:  33%|███▎      | 121/364 [02:35<03:57,  1.02it/s]
Epoch 10/10:  34%|███▎      | 122/364 [02:39<06:57,  1.73s/it]
Epoch 10/10:  34%|███▍      | 123/364 [02:39<05:23,  1.34s/it]
Epoch 10/10:  34%|███▍      | 124/364 [02:40<04:15,  1.06s/it]
Epoch 10/10:  34%|███▍      | 125/364 [02:41<03:52,  1.03it/s]
Epoch 10/10:  35%|███▍      | 126/364 [02:44<07:14,  1.83s/it]
Epoch 10/10:  35%|███▍      | 127/364 [02:45<05:33,  1.41s/it]
Epoch 10/10:  35%|███▌      | 128/364 [02:45<04:22,  1.11s/it]
Epoch 10/10:  35%|███▌      | 129/364 [02:46<03:48,  1.03it/s]
Epoch 10/10:  36%|███▌      | 130/364 [02:49<06:40,  1.71s/it]
Epoch 10/10:  36%|███▌      | 131/364 [02:50<05:10,  1.33s/it]
Epoch 10/10:  36%|███▋      | 132/364 [02:50<04:06,  1.06s/it]
Epoch 10/10:  37%|███▋      | 133/364 [02:51<03:48,  1.01it/s]
Epoch 10/10:  37%|███▋      | 134/364 [02:54<06:22,  1.66s/it]
Epoch 10/10:  37%|███▋      | 135/364 [02:55<04:54,  1.29s/it]
Epoch 10/10:  37%|███▋      | 136/364 [02:55<03:55,  1.03s/it]
Epoch 10/10:  38%|███▊      | 137/364 [02:56<03:42,  1.02it/s]
Epoch 10/10:  38%|███▊      | 138/364 [02:59<06:15,  1.66s/it]
Epoch 10/10:  38%|███▊      | 139/364 [03:00<04:48,  1.28s/it]
Epoch 10/10:  38%|███▊      | 140/364 [03:00<03:49,  1.02s/it]
Epoch 10/10:  39%|███▊      | 141/364 [03:01<03:32,  1.05it/s]
Epoch 10/10:  39%|███▉      | 142/364 [03:04<06:04,  1.64s/it]
Epoch 10/10:  39%|███▉      | 143/364 [03:04<04:41,  1.28s/it]
Epoch 10/10:  40%|███▉      | 144/364 [03:05<03:45,  1.02s/it]
Epoch 10/10:  40%|███▉      | 145/364 [03:06<04:04,  1.12s/it]
Epoch 10/10:  40%|████      | 146/364 [03:09<06:11,  1.70s/it]
Epoch 10/10:  40%|████      | 147/364 [03:10<04:46,  1.32s/it]
Epoch 10/10:  41%|████      | 148/364 [03:10<03:46,  1.05s/it]
Epoch 10/10:  41%|████      | 149/364 [03:12<04:12,  1.18s/it]
Epoch 10/10:  41%|████      | 150/364 [03:14<06:01,  1.69s/it]
Epoch 10/10:  41%|████▏     | 151/364 [03:15<04:39,  1.31s/it]
Epoch 10/10:  42%|████▏     | 152/364 [03:15<03:42,  1.05s/it]
Epoch 10/10:  42%|████▏     | 153/364 [03:17<03:56,  1.12s/it]
Epoch 10/10:  42%|████▏     | 154/364 [03:20<06:04,  1.74s/it]
Epoch 10/10:  43%|████▎     | 155/364 [03:20<04:40,  1.34s/it]
Epoch 10/10:  43%|████▎     | 156/364 [03:21<03:42,  1.07s/it]
Epoch 10/10:  43%|████▎     | 157/364 [03:22<03:46,  1.09s/it]
Epoch 10/10:  43%|████▎     | 158/364 [03:25<05:49,  1.70s/it]
Epoch 10/10:  44%|████▎     | 159/364 [03:25<04:29,  1.31s/it]
Epoch 10/10:  44%|████▍     | 160/364 [03:26<03:34,  1.05s/it]
Epoch 10/10:  44%|████▍     | 161/364 [03:27<03:41,  1.09s/it]
Epoch 10/10:  45%|████▍     | 162/364 [03:30<05:34,  1.66s/it]
Epoch 10/10:  45%|████▍     | 163/364 [03:30<04:19,  1.29s/it]
Epoch 10/10:  45%|████▌     | 164/364 [03:31<03:26,  1.03s/it]
Epoch 10/10:  45%|████▌     | 165/364 [03:32<03:20,  1.01s/it]
Epoch 10/10:  46%|████▌     | 166/364 [03:35<05:42,  1.73s/it]
Epoch 10/10:  46%|████▌     | 167/364 [03:36<04:23,  1.34s/it]
Epoch 10/10:  46%|████▌     | 168/364 [03:36<03:27,  1.06s/it]
Epoch 10/10:  46%|████▋     | 169/364 [03:37<03:02,  1.07it/s]
Epoch 10/10:  47%|████▋     | 170/364 [03:40<05:27,  1.69s/it]
Epoch 10/10:  47%|████▋     | 171/364 [03:40<04:13,  1.31s/it]
Epoch 10/10:  47%|████▋     | 172/364 [03:41<03:21,  1.05s/it]
Epoch 10/10:  48%|████▊     | 173/364 [03:42<03:08,  1.01it/s]
Epoch 10/10:  48%|████▊     | 174/364 [03:45<05:08,  1.63s/it]
Epoch 10/10:  48%|████▊     | 175/364 [03:45<03:59,  1.27s/it]
Epoch 10/10:  48%|████▊     | 176/364 [03:46<03:10,  1.02s/it]
Epoch 10/10:  49%|████▊     | 177/364 [03:46<02:48,  1.11it/s]
Epoch 10/10:  49%|████▉     | 178/364 [03:50<05:11,  1.67s/it]
Epoch 10/10:  49%|████▉     | 179/364 [03:50<03:59,  1.29s/it]
Epoch 10/10:  49%|████▉     | 180/364 [03:51<03:09,  1.03s/it]
Epoch 10/10:  50%|████▉     | 181/364 [03:52<03:08,  1.03s/it]
Epoch 10/10:  50%|█████     | 182/364 [03:55<05:35,  1.85s/it]
Epoch 10/10:  50%|█████     | 183/364 [03:56<04:17,  1.42s/it]
Epoch 10/10:  51%|█████     | 184/364 [03:56<03:21,  1.12s/it]
Epoch 10/10:  51%|█████     | 185/364 [03:57<02:44,  1.09it/s]
Epoch 10/10:  51%|█████     | 186/364 [04:00<05:03,  1.70s/it]
Epoch 10/10:  51%|█████▏    | 187/364 [04:01<03:53,  1.32s/it]
Epoch 10/10:  52%|█████▏    | 188/364 [04:01<03:05,  1.05s/it]
Epoch 10/10:  52%|█████▏    | 189/364 [04:02<02:40,  1.09it/s]
Epoch 10/10:  52%|█████▏    | 190/364 [04:06<05:11,  1.79s/it]
Epoch 10/10:  52%|█████▏    | 191/364 [04:06<03:58,  1.38s/it]
Epoch 10/10:  53%|█████▎    | 192/364 [04:06<03:08,  1.09s/it]
Epoch 10/10:  53%|█████▎    | 193/364 [04:07<02:33,  1.12it/s]
Epoch 10/10:  53%|█████▎    | 194/364 [04:11<05:04,  1.79s/it]
Epoch 10/10:  54%|█████▎    | 195/364 [04:11<03:53,  1.38s/it]
Epoch 10/10:  54%|█████▍    | 196/364 [04:12<03:04,  1.10s/it]
Epoch 10/10:  54%|█████▍    | 197/364 [04:12<02:29,  1.11it/s]
Epoch 10/10:  54%|█████▍    | 198/364 [04:16<04:53,  1.77s/it]
Epoch 10/10:  55%|█████▍    | 199/364 [04:16<03:45,  1.37s/it]
Epoch 10/10:  55%|█████▍    | 200/364 [04:17<02:58,  1.09s/it]
Epoch 10/10:  55%|█████▌    | 201/364 [04:17<02:26,  1.11it/s]
Epoch 10/10:  55%|█████▌    | 202/364 [04:21<04:51,  1.80s/it]
Epoch 10/10:  56%|█████▌    | 203/364 [04:21<03:42,  1.38s/it]
Epoch 10/10:  56%|█████▌    | 204/364 [04:22<02:54,  1.09s/it]
Epoch 10/10:  56%|█████▋    | 205/364 [04:22<02:23,  1.11it/s]
Epoch 10/10:  57%|█████▋    | 206/364 [04:26<04:37,  1.75s/it]
Epoch 10/10:  57%|█████▋    | 207/364 [04:27<03:33,  1.36s/it]
Epoch 10/10:  57%|█████▋    | 208/364 [04:27<02:48,  1.08s/it]
Epoch 10/10:  57%|█████▋    | 209/364 [04:27<02:17,  1.13it/s]
Epoch 10/10:  58%|█████▊    | 210/364 [04:31<04:27,  1.74s/it]
Epoch 10/10:  58%|█████▊    | 211/364 [04:32<03:26,  1.35s/it]
Epoch 10/10:  58%|█████▊    | 212/364 [04:32<02:42,  1.07s/it]
Epoch 10/10:  59%|█████▊    | 213/364 [04:32<02:12,  1.14it/s]
Epoch 10/10:  59%|█████▉    | 214/364 [04:36<04:28,  1.79s/it]
Epoch 10/10:  59%|█████▉    | 215/364 [04:37<03:26,  1.38s/it]
Epoch 10/10:  59%|█████▉    | 216/364 [04:37<02:42,  1.10s/it]
Epoch 10/10:  60%|█████▉    | 217/364 [04:38<02:12,  1.11it/s]
Epoch 10/10:  60%|█████▉    | 218/364 [04:41<04:14,  1.74s/it]
Epoch 10/10:  60%|██████    | 219/364 [04:42<03:15,  1.35s/it]
Epoch 10/10:  60%|██████    | 220/364 [04:42<02:34,  1.07s/it]
Epoch 10/10:  61%|██████    | 221/364 [04:43<02:06,  1.13it/s]
Epoch 10/10:  61%|██████    | 222/364 [04:46<03:59,  1.69s/it]
Epoch 10/10:  61%|██████▏   | 223/364 [04:47<03:05,  1.31s/it]
Epoch 10/10:  62%|██████▏   | 224/364 [04:47<02:26,  1.05s/it]
Epoch 10/10:  62%|██████▏   | 225/364 [04:47<01:59,  1.16it/s]
Epoch 10/10:  62%|██████▏   | 226/364 [04:51<04:00,  1.74s/it]
Epoch 10/10:  62%|██████▏   | 227/364 [04:52<03:06,  1.36s/it]
Epoch 10/10:  63%|██████▎   | 228/364 [04:52<02:27,  1.08s/it]
Epoch 10/10:  63%|██████▎   | 229/364 [04:53<01:59,  1.13it/s]
Epoch 10/10:  63%|██████▎   | 230/364 [04:56<03:54,  1.75s/it]
Epoch 10/10:  63%|██████▎   | 231/364 [04:57<03:02,  1.37s/it]
Epoch 10/10:  64%|██████▎   | 232/364 [04:58<02:33,  1.16s/it]
Epoch 10/10:  64%|██████▍   | 233/364 [04:58<02:03,  1.06it/s]
Epoch 10/10:  64%|██████▍   | 234/364 [05:01<03:41,  1.70s/it]
Epoch 10/10:  65%|██████▍   | 235/364 [05:02<02:50,  1.32s/it]
Epoch 10/10:  65%|██████▍   | 236/364 [05:02<02:14,  1.05s/it]
Epoch 10/10:  65%|██████▌   | 237/364 [05:03<01:49,  1.16it/s]
Epoch 10/10:  65%|██████▌   | 238/364 [05:06<03:20,  1.59s/it]
Epoch 10/10:  66%|██████▌   | 239/364 [05:07<03:08,  1.51s/it]
Epoch 10/10:  66%|██████▌   | 240/364 [05:08<02:27,  1.19s/it]
Epoch 10/10:  66%|██████▌   | 241/364 [05:08<01:58,  1.04it/s]
Epoch 10/10:  66%|██████▋   | 242/364 [05:11<03:07,  1.53s/it]
Epoch 10/10:  67%|██████▋   | 243/364 [05:13<03:04,  1.53s/it]
Epoch 10/10:  67%|██████▋   | 244/364 [05:13<02:23,  1.20s/it]
Epoch 10/10:  67%|██████▋   | 245/364 [05:13<01:54,  1.04it/s]
Epoch 10/10:  68%|██████▊   | 246/364 [05:16<02:58,  1.51s/it]
Epoch 10/10:  68%|██████▊   | 247/364 [05:18<02:59,  1.54s/it]
Epoch 10/10:  68%|██████▊   | 248/364 [05:18<02:19,  1.20s/it]
Epoch 10/10:  68%|██████▊   | 249/364 [05:19<01:51,  1.03it/s]
Epoch 10/10:  69%|██████▊   | 250/364 [05:21<02:41,  1.42s/it]
Epoch 10/10:  69%|██████▉   | 251/364 [05:23<02:47,  1.49s/it]
Epoch 10/10:  69%|██████▉   | 252/364 [05:23<02:10,  1.17s/it]
Epoch 10/10:  70%|██████▉   | 253/364 [05:24<01:44,  1.06it/s]
Epoch 10/10:  70%|██████▉   | 254/364 [05:26<02:43,  1.49s/it]
Epoch 10/10:  70%|███████   | 255/364 [05:28<02:49,  1.55s/it]
Epoch 10/10:  70%|███████   | 256/364 [05:29<02:11,  1.21s/it]
Epoch 10/10:  71%|███████   | 257/364 [05:29<01:44,  1.02it/s]
Epoch 10/10:  71%|███████   | 258/364 [05:31<02:26,  1.39s/it]
Epoch 10/10:  71%|███████   | 259/364 [05:33<02:49,  1.62s/it]
Epoch 10/10:  71%|███████▏  | 260/364 [05:34<02:11,  1.26s/it]
Epoch 10/10:  72%|███████▏  | 261/364 [05:34<01:44,  1.01s/it]
Epoch 10/10:  72%|███████▏  | 262/364 [05:36<02:10,  1.28s/it]
Epoch 10/10:  72%|███████▏  | 263/364 [05:39<02:46,  1.65s/it]
Epoch 10/10:  73%|███████▎  | 264/364 [05:39<02:08,  1.28s/it]
Epoch 10/10:  73%|███████▎  | 265/364 [05:40<01:41,  1.03s/it]
Epoch 10/10:  73%|███████▎  | 266/364 [05:41<01:54,  1.17s/it]
Epoch 10/10:  73%|███████▎  | 267/364 [05:44<02:40,  1.66s/it]
Epoch 10/10:  74%|███████▎  | 268/364 [05:44<02:04,  1.29s/it]
Epoch 10/10:  74%|███████▍  | 269/364 [05:45<01:37,  1.03s/it]
Epoch 10/10:  74%|███████▍  | 270/364 [05:46<01:37,  1.03s/it]
Epoch 10/10:  74%|███████▍  | 271/364 [05:49<02:41,  1.74s/it]
Epoch 10/10:  75%|███████▍  | 272/364 [05:50<02:03,  1.35s/it]
Epoch 10/10:  75%|███████▌  | 273/364 [05:50<01:37,  1.07s/it]
Epoch 10/10:  75%|███████▌  | 274/364 [05:51<01:26,  1.04it/s]
Epoch 10/10:  76%|███████▌  | 275/364 [05:54<02:31,  1.70s/it]
Epoch 10/10:  76%|███████▌  | 276/364 [05:55<01:55,  1.32s/it]
Epoch 10/10:  76%|███████▌  | 277/364 [05:55<01:30,  1.05s/it]
Epoch 10/10:  76%|███████▋  | 278/364 [05:56<01:26,  1.01s/it]
Epoch 10/10:  77%|███████▋  | 279/364 [05:59<02:30,  1.78s/it]
Epoch 10/10:  77%|███████▋  | 280/364 [06:00<01:54,  1.37s/it]
Epoch 10/10:  77%|███████▋  | 281/364 [06:00<01:29,  1.08s/it]
Epoch 10/10:  77%|███████▋  | 282/364 [06:01<01:19,  1.03it/s]
Epoch 10/10:  78%|███████▊  | 283/364 [06:04<02:18,  1.71s/it]
Epoch 10/10:  78%|███████▊  | 284/364 [06:05<01:45,  1.32s/it]
Epoch 10/10:  78%|███████▊  | 285/364 [06:05<01:22,  1.05s/it]
Epoch 10/10:  79%|███████▊  | 286/364 [06:06<01:24,  1.09s/it]
Epoch 10/10:  79%|███████▉  | 287/364 [06:10<02:12,  1.72s/it]
Epoch 10/10:  79%|███████▉  | 288/364 [06:10<01:40,  1.33s/it]
Epoch 10/10:  79%|███████▉  | 289/364 [06:10<01:18,  1.05s/it]
Epoch 10/10:  80%|███████▉  | 290/364 [06:12<01:22,  1.12s/it]
Epoch 10/10:  80%|███████▉  | 291/364 [06:15<02:09,  1.77s/it]
Epoch 10/10:  80%|████████  | 292/364 [06:15<01:38,  1.37s/it]
Epoch 10/10:  80%|████████  | 293/364 [06:16<01:17,  1.09s/it]
Epoch 10/10:  81%|████████  | 294/364 [06:17<01:07,  1.03it/s]
Epoch 10/10:  81%|████████  | 295/364 [06:20<02:00,  1.75s/it]
Epoch 10/10:  81%|████████▏ | 296/364 [06:21<01:32,  1.36s/it]
Epoch 10/10:  82%|████████▏ | 297/364 [06:21<01:12,  1.08s/it]
Epoch 10/10:  82%|████████▏ | 298/364 [06:21<00:58,  1.13it/s]
Epoch 10/10:  82%|████████▏ | 299/364 [06:25<01:47,  1.65s/it]
Epoch 10/10:  82%|████████▏ | 300/364 [06:25<01:21,  1.28s/it]
Epoch 10/10:  83%|████████▎ | 301/364 [06:26<01:04,  1.02s/it]
Epoch 10/10:  83%|████████▎ | 302/364 [06:27<00:58,  1.06it/s]
Epoch 10/10:  83%|████████▎ | 303/364 [06:30<01:44,  1.71s/it]
Epoch 10/10:  84%|████████▎ | 304/364 [06:30<01:19,  1.32s/it]
Epoch 10/10:  84%|████████▍ | 305/364 [06:31<01:02,  1.06s/it]
Epoch 10/10:  84%|████████▍ | 306/364 [06:31<00:53,  1.09it/s]
Epoch 10/10:  84%|████████▍ | 307/364 [06:35<01:34,  1.65s/it]
Epoch 10/10:  85%|████████▍ | 308/364 [06:35<01:11,  1.28s/it]
Epoch 10/10:  85%|████████▍ | 309/364 [06:36<00:56,  1.03s/it]
Epoch 10/10:  85%|████████▌ | 310/364 [06:37<00:53,  1.00it/s]
Epoch 10/10:  85%|████████▌ | 311/364 [06:40<01:26,  1.63s/it]
Epoch 10/10:  86%|████████▌ | 312/364 [06:40<01:06,  1.27s/it]
Epoch 10/10:  86%|████████▌ | 313/364 [06:41<00:51,  1.02s/it]
Epoch 10/10:  86%|████████▋ | 314/364 [06:42<00:51,  1.04s/it]
Epoch 10/10:  87%|████████▋ | 315/364 [06:45<01:18,  1.59s/it]
Epoch 10/10:  87%|████████▋ | 316/364 [06:45<00:59,  1.24s/it]
Epoch 10/10:  87%|████████▋ | 317/364 [06:45<00:46,  1.01it/s]
Epoch 10/10:  87%|████████▋ | 318/364 [06:47<00:52,  1.14s/it]
Epoch 10/10:  88%|████████▊ | 319/364 [06:50<01:13,  1.64s/it]
Epoch 10/10:  88%|████████▊ | 320/364 [06:50<00:56,  1.28s/it]
Epoch 10/10:  88%|████████▊ | 321/364 [06:51<00:43,  1.02s/it]
Epoch 10/10:  88%|████████▊ | 322/364 [06:52<00:48,  1.14s/it]
Epoch 10/10:  89%|████████▊ | 323/364 [06:54<00:59,  1.44s/it]
Epoch 10/10:  89%|████████▉ | 324/364 [06:55<00:45,  1.14s/it]
Epoch 10/10:  89%|████████▉ | 325/364 [06:55<00:36,  1.08it/s]
Epoch 10/10:  90%|████████▉ | 326/364 [06:57<00:47,  1.24s/it]
Epoch 10/10:  90%|████████▉ | 327/364 [06:59<00:58,  1.58s/it]
Epoch 10/10:  90%|█████████ | 328/364 [07:00<00:44,  1.24s/it]
Epoch 10/10:  90%|█████████ | 329/364 [07:00<00:34,  1.00it/s]
Epoch 10/10:  91%|█████████ | 330/364 [07:02<00:41,  1.22s/it]
Epoch 10/10:  91%|█████████ | 331/364 [07:04<00:53,  1.62s/it]
Epoch 10/10:  91%|█████████ | 332/364 [07:05<00:40,  1.26s/it]
Epoch 10/10:  91%|█████████▏| 333/364 [07:05<00:31,  1.01s/it]
Epoch 10/10:  92%|█████████▏| 334/364 [07:07<00:37,  1.25s/it]
Epoch 10/10:  92%|█████████▏| 335/364 [07:10<00:49,  1.69s/it]
Epoch 10/10:  92%|█████████▏| 336/364 [07:10<00:36,  1.31s/it]
Epoch 10/10:  93%|█████████▎| 337/364 [07:11<00:28,  1.05s/it]
Epoch 10/10:  93%|█████████▎| 338/364 [07:12<00:30,  1.18s/it]
Epoch 10/10:  93%|█████████▎| 339/364 [07:15<00:39,  1.59s/it]
Epoch 10/10:  93%|█████████▎| 340/364 [07:15<00:29,  1.23s/it]
Epoch 10/10:  94%|█████████▎| 341/364 [07:16<00:22,  1.01it/s]
Epoch 10/10:  94%|█████████▍| 342/364 [07:17<00:26,  1.22s/it]
Epoch 10/10:  94%|█████████▍| 343/364 [07:20<00:32,  1.55s/it]
Epoch 10/10:  95%|█████████▍| 344/364 [07:20<00:24,  1.21s/it]
Epoch 10/10:  95%|█████████▍| 345/364 [07:21<00:18,  1.02it/s]
Epoch 10/10:  95%|█████████▌| 346/364 [07:23<00:23,  1.30s/it]
Epoch 10/10:  95%|█████████▌| 347/364 [07:25<00:25,  1.51s/it]
Epoch 10/10:  96%|█████████▌| 348/364 [07:25<00:19,  1.19s/it]
Epoch 10/10:  96%|█████████▌| 349/364 [07:26<00:15,  1.04s/it]
Epoch 10/10:  96%|█████████▌| 350/364 [07:28<00:18,  1.33s/it]
Epoch 10/10:  96%|█████████▋| 351/364 [07:30<00:19,  1.48s/it]
Epoch 10/10:  97%|█████████▋| 352/364 [07:30<00:13,  1.16s/it]
Epoch 10/10:  97%|█████████▋| 353/364 [07:31<00:11,  1.05s/it]
Epoch 10/10:  97%|█████████▋| 354/364 [07:32<00:12,  1.24s/it]
Epoch 10/10:  98%|█████████▊| 355/364 [07:34<00:13,  1.46s/it]
Epoch 10/10:  98%|█████████▊| 356/364 [07:35<00:09,  1.15s/it]
Epoch 10/10:  98%|█████████▊| 357/364 [07:36<00:07,  1.14s/it]
Epoch 10/10:  98%|█████████▊| 358/364 [07:37<00:07,  1.21s/it]
Epoch 10/10:  99%|█████████▊| 359/364 [07:39<00:07,  1.49s/it]
Epoch 10/10:  99%|█████████▉| 360/364 [07:40<00:04,  1.17s/it]
Epoch 10/10:  99%|█████████▉| 361/364 [07:41<00:03,  1.11s/it]
Epoch 10/10:  99%|█████████▉| 362/364 [07:42<00:02,  1.04s/it]
Epoch 10/10: 100%|█████████▉| 363/364 [07:43<00:01,  1.01s/it]
Epoch 10/10: 100%|██████████| 364/364 [07:43<00:00,  1.32it/s]
Epoch 10/10: 100%|██████████| 364/364 [07:43<00:00,  1.27s/it]

Validating:   0%|          | 0/57 [00:00<?, ?it/s]
Validating:   2%|▏         | 1/57 [00:04<04:28,  4.80s/it]
Validating:   4%|▎         | 2/57 [00:04<01:53,  2.07s/it]
Validating:   5%|▌         | 3/57 [00:05<01:04,  1.20s/it]
Validating:   7%|▋         | 4/57 [00:05<00:41,  1.28it/s]
Validating:   9%|▉         | 5/57 [00:09<01:42,  1.97s/it]
Validating:  11%|█         | 6/57 [00:09<01:09,  1.36s/it]
Validating:  12%|█▏        | 7/57 [00:09<00:47,  1.04it/s]
Validating:  14%|█▍        | 8/57 [00:09<00:34,  1.44it/s]
Validating:  16%|█▌        | 9/57 [00:13<01:22,  1.71s/it]
Validating:  18%|█▊        | 10/57 [00:13<00:57,  1.23s/it]
Validating:  19%|█▉        | 11/57 [00:14<00:41,  1.11it/s]
Validating:  21%|██        | 12/57 [00:14<00:30,  1.50it/s]
Validating:  23%|██▎       | 13/57 [00:18<01:16,  1.74s/it]
Validating:  25%|██▍       | 14/57 [00:18<00:54,  1.27s/it]
Validating:  26%|██▋       | 15/57 [00:18<00:39,  1.07it/s]
Validating:  28%|██▊       | 16/57 [00:18<00:28,  1.44it/s]
Validating:  30%|██▉       | 17/57 [00:22<01:04,  1.62s/it]
Validating:  32%|███▏      | 18/57 [00:22<00:46,  1.18s/it]
Validating:  33%|███▎      | 19/57 [00:22<00:33,  1.14it/s]
Validating:  35%|███▌      | 20/57 [00:23<00:24,  1.52it/s]
Validating:  37%|███▋      | 21/57 [00:26<00:57,  1.60s/it]
Validating:  39%|███▊      | 22/57 [00:27<00:40,  1.17s/it]
Validating:  40%|████      | 23/57 [00:27<00:29,  1.16it/s]
Validating:  42%|████▏     | 24/57 [00:27<00:21,  1.54it/s]
Validating:  44%|████▍     | 25/57 [00:31<00:52,  1.66s/it]
Validating:  46%|████▌     | 26/57 [00:31<00:37,  1.20s/it]
Validating:  47%|████▋     | 27/57 [00:31<00:26,  1.13it/s]
Validating:  49%|████▉     | 28/57 [00:31<00:18,  1.53it/s]
Validating:  51%|█████     | 29/57 [00:35<00:45,  1.64s/it]
Validating:  53%|█████▎    | 30/57 [00:35<00:32,  1.20s/it]
Validating:  54%|█████▍    | 31/57 [00:36<00:22,  1.13it/s]
Validating:  56%|█████▌    | 32/57 [00:36<00:16,  1.51it/s]
Validating:  58%|█████▊    | 33/57 [00:40<00:39,  1.66s/it]
Validating:  60%|█████▉    | 34/57 [00:40<00:27,  1.20s/it]
Validating:  61%|██████▏   | 35/57 [00:40<00:19,  1.13it/s]
Validating:  63%|██████▎   | 36/57 [00:40<00:13,  1.50it/s]
Validating:  65%|██████▍   | 37/57 [00:44<00:35,  1.77s/it]
Validating:  67%|██████▋   | 38/57 [00:45<00:24,  1.29s/it]
Validating:  68%|██████▊   | 39/57 [00:45<00:17,  1.06it/s]
Validating:  70%|███████   | 40/57 [00:45<00:11,  1.42it/s]
Validating:  72%|███████▏  | 41/57 [00:49<00:27,  1.75s/it]
Validating:  74%|███████▎  | 42/57 [00:49<00:19,  1.27s/it]
Validating:  75%|███████▌  | 43/57 [00:49<00:12,  1.08it/s]
Validating:  77%|███████▋  | 44/57 [00:49<00:09,  1.44it/s]
Validating:  79%|███████▉  | 45/57 [00:54<00:20,  1.71s/it]
Validating:  81%|████████  | 46/57 [00:54<00:13,  1.25s/it]
Validating:  82%|████████▏ | 47/57 [00:54<00:09,  1.09it/s]
Validating:  84%|████████▍ | 48/57 [00:54<00:06,  1.46it/s]
Validating:  86%|████████▌ | 49/57 [00:58<00:12,  1.56s/it]
Validating:  88%|████████▊ | 50/57 [00:58<00:08,  1.18s/it]
Validating:  89%|████████▉ | 51/57 [00:58<00:05,  1.15it/s]
Validating:  91%|█████████ | 52/57 [00:58<00:03,  1.54it/s]
Validating:  93%|█████████▎| 53/57 [01:02<00:05,  1.47s/it]
Validating:  95%|█████████▍| 54/57 [01:02<00:03,  1.06s/it]
Validating:  96%|█████████▋| 55/57 [01:02<00:01,  1.28it/s]
Validating:  98%|█████████▊| 56/57 [01:02<00:00,  1.72it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  2.27it/s]
Validating: 100%|██████████| 57/57 [01:02<00:00,  1.10s/it]
Epoch 10: Train Loss: 0.0321, Train Acc: 0.9916 | Val Loss: 0.3719, Val Acc: 0.8911
Evaluating on test set...

Evaluating:   0%|          | 0/108 [00:00<?, ?it/s]
Evaluating:   1%|          | 1/108 [00:04<08:53,  4.98s/it]
Evaluating:   2%|▏         | 2/108 [00:05<03:45,  2.12s/it]
Evaluating:   3%|▎         | 3/108 [00:05<02:08,  1.22s/it]
Evaluating:   4%|▎         | 4/108 [00:05<01:21,  1.28it/s]
Evaluating:   5%|▍         | 5/108 [00:09<03:27,  2.02s/it]
Evaluating:   6%|▌         | 6/108 [00:09<02:20,  1.38s/it]
Evaluating:   6%|▋         | 7/108 [00:09<01:39,  1.01it/s]
Evaluating:   7%|▋         | 8/108 [00:10<01:11,  1.40it/s]
Evaluating:   8%|▊         | 9/108 [00:14<02:53,  1.75s/it]
Evaluating:   9%|▉         | 10/108 [00:14<02:09,  1.32s/it]
Evaluating:  10%|█         | 11/108 [00:14<01:33,  1.03it/s]
Evaluating:  11%|█         | 12/108 [00:14<01:08,  1.39it/s]
Evaluating:  12%|█▏        | 13/108 [00:18<02:48,  1.77s/it]
Evaluating:  13%|█▎        | 14/108 [00:19<02:00,  1.28s/it]
Evaluating:  14%|█▍        | 15/108 [00:19<01:27,  1.07it/s]
Evaluating:  15%|█▍        | 16/108 [00:19<01:03,  1.44it/s]
Evaluating:  16%|█▌        | 17/108 [00:23<02:39,  1.76s/it]
Evaluating:  17%|█▋        | 18/108 [00:23<01:54,  1.27s/it]
Evaluating:  18%|█▊        | 19/108 [00:23<01:22,  1.07it/s]
Evaluating:  19%|█▊        | 20/108 [00:23<01:00,  1.45it/s]
Evaluating:  19%|█▉        | 21/108 [00:28<02:29,  1.71s/it]
Evaluating:  20%|██        | 22/108 [00:28<01:46,  1.24s/it]
Evaluating:  21%|██▏       | 23/108 [00:28<01:18,  1.09it/s]
Evaluating:  22%|██▏       | 24/108 [00:28<00:58,  1.45it/s]
Evaluating:  23%|██▎       | 25/108 [00:32<02:23,  1.73s/it]
Evaluating:  24%|██▍       | 26/108 [00:32<01:42,  1.25s/it]
Evaluating:  25%|██▌       | 27/108 [00:32<01:14,  1.08it/s]
Evaluating:  26%|██▌       | 28/108 [00:33<00:55,  1.45it/s]
Evaluating:  27%|██▋       | 29/108 [00:37<02:15,  1.71s/it]
Evaluating:  28%|██▊       | 30/108 [00:37<01:36,  1.24s/it]
Evaluating:  29%|██▊       | 31/108 [00:37<01:09,  1.11it/s]
Evaluating:  30%|██▉       | 32/108 [00:37<00:51,  1.49it/s]
Evaluating:  31%|███       | 33/108 [00:41<02:11,  1.75s/it]
Evaluating:  31%|███▏      | 34/108 [00:42<01:33,  1.27s/it]
Evaluating:  32%|███▏      | 35/108 [00:42<01:08,  1.07it/s]
Evaluating:  33%|███▎      | 36/108 [00:42<00:50,  1.44it/s]
Evaluating:  34%|███▍      | 37/108 [00:46<01:59,  1.69s/it]
Evaluating:  35%|███▌      | 38/108 [00:46<01:25,  1.23s/it]
Evaluating:  36%|███▌      | 39/108 [00:46<01:02,  1.11it/s]
Evaluating:  37%|███▋      | 40/108 [00:46<00:45,  1.49it/s]
Evaluating:  38%|███▊      | 41/108 [00:50<01:55,  1.72s/it]
Evaluating:  39%|███▉      | 42/108 [00:51<01:22,  1.25s/it]
Evaluating:  40%|███▉      | 43/108 [00:51<00:59,  1.09it/s]
Evaluating:  41%|████      | 44/108 [00:51<00:43,  1.48it/s]
Evaluating:  42%|████▏     | 45/108 [00:55<01:44,  1.67s/it]
Evaluating:  43%|████▎     | 46/108 [00:55<01:15,  1.21s/it]
Evaluating:  44%|████▎     | 47/108 [00:55<00:54,  1.12it/s]
Evaluating:  44%|████▍     | 48/108 [00:55<00:39,  1.52it/s]
Evaluating:  45%|████▌     | 49/108 [00:59<01:40,  1.71s/it]
Evaluating:  46%|████▋     | 50/108 [01:00<01:12,  1.25s/it]
Evaluating:  47%|████▋     | 51/108 [01:00<00:51,  1.10it/s]
Evaluating:  48%|████▊     | 52/108 [01:00<00:37,  1.49it/s]
Evaluating:  49%|████▉     | 53/108 [01:04<01:34,  1.72s/it]
Evaluating:  50%|█████     | 54/108 [01:04<01:07,  1.25s/it]
Evaluating:  51%|█████     | 55/108 [01:04<00:49,  1.08it/s]
Evaluating:  52%|█████▏    | 56/108 [01:04<00:35,  1.45it/s]
Evaluating:  53%|█████▎    | 57/108 [01:08<01:23,  1.63s/it]
Evaluating:  54%|█████▎    | 58/108 [01:08<00:59,  1.19s/it]
Evaluating:  55%|█████▍    | 59/108 [01:09<00:42,  1.15it/s]
Evaluating:  56%|█████▌    | 60/108 [01:09<00:31,  1.52it/s]
Evaluating:  56%|█████▋    | 61/108 [01:13<01:23,  1.78s/it]
Evaluating:  57%|█████▋    | 62/108 [01:13<00:59,  1.29s/it]
Evaluating:  58%|█████▊    | 63/108 [01:13<00:42,  1.06it/s]
Evaluating:  59%|█████▉    | 64/108 [01:14<00:30,  1.42it/s]
Evaluating:  60%|██████    | 65/108 [01:18<01:17,  1.80s/it]
Evaluating:  61%|██████    | 66/108 [01:18<00:54,  1.30s/it]
Evaluating:  62%|██████▏   | 67/108 [01:18<00:39,  1.04it/s]
Evaluating:  63%|██████▎   | 68/108 [01:18<00:28,  1.40it/s]
Evaluating:  64%|██████▍   | 69/108 [01:23<01:11,  1.83s/it]
Evaluating:  65%|██████▍   | 70/108 [01:23<00:50,  1.33s/it]
Evaluating:  66%|██████▌   | 71/108 [01:23<00:35,  1.04it/s]
Evaluating:  67%|██████▋   | 72/108 [01:23<00:25,  1.39it/s]
Evaluating:  68%|██████▊   | 73/108 [01:27<00:59,  1.69s/it]
Evaluating:  69%|██████▊   | 74/108 [01:27<00:41,  1.23s/it]
Evaluating:  69%|██████▉   | 75/108 [01:27<00:30,  1.10it/s]
Evaluating:  70%|███████   | 76/108 [01:28<00:21,  1.47it/s]
Evaluating:  71%|███████▏  | 77/108 [01:32<00:52,  1.69s/it]
Evaluating:  72%|███████▏  | 78/108 [01:32<00:37,  1.24s/it]
Evaluating:  73%|███████▎  | 79/108 [01:32<00:26,  1.10it/s]
Evaluating:  74%|███████▍  | 80/108 [01:32<00:18,  1.48it/s]
Evaluating:  75%|███████▌  | 81/108 [01:36<00:45,  1.70s/it]
Evaluating:  76%|███████▌  | 82/108 [01:36<00:32,  1.23s/it]
Evaluating:  77%|███████▋  | 83/108 [01:36<00:22,  1.10it/s]
Evaluating:  78%|███████▊  | 84/108 [01:37<00:16,  1.46it/s]
Evaluating:  79%|███████▊  | 85/108 [01:41<00:38,  1.69s/it]
Evaluating:  80%|███████▉  | 86/108 [01:41<00:26,  1.22s/it]
Evaluating:  81%|████████  | 87/108 [01:41<00:18,  1.11it/s]
Evaluating:  81%|████████▏ | 88/108 [01:41<00:13,  1.49it/s]
Evaluating:  82%|████████▏ | 89/108 [01:45<00:33,  1.75s/it]
Evaluating:  83%|████████▎ | 90/108 [01:46<00:22,  1.27s/it]
Evaluating:  84%|████████▍ | 91/108 [01:46<00:15,  1.07it/s]
Evaluating:  85%|████████▌ | 92/108 [01:46<00:11,  1.44it/s]
Evaluating:  86%|████████▌ | 93/108 [01:50<00:25,  1.69s/it]
Evaluating:  87%|████████▋ | 94/108 [01:50<00:17,  1.23s/it]
Evaluating:  88%|████████▊ | 95/108 [01:50<00:11,  1.11it/s]
Evaluating:  89%|████████▉ | 96/108 [01:50<00:07,  1.50it/s]
Evaluating:  90%|████████▉ | 97/108 [01:54<00:17,  1.60s/it]
Evaluating:  91%|█████████ | 98/108 [01:54<00:11,  1.17s/it]
Evaluating:  92%|█████████▏| 99/108 [01:54<00:07,  1.17it/s]
Evaluating:  93%|█████████▎| 100/108 [01:54<00:05,  1.56it/s]
Evaluating:  94%|█████████▎| 101/108 [01:58<00:11,  1.60s/it]
Evaluating:  94%|█████████▍| 102/108 [01:58<00:06,  1.16s/it]
Evaluating:  95%|█████████▌| 103/108 [01:59<00:04,  1.17it/s]
Evaluating:  96%|█████████▋| 104/108 [01:59<00:02,  1.58it/s]
Evaluating:  97%|█████████▋| 105/108 [02:01<00:03,  1.24s/it]
Evaluating:  98%|█████████▊| 106/108 [02:01<00:01,  1.10it/s]
Evaluating:  99%|█████████▉| 107/108 [02:02<00:00,  1.49it/s]
Evaluating: 100%|██████████| 108/108 [02:02<00:00,  1.13s/it]
Test Accuracy: 0.8494
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.67      0.74       695
           1       0.78      0.94      0.85      1086
           2       0.95      0.93      0.94       799
           3       0.89      0.81      0.85       852

    accuracy                           0.85      3432
   macro avg       0.86      0.84      0.84      3432
weighted avg       0.85      0.85      0.85      3432

